# Processing Montserrat CSV catalog with AAA
---

**Last update:** 12/2021 - Glenn THOMPSON  
**Contact:** thompsong@usf.edu   
---

In [ ]:
################################
# General setup                #
################################

import os
import pandas as pd
import pickle
import numpy as np


# Change if you want your screen to keep quiet
# 0 = quiet
# 1 = in between
# 2 = detailed information
verbatim = 2

### PREPARE THE CATALOG DataFrame ###
SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO') # e.g. /home/user/seismo
pandaSeisDir = os.path.join(SEISAN_DATA, 'miniseed_c') # e.g. /home/user/seismo/pandaSeis
SEISAN_DB = 'MVOE_' # e.g. the seisan database name (e.g. MVOE_)
PROJECTDIR = os.path.join(os.getenv('HOME'),'src', 'kitchensinkGT', 'PROJECTS', 'MontserratML') # this dir
#csvfile_external = os.path.join(PROJECTDIR, 'MVO_labelled_events.csv')
csvfile_external = os.path.join(SEISAN_DATA, 'MachineLearning', SEISAN_DB, 'runAAA', 'MVOE_11_labelled_events.csv')
#csvfile_internal = './catalog/MVO_labelled_events_filtered.csv'
csvfile_internal = 'catalog/30_MVO_labelled_events_filtered.csv' # has to match that in AAA-master/config/general/newsettings_10.json
csvfile_internal = './AAA-master/MONTSERRAT/' + csvfile_internal
output_path_cat = csvfile_internal.replace('.csv', '.pd')
alltraces_file = '30_alltraceDFs.csv'

################################
# Machine learning and testing #
################################
import sys
sys.path.insert(0, './AAA-master/automatic_processing')
#import tools
from config import Config
from analyzer import Analyzer

# Change if you want your screen to keep quiet
# 0 = quiet
# 1 = in between
# 2 = detailed information
verbatim = 2

# Init project with configuration file
config = Config('./AAA-master/config/general/newsettings_10.json', verbatim=verbatim)
config.readAndCheck()  

##########################
# Variables to loop over #
##########################
alltraces = pd.read_csv(alltraces_file)

traceIDs = ['MV.MBWH..SHZ', 'MV.MBLG..SHZ', 'MV.MBRY..SHZ']
minWeights = range(4)
classes_to_include = [ ['l', 't'], ['e', 'r'], ['h', 'l', 't'], ['h', 'l', 't', 'r'], ['e', 'h', 'l', 't', 'r'] ]

traceIDs = ['MV.MBWH..SHZ']
minWeights = range(1)
classes_to_include = [ ['l', 't'] ]


#############
# Functions #
#############

def cat_filter_traceID(cat, alltraces, traceID):
    # subset catalog based on traceID
    matchingEvents = alltraces[alltraces['id']==traceID]
    for i, row in cat.iterrows():
        matching_indices = matchingEvents.index[matchingEvents['filetime']==row['filetime']].tolist()
        if len(matching_indices)==1:
            pass
        else:
            cat.drop(i, inplace=True)
    N = len(cat.index)
    #print('%d events after matching against traceID' % N)
    return N

def cat_filter_classes(cat, remove_classes):
    """
    for rmclass in remove_classes:
        print('Removing %s' % rmclass)
        cat = cat[cat['class']!=rmclass]
    """
    cat=cat[cat["class"].isin(remove_classes)]
    N = len(cat.index)
    #print('%d events after removing classes' % N)
    return cat, N

def cat_filter_weight(cat, minWeight):
    #if minWeight>0:
    #    cat = cat[cat['weight']>=minWeight]
    cat=cat[cat["weight"].isin(range(minWeight,13))]
    N = len(cat.index)
    #print('%d events after filtering above %d' % (N, minWeight))
    return cat, N

def cat_check_numbers(cat, minthresh = 20):
    df = cat.copy()
    tooSmall = False
    lengths = []
    for subclass in df['class'].unique():
        dfs = df[df['class']==subclass]
        N = len(dfs.index)
        if N<minthresh:
            tooSmall=True
        lengths.append(N)
    return tooSmall, lengths


#######################
# Looping starts here #
#######################
minPerClass = 30
counter = 0
results_list = []
for traceID in traceIDs:
    
    # what traceID are we looking for - read_montserrat needs this - should figure out how to write this into the config
    fptr = open('./AAA-master/MONTSERRAT/current_traceID.txt','w')
    fptr.write(traceID)
    fptr.close()
    
    for minWeight in minWeights:
        for include_classes in classes_to_include:
            # reload cat because we filter it down each time
            #cat = pickle.load(open(output_path_cat,'rb'))
            cat = pd.read_csv(csvfile_internal)
            print(cat['class'].value_counts())
            print(traceID, include_classes, minWeight)
            
            results_dict = {}
            results_dict['traceID'] = traceID
            results_dict['classes'] = ','.join(include_classes)
            results_dict['minWeight'] = minWeight
            results_dict['NtraceID'] = cat_filter_traceID(cat, alltraces, traceID)
            cat, N = cat_filter_classes(cat, include_classes) 
            results_dict['Nclasses'] = N
            cat, N = cat_filter_weight(cat, minWeight)
            results_dict['Nweight'] = N
            tooSmall, lengths = cat_check_numbers(cat, minPerClass)
            results_dict['counts'] = str(lengths)
            
            results_dict['acc_mean'] = None
            results_dict['acc_std'] = None
            
            if N>=minPerClass*len(include_classes) and not tooSmall:
                #try:
                    print(cat.groupby('class').size())
                    analyzer = Analyzer(config, verbatim=verbatim, catalog=cat)
                    allData, allLabels, acc = analyzer.learn(config, returnData=True, verbatim=3) # If you want the data
                    results_dict['acc_mean'] = np.round(np.mean(acc)*100, 1)
                    results_dict['acc_std'] = np.round(np.std(acc)*100, 1)                        
                    cat['predicted_class'] = allLabels
                    cat.to_csv(csvfile_internal.replace('.csv','_predicted_%d.csv' % counter)) 
            results_list.append(results_dict)
            counter += 1
            
resultsDF = pd.DataFrame(results_list)
resultsDF.to_csv('new_results.csv')################################

# General setup                #

################################

​

import os

import pandas as pd

import pickle

import numpy as np

​

​

# Change if you want your screen to keep quiet

# 0 = quiet

# 1 = in between

# 2 = detailed information

verbatim = 2

​

### PREPARE THE CATALOG DataFrame ###

SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO') # e.g. /home/user/seismo

pandaSeisDir = os.path.join(SEISAN_DATA, 'miniseed_c') # e.g. /home/user/seismo/pandaSeis

SEISAN_DB = 'MVOE_' # e.g. the seisan database name (e.g. MVOE_)

PROJECTDIR = os.path.join(os.getenv('HOME'),'src', 'kitchensinkGT', 'PROJECTS', 'MontserratML') # this dir

#csvfile_external = os.path.join(PROJECTDIR, 'MVO_labelled_events.csv')

csvfile_external = os.path.join(SEISAN_DATA, 'MachineLearning', SEISAN_DB, 'runAAA', 'MVOE_11_labelled_events.csv')

#csvfile_internal = './catalog/MVO_labelled_events_filtered.csv'

csvfile_internal = 'catalog/30_MVO_labelled_events_filtered.csv' # has to match that in AAA-master/config/general/newsettings_10.json

csvfile_internal = './AAA-master/MONTSERRAT/' + csvfile_internal

output_path_cat = csvfile_internal.replace('.csv', '.pd')

alltraces_file = '30_alltraceDFs.csv'

​

################################

# Machine learning and testing #

################################

import sys

sys.path.insert(0, './AAA-master/automatic_processing')

#import tools

from config import Config

from analyzer import Analyzer

​

# Change if you want your screen to keep quiet

# 0 = quiet

# 1 = in between

# 2 = detailed information

verbatim = 2

​

# Init project with configuration file

config = Config('./AAA-master/config/general/newsettings_10.json', verbatim=verbatim)

config.readAndCheck()  

​

##########################

# Variables to loop over #

##########################

alltraces = pd.read_csv(alltraces_file)

​

traceIDs = ['MV.MBWH..SHZ', 'MV.MBLG..SHZ', 'MV.MBRY..SHZ']

minWeights = range(4)

classes_to_include = [ ['l', 't'], ['e', 'r'], ['h', 'l', 't'], ['h', 'l', 't', 'r'], ['e', 'h', 'l', 't', 'r'] ]

​

traceIDs = ['MV.MBWH..SHZ']

minWeights = range(1)

classes_to_include = [ ['l', 't'] ]

​

​

#############

# Functions #

#############

​

def cat_filter_traceID(cat, alltraces, traceID):

    # subset catalog based on traceID

    matchingEvents = alltraces[alltraces['id']==traceID]

    for i, row in cat.iterrows():

        matching_indices = matchingEvents.index[matchingEvents['filetime']==row['filetime']].tolist()

        if len(matching_indices)==1:

            pass

        else:

            cat.drop(i, inplace=True)

    N = len(cat.index)

    #print('%d events after matching against traceID' % N)

    return N

​

def cat_filter_classes(cat, remove_classes):

    """

    for rmclass in remove_classes:

        print('Removing %s' % rmclass)

        cat = cat[cat['class']!=rmclass]

    """

    cat=cat[cat["class"].isin(remove_classes)]

    N = len(cat.index)

    #print('%d events after removing classes' % N)

    return cat, N

​

def cat_filter_weight(cat, minWeight):

    #if minWeight>0:

    #    cat = cat[cat['weight']>=minWeight]

    cat=cat[cat["weight"].isin(range(minWeight,13))]

    N = len(cat.index)

    #print('%d events after filtering above %d' % (N, minWeight))

    return cat, N

​

def cat_check_numbers(cat, minthresh = 20):

    df = cat.copy()

    tooSmall = False

    lengths = []

    for subclass in df['class'].unique():

        dfs = df[df['class']==subclass]

        N = len(dfs.index)

        if N<minthresh:

            tooSmall=True

        lengths.append(N)

    return tooSmall, lengths

​

​

#######################

# Looping starts here #

#######################

minPerClass = 30

counter = 0

results_list = []

for traceID in traceIDs:

    

    # what traceID are we looking for - read_montserrat needs this - should figure out how to write this into the config

    fptr = open('./AAA-master/MONTSERRAT/current_traceID.txt','w')

    fptr.write(traceID)

    fptr.close()

    

    for minWeight in minWeights:

        for include_classes in classes_to_include:

            # reload cat because we filter it down each time

            #cat = pickle.load(open(output_path_cat,'rb'))

            cat = pd.read_csv(csvfile_internal)

            print(cat['class'].value_counts())

            print(traceID, include_classes, minWeight)

            

            results_dict = {}

            results_dict['traceID'] = traceID

            results_dict['classes'] = ','.join(include_classes)

            results_dict['minWeight'] = minWeight

            results_dict['NtraceID'] = cat_filter_traceID(cat, alltraces, traceID)

            cat, N = cat_filter_classes(cat, include_classes) 

            results_dict['Nclasses'] = N

            cat, N = cat_filter_weight(cat, minWeight)

            results_dict['Nweight'] = N

            tooSmall, lengths = cat_check_numbers(cat, minPerClass)

            results_dict['counts'] = str(lengths)

            

            results_dict['acc_mean'] = None

            results_dict['acc_std'] = None

            

            if N>=minPerClass*len(include_classes) and not tooSmall:

                #try:

                    print(cat.groupby('class').size())

                    analyzer = Analyzer(config, verbatim=verbatim, catalog=cat)

                    allData, allLabels, acc = analyzer.learn(config, returnData=True, verbatim=3) # If you want the data

                    results_dict['acc_mean'] = np.round(np.mean(acc)*100, 1)

                    results_dict['acc_std'] = np.round(np.std(acc)*100, 1)                        

                    cat['predicted_class'] = allLabels

                    cat.to_csv(csvfile_internal.replace('.csv','_predicted_%d.csv' % counter)) 

            results_list.append(results_dict)

            counter += 1

            

resultsDF = pd.DataFrame(results_list)

resultsDF.to_csv('new_results.csv')

print('Done')

​

​

./AAA-master/config/specific/usecase1_continuous_classification/usecase1_EXAMPLE.json
Welcome to this automatic analysis architecture
Copyright: Marielle MALFANTE - GIPSA-Lab
Univ. Grenoble Alpes, CNRS, Grenoble INP, GIPSA-lab, 38000 Grenoble, France

 *** PROJECT CONFIGURATION 10 ***  
Configuration object from <path> ./AAA-master/config/general/newsettings_10.json, <configuration_number> 10,
 <general> {'project_root': './AAA-master/', 'analysis_type': 'continuous', 'path_to_specific_settings_file': 'config/specific/usecase1_continuous_classification/usecase1_EXAMPLE.json', 'path_to_res': 'res/', 'path_to_visuals': 'fig/', 'path_to_res_to_review': 'res_to_review/'},
 <application> {'name': 'montserrat'},
 <preprocessing> {'energy_norm': True},
 <learning> {'algo': RandomForestClassifier(criterion='entropy'), 'cv': StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5), 'path_to_catalogue': 'catalog/30_MVO_labelled_events_filtered.pd'},
 <features> {'path_to_config': 'config/specific/features/features_montserrat.json', 'computation_domains': 'time spectral cepstral', 'thresholding': True, 'thresholds': [0.8, 0.8, 0.8, 0.8, 0.8]},
 <data_to_analyze> {'path_to_data': './miniseed_c', 'data_files': '*.mseed', 'reading_function': <function read_montserrat at 0x7f24d7bb8b70>, 'reading_arguments': {}},
 <analysis> {'n_window': 1, 'window_length': 15.0, 'delta': 100, 'bandwidth': {'f_min': [0.5], 'f_max': [25]}, 'butter_order': 2, 'spectro_window_size': 256, 'f_max': 25, 'nBands': 1}

r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['l', 't'] 0
class
l    103
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 204
75.0 3975
Processing waveform 1 of 204
75.0 5025
Processing waveform 2 of 204
75.0 6825
Processing waveform 3 of 204
75.0 9075
Processing waveform 4 of 204
75.0 4500
Processing waveform 5 of 204
75.0 2175
Processing waveform 6 of 204
75.0 3300
Processing waveform 7 of 204
75.0 4275
Processing waveform 8 of 204
75.0 2850
Processing waveform 9 of 204
75.0 2850
Processing waveform 10 of 204
75.0 2700
Processing waveform 11 of 204
75.0 5250
Processing waveform 12 of 204
75.0 3000
Processing waveform 13 of 204
75.0 2775
Processing waveform 14 of 204
75.0 6450
Processing waveform 15 of 204
75.0 3150
Processing waveform 16 of 204
75.0 1650
Processing waveform 17 of 204
75.0 4500
Processing waveform 18 of 204
75.0 2250
Processing waveform 19 of 204
75.0 2175
Processing waveform 20 of 204
75.0 3225
Processing waveform 21 of 204
75.0 4050
Processing waveform 22 of 204
75.0 3300
Processing waveform 23 of 204
75.0 2850
Processing waveform 24 of 204
75.0 2025
Processing waveform 25 of 204
75.0 4800
Processing waveform 26 of 204
75.0 2325
Processing waveform 27 of 204
75.0 6150
Processing waveform 28 of 204
75.0 2775
Processing waveform 29 of 204
75.0 2850
Processing waveform 30 of 204
75.0 4425
Processing waveform 31 of 204
75.0 2250
Processing waveform 32 of 204
75.0 3075
Processing waveform 33 of 204
75.0 2775
Processing waveform 34 of 204
75.0 1875
Processing waveform 35 of 204
75.0 3300
Processing waveform 36 of 204
75.0 1875
Processing waveform 37 of 204
75.0 5550
Processing waveform 38 of 204
75.0 9075
Processing waveform 39 of 204
75.0 4650
Processing waveform 40 of 204
75.0 6525
Processing waveform 41 of 204
75.0 9075
Processing waveform 42 of 204
75.0 3225
Processing waveform 43 of 204
75.0 3150
Processing waveform 44 of 204
75.0 4275
Processing waveform 45 of 204
75.0 4050
Processing waveform 46 of 204
75.0 5400
Processing waveform 47 of 204
75.0 4725
Processing waveform 48 of 204
75.0 3450
Processing waveform 49 of 204
75.0 6150
Processing waveform 50 of 204
75.0 2250
Processing waveform 51 of 204
75.0 9000
Processing waveform 52 of 204
75.0 3150
Processing waveform 53 of 204
75.0 4950
Processing waveform 54 of 204
75.0 6750
Processing waveform 55 of 204
75.0 3150
Processing waveform 56 of 204
75.0 7050
Processing waveform 57 of 204
75.0 3675
Processing waveform 58 of 204
75.0 3225
Processing waveform 59 of 204
75.0 2325
Processing waveform 60 of 204
75.0 7500
Processing waveform 61 of 204
75.0 3975
Processing waveform 62 of 204
75.0 6150
Processing waveform 63 of 204
75.0 7875
Processing waveform 64 of 204
75.0 7275
Processing waveform 65 of 204
75.0 6075
Processing waveform 66 of 204
75.0 3900
Processing waveform 67 of 204
75.0 3000
Processing waveform 68 of 204
75.0 3900
Processing waveform 69 of 204
75.0 6525
Processing waveform 70 of 204
75.0 2925
Processing waveform 71 of 204
75.0 2700
Processing waveform 72 of 204
75.0 2250
Processing waveform 73 of 204
75.0 4425
Processing waveform 74 of 204
75.0 7500
Processing waveform 75 of 204
75.0 2250
Processing waveform 76 of 204
75.0 9600
Processing waveform 77 of 204
75.0 4650
Processing waveform 78 of 204
75.0 6750
Processing waveform 79 of 204
75.0 8925
Processing waveform 80 of 204
75.0 7125
Processing waveform 81 of 204
75.0 3000
Processing waveform 82 of 204
75.0 10200
Processing waveform 83 of 204
75.0 1800
Processing waveform 84 of 204
75.0 4425
Processing waveform 85 of 204
75.0 2250
Processing waveform 86 of 204
75.0 6150
Processing waveform 87 of 204
75.0 7125
Processing waveform 88 of 204
75.0 3600
Processing waveform 89 of 204
75.0 6000
Processing waveform 90 of 204
75.0 4800
Processing waveform 91 of 204
75.0 7200
Processing waveform 92 of 204
75.0 3000
Processing waveform 93 of 204
75.0 4500
Processing waveform 94 of 204
75.0 3600
Processing waveform 95 of 204
75.0 4875
Processing waveform 96 of 204
75.0 5625
Processing waveform 97 of 204
75.0 3150
Processing waveform 98 of 204
75.0 3975
Processing waveform 99 of 204
75.0 5325
Processing waveform 100 of 204
75.0 2850
Processing waveform 101 of 204
75.0 9450
Processing waveform 102 of 204
75.0 6525
Processing waveform 103 of 204
75.0 2250
Processing waveform 104 of 204
75.0 3600
Processing waveform 105 of 204
75.0 5775
Processing waveform 106 of 204
75.0 2325
Processing waveform 107 of 204
75.0 2325
Processing waveform 108 of 204
75.0 3525
Processing waveform 109 of 204
75.0 4500
Processing waveform 110 of 204
75.0 4275
Processing waveform 111 of 204
75.0 3450
Processing waveform 112 of 204
75.0 2550
Processing waveform 113 of 204
75.0 3300
Processing waveform 114 of 204
75.0 9075
Processing waveform 115 of 204
75.0 9075
Processing waveform 116 of 204
75.0 6525
Processing waveform 117 of 204
75.0 6975
Processing waveform 118 of 204
75.0 6750
Processing waveform 119 of 204
75.0 7275
Processing waveform 120 of 204
75.0 2400
Processing waveform 121 of 204
75.0 4125
Processing waveform 122 of 204
75.0 3525
Processing waveform 123 of 204
75.0 5175
Processing waveform 124 of 204
75.0 3750
Processing waveform 125 of 204
75.0 4875
Processing waveform 126 of 204
75.0 2250
Processing waveform 127 of 204
75.0 4125
Processing waveform 128 of 204
75.0 4425
Processing waveform 129 of 204
75.0 5250
Processing waveform 130 of 204
75.0 9075
Processing waveform 131 of 204
75.0 4725
Processing waveform 132 of 204
75.0 2475
Processing waveform 133 of 204
75.0 2925
Processing waveform 134 of 204
75.0 7350
Processing waveform 135 of 204
75.0 2550
Processing waveform 136 of 204
75.0 2850
Processing waveform 137 of 204
75.0 2550
Processing waveform 138 of 204
75.0 4425
Processing waveform 139 of 204
75.0 5775
Processing waveform 140 of 204
75.0 4200
Processing waveform 141 of 204
75.0 2175
Processing waveform 142 of 204
75.0 2925
Processing waveform 143 of 204
75.0 10350
Processing waveform 144 of 204
75.0 15600
Processing waveform 145 of 204
75.0 6450
Processing waveform 146 of 204
75.0 3450
Processing waveform 147 of 204
75.0 11175
Processing waveform 148 of 204
75.0 4725
Processing waveform 149 of 204
75.0 4200
Processing waveform 150 of 204
75.0 3750
Processing waveform 151 of 204
75.0 3975
Processing waveform 152 of 204
75.0 4575
Processing waveform 153 of 204
75.0 3225
Processing waveform 154 of 204
75.0 13500
Processing waveform 155 of 204
75.0 4050
Processing waveform 156 of 204
75.0 5250
Processing waveform 157 of 204
75.0 7050
Processing waveform 158 of 204
75.0 3450
Processing waveform 159 of 204
75.0 6675

Processing waveform 160 of 204
75.0 2850
Processing waveform 161 of 204
75.0 6750
Processing waveform 162 of 204
75.0 2475
Processing waveform 163 of 204
75.0 7650
Processing waveform 164 of 204
75.0 6750
Processing waveform 165 of 204
75.0 4125
Processing waveform 166 of 204
75.0 7425
Processing waveform 167 of 204
75.0 5400
Processing waveform 168 of 204
75.0 4800
Processing waveform 169 of 204
75.0 10200
Processing waveform 170 of 204
75.0 18225
Processing waveform 171 of 204
75.0 5925
Processing waveform 172 of 204
75.0 3525
Processing waveform 173 of 204
75.0 3300
Processing waveform 174 of 204
75.0 3600
Processing waveform 175 of 204
75.0 4650
Processing waveform 176 of 204
75.0 7875
Processing waveform 177 of 204
75.0 6300
Processing waveform 178 of 204
75.0 6750
Processing waveform 179 of 204
75.0 2925
Processing waveform 180 of 204
75.0 3525
Processing waveform 181 of 204
75.0 7875
Processing waveform 182 of 204
75.0 8925
Processing waveform 183 of 204
75.0 2850
Processing waveform 184 of 204
75.0 2475
Processing waveform 185 of 204
75.0 2325
Processing waveform 186 of 204
75.0 2250
Processing waveform 187 of 204
75.0 2700
Processing waveform 188 of 204
75.0 3000
Processing waveform 189 of 204
75.0 3300
Processing waveform 190 of 204
75.0 4425
Processing waveform 191 of 204
75.0 4500
Processing waveform 192 of 204
75.0 7500
Processing waveform 193 of 204
75.0 4200
Processing waveform 194 of 204
75.0 4800
Processing waveform 195 of 204
75.0 3000
Processing waveform 196 of 204
75.0 2475
Processing waveform 197 of 204
75.0 1950
Processing waveform 198 of 204
75.0 2625
Processing waveform 199 of 204
75.0 4650
Processing waveform 200 of 204
75.0 5475
Processing waveform 201 of 204
75.0 4950
Processing waveform 202 of 204
75.0 6375
Processing waveform 203 of 204
75.0 7500
Training data have been read and features have been extracted  (204, 120)
Computation time:  284.4824905395508
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.16238975524902344
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l   103       
        t         101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  52.7124183007  +/-  3.90259571585  %
          Predicted class
              l     t 
        l    29    23 
        t    26    25 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'r'] 0
class
e     93
r    110
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 203
75.0 5325
Processing waveform 1 of 203
75.0 2700
Processing waveform 2 of 203
75.0 4350
Processing waveform 3 of 203
75.0 2775
Processing waveform 4 of 203
75.0 1950
Processing waveform 5 of 203
75.0 2250
Processing waveform 6 of 203
75.0 2250
Processing waveform 7 of 203
75.0 2250
Processing waveform 8 of 203
75.0 5100
Processing waveform 9 of 203
75.0 2475
Processing waveform 10 of 203
75.0 3300
Processing waveform 11 of 203
75.0 2175
Processing waveform 12 of 203
75.0 6300
Processing waveform 13 of 203
75.0 9075
Processing waveform 14 of 203
75.0 3225
Processing waveform 15 of 203
75.0 4125
Processing waveform 16 of 203
75.0 3300
Processing waveform 17 of 203
75.0 3225
Processing waveform 18 of 203
75.0 2850
Processing waveform 19 of 203
75.0 5400
Processing waveform 20 of 203
75.0 2400
Processing waveform 21 of 203
75.0 7725
Processing waveform 22 of 203
75.0 3900
Processing waveform 23 of 203
75.0 2775
Processing waveform 24 of 203
75.0 2325
Processing waveform 25 of 203
75.0 4425
Processing waveform 26 of 203
75.0 3525
Processing waveform 27 of 203
75.0 6300
Processing waveform 28 of 203
75.0 2625
Processing waveform 29 of 203
75.0 4050
Processing waveform 30 of 203
75.0 2775
Processing waveform 31 of 203
75.0 2400
Processing waveform 32 of 203
75.0 2850
Processing waveform 33 of 203
75.0 2250
Processing waveform 34 of 203
75.0 2325
Processing waveform 35 of 203
75.0 3750
Processing waveform 36 of 203
75.0 3525
Processing waveform 37 of 203
75.0 5700
Processing waveform 38 of 203
75.0 2700
Processing waveform 39 of 203
75.0 3000
Processing waveform 40 of 203
75.0 4350
Processing waveform 41 of 203
75.0 4950
Processing waveform 42 of 203
75.0 2925
Processing waveform 43 of 203
75.0 2775
Processing waveform 44 of 203
75.0 3450
Processing waveform 45 of 203
75.0 4875
Processing waveform 46 of 203
75.0 9075
Processing waveform 47 of 203
75.0 5175
Processing waveform 48 of 203
75.0 3075
Processing waveform 49 of 203
75.0 4125
Processing waveform 50 of 203
75.0 9075
Processing waveform 51 of 203
75.0 3225
Processing waveform 52 of 203
75.0 2775
Processing waveform 53 of 203
75.0 4575
Processing waveform 54 of 203
75.0 2700
Processing waveform 55 of 203
75.0 1950
Processing waveform 56 of 203
75.0 2100
Processing waveform 57 of 203
75.0 4575
Processing waveform 58 of 203
75.0 5850
Processing waveform 59 of 203
75.0 3150
Processing waveform 60 of 203
75.0 4275
Processing waveform 61 of 203
75.0 9075
Processing waveform 62 of 203
75.0 3600
Processing waveform 63 of 203
75.0 2625
Processing waveform 64 of 203
75.0 2700
Processing waveform 65 of 203
75.0 2700
Processing waveform 66 of 203
75.0 3525
Processing waveform 67 of 203
75.0 4200
Processing waveform 68 of 203
75.0 4200
Processing waveform 69 of 203
75.0 1950
Processing waveform 70 of 203
75.0 5775
Processing waveform 71 of 203
75.0 9075
Processing waveform 72 of 203
75.0 4200
Processing waveform 73 of 203
75.0 2625
Processing waveform 74 of 203
75.0 4875
Processing waveform 75 of 203
75.0 5025
Processing waveform 76 of 203
75.0 14625
Processing waveform 77 of 203
75.0 11025
Processing waveform 78 of 203
75.0 1950
Processing waveform 79 of 203
75.0 3975
Processing waveform 80 of 203
75.0 2550
Processing waveform 81 of 203
75.0 7500
Processing waveform 82 of 203
75.0 14925
Processing waveform 83 of 203
75.0 13275
Processing waveform 84 of 203
75.0 1725
Processing waveform 85 of 203
75.0 2400
Processing waveform 86 of 203
75.0 6375
Processing waveform 87 of 203
75.0 6450
Processing waveform 88 of 203
75.0 4125
Processing waveform 89 of 203
75.0 3975
Processing waveform 90 of 203
75.0 1725
Processing waveform 91 of 203
75.0 2700
Processing waveform 92 of 203
75.0 4800
Processing waveform 93 of 203
75.0 7125
Processing waveform 94 of 203
75.0 5325
Processing waveform 95 of 203
75.0 5400
Processing waveform 96 of 203
75.0 6825
Processing waveform 97 of 203
75.0 7050
Processing waveform 98 of 203
75.0 4500
Processing waveform 99 of 203
75.0 5625
Processing waveform 100 of 203
75.0 4275
Processing waveform 101 of 203
75.0 2925
Processing waveform 102 of 203
75.0 3150
Processing waveform 103 of 203
75.0 6000
Processing waveform 104 of 203
75.0 3150
Processing waveform 105 of 203
75.0 6000
Processing waveform 106 of 203
75.0 12600
Processing waveform 107 of 203
75.0 7425
Processing waveform 108 of 203
75.0 16725
Processing waveform 109 of 203
75.0 6075
Processing waveform 110 of 203
75.0 3450
Processing waveform 111 of 203
75.0 2325
Processing waveform 112 of 203
75.0 2925
Processing waveform 113 of 203
75.0 4575
Processing waveform 114 of 203
75.0 7050
Processing waveform 115 of 203
75.0 2025
Processing waveform 116 of 203
75.0 2925
Processing waveform 117 of 203
75.0 2850
Processing waveform 118 of 203
75.0 3600
Processing waveform 119 of 203
75.0 3750
Processing waveform 120 of 203
75.0 2100
Processing waveform 121 of 203
75.0 1650
Processing waveform 122 of 203
75.0 4200
Processing waveform 123 of 203
75.0 9075
Processing waveform 124 of 203
75.0 4125
Processing waveform 125 of 203
75.0 2925
Processing waveform 126 of 203
75.0 2250
Processing waveform 127 of 203
75.0 5475
Processing waveform 128 of 203
75.0 2625
Processing waveform 129 of 203
75.0 2400
Processing waveform 130 of 203
75.0 2700
Processing waveform 131 of 203
75.0 9075
Processing waveform 132 of 203
75.0 3300
Processing waveform 133 of 203
75.0 9075
Processing waveform 134 of 203
75.0 6225
Processing waveform 135 of 203
75.0 7125

Processing waveform 136 of 203
75.0 5850
Processing waveform 137 of 203
75.0 9075
Processing waveform 138 of 203
75.0 3225
Processing waveform 139 of 203
75.0 8100
Processing waveform 140 of 203
75.0 4575
Processing waveform 141 of 203
75.0 2850
Processing waveform 142 of 203
75.0 3375
Processing waveform 143 of 203
75.0 2100
Processing waveform 144 of 203
75.0 9075
Processing waveform 145 of 203
75.0 5850
Processing waveform 146 of 203
75.0 2775
Processing waveform 147 of 203
75.0 2700
Processing waveform 148 of 203
75.0 2100
Processing waveform 149 of 203
75.0 9075
Processing waveform 150 of 203
75.0 3225
Processing waveform 151 of 203
75.0 3300
Processing waveform 152 of 203
75.0 2475
Processing waveform 153 of 203
75.0 3075
Processing waveform 154 of 203
75.0 6450
Processing waveform 155 of 203
75.0 3600
Processing waveform 156 of 203
75.0 4650
Processing waveform 157 of 203
75.0 2475
Processing waveform 158 of 203
75.0 1875
Processing waveform 159 of 203
75.0 3450
Processing waveform 160 of 203
75.0 1500
Processing waveform 161 of 203
75.0 2625
Processing waveform 162 of 203
75.0 9075
Processing waveform 163 of 203
75.0 3525
Processing waveform 164 of 203
75.0 5625
Processing waveform 165 of 203
75.0 3675
Processing waveform 166 of 203
75.0 9075
Processing waveform 167 of 203
75.0 2700
Processing waveform 168 of 203
75.0 9075
Processing waveform 169 of 203
75.0 3000
Processing waveform 170 of 203
75.0 3525
Processing waveform 171 of 203
75.0 7200
Processing waveform 172 of 203
75.0 4875
Processing waveform 173 of 203
75.0 6075
Processing waveform 174 of 203
75.0 2700
Processing waveform 175 of 203
75.0 5250
Processing waveform 176 of 203
75.0 12750
Processing waveform 177 of 203
75.0 5100
Processing waveform 178 of 203
75.0 2475
Processing waveform 179 of 203
75.0 6525
Processing waveform 180 of 203
75.0 7200
Processing waveform 181 of 203
75.0 4500
Processing waveform 182 of 203
75.0 4950
Processing waveform 183 of 203
75.0 8250
Processing waveform 184 of 203
75.0 3150
Processing waveform 185 of 203
75.0 5850
Processing waveform 186 of 203
75.0 2325
Processing waveform 187 of 203
75.0 4200
Processing waveform 188 of 203
75.0 3300
Processing waveform 189 of 203
75.0 2325
Processing waveform 190 of 203
75.0 13500
Processing waveform 191 of 203
75.0 3525
Processing waveform 192 of 203
75.0 3075
Processing waveform 193 of 203
75.0 6600
Processing waveform 194 of 203
75.0 6225
Processing waveform 195 of 203
75.0 9000
Processing waveform 196 of 203
75.0 5100
Processing waveform 197 of 203
75.0 6975
Processing waveform 198 of 203
75.0 4425
Processing waveform 199 of 203
75.0 5400
Processing waveform 200 of 203
75.0 13350
Processing waveform 201 of 203
75.0 9525
Processing waveform 202 of 203
75.0 2325
Training data have been read and features have been extracted  (203, 120)
Computation time:  292.37775206565857
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.16533493995666504
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    93       
        r         110 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  52.5163398693  +/-  4.10270658104  %
          Predicted class
              e     r 
        e    20    27 
        r    21    34 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't'] 0
class
h    106
l    103
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 310
75.0 3975
Processing waveform 1 of 310
75.0 5025
Processing waveform 2 of 310
75.0 6825
Processing waveform 3 of 310
75.0 9075
Processing waveform 4 of 310
75.0 4500
Processing waveform 5 of 310
75.0 2175
Processing waveform 6 of 310
75.0 3300
Processing waveform 7 of 310
75.0 4275
Processing waveform 8 of 310
75.0 2850
Processing waveform 9 of 310
75.0 2850
Processing waveform 10 of 310
75.0 2700
Processing waveform 11 of 310
75.0 5250
Processing waveform 12 of 310
75.0 3000
Processing waveform 13 of 310
75.0 2775
Processing waveform 14 of 310
75.0 6450
Processing waveform 15 of 310
75.0 3150
Processing waveform 16 of 310
75.0 1650
Processing waveform 17 of 310
75.0 4500
Processing waveform 18 of 310
75.0 2250
Processing waveform 19 of 310
75.0 2175
Processing waveform 20 of 310
75.0 3225
Processing waveform 21 of 310
75.0 4050
Processing waveform 22 of 310
75.0 3300
Processing waveform 23 of 310
75.0 2850
Processing waveform 24 of 310
75.0 2025
Processing waveform 25 of 310
75.0 4800
Processing waveform 26 of 310
75.0 2325
Processing waveform 27 of 310
75.0 6150
Processing waveform 28 of 310
75.0 2775
Processing waveform 29 of 310
75.0 2850
Processing waveform 30 of 310
75.0 4425
Processing waveform 31 of 310
75.0 2250
Processing waveform 32 of 310
75.0 3075
Processing waveform 33 of 310
75.0 2775
Processing waveform 34 of 310
75.0 1875
Processing waveform 35 of 310
75.0 3300
Processing waveform 36 of 310
75.0 1875
Processing waveform 37 of 310
75.0 5550
Processing waveform 38 of 310
75.0 9075
Processing waveform 39 of 310
75.0 4650
Processing waveform 40 of 310
75.0 6525
Processing waveform 41 of 310
75.0 9075
Processing waveform 42 of 310
75.0 3225
Processing waveform 43 of 310
75.0 3150
Processing waveform 44 of 310
75.0 4275
Processing waveform 45 of 310
75.0 4050
Processing waveform 46 of 310
75.0 5400
Processing waveform 47 of 310
75.0 4725
Processing waveform 48 of 310
75.0 3450
Processing waveform 49 of 310
75.0 6150
Processing waveform 50 of 310
75.0 2250
Processing waveform 51 of 310
75.0 9000
Processing waveform 52 of 310
75.0 3150
Processing waveform 53 of 310
75.0 4950
Processing waveform 54 of 310
75.0 6750
Processing waveform 55 of 310
75.0 3150
Processing waveform 56 of 310
75.0 7050
Processing waveform 57 of 310
75.0 3675
Processing waveform 58 of 310
75.0 3225
Processing waveform 59 of 310
75.0 2325
Processing waveform 60 of 310
75.0 7500
Processing waveform 61 of 310
75.0 3975
Processing waveform 62 of 310
75.0 6150
Processing waveform 63 of 310
75.0 7875
Processing waveform 64 of 310
75.0 7275
Processing waveform 65 of 310
75.0 6075
Processing waveform 66 of 310
75.0 3900
Processing waveform 67 of 310
75.0 3000
Processing waveform 68 of 310
75.0 3900
Processing waveform 69 of 310
75.0 6525
Processing waveform 70 of 310
75.0 2925
Processing waveform 71 of 310
75.0 2700
Processing waveform 72 of 310
75.0 2250
Processing waveform 73 of 310
75.0 4425
Processing waveform 74 of 310
75.0 7500
Processing waveform 75 of 310
75.0 2250
Processing waveform 76 of 310
75.0 9600
Processing waveform 77 of 310
75.0 4650
Processing waveform 78 of 310
75.0 6750
Processing waveform 79 of 310
75.0 8925
Processing waveform 80 of 310
75.0 7125
Processing waveform 81 of 310
75.0 3000
Processing waveform 82 of 310
75.0 10200
Processing waveform 83 of 310
75.0 1800
Processing waveform 84 of 310
75.0 4425
Processing waveform 85 of 310
75.0 2250
Processing waveform 86 of 310
75.0 6150
Processing waveform 87 of 310
75.0 7125
Processing waveform 88 of 310
75.0 3600
Processing waveform 89 of 310
75.0 6000
Processing waveform 90 of 310
75.0 4800
Processing waveform 91 of 310
75.0 7200
Processing waveform 92 of 310
75.0 3000
Processing waveform 93 of 310
75.0 4500
Processing waveform 94 of 310
75.0 3600
Processing waveform 95 of 310
75.0 4875
Processing waveform 96 of 310
75.0 5625
Processing waveform 97 of 310
75.0 3150
Processing waveform 98 of 310
75.0 3975
Processing waveform 99 of 310
75.0 5325
Processing waveform 100 of 310
75.0 2850
Processing waveform 101 of 310
75.0 9450
Processing waveform 102 of 310
75.0 6525
Processing waveform 103 of 310
75.0 3225
Processing waveform 104 of 310
75.0 2100
Processing waveform 105 of 310
75.0 2700
Processing waveform 106 of 310
75.0 2325
Processing waveform 107 of 310
75.0 2775
Processing waveform 108 of 310
75.0 2625
Processing waveform 109 of 310
75.0 5850
Processing waveform 110 of 310
75.0 3825
Processing waveform 111 of 310
75.0 6000
Processing waveform 112 of 310
75.0 2025

Processing waveform 113 of 310
75.0 1725
Processing waveform 114 of 310
75.0 2175
Processing waveform 115 of 310
75.0 5400
Processing waveform 116 of 310
75.0 5325
Processing waveform 117 of 310
75.0 3750
Processing waveform 118 of 310
75.0 3075
Processing waveform 119 of 310
75.0 5550
Processing waveform 120 of 310
75.0 7350
Processing waveform 121 of 310
75.0 4650
Processing waveform 122 of 310
75.0 2775
Processing waveform 123 of 310
75.0 2325
Processing waveform 124 of 310
75.0 2925
Processing waveform 125 of 310
75.0 3075
Processing waveform 126 of 310
75.0 6375
Processing waveform 127 of 310
75.0 2475
Processing waveform 128 of 310
75.0 3975
Processing waveform 129 of 310
75.0 3150
Processing waveform 130 of 310
75.0 5925
Processing waveform 131 of 310
75.0 5400
Processing waveform 132 of 310
75.0 1875
Processing waveform 133 of 310
75.0 5850
Processing waveform 134 of 310
75.0 1725
Processing waveform 135 of 310
75.0 7650
Processing waveform 136 of 310
75.0 2700
Processing waveform 137 of 310
75.0 9075
Processing waveform 138 of 310
75.0 6975
Processing waveform 139 of 310
75.0 4050
Processing waveform 140 of 310
75.0 3450
Processing waveform 141 of 310
75.0 5925
Processing waveform 142 of 310
75.0 2250
Processing waveform 143 of 310
75.0 3675
Processing waveform 144 of 310
75.0 2625
Processing waveform 145 of 310
75.0 4575
Processing waveform 146 of 310
75.0 2025
Processing waveform 147 of 310
75.0 5175
Processing waveform 148 of 310
75.0 3225
Processing waveform 149 of 310
75.0 4950
Processing waveform 150 of 310
75.0 2925
Processing waveform 151 of 310
75.0 1800
Processing waveform 152 of 310
75.0 8925
Processing waveform 153 of 310
75.0 7350
Processing waveform 154 of 310
75.0 4125
Processing waveform 155 of 310
75.0 3225
Processing waveform 156 of 310
75.0 9075
Processing waveform 157 of 310
75.0 5025
Processing waveform 158 of 310
75.0 8925
Processing waveform 159 of 310
75.0 4800
Processing waveform 160 of 310
75.0 7800
Processing waveform 161 of 310
75.0 2775
Processing waveform 162 of 310
75.0 6600
Processing waveform 163 of 310
75.0 1800
Processing waveform 164 of 310
75.0 2625
Processing waveform 165 of 310
75.0 9075
Processing waveform 166 of 310
75.0 7200
Processing waveform 167 of 310
75.0 5550
Processing waveform 168 of 310
75.0 2775
Processing waveform 169 of 310
75.0 5250
Processing waveform 170 of 310
75.0 1875
Processing waveform 171 of 310
75.0 1875
Processing waveform 172 of 310
75.0 5025
Processing waveform 173 of 310
75.0 3600
Processing waveform 174 of 310
75.0 3375
Processing waveform 175 of 310
75.0 4050
Processing waveform 176 of 310
75.0 3375
Processing waveform 177 of 310
75.0 2925
Processing waveform 178 of 310
75.0 6675
Processing waveform 179 of 310
75.0 3750
Processing waveform 180 of 310
75.0 4575
Processing waveform 181 of 310
75.0 9075
Processing waveform 182 of 310
75.0 3900
Processing waveform 183 of 310
75.0 9075
Processing waveform 184 of 310
75.0 4875
Processing waveform 185 of 310
75.0 3075
Processing waveform 186 of 310
75.0 3150
Processing waveform 187 of 310
75.0 2550
Processing waveform 188 of 310
75.0 9075
Processing waveform 189 of 310
75.0 7425
Processing waveform 190 of 310
75.0 5025
Processing waveform 191 of 310
75.0 2850
Processing waveform 192 of 310
75.0 1875
Processing waveform 193 of 310
75.0 7125
Processing waveform 194 of 310
75.0 4500
Processing waveform 195 of 310
75.0 6075
Processing waveform 196 of 310
75.0 2475
Processing waveform 197 of 310
75.0 4125
Processing waveform 198 of 310
75.0 4500
Processing waveform 199 of 310
75.0 2100
Processing waveform 200 of 310
75.0 3975
Processing waveform 201 of 310
75.0 3225
Processing waveform 202 of 310
75.0 2550
Processing waveform 203 of 310
75.0 6975
Processing waveform 204 of 310
75.0 3075
Processing waveform 205 of 310
75.0 3675
Processing waveform 206 of 310
75.0 4800
Processing waveform 207 of 310
75.0 5850
Processing waveform 208 of 310
75.0 3675
Processing waveform 209 of 310
75.0 2250
Processing waveform 210 of 310
75.0 3600
Processing waveform 211 of 310
75.0 5775
Processing waveform 212 of 310
75.0 2325
Processing waveform 213 of 310
75.0 2325
Processing waveform 214 of 310
75.0 3525
Processing waveform 215 of 310
75.0 4500
Processing waveform 216 of 310
75.0 4275
Processing waveform 217 of 310
75.0 3450
Processing waveform 218 of 310
75.0 2550
Processing waveform 219 of 310
75.0 3300
Processing waveform 220 of 310
75.0 9075
Processing waveform 221 of 310
75.0 9075
Processing waveform 222 of 310
75.0 6525
Processing waveform 223 of 310
75.0 6975
Processing waveform 224 of 310
75.0 6750
Processing waveform 225 of 310
75.0 7275
Processing waveform 226 of 310
75.0 2400
Processing waveform 227 of 310
75.0 4125
Processing waveform 228 of 310
75.0 3525
Processing waveform 229 of 310
75.0 5175
Processing waveform 230 of 310
75.0 3750
Processing waveform 231 of 310
75.0 4875
Processing waveform 232 of 310
75.0 2250
Processing waveform 233 of 310
75.0 4125
Processing waveform 234 of 310
75.0 4425
Processing waveform 235 of 310
75.0 5250
Processing waveform 236 of 310
75.0 9075
Processing waveform 237 of 310
75.0 4725
Processing waveform 238 of 310
75.0 2475
Processing waveform 239 of 310
75.0 2925
Processing waveform 240 of 310
75.0 7350
Processing waveform 241 of 310
75.0 2550
Processing waveform 242 of 310
75.0 2850
Processing waveform 243 of 310
75.0 2550
Processing waveform 244 of 310
75.0 4425
Processing waveform 245 of 310
75.0 5775
Processing waveform 246 of 310
75.0 4200
Processing waveform 247 of 310
75.0 2175
Processing waveform 248 of 310
75.0 2925
Processing waveform 249 of 310
75.0 10350
Processing waveform 250 of 310
75.0 15600
Processing waveform 251 of 310
75.0 6450
Processing waveform 252 of 310
75.0 3450
Processing waveform 253 of 310
75.0 11175
Processing waveform 254 of 310
75.0 4725
Processing waveform 255 of 310
75.0 4200
Processing waveform 256 of 310
75.0 3750
Processing waveform 257 of 310
75.0 3975
Processing waveform 258 of 310
75.0 4575
Processing waveform 259 of 310
75.0 3225
Processing waveform 260 of 310
75.0 13500
Processing waveform 261 of 310
75.0 4050
Processing waveform 262 of 310
75.0 5250
Processing waveform 263 of 310
75.0 7050
Processing waveform 264 of 310
75.0 3450
Processing waveform 265 of 310
75.0 6675
Processing waveform 266 of 310
75.0 2850
Processing waveform 267 of 310
75.0 6750
Processing waveform 268 of 310
75.0 2475
Processing waveform 269 of 310
75.0 7650
Processing waveform 270 of 310
75.0 6750
Processing waveform 271 of 310
75.0 4125
Processing waveform 272 of 310
75.0 7425
Processing waveform 273 of 310
75.0 5400
Processing waveform 274 of 310
75.0 4800
Processing waveform 275 of 310
75.0 10200
Processing waveform 276 of 310
75.0 18225
Processing waveform 277 of 310
75.0 5925
Processing waveform 278 of 310
75.0 3525
Processing waveform 279 of 310
75.0 3300
Processing waveform 280 of 310
75.0 3600
Processing waveform 281 of 310
75.0 4650
Processing waveform 282 of 310
75.0 7875
Processing waveform 283 of 310
75.0 6300
Processing waveform 284 of 310
75.0 6750
Processing waveform 285 of 310
75.0 2925
Processing waveform 286 of 310
75.0 3525
Processing waveform 287 of 310
75.0 7875
Processing waveform 288 of 310
75.0 8925
Processing waveform 289 of 310
75.0 2850
Processing waveform 290 of 310
75.0 2475
Processing waveform 291 of 310
75.0 2325
Processing waveform 292 of 310
75.0 2250
Processing waveform 293 of 310
75.0 2700
Processing waveform 294 of 310
75.0 3000
Processing waveform 295 of 310
75.0 3300
Processing waveform 296 of 310
75.0 4425
Processing waveform 297 of 310
75.0 4500
Processing waveform 298 of 310
75.0 7500
Processing waveform 299 of 310
75.0 4200
Processing waveform 300 of 310
75.0 4800
Processing waveform 301 of 310
75.0 3000
Processing waveform 302 of 310
75.0 2475
Processing waveform 303 of 310
75.0 1950
Processing waveform 304 of 310
75.0 2625
Processing waveform 305 of 310
75.0 4650
Processing waveform 306 of 310
75.0 5475
Processing waveform 307 of 310
75.0 4950
Processing waveform 308 of 310
75.0 6375
Processing waveform 309 of 310
75.0 7500
Training data have been read and features have been extracted  (310, 120)
Computation time:  403.237916469574
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled

Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.32564544677734375
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h   106             
        l         103       
        t               101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  40.1290322581  +/-  3.42766286295  %
          Predicted class
              h     l     t 
        h    26    13    14 
        l    19    17    16 
        t    16    16    19 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't', 'r'] 0
class
h    106
l    103
r    110
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 420
75.0 5325
Processing waveform 1 of 420
75.0 2700
Processing waveform 2 of 420
75.0 4350
Processing waveform 3 of 420
75.0 2775
Processing waveform 4 of 420
75.0 1950
Processing waveform 5 of 420
75.0 2250
Processing waveform 6 of 420
75.0 2250
Processing waveform 7 of 420
75.0 2250
Processing waveform 8 of 420
75.0 5100
Processing waveform 9 of 420
75.0 2475
Processing waveform 10 of 420
75.0 3300
Processing waveform 11 of 420
75.0 2175
Processing waveform 12 of 420
75.0 6300
Processing waveform 13 of 420
75.0 9075
Processing waveform 14 of 420
75.0 3225
Processing waveform 15 of 420
75.0 4125
Processing waveform 16 of 420
75.0 3300
Processing waveform 17 of 420
75.0 3225
Processing waveform 18 of 420
75.0 2850
Processing waveform 19 of 420
75.0 5400
Processing waveform 20 of 420
75.0 2400
Processing waveform 21 of 420
75.0 7725
Processing waveform 22 of 420
75.0 3900
Processing waveform 23 of 420
75.0 2775
Processing waveform 24 of 420
75.0 2325
Processing waveform 25 of 420
75.0 4425
Processing waveform 26 of 420
75.0 3525
Processing waveform 27 of 420
75.0 6300
Processing waveform 28 of 420
75.0 2625
Processing waveform 29 of 420
75.0 4050
Processing waveform 30 of 420
75.0 2775
Processing waveform 31 of 420
75.0 2400
Processing waveform 32 of 420
75.0 2850
Processing waveform 33 of 420
75.0 2250
Processing waveform 34 of 420
75.0 2325
Processing waveform 35 of 420
75.0 3750
Processing waveform 36 of 420
75.0 3525
Processing waveform 37 of 420
75.0 5700
Processing waveform 38 of 420
75.0 2700
Processing waveform 39 of 420
75.0 3000
Processing waveform 40 of 420
75.0 4350
Processing waveform 41 of 420
75.0 4950
Processing waveform 42 of 420
75.0 2925
Processing waveform 43 of 420
75.0 2775
Processing waveform 44 of 420
75.0 3450
Processing waveform 45 of 420
75.0 4875
Processing waveform 46 of 420
75.0 9075
Processing waveform 47 of 420
75.0 5175
Processing waveform 48 of 420
75.0 3075
Processing waveform 49 of 420
75.0 4125
Processing waveform 50 of 420
75.0 9075
Processing waveform 51 of 420
75.0 3225
Processing waveform 52 of 420
75.0 2775
Processing waveform 53 of 420
75.0 4575
Processing waveform 54 of 420
75.0 2700
Processing waveform 55 of 420
75.0 1950
Processing waveform 56 of 420
75.0 2100
Processing waveform 57 of 420
75.0 4575
Processing waveform 58 of 420
75.0 5850
Processing waveform 59 of 420
75.0 3150
Processing waveform 60 of 420
75.0 4275
Processing waveform 61 of 420
75.0 9075
Processing waveform 62 of 420
75.0 3600
Processing waveform 63 of 420
75.0 2625
Processing waveform 64 of 420
75.0 2700
Processing waveform 65 of 420
75.0 2700
Processing waveform 66 of 420
75.0 3525
Processing waveform 67 of 420
75.0 4200
Processing waveform 68 of 420
75.0 4200
Processing waveform 69 of 420
75.0 1950
Processing waveform 70 of 420
75.0 5775
Processing waveform 71 of 420
75.0 9075
Processing waveform 72 of 420
75.0 4200
Processing waveform 73 of 420
75.0 2625
Processing waveform 74 of 420
75.0 4875
Processing waveform 75 of 420
75.0 5025
Processing waveform 76 of 420
75.0 14625
Processing waveform 77 of 420
75.0 11025
Processing waveform 78 of 420
75.0 1950
Processing waveform 79 of 420
75.0 3975
Processing waveform 80 of 420
75.0 2550
Processing waveform 81 of 420
75.0 7500
Processing waveform 82 of 420
75.0 14925
Processing waveform 83 of 420
75.0 13275
Processing waveform 84 of 420
75.0 1725
Processing waveform 85 of 420
75.0 2400
Processing waveform 86 of 420
75.0 6375
Processing waveform 87 of 420
75.0 6450
Processing waveform 88 of 420
75.0 4125
Processing waveform 89 of 420
75.0 3975
Processing waveform 90 of 420
75.0 1725
Processing waveform 91 of 420
75.0 2700
Processing waveform 92 of 420
75.0 4800
Processing waveform 93 of 420
75.0 7125
Processing waveform 94 of 420
75.0 5325
Processing waveform 95 of 420
75.0 5400
Processing waveform 96 of 420
75.0 6825
Processing waveform 97 of 420
75.0 7050
Processing waveform 98 of 420
75.0 4500
Processing waveform 99 of 420
75.0 5625
Processing waveform 100 of 420
75.0 4275
Processing waveform 101 of 420
75.0 2925
Processing waveform 102 of 420
75.0 3150
Processing waveform 103 of 420
75.0 6000
Processing waveform 104 of 420
75.0 3150
Processing waveform 105 of 420
75.0 6000
Processing waveform 106 of 420
75.0 12600
Processing waveform 107 of 420
75.0 7425
Processing waveform 108 of 420
75.0 16725
Processing waveform 109 of 420
75.0 6075
Processing waveform 110 of 420
75.0 3975
Processing waveform 111 of 420
75.0 5025
Processing waveform 112 of 420
75.0 6825
Processing waveform 113 of 420
75.0 9075
Processing waveform 114 of 420
75.0 4500
Processing waveform 115 of 420
75.0 2175
Processing waveform 116 of 420
75.0 3300
Processing waveform 117 of 420
75.0 4275
Processing waveform 118 of 420
75.0 2850
Processing waveform 119 of 420
75.0 2850
Processing waveform 120 of 420
75.0 2700
Processing waveform 121 of 420
75.0 5250
Processing waveform 122 of 420
75.0 3000
Processing waveform 123 of 420
75.0 2775
Processing waveform 124 of 420
75.0 6450
Processing waveform 125 of 420
75.0 3150
Processing waveform 126 of 420
75.0 1650
Processing waveform 127 of 420
75.0 4500
Processing waveform 128 of 420
75.0 2250
Processing waveform 129 of 420
75.0 2175
Processing waveform 130 of 420
75.0 3225
Processing waveform 131 of 420
75.0 4050
Processing waveform 132 of 420
75.0 3300
Processing waveform 133 of 420
75.0 2850
Processing waveform 134 of 420
75.0 2025
Processing waveform 135 of 420
75.0 4800
Processing waveform 136 of 420
75.0 2325
Processing waveform 137 of 420
75.0 6150
Processing waveform 138 of 420
75.0 2775
Processing waveform 139 of 420
75.0 2850
Processing waveform 140 of 420
75.0 4425
Processing waveform 141 of 420
75.0 2250
Processing waveform 142 of 420
75.0 3075
Processing waveform 143 of 420
75.0 2775
Processing waveform 144 of 420
75.0 1875
Processing waveform 145 of 420
75.0 3300
Processing waveform 146 of 420
75.0 1875
Processing waveform 147 of 420
75.0 5550
Processing waveform 148 of 420
75.0 9075
Processing waveform 149 of 420
75.0 4650
Processing waveform 150 of 420
75.0 6525
Processing waveform 151 of 420
75.0 9075
Processing waveform 152 of 420
75.0 3225
Processing waveform 153 of 420
75.0 3150
Processing waveform 154 of 420
75.0 4275
Processing waveform 155 of 420
75.0 4050
Processing waveform 156 of 420
75.0 5400
Processing waveform 157 of 420
75.0 4725
Processing waveform 158 of 420
75.0 3450
Processing waveform 159 of 420
75.0 6150
Processing waveform 160 of 420
75.0 2250
Processing waveform 161 of 420
75.0 9000
Processing waveform 162 of 420
75.0 3150
Processing waveform 163 of 420
75.0 4950
Processing waveform 164 of 420
75.0 6750
Processing waveform 165 of 420
75.0 3150
Processing waveform 166 of 420
75.0 7050
Processing waveform 167 of 420
75.0 3675
Processing waveform 168 of 420
75.0 3225
Processing waveform 169 of 420
75.0 2325
Processing waveform 170 of 420
75.0 7500
Processing waveform 171 of 420
75.0 3975
Processing waveform 172 of 420
75.0 6150
Processing waveform 173 of 420
75.0 7875
Processing waveform 174 of 420
75.0 7275
Processing waveform 175 of 420
75.0 6075
Processing waveform 176 of 420
75.0 3900
Processing waveform 177 of 420
75.0 3000
Processing waveform 178 of 420
75.0 3900
Processing waveform 179 of 420
75.0 6525
Processing waveform 180 of 420
75.0 2925
Processing waveform 181 of 420
75.0 2700

Processing waveform 182 of 420
75.0 2250
Processing waveform 183 of 420
75.0 4425
Processing waveform 184 of 420
75.0 7500
Processing waveform 185 of 420
75.0 2250
Processing waveform 186 of 420
75.0 9600
Processing waveform 187 of 420
75.0 4650
Processing waveform 188 of 420
75.0 6750
Processing waveform 189 of 420
75.0 8925
Processing waveform 190 of 420
75.0 7125
Processing waveform 191 of 420
75.0 3000
Processing waveform 192 of 420
75.0 10200
Processing waveform 193 of 420
75.0 1800
Processing waveform 194 of 420
75.0 4425
Processing waveform 195 of 420
75.0 2250
Processing waveform 196 of 420
75.0 6150
Processing waveform 197 of 420
75.0 7125
Processing waveform 198 of 420
75.0 3600
Processing waveform 199 of 420
75.0 6000
Processing waveform 200 of 420
75.0 4800
Processing waveform 201 of 420
75.0 7200
Processing waveform 202 of 420
75.0 3000
Processing waveform 203 of 420
75.0 4500
Processing waveform 204 of 420
75.0 3600
Processing waveform 205 of 420
75.0 4875
Processing waveform 206 of 420
75.0 5625
Processing waveform 207 of 420
75.0 3150
Processing waveform 208 of 420
75.0 3975
Processing waveform 209 of 420
75.0 5325
Processing waveform 210 of 420
75.0 2850
Processing waveform 211 of 420
75.0 9450
Processing waveform 212 of 420
75.0 6525
Processing waveform 213 of 420
75.0 3225
Processing waveform 214 of 420
75.0 2100
Processing waveform 215 of 420
75.0 2700
Processing waveform 216 of 420
75.0 2325
Processing waveform 217 of 420
75.0 2775
Processing waveform 218 of 420
75.0 2625
Processing waveform 219 of 420
75.0 5850
Processing waveform 220 of 420
75.0 3825
Processing waveform 221 of 420
75.0 6000
Processing waveform 222 of 420
75.0 2025
Processing waveform 223 of 420
75.0 1725
Processing waveform 224 of 420
75.0 2175
Processing waveform 225 of 420
75.0 5400
Processing waveform 226 of 420
75.0 5325
Processing waveform 227 of 420
75.0 3750
Processing waveform 228 of 420
75.0 3075
Processing waveform 229 of 420
75.0 5550
Processing waveform 230 of 420
75.0 7350
Processing waveform 231 of 420
75.0 4650
Processing waveform 232 of 420
75.0 2775
Processing waveform 233 of 420
75.0 2325
Processing waveform 234 of 420
75.0 2925
Processing waveform 235 of 420
75.0 3075
Processing waveform 236 of 420
75.0 6375
Processing waveform 237 of 420
75.0 2475
Processing waveform 238 of 420
75.0 3975
Processing waveform 239 of 420
75.0 3150
Processing waveform 240 of 420
75.0 5925
Processing waveform 241 of 420
75.0 5400
Processing waveform 242 of 420
75.0 1875
Processing waveform 243 of 420
75.0 5850
Processing waveform 244 of 420
75.0 1725
Processing waveform 245 of 420
75.0 7650
Processing waveform 246 of 420
75.0 2700
Processing waveform 247 of 420
75.0 9075
Processing waveform 248 of 420
75.0 6975
Processing waveform 249 of 420
75.0 4050
Processing waveform 250 of 420
75.0 3450
Processing waveform 251 of 420
75.0 5925
Processing waveform 252 of 420
75.0 2250
Processing waveform 253 of 420
75.0 3675
Processing waveform 254 of 420
75.0 2625
Processing waveform 255 of 420
75.0 4575
Processing waveform 256 of 420
75.0 2025
Processing waveform 257 of 420
75.0 5175
Processing waveform 258 of 420
75.0 3225
Processing waveform 259 of 420
75.0 4950
Processing waveform 260 of 420
75.0 2925
Processing waveform 261 of 420
75.0 1800
Processing waveform 262 of 420
75.0 8925
Processing waveform 263 of 420
75.0 7350
Processing waveform 264 of 420
75.0 4125
Processing waveform 265 of 420
75.0 3225
Processing waveform 266 of 420
75.0 9075
Processing waveform 267 of 420
75.0 5025
Processing waveform 268 of 420
75.0 8925
Processing waveform 269 of 420
75.0 4800
Processing waveform 270 of 420
75.0 7800
Processing waveform 271 of 420
75.0 2775
Processing waveform 272 of 420
75.0 6600
Processing waveform 273 of 420
75.0 1800
Processing waveform 274 of 420
75.0 2625
Processing waveform 275 of 420
75.0 9075
Processing waveform 276 of 420
75.0 7200
Processing waveform 277 of 420
75.0 5550
Processing waveform 278 of 420
75.0 2775
Processing waveform 279 of 420
75.0 5250
Processing waveform 280 of 420
75.0 1875
Processing waveform 281 of 420
75.0 1875
Processing waveform 282 of 420
75.0 5025
Processing waveform 283 of 420
75.0 3600
Processing waveform 284 of 420
75.0 3375
Processing waveform 285 of 420
75.0 4050
Processing waveform 286 of 420
75.0 3375
Processing waveform 287 of 420
75.0 2925
Processing waveform 288 of 420
75.0 6675
Processing waveform 289 of 420
75.0 3750
Processing waveform 290 of 420
75.0 4575
Processing waveform 291 of 420
75.0 9075
Processing waveform 292 of 420
75.0 3900
Processing waveform 293 of 420
75.0 9075
Processing waveform 294 of 420
75.0 4875
Processing waveform 295 of 420
75.0 3075
Processing waveform 296 of 420
75.0 3150
Processing waveform 297 of 420
75.0 2550
Processing waveform 298 of 420
75.0 9075
Processing waveform 299 of 420
75.0 7425
Processing waveform 300 of 420
75.0 5025
Processing waveform 301 of 420
75.0 2850
Processing waveform 302 of 420
75.0 1875
Processing waveform 303 of 420
75.0 7125
Processing waveform 304 of 420
75.0 4500
Processing waveform 305 of 420
75.0 6075
Processing waveform 306 of 420
75.0 2475
Processing waveform 307 of 420
75.0 4125
Processing waveform 308 of 420
75.0 4500
Processing waveform 309 of 420
75.0 2100
Processing waveform 310 of 420
75.0 3975
Processing waveform 311 of 420
75.0 3225
Processing waveform 312 of 420
75.0 2550
Processing waveform 313 of 420
75.0 6975
Processing waveform 314 of 420
75.0 3075
Processing waveform 315 of 420
75.0 3675
Processing waveform 316 of 420
75.0 4800
Processing waveform 317 of 420
75.0 5850
Processing waveform 318 of 420
75.0 3675
Processing waveform 319 of 420
75.0 2250
Processing waveform 320 of 420
75.0 3600
Processing waveform 321 of 420
75.0 5775
Processing waveform 322 of 420
75.0 2325
Processing waveform 323 of 420
75.0 2325
Processing waveform 324 of 420
75.0 3525
Processing waveform 325 of 420
75.0 4500
Processing waveform 326 of 420
75.0 4275
Processing waveform 327 of 420
75.0 3450
Processing waveform 328 of 420
75.0 2550
Processing waveform 329 of 420
75.0 3300
Processing waveform 330 of 420
75.0 9075
Processing waveform 331 of 420
75.0 9075
Processing waveform 332 of 420
75.0 6525
Processing waveform 333 of 420
75.0 6975
Processing waveform 334 of 420
75.0 6750
Processing waveform 335 of 420
75.0 7275
Processing waveform 336 of 420
75.0 2400
Processing waveform 337 of 420
75.0 4125
Processing waveform 338 of 420
75.0 3525
Processing waveform 339 of 420
75.0 5175
Processing waveform 340 of 420
75.0 3750
Processing waveform 341 of 420
75.0 4875
Processing waveform 342 of 420
75.0 2250
Processing waveform 343 of 420
75.0 4125
Processing waveform 344 of 420
75.0 4425
Processing waveform 345 of 420
75.0 5250
Processing waveform 346 of 420
75.0 9075
Processing waveform 347 of 420
75.0 4725
Processing waveform 348 of 420
75.0 2475
Processing waveform 349 of 420
75.0 2925
Processing waveform 350 of 420
75.0 7350
Processing waveform 351 of 420
75.0 2550
Processing waveform 352 of 420
75.0 2850
Processing waveform 353 of 420
75.0 2550
Processing waveform 354 of 420
75.0 4425
Processing waveform 355 of 420
75.0 5775
Processing waveform 356 of 420
75.0 4200
Processing waveform 357 of 420
75.0 2175
Processing waveform 358 of 420
75.0 2925
Processing waveform 359 of 420
75.0 10350
Processing waveform 360 of 420
75.0 15600
Processing waveform 361 of 420
75.0 6450
Processing waveform 362 of 420
75.0 3450
Processing waveform 363 of 420
75.0 11175
Processing waveform 364 of 420
75.0 4725
Processing waveform 365 of 420
75.0 4200
Processing waveform 366 of 420
75.0 3750
Processing waveform 367 of 420
75.0 3975
Processing waveform 368 of 420
75.0 4575
Processing waveform 369 of 420
75.0 3225
Processing waveform 370 of 420
75.0 13500
Processing waveform 371 of 420
75.0 4050
Processing waveform 372 of 420
75.0 5250
Processing waveform 373 of 420
75.0 7050
Processing waveform 374 of 420
75.0 3450
Processing waveform 375 of 420
75.0 6675
Processing waveform 376 of 420
75.0 2850
Processing waveform 377 of 420
75.0 6750
Processing waveform 378 of 420
75.0 2475
Processing waveform 379 of 420
75.0 7650
Processing waveform 380 of 420
75.0 6750
Processing waveform 381 of 420
75.0 4125

Processing waveform 382 of 420
75.0 7425
Processing waveform 383 of 420
75.0 5400
Processing waveform 384 of 420
75.0 4800
Processing waveform 385 of 420
75.0 10200
Processing waveform 386 of 420
75.0 18225
Processing waveform 387 of 420
75.0 5925
Processing waveform 388 of 420
75.0 3525
Processing waveform 389 of 420
75.0 3300
Processing waveform 390 of 420
75.0 3600
Processing waveform 391 of 420
75.0 4650
Processing waveform 392 of 420
75.0 7875
Processing waveform 393 of 420
75.0 6300
Processing waveform 394 of 420
75.0 6750
Processing waveform 395 of 420
75.0 2925
Processing waveform 396 of 420
75.0 3525
Processing waveform 397 of 420
75.0 7875
Processing waveform 398 of 420
75.0 8925
Processing waveform 399 of 420
75.0 2850
Processing waveform 400 of 420
75.0 2475
Processing waveform 401 of 420
75.0 2325
Processing waveform 402 of 420
75.0 2250
Processing waveform 403 of 420
75.0 2700
Processing waveform 404 of 420
75.0 3000
Processing waveform 405 of 420
75.0 3300
Processing waveform 406 of 420
75.0 4425
Processing waveform 407 of 420
75.0 4500
Processing waveform 408 of 420
75.0 7500
Processing waveform 409 of 420
75.0 4200
Processing waveform 410 of 420
75.0 4800
Processing waveform 411 of 420
75.0 3000
Processing waveform 412 of 420
75.0 2475
Processing waveform 413 of 420
75.0 1950
Processing waveform 414 of 420
75.0 2625
Processing waveform 415 of 420
75.0 4650
Processing waveform 416 of 420
75.0 5475
Processing waveform 417 of 420
75.0 4950
Processing waveform 418 of 420
75.0 6375
Processing waveform 419 of 420
75.0 7500
Training data have been read and features have been extracted  (420, 120)
Computation time:  559.7656652927399
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.5367674827575684
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h   106                   
        l         103             
        r               110       
        t                     101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  28.8095238095  +/-  2.2795883518  %
          Predicted class
              h     l     r     t 
        h    19    10    15    10 
        l    14    12    13    12 
        r    18    10    16    11 
        t    11    12    14    14 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'h', 'l', 't', 'r'] 0
class
e     93
h    106
l    103
r    110
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 513
75.0 5325
Processing waveform 1 of 513
75.0 2700
Processing waveform 2 of 513
75.0 4350
Processing waveform 3 of 513
75.0 2775
Processing waveform 4 of 513
75.0 1950
Processing waveform 5 of 513
75.0 2250
Processing waveform 6 of 513
75.0 2250
Processing waveform 7 of 513
75.0 2250
Processing waveform 8 of 513
75.0 5100
Processing waveform 9 of 513
75.0 2475
Processing waveform 10 of 513
75.0 3300
Processing waveform 11 of 513
75.0 2175
Processing waveform 12 of 513
75.0 6300
Processing waveform 13 of 513
75.0 9075
Processing waveform 14 of 513
75.0 3225
Processing waveform 15 of 513
75.0 4125
Processing waveform 16 of 513
75.0 3300
Processing waveform 17 of 513
75.0 3225
Processing waveform 18 of 513
75.0 2850
Processing waveform 19 of 513
75.0 5400
Processing waveform 20 of 513
75.0 2400
Processing waveform 21 of 513
75.0 7725
Processing waveform 22 of 513
75.0 3900
Processing waveform 23 of 513
75.0 2775
Processing waveform 24 of 513
75.0 2325
Processing waveform 25 of 513
75.0 4425
Processing waveform 26 of 513
75.0 3525
Processing waveform 27 of 513
75.0 6300
Processing waveform 28 of 513
75.0 2625
Processing waveform 29 of 513
75.0 4050
Processing waveform 30 of 513
75.0 2775
Processing waveform 31 of 513
75.0 2400
Processing waveform 32 of 513
75.0 2850
Processing waveform 33 of 513
75.0 2250
Processing waveform 34 of 513
75.0 2325
Processing waveform 35 of 513
75.0 3750
Processing waveform 36 of 513
75.0 3525
Processing waveform 37 of 513
75.0 5700
Processing waveform 38 of 513
75.0 2700
Processing waveform 39 of 513
75.0 3000
Processing waveform 40 of 513
75.0 4350
Processing waveform 41 of 513
75.0 4950
Processing waveform 42 of 513
75.0 2925
Processing waveform 43 of 513
75.0 2775
Processing waveform 44 of 513
75.0 3450
Processing waveform 45 of 513
75.0 4875
Processing waveform 46 of 513
75.0 9075
Processing waveform 47 of 513
75.0 5175
Processing waveform 48 of 513
75.0 3075
Processing waveform 49 of 513
75.0 4125
Processing waveform 50 of 513
75.0 9075
Processing waveform 51 of 513
75.0 3225
Processing waveform 52 of 513
75.0 2775
Processing waveform 53 of 513
75.0 4575
Processing waveform 54 of 513
75.0 2700
Processing waveform 55 of 513
75.0 1950
Processing waveform 56 of 513
75.0 2100
Processing waveform 57 of 513
75.0 4575
Processing waveform 58 of 513
75.0 5850
Processing waveform 59 of 513
75.0 3150
Processing waveform 60 of 513
75.0 4275
Processing waveform 61 of 513
75.0 9075
Processing waveform 62 of 513
75.0 3600
Processing waveform 63 of 513
75.0 2625
Processing waveform 64 of 513
75.0 2700
Processing waveform 65 of 513
75.0 2700
Processing waveform 66 of 513
75.0 3525
Processing waveform 67 of 513
75.0 4200
Processing waveform 68 of 513
75.0 4200
Processing waveform 69 of 513
75.0 1950
Processing waveform 70 of 513
75.0 5775
Processing waveform 71 of 513
75.0 9075
Processing waveform 72 of 513
75.0 4200
Processing waveform 73 of 513
75.0 2625
Processing waveform 74 of 513
75.0 4875
Processing waveform 75 of 513
75.0 5025
Processing waveform 76 of 513
75.0 14625
Processing waveform 77 of 513
75.0 11025
Processing waveform 78 of 513
75.0 1950
Processing waveform 79 of 513
75.0 3975
Processing waveform 80 of 513
75.0 2550
Processing waveform 81 of 513
75.0 7500
Processing waveform 82 of 513
75.0 14925
Processing waveform 83 of 513
75.0 13275
Processing waveform 84 of 513
75.0 1725
Processing waveform 85 of 513
75.0 2400
Processing waveform 86 of 513
75.0 6375
Processing waveform 87 of 513
75.0 6450
Processing waveform 88 of 513
75.0 4125
Processing waveform 89 of 513
75.0 3975
Processing waveform 90 of 513
75.0 1725
Processing waveform 91 of 513
75.0 2700
Processing waveform 92 of 513
75.0 4800
Processing waveform 93 of 513
75.0 7125
Processing waveform 94 of 513
75.0 5325
Processing waveform 95 of 513
75.0 5400
Processing waveform 96 of 513
75.0 6825
Processing waveform 97 of 513
75.0 7050
Processing waveform 98 of 513
75.0 4500
Processing waveform 99 of 513
75.0 5625
Processing waveform 100 of 513
75.0 4275
Processing waveform 101 of 513
75.0 2925
Processing waveform 102 of 513
75.0 3150
Processing waveform 103 of 513
75.0 6000
Processing waveform 104 of 513
75.0 3150
Processing waveform 105 of 513
75.0 6000
Processing waveform 106 of 513
75.0 12600
Processing waveform 107 of 513
75.0 7425
Processing waveform 108 of 513
75.0 16725
Processing waveform 109 of 513
75.0 6075
Processing waveform 110 of 513
75.0 3450
Processing waveform 111 of 513
75.0 2325
Processing waveform 112 of 513
75.0 2925
Processing waveform 113 of 513
75.0 4575
Processing waveform 114 of 513
75.0 7050
Processing waveform 115 of 513
75.0 2025
Processing waveform 116 of 513
75.0 2925
Processing waveform 117 of 513
75.0 2850
Processing waveform 118 of 513
75.0 3600
Processing waveform 119 of 513
75.0 3750
Processing waveform 120 of 513
75.0 2100
Processing waveform 121 of 513
75.0 1650
Processing waveform 122 of 513
75.0 4200
Processing waveform 123 of 513
75.0 9075
Processing waveform 124 of 513
75.0 4125
Processing waveform 125 of 513
75.0 2925
Processing waveform 126 of 513
75.0 2250
Processing waveform 127 of 513
75.0 5475
Processing waveform 128 of 513
75.0 2625
Processing waveform 129 of 513
75.0 2400
Processing waveform 130 of 513
75.0 2700
Processing waveform 131 of 513
75.0 9075
Processing waveform 132 of 513
75.0 3300
Processing waveform 133 of 513
75.0 9075
Processing waveform 134 of 513
75.0 6225
Processing waveform 135 of 513
75.0 7125

Processing waveform 136 of 513
75.0 5850
Processing waveform 137 of 513
75.0 9075
Processing waveform 138 of 513
75.0 3225
Processing waveform 139 of 513
75.0 8100
Processing waveform 140 of 513
75.0 4575
Processing waveform 141 of 513
75.0 2850
Processing waveform 142 of 513
75.0 3375
Processing waveform 143 of 513
75.0 2100
Processing waveform 144 of 513
75.0 9075
Processing waveform 145 of 513
75.0 5850
Processing waveform 146 of 513
75.0 2775
Processing waveform 147 of 513
75.0 2700
Processing waveform 148 of 513
75.0 2100
Processing waveform 149 of 513
75.0 9075
Processing waveform 150 of 513
75.0 3225
Processing waveform 151 of 513
75.0 3300
Processing waveform 152 of 513
75.0 2475
Processing waveform 153 of 513
75.0 3075
Processing waveform 154 of 513
75.0 6450
Processing waveform 155 of 513
75.0 3600
Processing waveform 156 of 513
75.0 4650
Processing waveform 157 of 513
75.0 2475
Processing waveform 158 of 513
75.0 1875
Processing waveform 159 of 513
75.0 3450
Processing waveform 160 of 513
75.0 1500
Processing waveform 161 of 513
75.0 2625
Processing waveform 162 of 513
75.0 9075
Processing waveform 163 of 513
75.0 3525
Processing waveform 164 of 513
75.0 5625
Processing waveform 165 of 513
75.0 3675
Processing waveform 166 of 513
75.0 9075
Processing waveform 167 of 513
75.0 2700
Processing waveform 168 of 513
75.0 9075
Processing waveform 169 of 513
75.0 3000
Processing waveform 170 of 513
75.0 3525
Processing waveform 171 of 513
75.0 7200
Processing waveform 172 of 513
75.0 4875
Processing waveform 173 of 513
75.0 6075
Processing waveform 174 of 513
75.0 2700
Processing waveform 175 of 513
75.0 5250
Processing waveform 176 of 513
75.0 12750
Processing waveform 177 of 513
75.0 5100
Processing waveform 178 of 513
75.0 2475
Processing waveform 179 of 513
75.0 6525
Processing waveform 180 of 513
75.0 7200
Processing waveform 181 of 513
75.0 4500
Processing waveform 182 of 513
75.0 4950
Processing waveform 183 of 513
75.0 8250
Processing waveform 184 of 513
75.0 3150
Processing waveform 185 of 513
75.0 5850
Processing waveform 186 of 513
75.0 2325
Processing waveform 187 of 513
75.0 4200
Processing waveform 188 of 513
75.0 3300
Processing waveform 189 of 513
75.0 2325
Processing waveform 190 of 513
75.0 13500
Processing waveform 191 of 513
75.0 3525
Processing waveform 192 of 513
75.0 3075
Processing waveform 193 of 513
75.0 6600
Processing waveform 194 of 513
75.0 6225
Processing waveform 195 of 513
75.0 9000
Processing waveform 196 of 513
75.0 5100
Processing waveform 197 of 513
75.0 6975
Processing waveform 198 of 513
75.0 4425
Processing waveform 199 of 513
75.0 5400
Processing waveform 200 of 513
75.0 13350
Processing waveform 201 of 513
75.0 9525
Processing waveform 202 of 513
75.0 2325
Processing waveform 203 of 513
75.0 3975
Processing waveform 204 of 513
75.0 5025
Processing waveform 205 of 513
75.0 6825
Processing waveform 206 of 513
75.0 9075
Processing waveform 207 of 513
75.0 4500
Processing waveform 208 of 513
75.0 2175
Processing waveform 209 of 513
75.0 3300
Processing waveform 210 of 513
75.0 4275
Processing waveform 211 of 513
75.0 2850
Processing waveform 212 of 513
75.0 2850
Processing waveform 213 of 513
75.0 2700
Processing waveform 214 of 513
75.0 5250
Processing waveform 215 of 513
75.0 3000
Processing waveform 216 of 513
75.0 2775
Processing waveform 217 of 513
75.0 6450
Processing waveform 218 of 513
75.0 3150
Processing waveform 219 of 513
75.0 1650
Processing waveform 220 of 513
75.0 4500
Processing waveform 221 of 513
75.0 2250
Processing waveform 222 of 513
75.0 2175
Processing waveform 223 of 513
75.0 3225
Processing waveform 224 of 513
75.0 4050
Processing waveform 225 of 513
75.0 3300
Processing waveform 226 of 513
75.0 2850
Processing waveform 227 of 513
75.0 2025
Processing waveform 228 of 513
75.0 4800
Processing waveform 229 of 513
75.0 2325
Processing waveform 230 of 513
75.0 6150
Processing waveform 231 of 513
75.0 2775
Processing waveform 232 of 513
75.0 2850
Processing waveform 233 of 513
75.0 4425
Processing waveform 234 of 513
75.0 2250
Processing waveform 235 of 513
75.0 3075
Processing waveform 236 of 513
75.0 2775
Processing waveform 237 of 513
75.0 1875
Processing waveform 238 of 513
75.0 3300
Processing waveform 239 of 513
75.0 1875
Processing waveform 240 of 513
75.0 5550
Processing waveform 241 of 513
75.0 9075
Processing waveform 242 of 513
75.0 4650
Processing waveform 243 of 513
75.0 6525
Processing waveform 244 of 513
75.0 9075
Processing waveform 245 of 513
75.0 3225
Processing waveform 246 of 513
75.0 3150
Processing waveform 247 of 513
75.0 4275
Processing waveform 248 of 513
75.0 4050
Processing waveform 249 of 513
75.0 5400
Processing waveform 250 of 513
75.0 4725
Processing waveform 251 of 513
75.0 3450
Processing waveform 252 of 513
75.0 6150
Processing waveform 253 of 513
75.0 2250
Processing waveform 254 of 513
75.0 9000
Processing waveform 255 of 513
75.0 3150
Processing waveform 256 of 513
75.0 4950
Processing waveform 257 of 513
75.0 6750
Processing waveform 258 of 513
75.0 3150
Processing waveform 259 of 513
75.0 7050
Processing waveform 260 of 513
75.0 3675
Processing waveform 261 of 513
75.0 3225
Processing waveform 262 of 513
75.0 2325
Processing waveform 263 of 513
75.0 7500
Processing waveform 264 of 513
75.0 3975
Processing waveform 265 of 513
75.0 6150
Processing waveform 266 of 513
75.0 7875
Processing waveform 267 of 513
75.0 7275
Processing waveform 268 of 513
75.0 6075
Processing waveform 269 of 513
75.0 3900
Processing waveform 270 of 513
75.0 3000
Processing waveform 271 of 513
75.0 3900
Processing waveform 272 of 513
75.0 6525
Processing waveform 273 of 513
75.0 2925
Processing waveform 274 of 513
75.0 2700
Processing waveform 275 of 513
75.0 2250
Processing waveform 276 of 513
75.0 4425
Processing waveform 277 of 513
75.0 7500
Processing waveform 278 of 513
75.0 2250
Processing waveform 279 of 513
75.0 9600
Processing waveform 280 of 513
75.0 4650
Processing waveform 281 of 513
75.0 6750
Processing waveform 282 of 513
75.0 8925
Processing waveform 283 of 513
75.0 7125
Processing waveform 284 of 513
75.0 3000
Processing waveform 285 of 513
75.0 10200
Processing waveform 286 of 513
75.0 1800
Processing waveform 287 of 513
75.0 4425
Processing waveform 288 of 513
75.0 2250
Processing waveform 289 of 513
75.0 6150
Processing waveform 290 of 513
75.0 7125
Processing waveform 291 of 513
75.0 3600
Processing waveform 292 of 513
75.0 6000
Processing waveform 293 of 513
75.0 4800
Processing waveform 294 of 513
75.0 7200
Processing waveform 295 of 513
75.0 3000
Processing waveform 296 of 513
75.0 4500
Processing waveform 297 of 513
75.0 3600
Processing waveform 298 of 513
75.0 4875
Processing waveform 299 of 513
75.0 5625
Processing waveform 300 of 513
75.0 3150
Processing waveform 301 of 513
75.0 3975
Processing waveform 302 of 513
75.0 5325
Processing waveform 303 of 513
75.0 2850
Processing waveform 304 of 513
75.0 9450
Processing waveform 305 of 513
75.0 6525
Processing waveform 306 of 513
75.0 3225
Processing waveform 307 of 513
75.0 2100
Processing waveform 308 of 513
75.0 2700
Processing waveform 309 of 513
75.0 2325
Processing waveform 310 of 513
75.0 2775
Processing waveform 311 of 513
75.0 2625
Processing waveform 312 of 513
75.0 5850
Processing waveform 313 of 513
75.0 3825
Processing waveform 314 of 513
75.0 6000
Processing waveform 315 of 513
75.0 2025
Processing waveform 316 of 513
75.0 1725
Processing waveform 317 of 513
75.0 2175
Processing waveform 318 of 513
75.0 5400
Processing waveform 319 of 513
75.0 5325
Processing waveform 320 of 513
75.0 3750
Processing waveform 321 of 513
75.0 3075
Processing waveform 322 of 513
75.0 5550
Processing waveform 323 of 513
75.0 7350
Processing waveform 324 of 513
75.0 4650
Processing waveform 325 of 513
75.0 2775
Processing waveform 326 of 513
75.0 2325
Processing waveform 327 of 513
75.0 2925
Processing waveform 328 of 513
75.0 3075
Processing waveform 329 of 513
75.0 6375
Processing waveform 330 of 513
75.0 2475
Processing waveform 331 of 513
75.0 3975
Processing waveform 332 of 513
75.0 3150
Processing waveform 333 of 513
75.0 5925
Processing waveform 334 of 513
75.0 5400
Processing waveform 335 of 513
75.0 1875

Processing waveform 336 of 513
75.0 5850
Processing waveform 337 of 513
75.0 1725
Processing waveform 338 of 513
75.0 7650
Processing waveform 339 of 513
75.0 2700
Processing waveform 340 of 513
75.0 9075
Processing waveform 341 of 513
75.0 6975
Processing waveform 342 of 513
75.0 4050
Processing waveform 343 of 513
75.0 3450
Processing waveform 344 of 513
75.0 5925
Processing waveform 345 of 513
75.0 2250
Processing waveform 346 of 513
75.0 3675
Processing waveform 347 of 513
75.0 2625
Processing waveform 348 of 513
75.0 4575
Processing waveform 349 of 513
75.0 2025
Processing waveform 350 of 513
75.0 5175
Processing waveform 351 of 513
75.0 3225
Processing waveform 352 of 513
75.0 4950
Processing waveform 353 of 513
75.0 2925
Processing waveform 354 of 513
75.0 1800
Processing waveform 355 of 513
75.0 8925
Processing waveform 356 of 513
75.0 7350
Processing waveform 357 of 513
75.0 4125
Processing waveform 358 of 513
75.0 3225
Processing waveform 359 of 513
75.0 9075
Processing waveform 360 of 513
75.0 5025
Processing waveform 361 of 513
75.0 8925
Processing waveform 362 of 513
75.0 4800
Processing waveform 363 of 513
75.0 7800
Processing waveform 364 of 513
75.0 2775
Processing waveform 365 of 513
75.0 6600
Processing waveform 366 of 513
75.0 1800
Processing waveform 367 of 513
75.0 2625
Processing waveform 368 of 513
75.0 9075
Processing waveform 369 of 513
75.0 7200
Processing waveform 370 of 513
75.0 5550
Processing waveform 371 of 513
75.0 2775
Processing waveform 372 of 513
75.0 5250
Processing waveform 373 of 513
75.0 1875
Processing waveform 374 of 513
75.0 1875
Processing waveform 375 of 513
75.0 5025
Processing waveform 376 of 513
75.0 3600
Processing waveform 377 of 513
75.0 3375
Processing waveform 378 of 513
75.0 4050
Processing waveform 379 of 513
75.0 3375
Processing waveform 380 of 513
75.0 2925
Processing waveform 381 of 513
75.0 6675
Processing waveform 382 of 513
75.0 3750
Processing waveform 383 of 513
75.0 4575
Processing waveform 384 of 513
75.0 9075
Processing waveform 385 of 513
75.0 3900
Processing waveform 386 of 513
75.0 9075
Processing waveform 387 of 513
75.0 4875
Processing waveform 388 of 513
75.0 3075
Processing waveform 389 of 513
75.0 3150
Processing waveform 390 of 513
75.0 2550
Processing waveform 391 of 513
75.0 9075
Processing waveform 392 of 513
75.0 7425
Processing waveform 393 of 513
75.0 5025
Processing waveform 394 of 513
75.0 2850
Processing waveform 395 of 513
75.0 1875
Processing waveform 396 of 513
75.0 7125
Processing waveform 397 of 513
75.0 4500
Processing waveform 398 of 513
75.0 6075
Processing waveform 399 of 513
75.0 2475
Processing waveform 400 of 513
75.0 4125
Processing waveform 401 of 513
75.0 4500
Processing waveform 402 of 513
75.0 2100
Processing waveform 403 of 513
75.0 3975
Processing waveform 404 of 513
75.0 3225
Processing waveform 405 of 513
75.0 2550
Processing waveform 406 of 513
75.0 6975
Processing waveform 407 of 513
75.0 3075
Processing waveform 408 of 513
75.0 3675
Processing waveform 409 of 513
75.0 4800
Processing waveform 410 of 513
75.0 5850
Processing waveform 411 of 513
75.0 3675
Processing waveform 412 of 513
75.0 2250
Processing waveform 413 of 513
75.0 3600
Processing waveform 414 of 513
75.0 5775
Processing waveform 415 of 513
75.0 2325
Processing waveform 416 of 513
75.0 2325
Processing waveform 417 of 513
75.0 3525
Processing waveform 418 of 513
75.0 4500
Processing waveform 419 of 513
75.0 4275
Processing waveform 420 of 513
75.0 3450
Processing waveform 421 of 513
75.0 2550
Processing waveform 422 of 513
75.0 3300
Processing waveform 423 of 513
75.0 9075
Processing waveform 424 of 513
75.0 9075
Processing waveform 425 of 513
75.0 6525
Processing waveform 426 of 513
75.0 6975
Processing waveform 427 of 513
75.0 6750
Processing waveform 428 of 513
75.0 7275
Processing waveform 429 of 513
75.0 2400
Processing waveform 430 of 513
75.0 4125
Processing waveform 431 of 513
75.0 3525
Processing waveform 432 of 513
75.0 5175
Processing waveform 433 of 513
75.0 3750
Processing waveform 434 of 513
75.0 4875
Processing waveform 435 of 513
75.0 2250
Processing waveform 436 of 513
75.0 4125
Processing waveform 437 of 513
75.0 4425
Processing waveform 438 of 513
75.0 5250
Processing waveform 439 of 513
75.0 9075
Processing waveform 440 of 513
75.0 4725
Processing waveform 441 of 513
75.0 2475
Processing waveform 442 of 513
75.0 2925
Processing waveform 443 of 513
75.0 7350
Processing waveform 444 of 513
75.0 2550
Processing waveform 445 of 513
75.0 2850
Processing waveform 446 of 513
75.0 2550
Processing waveform 447 of 513
75.0 4425
Processing waveform 448 of 513
75.0 5775
Processing waveform 449 of 513
75.0 4200
Processing waveform 450 of 513
75.0 2175
Processing waveform 451 of 513
75.0 2925
Processing waveform 452 of 513
75.0 10350
Processing waveform 453 of 513
75.0 15600
Processing waveform 454 of 513
75.0 6450
Processing waveform 455 of 513
75.0 3450
Processing waveform 456 of 513
75.0 11175
Processing waveform 457 of 513
75.0 4725
Processing waveform 458 of 513
75.0 4200
Processing waveform 459 of 513
75.0 3750
Processing waveform 460 of 513
75.0 3975
Processing waveform 461 of 513
75.0 4575
Processing waveform 462 of 513
75.0 3225
Processing waveform 463 of 513
75.0 13500
Processing waveform 464 of 513
75.0 4050
Processing waveform 465 of 513
75.0 5250
Processing waveform 466 of 513
75.0 7050
Processing waveform 467 of 513
75.0 3450
Processing waveform 468 of 513
75.0 6675
Processing waveform 469 of 513
75.0 2850
Processing waveform 470 of 513
75.0 6750
Processing waveform 471 of 513
75.0 2475
Processing waveform 472 of 513
75.0 7650
Processing waveform 473 of 513
75.0 6750
Processing waveform 474 of 513
75.0 4125
Processing waveform 475 of 513
75.0 7425
Processing waveform 476 of 513
75.0 5400
Processing waveform 477 of 513
75.0 4800
Processing waveform 478 of 513
75.0 10200
Processing waveform 479 of 513
75.0 18225
Processing waveform 480 of 513
75.0 5925
Processing waveform 481 of 513
75.0 3525
Processing waveform 482 of 513
75.0 3300
Processing waveform 483 of 513
75.0 3600
Processing waveform 484 of 513
75.0 4650
Processing waveform 485 of 513
75.0 7875
Processing waveform 486 of 513
75.0 6300
Processing waveform 487 of 513
75.0 6750
Processing waveform 488 of 513
75.0 2925
Processing waveform 489 of 513
75.0 3525
Processing waveform 490 of 513
75.0 7875
Processing waveform 491 of 513
75.0 8925
Processing waveform 492 of 513
75.0 2850
Processing waveform 493 of 513
75.0 2475
Processing waveform 494 of 513
75.0 2325
Processing waveform 495 of 513
75.0 2250
Processing waveform 496 of 513
75.0 2700
Processing waveform 497 of 513
75.0 3000
Processing waveform 498 of 513
75.0 3300
Processing waveform 499 of 513
75.0 4425
Processing waveform 500 of 513
75.0 4500
Processing waveform 501 of 513
75.0 7500
Processing waveform 502 of 513
75.0 4200
Processing waveform 503 of 513
75.0 4800
Processing waveform 504 of 513
75.0 3000
Processing waveform 505 of 513
75.0 2475
Processing waveform 506 of 513
75.0 1950
Processing waveform 507 of 513
75.0 2625
Processing waveform 508 of 513
75.0 4650
Processing waveform 509 of 513
75.0 5475
Processing waveform 510 of 513
75.0 4950
Processing waveform 511 of 513
75.0 6375
Processing waveform 512 of 513
75.0 7500
Training data have been read and features have been extracted  (513, 120)
Computation time:  698.0176618099213
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.7517979145050049
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    93                         
        h         106                   
        l               103             
        r                     110       
        t                           101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  23.7743190661  +/-  2.27850832248  %
          Predicted class
              e     h     l     r     t 
        e     7    11     9    10    10 
        h     8    17     8    12     7 
        l     8    11    11    12    10 
        r     6    15     9    15    10 
        t     8     9    11    11    11 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['l', 't'] 1

class
l    103
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 204
75.0 3975
Processing waveform 1 of 204
75.0 5025
Processing waveform 2 of 204
75.0 6825
Processing waveform 3 of 204
75.0 9075
Processing waveform 4 of 204
75.0 4500
Processing waveform 5 of 204
75.0 2175
Processing waveform 6 of 204
75.0 3300
Processing waveform 7 of 204
75.0 4275
Processing waveform 8 of 204
75.0 2850
Processing waveform 9 of 204
75.0 2850
Processing waveform 10 of 204
75.0 2700
Processing waveform 11 of 204
75.0 5250
Processing waveform 12 of 204
75.0 3000
Processing waveform 13 of 204
75.0 2775
Processing waveform 14 of 204
75.0 6450
Processing waveform 15 of 204
75.0 3150
Processing waveform 16 of 204
75.0 1650
Processing waveform 17 of 204
75.0 4500
Processing waveform 18 of 204
75.0 2250
Processing waveform 19 of 204
75.0 2175
Processing waveform 20 of 204
75.0 3225
Processing waveform 21 of 204
75.0 4050
Processing waveform 22 of 204
75.0 3300
Processing waveform 23 of 204
75.0 2850
Processing waveform 24 of 204
75.0 2025
Processing waveform 25 of 204
75.0 4800
Processing waveform 26 of 204
75.0 2325
Processing waveform 27 of 204
75.0 6150
Processing waveform 28 of 204
75.0 2775
Processing waveform 29 of 204
75.0 2850
Processing waveform 30 of 204
75.0 4425
Processing waveform 31 of 204
75.0 2250
Processing waveform 32 of 204
75.0 3075
Processing waveform 33 of 204
75.0 2775
Processing waveform 34 of 204
75.0 1875
Processing waveform 35 of 204
75.0 3300
Processing waveform 36 of 204
75.0 1875
Processing waveform 37 of 204
75.0 5550
Processing waveform 38 of 204
75.0 9075
Processing waveform 39 of 204
75.0 4650
Processing waveform 40 of 204
75.0 6525
Processing waveform 41 of 204
75.0 9075
Processing waveform 42 of 204
75.0 3225
Processing waveform 43 of 204
75.0 3150
Processing waveform 44 of 204
75.0 4275
Processing waveform 45 of 204
75.0 4050
Processing waveform 46 of 204
75.0 5400
Processing waveform 47 of 204
75.0 4725
Processing waveform 48 of 204
75.0 3450
Processing waveform 49 of 204
75.0 6150
Processing waveform 50 of 204
75.0 2250
Processing waveform 51 of 204
75.0 9000
Processing waveform 52 of 204
75.0 3150
Processing waveform 53 of 204
75.0 4950
Processing waveform 54 of 204
75.0 6750
Processing waveform 55 of 204
75.0 3150
Processing waveform 56 of 204
75.0 7050
Processing waveform 57 of 204
75.0 3675
Processing waveform 58 of 204
75.0 3225
Processing waveform 59 of 204
75.0 2325
Processing waveform 60 of 204
75.0 7500
Processing waveform 61 of 204
75.0 3975
Processing waveform 62 of 204
75.0 6150
Processing waveform 63 of 204
75.0 7875
Processing waveform 64 of 204
75.0 7275
Processing waveform 65 of 204
75.0 6075
Processing waveform 66 of 204
75.0 3900
Processing waveform 67 of 204
75.0 3000
Processing waveform 68 of 204
75.0 3900
Processing waveform 69 of 204
75.0 6525
Processing waveform 70 of 204
75.0 2925
Processing waveform 71 of 204
75.0 2700
Processing waveform 72 of 204
75.0 2250
Processing waveform 73 of 204
75.0 4425
Processing waveform 74 of 204
75.0 7500
Processing waveform 75 of 204
75.0 2250
Processing waveform 76 of 204
75.0 9600
Processing waveform 77 of 204
75.0 4650
Processing waveform 78 of 204
75.0 6750
Processing waveform 79 of 204
75.0 8925
Processing waveform 80 of 204
75.0 7125
Processing waveform 81 of 204
75.0 3000
Processing waveform 82 of 204
75.0 10200
Processing waveform 83 of 204
75.0 1800
Processing waveform 84 of 204
75.0 4425
Processing waveform 85 of 204
75.0 2250
Processing waveform 86 of 204
75.0 6150
Processing waveform 87 of 204
75.0 7125
Processing waveform 88 of 204
75.0 3600
Processing waveform 89 of 204
75.0 6000
Processing waveform 90 of 204
75.0 4800
Processing waveform 91 of 204
75.0 7200
Processing waveform 92 of 204
75.0 3000
Processing waveform 93 of 204
75.0 4500
Processing waveform 94 of 204
75.0 3600
Processing waveform 95 of 204
75.0 4875
Processing waveform 96 of 204
75.0 5625
Processing waveform 97 of 204
75.0 3150
Processing waveform 98 of 204
75.0 3975
Processing waveform 99 of 204
75.0 5325
Processing waveform 100 of 204
75.0 2850
Processing waveform 101 of 204
75.0 9450
Processing waveform 102 of 204
75.0 6525
Processing waveform 103 of 204
75.0 2250
Processing waveform 104 of 204
75.0 3600
Processing waveform 105 of 204
75.0 5775
Processing waveform 106 of 204
75.0 2325
Processing waveform 107 of 204
75.0 2325
Processing waveform 108 of 204
75.0 3525
Processing waveform 109 of 204
75.0 4500
Processing waveform 110 of 204
75.0 4275
Processing waveform 111 of 204
75.0 3450
Processing waveform 112 of 204
75.0 2550
Processing waveform 113 of 204
75.0 3300
Processing waveform 114 of 204
75.0 9075
Processing waveform 115 of 204
75.0 9075
Processing waveform 116 of 204
75.0 6525
Processing waveform 117 of 204
75.0 6975
Processing waveform 118 of 204
75.0 6750
Processing waveform 119 of 204
75.0 7275
Processing waveform 120 of 204
75.0 2400
Processing waveform 121 of 204
75.0 4125
Processing waveform 122 of 204
75.0 3525
Processing waveform 123 of 204
75.0 5175
Processing waveform 124 of 204
75.0 3750
Processing waveform 125 of 204
75.0 4875
Processing waveform 126 of 204
75.0 2250
Processing waveform 127 of 204
75.0 4125
Processing waveform 128 of 204
75.0 4425
Processing waveform 129 of 204
75.0 5250
Processing waveform 130 of 204
75.0 9075
Processing waveform 131 of 204
75.0 4725
Processing waveform 132 of 204
75.0 2475
Processing waveform 133 of 204
75.0 2925
Processing waveform 134 of 204
75.0 7350
Processing waveform 135 of 204
75.0 2550
Processing waveform 136 of 204
75.0 2850
Processing waveform 137 of 204
75.0 2550
Processing waveform 138 of 204
75.0 4425
Processing waveform 139 of 204
75.0 5775
Processing waveform 140 of 204
75.0 4200
Processing waveform 141 of 204
75.0 2175
Processing waveform 142 of 204
75.0 2925
Processing waveform 143 of 204
75.0 10350
Processing waveform 144 of 204
75.0 15600
Processing waveform 145 of 204
75.0 6450
Processing waveform 146 of 204
75.0 3450
Processing waveform 147 of 204
75.0 11175
Processing waveform 148 of 204
75.0 4725
Processing waveform 149 of 204
75.0 4200
Processing waveform 150 of 204
75.0 3750
Processing waveform 151 of 204
75.0 3975
Processing waveform 152 of 204
75.0 4575
Processing waveform 153 of 204
75.0 3225
Processing waveform 154 of 204
75.0 13500
Processing waveform 155 of 204
75.0 4050
Processing waveform 156 of 204
75.0 5250
Processing waveform 157 of 204
75.0 7050
Processing waveform 158 of 204
75.0 3450
Processing waveform 159 of 204
75.0 6675
Processing waveform 160 of 204
75.0 2850
Processing waveform 161 of 204
75.0 6750
Processing waveform 162 of 204
75.0 2475
Processing waveform 163 of 204
75.0 7650
Processing waveform 164 of 204
75.0 6750
Processing waveform 165 of 204
75.0 4125
Processing waveform 166 of 204
75.0 7425
Processing waveform 167 of 204
75.0 5400
Processing waveform 168 of 204
75.0 4800
Processing waveform 169 of 204
75.0 10200
Processing waveform 170 of 204
75.0 18225
Processing waveform 171 of 204
75.0 5925
Processing waveform 172 of 204
75.0 3525
Processing waveform 173 of 204
75.0 3300
Processing waveform 174 of 204
75.0 3600
Processing waveform 175 of 204
75.0 4650
Processing waveform 176 of 204
75.0 7875
Processing waveform 177 of 204
75.0 6300
Processing waveform 178 of 204
75.0 6750
Processing waveform 179 of 204
75.0 2925
Processing waveform 180 of 204
75.0 3525
Processing waveform 181 of 204
75.0 7875
Processing waveform 182 of 204
75.0 8925
Processing waveform 183 of 204
75.0 2850
Processing waveform 184 of 204
75.0 2475
Processing waveform 185 of 204
75.0 2325
Processing waveform 186 of 204
75.0 2250
Processing waveform 187 of 204
75.0 2700
Processing waveform 188 of 204
75.0 3000
Processing waveform 189 of 204
75.0 3300
Processing waveform 190 of 204
75.0 4425
Processing waveform 191 of 204
75.0 4500
Processing waveform 192 of 204
75.0 7500
Processing waveform 193 of 204
75.0 4200
Processing waveform 194 of 204
75.0 4800
Processing waveform 195 of 204
75.0 3000
Processing waveform 196 of 204
75.0 2475
Processing waveform 197 of 204
75.0 1950
Processing waveform 198 of 204
75.0 2625
Processing waveform 199 of 204
75.0 4650

Processing waveform 200 of 204
75.0 5475
Processing waveform 201 of 204
75.0 4950
Processing waveform 202 of 204
75.0 6375
Processing waveform 203 of 204
75.0 7500
Training data have been read and features have been extracted  (204, 120)
Computation time:  282.4353222846985
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.16326045989990234
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l   103       
        t         101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  51.8300653595  +/-  3.49596839747  %
          Predicted class
              l     t 
        l    28    23 
        t    26    25 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'r'] 1
class
e     93
r    110
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 203
75.0 5325
Processing waveform 1 of 203
75.0 2700
Processing waveform 2 of 203
75.0 4350
Processing waveform 3 of 203
75.0 2775
Processing waveform 4 of 203
75.0 1950
Processing waveform 5 of 203
75.0 2250
Processing waveform 6 of 203
75.0 2250
Processing waveform 7 of 203
75.0 2250
Processing waveform 8 of 203
75.0 5100
Processing waveform 9 of 203
75.0 2475
Processing waveform 10 of 203
75.0 3300
Processing waveform 11 of 203
75.0 2175
Processing waveform 12 of 203
75.0 6300
Processing waveform 13 of 203
75.0 9075
Processing waveform 14 of 203
75.0 3225
Processing waveform 15 of 203
75.0 4125
Processing waveform 16 of 203
75.0 3300
Processing waveform 17 of 203
75.0 3225
Processing waveform 18 of 203
75.0 2850
Processing waveform 19 of 203
75.0 5400
Processing waveform 20 of 203
75.0 2400
Processing waveform 21 of 203
75.0 7725
Processing waveform 22 of 203
75.0 3900
Processing waveform 23 of 203
75.0 2775
Processing waveform 24 of 203
75.0 2325
Processing waveform 25 of 203
75.0 4425
Processing waveform 26 of 203
75.0 3525
Processing waveform 27 of 203
75.0 6300
Processing waveform 28 of 203
75.0 2625
Processing waveform 29 of 203
75.0 4050
Processing waveform 30 of 203
75.0 2775
Processing waveform 31 of 203
75.0 2400
Processing waveform 32 of 203
75.0 2850
Processing waveform 33 of 203
75.0 2250
Processing waveform 34 of 203
75.0 2325
Processing waveform 35 of 203
75.0 3750
Processing waveform 36 of 203
75.0 3525
Processing waveform 37 of 203
75.0 5700
Processing waveform 38 of 203
75.0 2700
Processing waveform 39 of 203
75.0 3000
Processing waveform 40 of 203
75.0 4350
Processing waveform 41 of 203
75.0 4950
Processing waveform 42 of 203
75.0 2925
Processing waveform 43 of 203
75.0 2775
Processing waveform 44 of 203
75.0 3450
Processing waveform 45 of 203
75.0 4875
Processing waveform 46 of 203
75.0 9075
Processing waveform 47 of 203
75.0 5175
Processing waveform 48 of 203
75.0 3075
Processing waveform 49 of 203
75.0 4125
Processing waveform 50 of 203
75.0 9075
Processing waveform 51 of 203
75.0 3225
Processing waveform 52 of 203
75.0 2775
Processing waveform 53 of 203
75.0 4575
Processing waveform 54 of 203
75.0 2700
Processing waveform 55 of 203
75.0 1950
Processing waveform 56 of 203
75.0 2100
Processing waveform 57 of 203
75.0 4575
Processing waveform 58 of 203
75.0 5850
Processing waveform 59 of 203
75.0 3150
Processing waveform 60 of 203
75.0 4275
Processing waveform 61 of 203
75.0 9075
Processing waveform 62 of 203
75.0 3600
Processing waveform 63 of 203
75.0 2625
Processing waveform 64 of 203
75.0 2700
Processing waveform 65 of 203
75.0 2700
Processing waveform 66 of 203
75.0 3525
Processing waveform 67 of 203
75.0 4200
Processing waveform 68 of 203
75.0 4200
Processing waveform 69 of 203
75.0 1950
Processing waveform 70 of 203
75.0 5775
Processing waveform 71 of 203
75.0 9075
Processing waveform 72 of 203
75.0 4200
Processing waveform 73 of 203
75.0 2625
Processing waveform 74 of 203
75.0 4875
Processing waveform 75 of 203
75.0 5025
Processing waveform 76 of 203
75.0 14625
Processing waveform 77 of 203
75.0 11025
Processing waveform 78 of 203
75.0 1950
Processing waveform 79 of 203
75.0 3975
Processing waveform 80 of 203
75.0 2550
Processing waveform 81 of 203
75.0 7500
Processing waveform 82 of 203
75.0 14925
Processing waveform 83 of 203
75.0 13275
Processing waveform 84 of 203
75.0 1725
Processing waveform 85 of 203
75.0 2400
Processing waveform 86 of 203
75.0 6375
Processing waveform 87 of 203
75.0 6450
Processing waveform 88 of 203
75.0 4125
Processing waveform 89 of 203
75.0 3975
Processing waveform 90 of 203
75.0 1725
Processing waveform 91 of 203
75.0 2700
Processing waveform 92 of 203
75.0 4800
Processing waveform 93 of 203
75.0 7125
Processing waveform 94 of 203
75.0 5325
Processing waveform 95 of 203
75.0 5400
Processing waveform 96 of 203
75.0 6825
Processing waveform 97 of 203
75.0 7050
Processing waveform 98 of 203
75.0 4500
Processing waveform 99 of 203
75.0 5625
Processing waveform 100 of 203
75.0 4275
Processing waveform 101 of 203
75.0 2925
Processing waveform 102 of 203
75.0 3150
Processing waveform 103 of 203
75.0 6000
Processing waveform 104 of 203
75.0 3150
Processing waveform 105 of 203
75.0 6000
Processing waveform 106 of 203
75.0 12600
Processing waveform 107 of 203
75.0 7425
Processing waveform 108 of 203
75.0 16725
Processing waveform 109 of 203
75.0 6075
Processing waveform 110 of 203
75.0 3450
Processing waveform 111 of 203
75.0 2325
Processing waveform 112 of 203
75.0 2925
Processing waveform 113 of 203
75.0 4575
Processing waveform 114 of 203
75.0 7050
Processing waveform 115 of 203
75.0 2025
Processing waveform 116 of 203
75.0 2925
Processing waveform 117 of 203
75.0 2850
Processing waveform 118 of 203
75.0 3600
Processing waveform 119 of 203
75.0 3750
Processing waveform 120 of 203
75.0 2100
Processing waveform 121 of 203
75.0 1650
Processing waveform 122 of 203
75.0 4200
Processing waveform 123 of 203
75.0 9075
Processing waveform 124 of 203
75.0 4125
Processing waveform 125 of 203
75.0 2925
Processing waveform 126 of 203
75.0 2250
Processing waveform 127 of 203
75.0 5475
Processing waveform 128 of 203
75.0 2625
Processing waveform 129 of 203
75.0 2400
Processing waveform 130 of 203
75.0 2700
Processing waveform 131 of 203
75.0 9075
Processing waveform 132 of 203
75.0 3300
Processing waveform 133 of 203
75.0 9075
Processing waveform 134 of 203
75.0 6225
Processing waveform 135 of 203
75.0 7125
Processing waveform 136 of 203
75.0 5850
Processing waveform 137 of 203
75.0 9075
Processing waveform 138 of 203
75.0 3225
Processing waveform 139 of 203
75.0 8100
Processing waveform 140 of 203
75.0 4575
Processing waveform 141 of 203
75.0 2850
Processing waveform 142 of 203
75.0 3375
Processing waveform 143 of 203
75.0 2100
Processing waveform 144 of 203
75.0 9075
Processing waveform 145 of 203
75.0 5850
Processing waveform 146 of 203
75.0 2775
Processing waveform 147 of 203
75.0 2700
Processing waveform 148 of 203
75.0 2100
Processing waveform 149 of 203
75.0 9075
Processing waveform 150 of 203
75.0 3225
Processing waveform 151 of 203
75.0 3300
Processing waveform 152 of 203
75.0 2475
Processing waveform 153 of 203
75.0 3075
Processing waveform 154 of 203
75.0 6450
Processing waveform 155 of 203
75.0 3600
Processing waveform 156 of 203
75.0 4650
Processing waveform 157 of 203
75.0 2475
Processing waveform 158 of 203
75.0 1875
Processing waveform 159 of 203
75.0 3450
Processing waveform 160 of 203
75.0 1500
Processing waveform 161 of 203
75.0 2625
Processing waveform 162 of 203
75.0 9075
Processing waveform 163 of 203
75.0 3525
Processing waveform 164 of 203
75.0 5625
Processing waveform 165 of 203
75.0 3675
Processing waveform 166 of 203
75.0 9075
Processing waveform 167 of 203
75.0 2700
Processing waveform 168 of 203
75.0 9075
Processing waveform 169 of 203
75.0 3000
Processing waveform 170 of 203
75.0 3525
Processing waveform 171 of 203
75.0 7200
Processing waveform 172 of 203
75.0 4875
Processing waveform 173 of 203
75.0 6075
Processing waveform 174 of 203
75.0 2700
Processing waveform 175 of 203
75.0 5250

Processing waveform 176 of 203
75.0 12750
Processing waveform 177 of 203
75.0 5100
Processing waveform 178 of 203
75.0 2475
Processing waveform 179 of 203
75.0 6525
Processing waveform 180 of 203
75.0 7200
Processing waveform 181 of 203
75.0 4500
Processing waveform 182 of 203
75.0 4950
Processing waveform 183 of 203
75.0 8250
Processing waveform 184 of 203
75.0 3150
Processing waveform 185 of 203
75.0 5850
Processing waveform 186 of 203
75.0 2325
Processing waveform 187 of 203
75.0 4200
Processing waveform 188 of 203
75.0 3300
Processing waveform 189 of 203
75.0 2325
Processing waveform 190 of 203
75.0 13500
Processing waveform 191 of 203
75.0 3525
Processing waveform 192 of 203
75.0 3075
Processing waveform 193 of 203
75.0 6600
Processing waveform 194 of 203
75.0 6225
Processing waveform 195 of 203
75.0 9000
Processing waveform 196 of 203
75.0 5100
Processing waveform 197 of 203
75.0 6975
Processing waveform 198 of 203
75.0 4425
Processing waveform 199 of 203
75.0 5400
Processing waveform 200 of 203
75.0 13350
Processing waveform 201 of 203
75.0 9525
Processing waveform 202 of 203
75.0 2325
Training data have been read and features have been extracted  (203, 120)
Computation time:  293.07095885276794
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.16646885871887207
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    93       
        r         110 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  53.3660130719  +/-  4.20300062263  %
          Predicted class
              e     r 
        e    20    27 
        r    20    35 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't'] 1
class
h    106
l    103
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 310
75.0 3975
Processing waveform 1 of 310
75.0 5025
Processing waveform 2 of 310
75.0 6825
Processing waveform 3 of 310
75.0 9075
Processing waveform 4 of 310
75.0 4500
Processing waveform 5 of 310
75.0 2175
Processing waveform 6 of 310
75.0 3300
Processing waveform 7 of 310
75.0 4275
Processing waveform 8 of 310
75.0 2850
Processing waveform 9 of 310
75.0 2850
Processing waveform 10 of 310
75.0 2700
Processing waveform 11 of 310
75.0 5250
Processing waveform 12 of 310
75.0 3000
Processing waveform 13 of 310
75.0 2775
Processing waveform 14 of 310
75.0 6450
Processing waveform 15 of 310
75.0 3150
Processing waveform 16 of 310
75.0 1650
Processing waveform 17 of 310
75.0 4500
Processing waveform 18 of 310
75.0 2250
Processing waveform 19 of 310
75.0 2175
Processing waveform 20 of 310
75.0 3225
Processing waveform 21 of 310
75.0 4050
Processing waveform 22 of 310
75.0 3300
Processing waveform 23 of 310
75.0 2850
Processing waveform 24 of 310
75.0 2025
Processing waveform 25 of 310
75.0 4800
Processing waveform 26 of 310
75.0 2325
Processing waveform 27 of 310
75.0 6150
Processing waveform 28 of 310
75.0 2775
Processing waveform 29 of 310
75.0 2850
Processing waveform 30 of 310
75.0 4425
Processing waveform 31 of 310
75.0 2250
Processing waveform 32 of 310
75.0 3075
Processing waveform 33 of 310
75.0 2775
Processing waveform 34 of 310
75.0 1875
Processing waveform 35 of 310
75.0 3300
Processing waveform 36 of 310
75.0 1875
Processing waveform 37 of 310
75.0 5550
Processing waveform 38 of 310
75.0 9075
Processing waveform 39 of 310
75.0 4650
Processing waveform 40 of 310
75.0 6525
Processing waveform 41 of 310
75.0 9075
Processing waveform 42 of 310
75.0 3225
Processing waveform 43 of 310
75.0 3150
Processing waveform 44 of 310
75.0 4275
Processing waveform 45 of 310
75.0 4050
Processing waveform 46 of 310
75.0 5400
Processing waveform 47 of 310
75.0 4725
Processing waveform 48 of 310
75.0 3450
Processing waveform 49 of 310
75.0 6150
Processing waveform 50 of 310
75.0 2250
Processing waveform 51 of 310
75.0 9000
Processing waveform 52 of 310
75.0 3150
Processing waveform 53 of 310
75.0 4950
Processing waveform 54 of 310
75.0 6750
Processing waveform 55 of 310
75.0 3150
Processing waveform 56 of 310
75.0 7050
Processing waveform 57 of 310
75.0 3675
Processing waveform 58 of 310
75.0 3225
Processing waveform 59 of 310
75.0 2325
Processing waveform 60 of 310
75.0 7500
Processing waveform 61 of 310
75.0 3975
Processing waveform 62 of 310
75.0 6150
Processing waveform 63 of 310
75.0 7875
Processing waveform 64 of 310
75.0 7275
Processing waveform 65 of 310
75.0 6075
Processing waveform 66 of 310
75.0 3900
Processing waveform 67 of 310
75.0 3000
Processing waveform 68 of 310
75.0 3900
Processing waveform 69 of 310
75.0 6525
Processing waveform 70 of 310
75.0 2925
Processing waveform 71 of 310
75.0 2700
Processing waveform 72 of 310
75.0 2250
Processing waveform 73 of 310
75.0 4425
Processing waveform 74 of 310
75.0 7500
Processing waveform 75 of 310
75.0 2250
Processing waveform 76 of 310
75.0 9600
Processing waveform 77 of 310
75.0 4650
Processing waveform 78 of 310
75.0 6750
Processing waveform 79 of 310
75.0 8925
Processing waveform 80 of 310
75.0 7125
Processing waveform 81 of 310
75.0 3000
Processing waveform 82 of 310
75.0 10200
Processing waveform 83 of 310
75.0 1800
Processing waveform 84 of 310
75.0 4425
Processing waveform 85 of 310
75.0 2250
Processing waveform 86 of 310
75.0 6150
Processing waveform 87 of 310
75.0 7125
Processing waveform 88 of 310
75.0 3600
Processing waveform 89 of 310
75.0 6000
Processing waveform 90 of 310
75.0 4800
Processing waveform 91 of 310
75.0 7200
Processing waveform 92 of 310
75.0 3000
Processing waveform 93 of 310
75.0 4500
Processing waveform 94 of 310
75.0 3600
Processing waveform 95 of 310
75.0 4875
Processing waveform 96 of 310
75.0 5625
Processing waveform 97 of 310
75.0 3150
Processing waveform 98 of 310
75.0 3975
Processing waveform 99 of 310
75.0 5325
Processing waveform 100 of 310
75.0 2850
Processing waveform 101 of 310
75.0 9450
Processing waveform 102 of 310
75.0 6525
Processing waveform 103 of 310
75.0 3225
Processing waveform 104 of 310
75.0 2100
Processing waveform 105 of 310
75.0 2700
Processing waveform 106 of 310
75.0 2325
Processing waveform 107 of 310
75.0 2775
Processing waveform 108 of 310
75.0 2625
Processing waveform 109 of 310
75.0 5850
Processing waveform 110 of 310
75.0 3825
Processing waveform 111 of 310
75.0 6000
Processing waveform 112 of 310
75.0 2025
Processing waveform 113 of 310
75.0 1725
Processing waveform 114 of 310
75.0 2175
Processing waveform 115 of 310
75.0 5400
Processing waveform 116 of 310
75.0 5325
Processing waveform 117 of 310
75.0 3750
Processing waveform 118 of 310
75.0 3075
Processing waveform 119 of 310
75.0 5550
Processing waveform 120 of 310
75.0 7350
Processing waveform 121 of 310
75.0 4650
Processing waveform 122 of 310
75.0 2775
Processing waveform 123 of 310
75.0 2325
Processing waveform 124 of 310
75.0 2925
Processing waveform 125 of 310
75.0 3075
Processing waveform 126 of 310
75.0 6375
Processing waveform 127 of 310
75.0 2475
Processing waveform 128 of 310
75.0 3975
Processing waveform 129 of 310
75.0 3150
Processing waveform 130 of 310
75.0 5925
Processing waveform 131 of 310
75.0 5400
Processing waveform 132 of 310
75.0 1875
Processing waveform 133 of 310
75.0 5850
Processing waveform 134 of 310
75.0 1725
Processing waveform 135 of 310
75.0 7650
Processing waveform 136 of 310
75.0 2700
Processing waveform 137 of 310
75.0 9075
Processing waveform 138 of 310
75.0 6975
Processing waveform 139 of 310
75.0 4050
Processing waveform 140 of 310
75.0 3450
Processing waveform 141 of 310
75.0 5925
Processing waveform 142 of 310
75.0 2250
Processing waveform 143 of 310
75.0 3675
Processing waveform 144 of 310
75.0 2625
Processing waveform 145 of 310
75.0 4575
Processing waveform 146 of 310
75.0 2025
Processing waveform 147 of 310
75.0 5175
Processing waveform 148 of 310
75.0 3225
Processing waveform 149 of 310
75.0 4950
Processing waveform 150 of 310
75.0 2925
Processing waveform 151 of 310
75.0 1800
Processing waveform 152 of 310
75.0 8925

Processing waveform 153 of 310
75.0 7350
Processing waveform 154 of 310
75.0 4125
Processing waveform 155 of 310
75.0 3225
Processing waveform 156 of 310
75.0 9075
Processing waveform 157 of 310
75.0 5025
Processing waveform 158 of 310
75.0 8925
Processing waveform 159 of 310
75.0 4800
Processing waveform 160 of 310
75.0 7800
Processing waveform 161 of 310
75.0 2775
Processing waveform 162 of 310
75.0 6600
Processing waveform 163 of 310
75.0 1800
Processing waveform 164 of 310
75.0 2625
Processing waveform 165 of 310
75.0 9075
Processing waveform 166 of 310
75.0 7200
Processing waveform 167 of 310
75.0 5550
Processing waveform 168 of 310
75.0 2775
Processing waveform 169 of 310
75.0 5250
Processing waveform 170 of 310
75.0 1875
Processing waveform 171 of 310
75.0 1875
Processing waveform 172 of 310
75.0 5025
Processing waveform 173 of 310
75.0 3600
Processing waveform 174 of 310
75.0 3375
Processing waveform 175 of 310
75.0 4050
Processing waveform 176 of 310
75.0 3375
Processing waveform 177 of 310
75.0 2925
Processing waveform 178 of 310
75.0 6675
Processing waveform 179 of 310
75.0 3750
Processing waveform 180 of 310
75.0 4575
Processing waveform 181 of 310
75.0 9075
Processing waveform 182 of 310
75.0 3900
Processing waveform 183 of 310
75.0 9075
Processing waveform 184 of 310
75.0 4875
Processing waveform 185 of 310
75.0 3075
Processing waveform 186 of 310
75.0 3150
Processing waveform 187 of 310
75.0 2550
Processing waveform 188 of 310
75.0 9075
Processing waveform 189 of 310
75.0 7425
Processing waveform 190 of 310
75.0 5025
Processing waveform 191 of 310
75.0 2850
Processing waveform 192 of 310
75.0 1875
Processing waveform 193 of 310
75.0 7125
Processing waveform 194 of 310
75.0 4500
Processing waveform 195 of 310
75.0 6075
Processing waveform 196 of 310
75.0 2475
Processing waveform 197 of 310
75.0 4125
Processing waveform 198 of 310
75.0 4500
Processing waveform 199 of 310
75.0 2100
Processing waveform 200 of 310
75.0 3975
Processing waveform 201 of 310
75.0 3225
Processing waveform 202 of 310
75.0 2550
Processing waveform 203 of 310
75.0 6975
Processing waveform 204 of 310
75.0 3075
Processing waveform 205 of 310
75.0 3675
Processing waveform 206 of 310
75.0 4800
Processing waveform 207 of 310
75.0 5850
Processing waveform 208 of 310
75.0 3675
Processing waveform 209 of 310
75.0 2250
Processing waveform 210 of 310
75.0 3600
Processing waveform 211 of 310
75.0 5775
Processing waveform 212 of 310
75.0 2325
Processing waveform 213 of 310
75.0 2325
Processing waveform 214 of 310
75.0 3525
Processing waveform 215 of 310
75.0 4500
Processing waveform 216 of 310
75.0 4275
Processing waveform 217 of 310
75.0 3450
Processing waveform 218 of 310
75.0 2550
Processing waveform 219 of 310
75.0 3300
Processing waveform 220 of 310
75.0 9075
Processing waveform 221 of 310
75.0 9075
Processing waveform 222 of 310
75.0 6525
Processing waveform 223 of 310
75.0 6975
Processing waveform 224 of 310
75.0 6750
Processing waveform 225 of 310
75.0 7275
Processing waveform 226 of 310
75.0 2400
Processing waveform 227 of 310
75.0 4125
Processing waveform 228 of 310
75.0 3525
Processing waveform 229 of 310
75.0 5175
Processing waveform 230 of 310
75.0 3750
Processing waveform 231 of 310
75.0 4875
Processing waveform 232 of 310
75.0 2250
Processing waveform 233 of 310
75.0 4125
Processing waveform 234 of 310
75.0 4425
Processing waveform 235 of 310
75.0 5250
Processing waveform 236 of 310
75.0 9075
Processing waveform 237 of 310
75.0 4725
Processing waveform 238 of 310
75.0 2475
Processing waveform 239 of 310
75.0 2925
Processing waveform 240 of 310
75.0 7350
Processing waveform 241 of 310
75.0 2550
Processing waveform 242 of 310
75.0 2850
Processing waveform 243 of 310
75.0 2550
Processing waveform 244 of 310
75.0 4425
Processing waveform 245 of 310
75.0 5775
Processing waveform 246 of 310
75.0 4200
Processing waveform 247 of 310
75.0 2175
Processing waveform 248 of 310
75.0 2925
Processing waveform 249 of 310
75.0 10350
Processing waveform 250 of 310
75.0 15600
Processing waveform 251 of 310
75.0 6450
Processing waveform 252 of 310
75.0 3450
Processing waveform 253 of 310
75.0 11175
Processing waveform 254 of 310
75.0 4725
Processing waveform 255 of 310
75.0 4200
Processing waveform 256 of 310
75.0 3750
Processing waveform 257 of 310
75.0 3975
Processing waveform 258 of 310
75.0 4575
Processing waveform 259 of 310
75.0 3225
Processing waveform 260 of 310
75.0 13500
Processing waveform 261 of 310
75.0 4050
Processing waveform 262 of 310
75.0 5250
Processing waveform 263 of 310
75.0 7050
Processing waveform 264 of 310
75.0 3450
Processing waveform 265 of 310
75.0 6675
Processing waveform 266 of 310
75.0 2850
Processing waveform 267 of 310
75.0 6750
Processing waveform 268 of 310
75.0 2475
Processing waveform 269 of 310
75.0 7650
Processing waveform 270 of 310
75.0 6750
Processing waveform 271 of 310
75.0 4125
Processing waveform 272 of 310
75.0 7425
Processing waveform 273 of 310
75.0 5400
Processing waveform 274 of 310
75.0 4800
Processing waveform 275 of 310
75.0 10200
Processing waveform 276 of 310
75.0 18225
Processing waveform 277 of 310
75.0 5925
Processing waveform 278 of 310
75.0 3525
Processing waveform 279 of 310
75.0 3300
Processing waveform 280 of 310
75.0 3600
Processing waveform 281 of 310
75.0 4650
Processing waveform 282 of 310
75.0 7875
Processing waveform 283 of 310
75.0 6300
Processing waveform 284 of 310
75.0 6750
Processing waveform 285 of 310
75.0 2925
Processing waveform 286 of 310
75.0 3525
Processing waveform 287 of 310
75.0 7875
Processing waveform 288 of 310
75.0 8925
Processing waveform 289 of 310
75.0 2850
Processing waveform 290 of 310
75.0 2475
Processing waveform 291 of 310
75.0 2325
Processing waveform 292 of 310
75.0 2250
Processing waveform 293 of 310
75.0 2700
Processing waveform 294 of 310
75.0 3000
Processing waveform 295 of 310
75.0 3300
Processing waveform 296 of 310
75.0 4425
Processing waveform 297 of 310
75.0 4500
Processing waveform 298 of 310
75.0 7500
Processing waveform 299 of 310
75.0 4200
Processing waveform 300 of 310
75.0 4800
Processing waveform 301 of 310
75.0 3000
Processing waveform 302 of 310
75.0 2475
Processing waveform 303 of 310
75.0 1950
Processing waveform 304 of 310
75.0 2625
Processing waveform 305 of 310
75.0 4650
Processing waveform 306 of 310
75.0 5475
Processing waveform 307 of 310
75.0 4950
Processing waveform 308 of 310
75.0 6375
Processing waveform 309 of 310
75.0 7500
Training data have been read and features have been extracted  (310, 120)
Computation time:  402.0796387195587
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.31955742835998535
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h   106             
        l         103       
        t               101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  39.0107526882  +/-  3.62568144786  %
          Predicted class
              h     l     t 
        h    26    14    13 
        l    20    17    15 
        t    17    15    18 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't', 'r'] 1
class
h    106
l    103
r    110
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 420
75.0 5325
Processing waveform 1 of 420
75.0 2700
Processing waveform 2 of 420
75.0 4350
Processing waveform 3 of 420
75.0 2775
Processing waveform 4 of 420
75.0 1950
Processing waveform 5 of 420
75.0 2250
Processing waveform 6 of 420
75.0 2250
Processing waveform 7 of 420
75.0 2250
Processing waveform 8 of 420
75.0 5100
Processing waveform 9 of 420
75.0 2475
Processing waveform 10 of 420
75.0 3300
Processing waveform 11 of 420
75.0 2175
Processing waveform 12 of 420
75.0 6300
Processing waveform 13 of 420
75.0 9075
Processing waveform 14 of 420
75.0 3225
Processing waveform 15 of 420
75.0 4125
Processing waveform 16 of 420
75.0 3300
Processing waveform 17 of 420
75.0 3225

Processing waveform 18 of 420
75.0 2850
Processing waveform 19 of 420
75.0 5400
Processing waveform 20 of 420
75.0 2400
Processing waveform 21 of 420
75.0 7725
Processing waveform 22 of 420
75.0 3900
Processing waveform 23 of 420
75.0 2775
Processing waveform 24 of 420
75.0 2325
Processing waveform 25 of 420
75.0 4425
Processing waveform 26 of 420
75.0 3525
Processing waveform 27 of 420
75.0 6300
Processing waveform 28 of 420
75.0 2625
Processing waveform 29 of 420
75.0 4050
Processing waveform 30 of 420
75.0 2775
Processing waveform 31 of 420
75.0 2400
Processing waveform 32 of 420
75.0 2850
Processing waveform 33 of 420
75.0 2250
Processing waveform 34 of 420
75.0 2325
Processing waveform 35 of 420
75.0 3750
Processing waveform 36 of 420
75.0 3525
Processing waveform 37 of 420
75.0 5700
Processing waveform 38 of 420
75.0 2700
Processing waveform 39 of 420
75.0 3000
Processing waveform 40 of 420
75.0 4350
Processing waveform 41 of 420
75.0 4950
Processing waveform 42 of 420
75.0 2925
Processing waveform 43 of 420
75.0 2775
Processing waveform 44 of 420
75.0 3450
Processing waveform 45 of 420
75.0 4875
Processing waveform 46 of 420
75.0 9075
Processing waveform 47 of 420
75.0 5175
Processing waveform 48 of 420
75.0 3075
Processing waveform 49 of 420
75.0 4125
Processing waveform 50 of 420
75.0 9075
Processing waveform 51 of 420
75.0 3225
Processing waveform 52 of 420
75.0 2775
Processing waveform 53 of 420
75.0 4575
Processing waveform 54 of 420
75.0 2700
Processing waveform 55 of 420
75.0 1950
Processing waveform 56 of 420
75.0 2100
Processing waveform 57 of 420
75.0 4575
Processing waveform 58 of 420
75.0 5850
Processing waveform 59 of 420
75.0 3150
Processing waveform 60 of 420
75.0 4275
Processing waveform 61 of 420
75.0 9075
Processing waveform 62 of 420
75.0 3600
Processing waveform 63 of 420
75.0 2625
Processing waveform 64 of 420
75.0 2700
Processing waveform 65 of 420
75.0 2700
Processing waveform 66 of 420
75.0 3525
Processing waveform 67 of 420
75.0 4200
Processing waveform 68 of 420
75.0 4200
Processing waveform 69 of 420
75.0 1950
Processing waveform 70 of 420
75.0 5775
Processing waveform 71 of 420
75.0 9075
Processing waveform 72 of 420
75.0 4200
Processing waveform 73 of 420
75.0 2625
Processing waveform 74 of 420
75.0 4875
Processing waveform 75 of 420
75.0 5025
Processing waveform 76 of 420
75.0 14625
Processing waveform 77 of 420
75.0 11025
Processing waveform 78 of 420
75.0 1950
Processing waveform 79 of 420
75.0 3975
Processing waveform 80 of 420
75.0 2550
Processing waveform 81 of 420
75.0 7500
Processing waveform 82 of 420
75.0 14925
Processing waveform 83 of 420
75.0 13275
Processing waveform 84 of 420
75.0 1725
Processing waveform 85 of 420
75.0 2400
Processing waveform 86 of 420
75.0 6375
Processing waveform 87 of 420
75.0 6450
Processing waveform 88 of 420
75.0 4125
Processing waveform 89 of 420
75.0 3975
Processing waveform 90 of 420
75.0 1725
Processing waveform 91 of 420
75.0 2700
Processing waveform 92 of 420
75.0 4800
Processing waveform 93 of 420
75.0 7125
Processing waveform 94 of 420
75.0 5325
Processing waveform 95 of 420
75.0 5400
Processing waveform 96 of 420
75.0 6825
Processing waveform 97 of 420
75.0 7050
Processing waveform 98 of 420
75.0 4500
Processing waveform 99 of 420
75.0 5625
Processing waveform 100 of 420
75.0 4275
Processing waveform 101 of 420
75.0 2925
Processing waveform 102 of 420
75.0 3150
Processing waveform 103 of 420
75.0 6000
Processing waveform 104 of 420
75.0 3150
Processing waveform 105 of 420
75.0 6000
Processing waveform 106 of 420
75.0 12600
Processing waveform 107 of 420
75.0 7425
Processing waveform 108 of 420
75.0 16725
Processing waveform 109 of 420
75.0 6075
Processing waveform 110 of 420
75.0 3975
Processing waveform 111 of 420
75.0 5025
Processing waveform 112 of 420
75.0 6825
Processing waveform 113 of 420
75.0 9075
Processing waveform 114 of 420
75.0 4500
Processing waveform 115 of 420
75.0 2175
Processing waveform 116 of 420
75.0 3300
Processing waveform 117 of 420
75.0 4275
Processing waveform 118 of 420
75.0 2850
Processing waveform 119 of 420
75.0 2850
Processing waveform 120 of 420
75.0 2700
Processing waveform 121 of 420
75.0 5250
Processing waveform 122 of 420
75.0 3000
Processing waveform 123 of 420
75.0 2775
Processing waveform 124 of 420
75.0 6450
Processing waveform 125 of 420
75.0 3150
Processing waveform 126 of 420
75.0 1650
Processing waveform 127 of 420
75.0 4500
Processing waveform 128 of 420
75.0 2250
Processing waveform 129 of 420
75.0 2175
Processing waveform 130 of 420
75.0 3225
Processing waveform 131 of 420
75.0 4050
Processing waveform 132 of 420
75.0 3300
Processing waveform 133 of 420
75.0 2850
Processing waveform 134 of 420
75.0 2025
Processing waveform 135 of 420
75.0 4800
Processing waveform 136 of 420
75.0 2325
Processing waveform 137 of 420
75.0 6150
Processing waveform 138 of 420
75.0 2775
Processing waveform 139 of 420
75.0 2850
Processing waveform 140 of 420
75.0 4425
Processing waveform 141 of 420
75.0 2250
Processing waveform 142 of 420
75.0 3075
Processing waveform 143 of 420
75.0 2775
Processing waveform 144 of 420
75.0 1875
Processing waveform 145 of 420
75.0 3300
Processing waveform 146 of 420
75.0 1875
Processing waveform 147 of 420
75.0 5550
Processing waveform 148 of 420
75.0 9075
Processing waveform 149 of 420
75.0 4650
Processing waveform 150 of 420
75.0 6525
Processing waveform 151 of 420
75.0 9075
Processing waveform 152 of 420
75.0 3225
Processing waveform 153 of 420
75.0 3150
Processing waveform 154 of 420
75.0 4275
Processing waveform 155 of 420
75.0 4050
Processing waveform 156 of 420
75.0 5400
Processing waveform 157 of 420
75.0 4725
Processing waveform 158 of 420
75.0 3450
Processing waveform 159 of 420
75.0 6150
Processing waveform 160 of 420
75.0 2250
Processing waveform 161 of 420
75.0 9000
Processing waveform 162 of 420
75.0 3150
Processing waveform 163 of 420
75.0 4950
Processing waveform 164 of 420
75.0 6750
Processing waveform 165 of 420
75.0 3150
Processing waveform 166 of 420
75.0 7050
Processing waveform 167 of 420
75.0 3675
Processing waveform 168 of 420
75.0 3225
Processing waveform 169 of 420
75.0 2325
Processing waveform 170 of 420
75.0 7500
Processing waveform 171 of 420
75.0 3975
Processing waveform 172 of 420
75.0 6150
Processing waveform 173 of 420
75.0 7875
Processing waveform 174 of 420
75.0 7275
Processing waveform 175 of 420
75.0 6075
Processing waveform 176 of 420
75.0 3900
Processing waveform 177 of 420
75.0 3000
Processing waveform 178 of 420
75.0 3900
Processing waveform 179 of 420
75.0 6525
Processing waveform 180 of 420
75.0 2925
Processing waveform 181 of 420
75.0 2700
Processing waveform 182 of 420
75.0 2250
Processing waveform 183 of 420
75.0 4425
Processing waveform 184 of 420
75.0 7500
Processing waveform 185 of 420
75.0 2250
Processing waveform 186 of 420
75.0 9600
Processing waveform 187 of 420
75.0 4650
Processing waveform 188 of 420
75.0 6750
Processing waveform 189 of 420
75.0 8925
Processing waveform 190 of 420
75.0 7125
Processing waveform 191 of 420
75.0 3000
Processing waveform 192 of 420
75.0 10200
Processing waveform 193 of 420
75.0 1800
Processing waveform 194 of 420
75.0 4425
Processing waveform 195 of 420
75.0 2250
Processing waveform 196 of 420
75.0 6150
Processing waveform 197 of 420
75.0 7125
Processing waveform 198 of 420
75.0 3600
Processing waveform 199 of 420
75.0 6000
Processing waveform 200 of 420
75.0 4800
Processing waveform 201 of 420
75.0 7200
Processing waveform 202 of 420
75.0 3000
Processing waveform 203 of 420
75.0 4500
Processing waveform 204 of 420
75.0 3600
Processing waveform 205 of 420
75.0 4875
Processing waveform 206 of 420
75.0 5625
Processing waveform 207 of 420
75.0 3150
Processing waveform 208 of 420
75.0 3975
Processing waveform 209 of 420
75.0 5325
Processing waveform 210 of 420
75.0 2850
Processing waveform 211 of 420
75.0 9450
Processing waveform 212 of 420
75.0 6525
Processing waveform 213 of 420
75.0 3225
Processing waveform 214 of 420
75.0 2100
Processing waveform 215 of 420
75.0 2700
Processing waveform 216 of 420
75.0 2325
Processing waveform 217 of 420
75.0 2775
Processing waveform 218 of 420
75.0 2625
Processing waveform 219 of 420
75.0 5850

Processing waveform 220 of 420
75.0 3825
Processing waveform 221 of 420
75.0 6000
Processing waveform 222 of 420
75.0 2025
Processing waveform 223 of 420
75.0 1725
Processing waveform 224 of 420
75.0 2175
Processing waveform 225 of 420
75.0 5400
Processing waveform 226 of 420
75.0 5325
Processing waveform 227 of 420
75.0 3750
Processing waveform 228 of 420
75.0 3075
Processing waveform 229 of 420
75.0 5550
Processing waveform 230 of 420
75.0 7350
Processing waveform 231 of 420
75.0 4650
Processing waveform 232 of 420
75.0 2775
Processing waveform 233 of 420
75.0 2325
Processing waveform 234 of 420
75.0 2925
Processing waveform 235 of 420
75.0 3075
Processing waveform 236 of 420
75.0 6375
Processing waveform 237 of 420
75.0 2475
Processing waveform 238 of 420
75.0 3975
Processing waveform 239 of 420
75.0 3150
Processing waveform 240 of 420
75.0 5925
Processing waveform 241 of 420
75.0 5400
Processing waveform 242 of 420
75.0 1875
Processing waveform 243 of 420
75.0 5850
Processing waveform 244 of 420
75.0 1725
Processing waveform 245 of 420
75.0 7650
Processing waveform 246 of 420
75.0 2700
Processing waveform 247 of 420
75.0 9075
Processing waveform 248 of 420
75.0 6975
Processing waveform 249 of 420
75.0 4050
Processing waveform 250 of 420
75.0 3450
Processing waveform 251 of 420
75.0 5925
Processing waveform 252 of 420
75.0 2250
Processing waveform 253 of 420
75.0 3675
Processing waveform 254 of 420
75.0 2625
Processing waveform 255 of 420
75.0 4575
Processing waveform 256 of 420
75.0 2025
Processing waveform 257 of 420
75.0 5175
Processing waveform 258 of 420
75.0 3225
Processing waveform 259 of 420
75.0 4950
Processing waveform 260 of 420
75.0 2925
Processing waveform 261 of 420
75.0 1800
Processing waveform 262 of 420
75.0 8925
Processing waveform 263 of 420
75.0 7350
Processing waveform 264 of 420
75.0 4125
Processing waveform 265 of 420
75.0 3225
Processing waveform 266 of 420
75.0 9075
Processing waveform 267 of 420
75.0 5025
Processing waveform 268 of 420
75.0 8925
Processing waveform 269 of 420
75.0 4800
Processing waveform 270 of 420
75.0 7800
Processing waveform 271 of 420
75.0 2775
Processing waveform 272 of 420
75.0 6600
Processing waveform 273 of 420
75.0 1800
Processing waveform 274 of 420
75.0 2625
Processing waveform 275 of 420
75.0 9075
Processing waveform 276 of 420
75.0 7200
Processing waveform 277 of 420
75.0 5550
Processing waveform 278 of 420
75.0 2775
Processing waveform 279 of 420
75.0 5250
Processing waveform 280 of 420
75.0 1875
Processing waveform 281 of 420
75.0 1875
Processing waveform 282 of 420
75.0 5025
Processing waveform 283 of 420
75.0 3600
Processing waveform 284 of 420
75.0 3375
Processing waveform 285 of 420
75.0 4050
Processing waveform 286 of 420
75.0 3375
Processing waveform 287 of 420
75.0 2925
Processing waveform 288 of 420
75.0 6675
Processing waveform 289 of 420
75.0 3750
Processing waveform 290 of 420
75.0 4575
Processing waveform 291 of 420
75.0 9075
Processing waveform 292 of 420
75.0 3900
Processing waveform 293 of 420
75.0 9075
Processing waveform 294 of 420
75.0 4875
Processing waveform 295 of 420
75.0 3075
Processing waveform 296 of 420
75.0 3150
Processing waveform 297 of 420
75.0 2550
Processing waveform 298 of 420
75.0 9075
Processing waveform 299 of 420
75.0 7425
Processing waveform 300 of 420
75.0 5025
Processing waveform 301 of 420
75.0 2850
Processing waveform 302 of 420
75.0 1875
Processing waveform 303 of 420
75.0 7125
Processing waveform 304 of 420
75.0 4500
Processing waveform 305 of 420
75.0 6075
Processing waveform 306 of 420
75.0 2475
Processing waveform 307 of 420
75.0 4125
Processing waveform 308 of 420
75.0 4500
Processing waveform 309 of 420
75.0 2100
Processing waveform 310 of 420
75.0 3975
Processing waveform 311 of 420
75.0 3225
Processing waveform 312 of 420
75.0 2550
Processing waveform 313 of 420
75.0 6975
Processing waveform 314 of 420
75.0 3075
Processing waveform 315 of 420
75.0 3675
Processing waveform 316 of 420
75.0 4800
Processing waveform 317 of 420
75.0 5850
Processing waveform 318 of 420
75.0 3675
Processing waveform 319 of 420
75.0 2250
Processing waveform 320 of 420
75.0 3600
Processing waveform 321 of 420
75.0 5775
Processing waveform 322 of 420
75.0 2325
Processing waveform 323 of 420
75.0 2325
Processing waveform 324 of 420
75.0 3525
Processing waveform 325 of 420
75.0 4500
Processing waveform 326 of 420
75.0 4275
Processing waveform 327 of 420
75.0 3450
Processing waveform 328 of 420
75.0 2550
Processing waveform 329 of 420
75.0 3300
Processing waveform 330 of 420
75.0 9075
Processing waveform 331 of 420
75.0 9075
Processing waveform 332 of 420
75.0 6525
Processing waveform 333 of 420
75.0 6975
Processing waveform 334 of 420
75.0 6750
Processing waveform 335 of 420
75.0 7275
Processing waveform 336 of 420
75.0 2400
Processing waveform 337 of 420
75.0 4125
Processing waveform 338 of 420
75.0 3525
Processing waveform 339 of 420
75.0 5175
Processing waveform 340 of 420
75.0 3750
Processing waveform 341 of 420
75.0 4875
Processing waveform 342 of 420
75.0 2250
Processing waveform 343 of 420
75.0 4125
Processing waveform 344 of 420
75.0 4425
Processing waveform 345 of 420
75.0 5250
Processing waveform 346 of 420
75.0 9075
Processing waveform 347 of 420
75.0 4725
Processing waveform 348 of 420
75.0 2475
Processing waveform 349 of 420
75.0 2925
Processing waveform 350 of 420
75.0 7350
Processing waveform 351 of 420
75.0 2550
Processing waveform 352 of 420
75.0 2850
Processing waveform 353 of 420
75.0 2550
Processing waveform 354 of 420
75.0 4425
Processing waveform 355 of 420
75.0 5775
Processing waveform 356 of 420
75.0 4200
Processing waveform 357 of 420
75.0 2175
Processing waveform 358 of 420
75.0 2925
Processing waveform 359 of 420
75.0 10350
Processing waveform 360 of 420
75.0 15600
Processing waveform 361 of 420
75.0 6450
Processing waveform 362 of 420
75.0 3450
Processing waveform 363 of 420
75.0 11175
Processing waveform 364 of 420
75.0 4725
Processing waveform 365 of 420
75.0 4200
Processing waveform 366 of 420
75.0 3750
Processing waveform 367 of 420
75.0 3975
Processing waveform 368 of 420
75.0 4575
Processing waveform 369 of 420
75.0 3225
Processing waveform 370 of 420
75.0 13500
Processing waveform 371 of 420
75.0 4050
Processing waveform 372 of 420
75.0 5250
Processing waveform 373 of 420
75.0 7050
Processing waveform 374 of 420
75.0 3450
Processing waveform 375 of 420
75.0 6675
Processing waveform 376 of 420
75.0 2850
Processing waveform 377 of 420
75.0 6750
Processing waveform 378 of 420
75.0 2475
Processing waveform 379 of 420
75.0 7650
Processing waveform 380 of 420
75.0 6750
Processing waveform 381 of 420
75.0 4125
Processing waveform 382 of 420
75.0 7425
Processing waveform 383 of 420
75.0 5400
Processing waveform 384 of 420
75.0 4800
Processing waveform 385 of 420
75.0 10200
Processing waveform 386 of 420
75.0 18225
Processing waveform 387 of 420
75.0 5925
Processing waveform 388 of 420
75.0 3525
Processing waveform 389 of 420
75.0 3300
Processing waveform 390 of 420
75.0 3600
Processing waveform 391 of 420
75.0 4650
Processing waveform 392 of 420
75.0 7875
Processing waveform 393 of 420
75.0 6300
Processing waveform 394 of 420
75.0 6750
Processing waveform 395 of 420
75.0 2925
Processing waveform 396 of 420
75.0 3525
Processing waveform 397 of 420
75.0 7875
Processing waveform 398 of 420
75.0 8925
Processing waveform 399 of 420
75.0 2850
Processing waveform 400 of 420
75.0 2475
Processing waveform 401 of 420
75.0 2325
Processing waveform 402 of 420
75.0 2250
Processing waveform 403 of 420
75.0 2700
Processing waveform 404 of 420
75.0 3000
Processing waveform 405 of 420
75.0 3300
Processing waveform 406 of 420
75.0 4425
Processing waveform 407 of 420
75.0 4500
Processing waveform 408 of 420
75.0 7500
Processing waveform 409 of 420
75.0 4200
Processing waveform 410 of 420
75.0 4800
Processing waveform 411 of 420
75.0 3000
Processing waveform 412 of 420
75.0 2475
Processing waveform 413 of 420
75.0 1950
Processing waveform 414 of 420
75.0 2625
Processing waveform 415 of 420
75.0 4650
Processing waveform 416 of 420
75.0 5475
Processing waveform 417 of 420
75.0 4950
Processing waveform 418 of 420
75.0 6375
Processing waveform 419 of 420
75.0 7500

Training data have been read and features have been extracted  (420, 120)
Computation time:  559.5278565883636
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.5785937309265137
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h   106                   
        l         103             
        r               110       
        t                     101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  30.8571428571  +/-  2.8209345894  %
          Predicted class
              h     l     r     t 
        h    19    10    14     9 
        l    13    14    13    12 
        r    17    10    16    11 
        t    11    12    13    15 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'h', 'l', 't', 'r'] 1
class
e     93
h    106
l    103
r    110
t    101
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 513
75.0 5325
Processing waveform 1 of 513
75.0 2700
Processing waveform 2 of 513
75.0 4350
Processing waveform 3 of 513
75.0 2775
Processing waveform 4 of 513
75.0 1950
Processing waveform 5 of 513
75.0 2250
Processing waveform 6 of 513
75.0 2250
Processing waveform 7 of 513
75.0 2250
Processing waveform 8 of 513
75.0 5100
Processing waveform 9 of 513
75.0 2475
Processing waveform 10 of 513
75.0 3300
Processing waveform 11 of 513
75.0 2175
Processing waveform 12 of 513
75.0 6300
Processing waveform 13 of 513
75.0 9075
Processing waveform 14 of 513
75.0 3225
Processing waveform 15 of 513
75.0 4125
Processing waveform 16 of 513
75.0 3300
Processing waveform 17 of 513
75.0 3225
Processing waveform 18 of 513
75.0 2850
Processing waveform 19 of 513
75.0 5400
Processing waveform 20 of 513
75.0 2400
Processing waveform 21 of 513
75.0 7725
Processing waveform 22 of 513
75.0 3900
Processing waveform 23 of 513
75.0 2775
Processing waveform 24 of 513
75.0 2325
Processing waveform 25 of 513
75.0 4425
Processing waveform 26 of 513
75.0 3525
Processing waveform 27 of 513
75.0 6300
Processing waveform 28 of 513
75.0 2625
Processing waveform 29 of 513
75.0 4050
Processing waveform 30 of 513
75.0 2775
Processing waveform 31 of 513
75.0 2400
Processing waveform 32 of 513
75.0 2850
Processing waveform 33 of 513
75.0 2250
Processing waveform 34 of 513
75.0 2325
Processing waveform 35 of 513
75.0 3750
Processing waveform 36 of 513
75.0 3525
Processing waveform 37 of 513
75.0 5700
Processing waveform 38 of 513
75.0 2700
Processing waveform 39 of 513
75.0 3000
Processing waveform 40 of 513
75.0 4350
Processing waveform 41 of 513
75.0 4950
Processing waveform 42 of 513
75.0 2925
Processing waveform 43 of 513
75.0 2775
Processing waveform 44 of 513
75.0 3450
Processing waveform 45 of 513
75.0 4875
Processing waveform 46 of 513
75.0 9075
Processing waveform 47 of 513
75.0 5175
Processing waveform 48 of 513
75.0 3075
Processing waveform 49 of 513
75.0 4125
Processing waveform 50 of 513
75.0 9075
Processing waveform 51 of 513
75.0 3225
Processing waveform 52 of 513
75.0 2775
Processing waveform 53 of 513
75.0 4575
Processing waveform 54 of 513
75.0 2700
Processing waveform 55 of 513
75.0 1950
Processing waveform 56 of 513
75.0 2100
Processing waveform 57 of 513
75.0 4575
Processing waveform 58 of 513
75.0 5850
Processing waveform 59 of 513
75.0 3150
Processing waveform 60 of 513
75.0 4275
Processing waveform 61 of 513
75.0 9075
Processing waveform 62 of 513
75.0 3600
Processing waveform 63 of 513
75.0 2625
Processing waveform 64 of 513
75.0 2700
Processing waveform 65 of 513
75.0 2700
Processing waveform 66 of 513
75.0 3525
Processing waveform 67 of 513
75.0 4200
Processing waveform 68 of 513
75.0 4200
Processing waveform 69 of 513
75.0 1950
Processing waveform 70 of 513
75.0 5775
Processing waveform 71 of 513
75.0 9075
Processing waveform 72 of 513
75.0 4200
Processing waveform 73 of 513
75.0 2625
Processing waveform 74 of 513
75.0 4875
Processing waveform 75 of 513
75.0 5025
Processing waveform 76 of 513
75.0 14625
Processing waveform 77 of 513
75.0 11025
Processing waveform 78 of 513
75.0 1950
Processing waveform 79 of 513
75.0 3975
Processing waveform 80 of 513
75.0 2550
Processing waveform 81 of 513
75.0 7500
Processing waveform 82 of 513
75.0 14925
Processing waveform 83 of 513
75.0 13275
Processing waveform 84 of 513
75.0 1725
Processing waveform 85 of 513
75.0 2400
Processing waveform 86 of 513
75.0 6375
Processing waveform 87 of 513
75.0 6450
Processing waveform 88 of 513
75.0 4125
Processing waveform 89 of 513
75.0 3975
Processing waveform 90 of 513
75.0 1725
Processing waveform 91 of 513
75.0 2700
Processing waveform 92 of 513
75.0 4800
Processing waveform 93 of 513
75.0 7125
Processing waveform 94 of 513
75.0 5325
Processing waveform 95 of 513
75.0 5400
Processing waveform 96 of 513
75.0 6825
Processing waveform 97 of 513
75.0 7050
Processing waveform 98 of 513
75.0 4500
Processing waveform 99 of 513
75.0 5625
Processing waveform 100 of 513
75.0 4275
Processing waveform 101 of 513
75.0 2925
Processing waveform 102 of 513
75.0 3150
Processing waveform 103 of 513
75.0 6000
Processing waveform 104 of 513
75.0 3150
Processing waveform 105 of 513
75.0 6000
Processing waveform 106 of 513
75.0 12600
Processing waveform 107 of 513
75.0 7425
Processing waveform 108 of 513
75.0 16725
Processing waveform 109 of 513
75.0 6075
Processing waveform 110 of 513
75.0 3450
Processing waveform 111 of 513
75.0 2325
Processing waveform 112 of 513
75.0 2925
Processing waveform 113 of 513
75.0 4575
Processing waveform 114 of 513
75.0 7050
Processing waveform 115 of 513
75.0 2025
Processing waveform 116 of 513
75.0 2925
Processing waveform 117 of 513
75.0 2850
Processing waveform 118 of 513
75.0 3600
Processing waveform 119 of 513
75.0 3750
Processing waveform 120 of 513
75.0 2100
Processing waveform 121 of 513
75.0 1650
Processing waveform 122 of 513
75.0 4200
Processing waveform 123 of 513
75.0 9075
Processing waveform 124 of 513
75.0 4125
Processing waveform 125 of 513
75.0 2925
Processing waveform 126 of 513
75.0 2250
Processing waveform 127 of 513
75.0 5475
Processing waveform 128 of 513
75.0 2625
Processing waveform 129 of 513
75.0 2400
Processing waveform 130 of 513
75.0 2700
Processing waveform 131 of 513
75.0 9075
Processing waveform 132 of 513
75.0 3300
Processing waveform 133 of 513
75.0 9075
Processing waveform 134 of 513
75.0 6225
Processing waveform 135 of 513
75.0 7125
Processing waveform 136 of 513
75.0 5850
Processing waveform 137 of 513
75.0 9075
Processing waveform 138 of 513
75.0 3225
Processing waveform 139 of 513
75.0 8100
Processing waveform 140 of 513
75.0 4575
Processing waveform 141 of 513
75.0 2850
Processing waveform 142 of 513
75.0 3375
Processing waveform 143 of 513
75.0 2100
Processing waveform 144 of 513
75.0 9075
Processing waveform 145 of 513
75.0 5850
Processing waveform 146 of 513
75.0 2775
Processing waveform 147 of 513
75.0 2700
Processing waveform 148 of 513
75.0 2100
Processing waveform 149 of 513
75.0 9075
Processing waveform 150 of 513
75.0 3225
Processing waveform 151 of 513
75.0 3300
Processing waveform 152 of 513
75.0 2475
Processing waveform 153 of 513
75.0 3075
Processing waveform 154 of 513
75.0 6450
Processing waveform 155 of 513
75.0 3600
Processing waveform 156 of 513
75.0 4650
Processing waveform 157 of 513
75.0 2475
Processing waveform 158 of 513
75.0 1875
Processing waveform 159 of 513
75.0 3450
Processing waveform 160 of 513
75.0 1500
Processing waveform 161 of 513
75.0 2625
Processing waveform 162 of 513
75.0 9075
Processing waveform 163 of 513
75.0 3525
Processing waveform 164 of 513
75.0 5625
Processing waveform 165 of 513
75.0 3675
Processing waveform 166 of 513
75.0 9075
Processing waveform 167 of 513
75.0 2700
Processing waveform 168 of 513
75.0 9075
Processing waveform 169 of 513
75.0 3000
Processing waveform 170 of 513
75.0 3525
Processing waveform 171 of 513
75.0 7200
Processing waveform 172 of 513
75.0 4875
Processing waveform 173 of 513
75.0 6075

Processing waveform 174 of 513
75.0 2700
Processing waveform 175 of 513
75.0 5250
Processing waveform 176 of 513
75.0 12750
Processing waveform 177 of 513
75.0 5100
Processing waveform 178 of 513
75.0 2475
Processing waveform 179 of 513
75.0 6525
Processing waveform 180 of 513
75.0 7200
Processing waveform 181 of 513
75.0 4500
Processing waveform 182 of 513
75.0 4950
Processing waveform 183 of 513
75.0 8250
Processing waveform 184 of 513
75.0 3150
Processing waveform 185 of 513
75.0 5850
Processing waveform 186 of 513
75.0 2325
Processing waveform 187 of 513
75.0 4200
Processing waveform 188 of 513
75.0 3300
Processing waveform 189 of 513
75.0 2325
Processing waveform 190 of 513
75.0 13500
Processing waveform 191 of 513
75.0 3525
Processing waveform 192 of 513
75.0 3075
Processing waveform 193 of 513
75.0 6600
Processing waveform 194 of 513
75.0 6225
Processing waveform 195 of 513
75.0 9000
Processing waveform 196 of 513
75.0 5100
Processing waveform 197 of 513
75.0 6975
Processing waveform 198 of 513
75.0 4425
Processing waveform 199 of 513
75.0 5400
Processing waveform 200 of 513
75.0 13350
Processing waveform 201 of 513
75.0 9525
Processing waveform 202 of 513
75.0 2325
Processing waveform 203 of 513
75.0 3975
Processing waveform 204 of 513
75.0 5025
Processing waveform 205 of 513
75.0 6825
Processing waveform 206 of 513
75.0 9075
Processing waveform 207 of 513
75.0 4500
Processing waveform 208 of 513
75.0 2175
Processing waveform 209 of 513
75.0 3300
Processing waveform 210 of 513
75.0 4275
Processing waveform 211 of 513
75.0 2850
Processing waveform 212 of 513
75.0 2850
Processing waveform 213 of 513
75.0 2700
Processing waveform 214 of 513
75.0 5250
Processing waveform 215 of 513
75.0 3000
Processing waveform 216 of 513
75.0 2775
Processing waveform 217 of 513
75.0 6450
Processing waveform 218 of 513
75.0 3150
Processing waveform 219 of 513
75.0 1650
Processing waveform 220 of 513
75.0 4500
Processing waveform 221 of 513
75.0 2250
Processing waveform 222 of 513
75.0 2175
Processing waveform 223 of 513
75.0 3225
Processing waveform 224 of 513
75.0 4050
Processing waveform 225 of 513
75.0 3300
Processing waveform 226 of 513
75.0 2850
Processing waveform 227 of 513
75.0 2025
Processing waveform 228 of 513
75.0 4800
Processing waveform 229 of 513
75.0 2325
Processing waveform 230 of 513
75.0 6150
Processing waveform 231 of 513
75.0 2775
Processing waveform 232 of 513
75.0 2850
Processing waveform 233 of 513
75.0 4425
Processing waveform 234 of 513
75.0 2250
Processing waveform 235 of 513
75.0 3075
Processing waveform 236 of 513
75.0 2775
Processing waveform 237 of 513
75.0 1875
Processing waveform 238 of 513
75.0 3300
Processing waveform 239 of 513
75.0 1875
Processing waveform 240 of 513
75.0 5550
Processing waveform 241 of 513
75.0 9075
Processing waveform 242 of 513
75.0 4650
Processing waveform 243 of 513
75.0 6525
Processing waveform 244 of 513
75.0 9075
Processing waveform 245 of 513
75.0 3225
Processing waveform 246 of 513
75.0 3150
Processing waveform 247 of 513
75.0 4275
Processing waveform 248 of 513
75.0 4050
Processing waveform 249 of 513
75.0 5400
Processing waveform 250 of 513
75.0 4725
Processing waveform 251 of 513
75.0 3450
Processing waveform 252 of 513
75.0 6150
Processing waveform 253 of 513
75.0 2250
Processing waveform 254 of 513
75.0 9000
Processing waveform 255 of 513
75.0 3150
Processing waveform 256 of 513
75.0 4950
Processing waveform 257 of 513
75.0 6750
Processing waveform 258 of 513
75.0 3150
Processing waveform 259 of 513
75.0 7050
Processing waveform 260 of 513
75.0 3675
Processing waveform 261 of 513
75.0 3225
Processing waveform 262 of 513
75.0 2325
Processing waveform 263 of 513
75.0 7500
Processing waveform 264 of 513
75.0 3975
Processing waveform 265 of 513
75.0 6150
Processing waveform 266 of 513
75.0 7875
Processing waveform 267 of 513
75.0 7275
Processing waveform 268 of 513
75.0 6075
Processing waveform 269 of 513
75.0 3900
Processing waveform 270 of 513
75.0 3000
Processing waveform 271 of 513
75.0 3900
Processing waveform 272 of 513
75.0 6525
Processing waveform 273 of 513
75.0 2925
Processing waveform 274 of 513
75.0 2700
Processing waveform 275 of 513
75.0 2250
Processing waveform 276 of 513
75.0 4425
Processing waveform 277 of 513
75.0 7500
Processing waveform 278 of 513
75.0 2250
Processing waveform 279 of 513
75.0 9600
Processing waveform 280 of 513
75.0 4650
Processing waveform 281 of 513
75.0 6750
Processing waveform 282 of 513
75.0 8925
Processing waveform 283 of 513
75.0 7125
Processing waveform 284 of 513
75.0 3000
Processing waveform 285 of 513
75.0 10200
Processing waveform 286 of 513
75.0 1800
Processing waveform 287 of 513
75.0 4425
Processing waveform 288 of 513
75.0 2250
Processing waveform 289 of 513
75.0 6150
Processing waveform 290 of 513
75.0 7125
Processing waveform 291 of 513
75.0 3600
Processing waveform 292 of 513
75.0 6000
Processing waveform 293 of 513
75.0 4800
Processing waveform 294 of 513
75.0 7200
Processing waveform 295 of 513
75.0 3000
Processing waveform 296 of 513
75.0 4500
Processing waveform 297 of 513
75.0 3600
Processing waveform 298 of 513
75.0 4875
Processing waveform 299 of 513
75.0 5625
Processing waveform 300 of 513
75.0 3150
Processing waveform 301 of 513
75.0 3975
Processing waveform 302 of 513
75.0 5325
Processing waveform 303 of 513
75.0 2850
Processing waveform 304 of 513
75.0 9450
Processing waveform 305 of 513
75.0 6525
Processing waveform 306 of 513
75.0 3225
Processing waveform 307 of 513
75.0 2100
Processing waveform 308 of 513
75.0 2700
Processing waveform 309 of 513
75.0 2325
Processing waveform 310 of 513
75.0 2775
Processing waveform 311 of 513
75.0 2625
Processing waveform 312 of 513
75.0 5850
Processing waveform 313 of 513
75.0 3825
Processing waveform 314 of 513
75.0 6000
Processing waveform 315 of 513
75.0 2025
Processing waveform 316 of 513
75.0 1725
Processing waveform 317 of 513
75.0 2175
Processing waveform 318 of 513
75.0 5400
Processing waveform 319 of 513
75.0 5325
Processing waveform 320 of 513
75.0 3750
Processing waveform 321 of 513
75.0 3075
Processing waveform 322 of 513
75.0 5550
Processing waveform 323 of 513
75.0 7350
Processing waveform 324 of 513
75.0 4650
Processing waveform 325 of 513
75.0 2775
Processing waveform 326 of 513
75.0 2325
Processing waveform 327 of 513
75.0 2925
Processing waveform 328 of 513
75.0 3075
Processing waveform 329 of 513
75.0 6375
Processing waveform 330 of 513
75.0 2475
Processing waveform 331 of 513
75.0 3975
Processing waveform 332 of 513
75.0 3150
Processing waveform 333 of 513
75.0 5925
Processing waveform 334 of 513
75.0 5400
Processing waveform 335 of 513
75.0 1875
Processing waveform 336 of 513
75.0 5850
Processing waveform 337 of 513
75.0 1725
Processing waveform 338 of 513
75.0 7650
Processing waveform 339 of 513
75.0 2700
Processing waveform 340 of 513
75.0 9075
Processing waveform 341 of 513
75.0 6975
Processing waveform 342 of 513
75.0 4050
Processing waveform 343 of 513
75.0 3450
Processing waveform 344 of 513
75.0 5925
Processing waveform 345 of 513
75.0 2250
Processing waveform 346 of 513
75.0 3675
Processing waveform 347 of 513
75.0 2625
Processing waveform 348 of 513
75.0 4575
Processing waveform 349 of 513
75.0 2025
Processing waveform 350 of 513
75.0 5175
Processing waveform 351 of 513
75.0 3225
Processing waveform 352 of 513
75.0 4950
Processing waveform 353 of 513
75.0 2925
Processing waveform 354 of 513
75.0 1800
Processing waveform 355 of 513
75.0 8925
Processing waveform 356 of 513
75.0 7350
Processing waveform 357 of 513
75.0 4125
Processing waveform 358 of 513
75.0 3225
Processing waveform 359 of 513
75.0 9075
Processing waveform 360 of 513
75.0 5025
Processing waveform 361 of 513
75.0 8925
Processing waveform 362 of 513
75.0 4800
Processing waveform 363 of 513
75.0 7800
Processing waveform 364 of 513
75.0 2775
Processing waveform 365 of 513
75.0 6600
Processing waveform 366 of 513
75.0 1800
Processing waveform 367 of 513
75.0 2625
Processing waveform 368 of 513
75.0 9075
Processing waveform 369 of 513
75.0 7200
Processing waveform 370 of 513
75.0 5550
Processing waveform 371 of 513
75.0 2775
Processing waveform 372 of 513
75.0 5250
Processing waveform 373 of 513
75.0 1875

Processing waveform 374 of 513
75.0 1875
Processing waveform 375 of 513
75.0 5025
Processing waveform 376 of 513
75.0 3600
Processing waveform 377 of 513
75.0 3375
Processing waveform 378 of 513
75.0 4050
Processing waveform 379 of 513
75.0 3375
Processing waveform 380 of 513
75.0 2925
Processing waveform 381 of 513
75.0 6675
Processing waveform 382 of 513
75.0 3750
Processing waveform 383 of 513
75.0 4575
Processing waveform 384 of 513
75.0 9075
Processing waveform 385 of 513
75.0 3900
Processing waveform 386 of 513
75.0 9075
Processing waveform 387 of 513
75.0 4875
Processing waveform 388 of 513
75.0 3075
Processing waveform 389 of 513
75.0 3150
Processing waveform 390 of 513
75.0 2550
Processing waveform 391 of 513
75.0 9075
Processing waveform 392 of 513
75.0 7425
Processing waveform 393 of 513
75.0 5025
Processing waveform 394 of 513
75.0 2850
Processing waveform 395 of 513
75.0 1875
Processing waveform 396 of 513
75.0 7125
Processing waveform 397 of 513
75.0 4500
Processing waveform 398 of 513
75.0 6075
Processing waveform 399 of 513
75.0 2475
Processing waveform 400 of 513
75.0 4125
Processing waveform 401 of 513
75.0 4500
Processing waveform 402 of 513
75.0 2100
Processing waveform 403 of 513
75.0 3975
Processing waveform 404 of 513
75.0 3225
Processing waveform 405 of 513
75.0 2550
Processing waveform 406 of 513
75.0 6975
Processing waveform 407 of 513
75.0 3075
Processing waveform 408 of 513
75.0 3675
Processing waveform 409 of 513
75.0 4800
Processing waveform 410 of 513
75.0 5850
Processing waveform 411 of 513
75.0 3675
Processing waveform 412 of 513
75.0 2250
Processing waveform 413 of 513
75.0 3600
Processing waveform 414 of 513
75.0 5775
Processing waveform 415 of 513
75.0 2325
Processing waveform 416 of 513
75.0 2325
Processing waveform 417 of 513
75.0 3525
Processing waveform 418 of 513
75.0 4500
Processing waveform 419 of 513
75.0 4275
Processing waveform 420 of 513
75.0 3450
Processing waveform 421 of 513
75.0 2550
Processing waveform 422 of 513
75.0 3300
Processing waveform 423 of 513
75.0 9075
Processing waveform 424 of 513
75.0 9075
Processing waveform 425 of 513
75.0 6525
Processing waveform 426 of 513
75.0 6975
Processing waveform 427 of 513
75.0 6750
Processing waveform 428 of 513
75.0 7275
Processing waveform 429 of 513
75.0 2400
Processing waveform 430 of 513
75.0 4125
Processing waveform 431 of 513
75.0 3525
Processing waveform 432 of 513
75.0 5175
Processing waveform 433 of 513
75.0 3750
Processing waveform 434 of 513
75.0 4875
Processing waveform 435 of 513
75.0 2250
Processing waveform 436 of 513
75.0 4125
Processing waveform 437 of 513
75.0 4425
Processing waveform 438 of 513
75.0 5250
Processing waveform 439 of 513
75.0 9075
Processing waveform 440 of 513
75.0 4725
Processing waveform 441 of 513
75.0 2475
Processing waveform 442 of 513
75.0 2925
Processing waveform 443 of 513
75.0 7350
Processing waveform 444 of 513
75.0 2550
Processing waveform 445 of 513
75.0 2850
Processing waveform 446 of 513
75.0 2550
Processing waveform 447 of 513
75.0 4425
Processing waveform 448 of 513
75.0 5775
Processing waveform 449 of 513
75.0 4200
Processing waveform 450 of 513
75.0 2175
Processing waveform 451 of 513
75.0 2925
Processing waveform 452 of 513
75.0 10350
Processing waveform 453 of 513
75.0 15600
Processing waveform 454 of 513
75.0 6450
Processing waveform 455 of 513
75.0 3450
Processing waveform 456 of 513
75.0 11175
Processing waveform 457 of 513
75.0 4725
Processing waveform 458 of 513
75.0 4200
Processing waveform 459 of 513
75.0 3750
Processing waveform 460 of 513
75.0 3975
Processing waveform 461 of 513
75.0 4575
Processing waveform 462 of 513
75.0 3225
Processing waveform 463 of 513
75.0 13500
Processing waveform 464 of 513
75.0 4050
Processing waveform 465 of 513
75.0 5250
Processing waveform 466 of 513
75.0 7050
Processing waveform 467 of 513
75.0 3450
Processing waveform 468 of 513
75.0 6675
Processing waveform 469 of 513
75.0 2850
Processing waveform 470 of 513
75.0 6750
Processing waveform 471 of 513
75.0 2475
Processing waveform 472 of 513
75.0 7650
Processing waveform 473 of 513
75.0 6750
Processing waveform 474 of 513
75.0 4125
Processing waveform 475 of 513
75.0 7425
Processing waveform 476 of 513
75.0 5400
Processing waveform 477 of 513
75.0 4800
Processing waveform 478 of 513
75.0 10200
Processing waveform 479 of 513
75.0 18225
Processing waveform 480 of 513
75.0 5925
Processing waveform 481 of 513
75.0 3525
Processing waveform 482 of 513
75.0 3300
Processing waveform 483 of 513
75.0 3600
Processing waveform 484 of 513
75.0 4650
Processing waveform 485 of 513
75.0 7875
Processing waveform 486 of 513
75.0 6300
Processing waveform 487 of 513
75.0 6750
Processing waveform 488 of 513
75.0 2925
Processing waveform 489 of 513
75.0 3525
Processing waveform 490 of 513
75.0 7875
Processing waveform 491 of 513
75.0 8925
Processing waveform 492 of 513
75.0 2850
Processing waveform 493 of 513
75.0 2475
Processing waveform 494 of 513
75.0 2325
Processing waveform 495 of 513
75.0 2250
Processing waveform 496 of 513
75.0 2700
Processing waveform 497 of 513
75.0 3000
Processing waveform 498 of 513
75.0 3300
Processing waveform 499 of 513
75.0 4425
Processing waveform 500 of 513
75.0 4500
Processing waveform 501 of 513
75.0 7500
Processing waveform 502 of 513
75.0 4200
Processing waveform 503 of 513
75.0 4800
Processing waveform 504 of 513
75.0 3000
Processing waveform 505 of 513
75.0 2475
Processing waveform 506 of 513
75.0 1950
Processing waveform 507 of 513
75.0 2625
Processing waveform 508 of 513
75.0 4650
Processing waveform 509 of 513
75.0 5475
Processing waveform 510 of 513
75.0 4950
Processing waveform 511 of 513
75.0 6375
Processing waveform 512 of 513
75.0 7500
Training data have been read and features have been extracted  (513, 120)
Computation time:  693.6631901264191
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.7518486976623535
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    93                         
        h         106                   
        l               103             
        r                     110       
        t                           101 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  23.9169909209  +/-  2.11870790975  %
          Predicted class
              e     h     l     r     t 
        e     7    11     9    11     9 
        h     7    16     9    15     7 
        l     8    11    12    12    10 
        r     7    14    10    15     9 
        t     8     9    10    12    12 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['l', 't'] 2
class
l     97
t    100
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 197
75.0 3975
Processing waveform 1 of 197
75.0 9075
Processing waveform 2 of 197
75.0 2175
Processing waveform 3 of 197
75.0 3300
Processing waveform 4 of 197
75.0 4275
Processing waveform 5 of 197
75.0 2850
Processing waveform 6 of 197
75.0 2850
Processing waveform 7 of 197
75.0 2700
Processing waveform 8 of 197
75.0 5250
Processing waveform 9 of 197
75.0 3000
Processing waveform 10 of 197
75.0 2775
Processing waveform 11 of 197
75.0 6450
Processing waveform 12 of 197
75.0 3150
Processing waveform 13 of 197
75.0 1650
Processing waveform 14 of 197
75.0 4500
Processing waveform 15 of 197
75.0 2250
Processing waveform 16 of 197
75.0 2175
Processing waveform 17 of 197
75.0 3225
Processing waveform 18 of 197
75.0 4050
Processing waveform 19 of 197
75.0 3300
Processing waveform 20 of 197
75.0 2850
Processing waveform 21 of 197
75.0 2025
Processing waveform 22 of 197
75.0 4800
Processing waveform 23 of 197
75.0 2325
Processing waveform 24 of 197
75.0 6150
Processing waveform 25 of 197
75.0 2775
Processing waveform 26 of 197
75.0 2850
Processing waveform 27 of 197
75.0 4425
Processing waveform 28 of 197
75.0 2250
Processing waveform 29 of 197
75.0 3075
Processing waveform 30 of 197
75.0 2775

Processing waveform 31 of 197
75.0 1875
Processing waveform 32 of 197
75.0 3300
Processing waveform 33 of 197
75.0 1875
Processing waveform 34 of 197
75.0 5550
Processing waveform 35 of 197
75.0 9075
Processing waveform 36 of 197
75.0 4650
Processing waveform 37 of 197
75.0 6525
Processing waveform 38 of 197
75.0 9075
Processing waveform 39 of 197
75.0 3225
Processing waveform 40 of 197
75.0 3150
Processing waveform 41 of 197
75.0 4275
Processing waveform 42 of 197
75.0 4050
Processing waveform 43 of 197
75.0 5400
Processing waveform 44 of 197
75.0 4725
Processing waveform 45 of 197
75.0 3450
Processing waveform 46 of 197
75.0 6150
Processing waveform 47 of 197
75.0 2250
Processing waveform 48 of 197
75.0 9000
Processing waveform 49 of 197
75.0 3150
Processing waveform 50 of 197
75.0 4950
Processing waveform 51 of 197
75.0 6750
Processing waveform 52 of 197
75.0 3150
Processing waveform 53 of 197
75.0 7050
Processing waveform 54 of 197
75.0 3675
Processing waveform 55 of 197
75.0 3225
Processing waveform 56 of 197
75.0 2325
Processing waveform 57 of 197
75.0 7500
Processing waveform 58 of 197
75.0 3975
Processing waveform 59 of 197
75.0 6150
Processing waveform 60 of 197
75.0 7875
Processing waveform 61 of 197
75.0 7275
Processing waveform 62 of 197
75.0 6075
Processing waveform 63 of 197
75.0 3900
Processing waveform 64 of 197
75.0 3000
Processing waveform 65 of 197
75.0 3900
Processing waveform 66 of 197
75.0 6525
Processing waveform 67 of 197
75.0 2925
Processing waveform 68 of 197
75.0 2700
Processing waveform 69 of 197
75.0 2250
Processing waveform 70 of 197
75.0 4425
Processing waveform 71 of 197
75.0 2250
Processing waveform 72 of 197
75.0 9600
Processing waveform 73 of 197
75.0 4650
Processing waveform 74 of 197
75.0 6750
Processing waveform 75 of 197
75.0 7125
Processing waveform 76 of 197
75.0 3000
Processing waveform 77 of 197
75.0 10200
Processing waveform 78 of 197
75.0 1800
Processing waveform 79 of 197
75.0 4425
Processing waveform 80 of 197
75.0 2250
Processing waveform 81 of 197
75.0 6150
Processing waveform 82 of 197
75.0 7125
Processing waveform 83 of 197
75.0 3600
Processing waveform 84 of 197
75.0 6000
Processing waveform 85 of 197
75.0 4800
Processing waveform 86 of 197
75.0 7200
Processing waveform 87 of 197
75.0 3000
Processing waveform 88 of 197
75.0 4500
Processing waveform 89 of 197
75.0 3600
Processing waveform 90 of 197
75.0 4875
Processing waveform 91 of 197
75.0 5625
Processing waveform 92 of 197
75.0 3150
Processing waveform 93 of 197
75.0 3975
Processing waveform 94 of 197
75.0 5325
Processing waveform 95 of 197
75.0 2850
Processing waveform 96 of 197
75.0 9450
Processing waveform 97 of 197
75.0 2250
Processing waveform 98 of 197
75.0 3600
Processing waveform 99 of 197
75.0 5775
Processing waveform 100 of 197
75.0 2325
Processing waveform 101 of 197
75.0 2325
Processing waveform 102 of 197
75.0 3525
Processing waveform 103 of 197
75.0 4500
Processing waveform 104 of 197
75.0 4275
Processing waveform 105 of 197
75.0 3450
Processing waveform 106 of 197
75.0 2550
Processing waveform 107 of 197
75.0 3300
Processing waveform 108 of 197
75.0 9075
Processing waveform 109 of 197
75.0 9075
Processing waveform 110 of 197
75.0 6525
Processing waveform 111 of 197
75.0 6975
Processing waveform 112 of 197
75.0 6750
Processing waveform 113 of 197
75.0 7275
Processing waveform 114 of 197
75.0 2400
Processing waveform 115 of 197
75.0 4125
Processing waveform 116 of 197
75.0 3525
Processing waveform 117 of 197
75.0 5175
Processing waveform 118 of 197
75.0 3750
Processing waveform 119 of 197
75.0 4875
Processing waveform 120 of 197
75.0 2250
Processing waveform 121 of 197
75.0 4125
Processing waveform 122 of 197
75.0 4425
Processing waveform 123 of 197
75.0 5250
Processing waveform 124 of 197
75.0 9075
Processing waveform 125 of 197
75.0 4725
Processing waveform 126 of 197
75.0 2475
Processing waveform 127 of 197
75.0 2925
Processing waveform 128 of 197
75.0 7350
Processing waveform 129 of 197
75.0 2550
Processing waveform 130 of 197
75.0 2850
Processing waveform 131 of 197
75.0 2550
Processing waveform 132 of 197
75.0 4425
Processing waveform 133 of 197
75.0 5775
Processing waveform 134 of 197
75.0 4200
Processing waveform 135 of 197
75.0 2175
Processing waveform 136 of 197
75.0 2925
Processing waveform 137 of 197
75.0 10350
Processing waveform 138 of 197
75.0 15600
Processing waveform 139 of 197
75.0 6450
Processing waveform 140 of 197
75.0 3450
Processing waveform 141 of 197
75.0 11175
Processing waveform 142 of 197
75.0 4725
Processing waveform 143 of 197
75.0 4200
Processing waveform 144 of 197
75.0 3750
Processing waveform 145 of 197
75.0 3975
Processing waveform 146 of 197
75.0 4575
Processing waveform 147 of 197
75.0 3225
Processing waveform 148 of 197
75.0 13500
Processing waveform 149 of 197
75.0 4050
Processing waveform 150 of 197
75.0 5250
Processing waveform 151 of 197
75.0 7050
Processing waveform 152 of 197
75.0 3450
Processing waveform 153 of 197
75.0 6675
Processing waveform 154 of 197
75.0 2850
Processing waveform 155 of 197
75.0 6750
Processing waveform 156 of 197
75.0 2475
Processing waveform 157 of 197
75.0 7650
Processing waveform 158 of 197
75.0 6750
Processing waveform 159 of 197
75.0 4125
Processing waveform 160 of 197
75.0 7425
Processing waveform 161 of 197
75.0 5400
Processing waveform 162 of 197
75.0 4800
Processing waveform 163 of 197
75.0 10200
Processing waveform 164 of 197
75.0 18225
Processing waveform 165 of 197
75.0 5925
Processing waveform 166 of 197
75.0 3525
Processing waveform 167 of 197
75.0 3300
Processing waveform 168 of 197
75.0 3600
Processing waveform 169 of 197
75.0 4650
Processing waveform 170 of 197
75.0 7875
Processing waveform 171 of 197
75.0 6300
Processing waveform 172 of 197
75.0 6750
Processing waveform 173 of 197
75.0 2925
Processing waveform 174 of 197
75.0 3525
Processing waveform 175 of 197
75.0 7875
Processing waveform 176 of 197
75.0 8925
Processing waveform 177 of 197
75.0 2850
Processing waveform 178 of 197
75.0 2475
Processing waveform 179 of 197
75.0 2325
Processing waveform 180 of 197
75.0 2250
Processing waveform 181 of 197
75.0 2700
Processing waveform 182 of 197
75.0 3000
Processing waveform 183 of 197
75.0 3300
Processing waveform 184 of 197
75.0 4425
Processing waveform 185 of 197
75.0 4500
Processing waveform 186 of 197
75.0 7500
Processing waveform 187 of 197
75.0 4200
Processing waveform 188 of 197
75.0 4800
Processing waveform 189 of 197
75.0 3000
Processing waveform 190 of 197
75.0 1950
Processing waveform 191 of 197
75.0 2625
Processing waveform 192 of 197
75.0 4650
Processing waveform 193 of 197
75.0 5475
Processing waveform 194 of 197
75.0 4950
Processing waveform 195 of 197
75.0 6375
Processing waveform 196 of 197
75.0 7500
Training data have been read and features have been extracted  (197, 120)
Computation time:  270.3844425678253
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.15511822700500488
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l    97       
        t         100 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  52.1548821549  +/-  4.7268938043  %
          Predicted class
              l     t 
        l    25    24 
        t    23    27 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'r'] 2
class
e    87
r    93
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 180
75.0 5325
Processing waveform 1 of 180
75.0 2700
Processing waveform 2 of 180
75.0 4350
Processing waveform 3 of 180
75.0 2775
Processing waveform 4 of 180
75.0 2250
Processing waveform 5 of 180
75.0 2250
Processing waveform 6 of 180
75.0 2475
Processing waveform 7 of 180
75.0 3300
Processing waveform 8 of 180
75.0 9075
Processing waveform 9 of 180
75.0 3225
Processing waveform 10 of 180
75.0 4125
Processing waveform 11 of 180
75.0 3300
Processing waveform 12 of 180
75.0 3225
Processing waveform 13 of 180
75.0 2850

Processing waveform 14 of 180
75.0 2400
Processing waveform 15 of 180
75.0 3900
Processing waveform 16 of 180
75.0 2775
Processing waveform 17 of 180
75.0 2325
Processing waveform 18 of 180
75.0 4425
Processing waveform 19 of 180
75.0 3525
Processing waveform 20 of 180
75.0 6300
Processing waveform 21 of 180
75.0 2625
Processing waveform 22 of 180
75.0 4050
Processing waveform 23 of 180
75.0 2775
Processing waveform 24 of 180
75.0 2400
Processing waveform 25 of 180
75.0 2850
Processing waveform 26 of 180
75.0 2250
Processing waveform 27 of 180
75.0 2325
Processing waveform 28 of 180
75.0 3750
Processing waveform 29 of 180
75.0 3525
Processing waveform 30 of 180
75.0 5700
Processing waveform 31 of 180
75.0 2700
Processing waveform 32 of 180
75.0 3000
Processing waveform 33 of 180
75.0 4350
Processing waveform 34 of 180
75.0 4950
Processing waveform 35 of 180
75.0 2925
Processing waveform 36 of 180
75.0 2775
Processing waveform 37 of 180
75.0 3450
Processing waveform 38 of 180
75.0 4875
Processing waveform 39 of 180
75.0 9075
Processing waveform 40 of 180
75.0 5175
Processing waveform 41 of 180
75.0 3075
Processing waveform 42 of 180
75.0 4125
Processing waveform 43 of 180
75.0 9075
Processing waveform 44 of 180
75.0 2775
Processing waveform 45 of 180
75.0 4575
Processing waveform 46 of 180
75.0 2700
Processing waveform 47 of 180
75.0 1950
Processing waveform 48 of 180
75.0 2100
Processing waveform 49 of 180
75.0 4575
Processing waveform 50 of 180
75.0 5850
Processing waveform 51 of 180
75.0 3150
Processing waveform 52 of 180
75.0 4275
Processing waveform 53 of 180
75.0 9075
Processing waveform 54 of 180
75.0 3600
Processing waveform 55 of 180
75.0 2625
Processing waveform 56 of 180
75.0 2700
Processing waveform 57 of 180
75.0 2700
Processing waveform 58 of 180
75.0 3525
Processing waveform 59 of 180
75.0 4200
Processing waveform 60 of 180
75.0 4200
Processing waveform 61 of 180
75.0 1950
Processing waveform 62 of 180
75.0 5775
Processing waveform 63 of 180
75.0 9075
Processing waveform 64 of 180
75.0 4875
Processing waveform 65 of 180
75.0 14625
Processing waveform 66 of 180
75.0 11025
Processing waveform 67 of 180
75.0 1950
Processing waveform 68 of 180
75.0 3975
Processing waveform 69 of 180
75.0 2550
Processing waveform 70 of 180
75.0 7500
Processing waveform 71 of 180
75.0 14925
Processing waveform 72 of 180
75.0 13275
Processing waveform 73 of 180
75.0 1725
Processing waveform 74 of 180
75.0 2400
Processing waveform 75 of 180
75.0 6375
Processing waveform 76 of 180
75.0 6450
Processing waveform 77 of 180
75.0 4125
Processing waveform 78 of 180
75.0 4800
Processing waveform 79 of 180
75.0 7125
Processing waveform 80 of 180
75.0 5325
Processing waveform 81 of 180
75.0 5400
Processing waveform 82 of 180
75.0 6825
Processing waveform 83 of 180
75.0 7050
Processing waveform 84 of 180
75.0 4500
Processing waveform 85 of 180
75.0 5625
Processing waveform 86 of 180
75.0 4275
Processing waveform 87 of 180
75.0 3150
Processing waveform 88 of 180
75.0 3150
Processing waveform 89 of 180
75.0 12600
Processing waveform 90 of 180
75.0 7425
Processing waveform 91 of 180
75.0 16725
Processing waveform 92 of 180
75.0 6075
Processing waveform 93 of 180
75.0 3450
Processing waveform 94 of 180
75.0 2325
Processing waveform 95 of 180
75.0 2925
Processing waveform 96 of 180
75.0 4575
Processing waveform 97 of 180
75.0 7050
Processing waveform 98 of 180
75.0 2025
Processing waveform 99 of 180
75.0 2925
Processing waveform 100 of 180
75.0 2850
Processing waveform 101 of 180
75.0 3600
Processing waveform 102 of 180
75.0 3750
Processing waveform 103 of 180
75.0 2100
Processing waveform 104 of 180
75.0 1650
Processing waveform 105 of 180
75.0 4200
Processing waveform 106 of 180
75.0 9075
Processing waveform 107 of 180
75.0 4125
Processing waveform 108 of 180
75.0 2925
Processing waveform 109 of 180
75.0 2250
Processing waveform 110 of 180
75.0 5475
Processing waveform 111 of 180
75.0 2625
Processing waveform 112 of 180
75.0 2400
Processing waveform 113 of 180
75.0 2700
Processing waveform 114 of 180
75.0 9075
Processing waveform 115 of 180
75.0 3300
Processing waveform 116 of 180
75.0 9075
Processing waveform 117 of 180
75.0 6225
Processing waveform 118 of 180
75.0 7125
Processing waveform 119 of 180
75.0 5850
Processing waveform 120 of 180
75.0 9075
Processing waveform 121 of 180
75.0 3225
Processing waveform 122 of 180
75.0 8100
Processing waveform 123 of 180
75.0 4575
Processing waveform 124 of 180
75.0 2850
Processing waveform 125 of 180
75.0 3375
Processing waveform 126 of 180
75.0 2100
Processing waveform 127 of 180
75.0 9075
Processing waveform 128 of 180
75.0 5850
Processing waveform 129 of 180
75.0 2775
Processing waveform 130 of 180
75.0 2700
Processing waveform 131 of 180
75.0 2100
Processing waveform 132 of 180
75.0 9075
Processing waveform 133 of 180
75.0 3225
Processing waveform 134 of 180
75.0 3300
Processing waveform 135 of 180
75.0 2475
Processing waveform 136 of 180
75.0 3075
Processing waveform 137 of 180
75.0 6450
Processing waveform 138 of 180
75.0 3600
Processing waveform 139 of 180
75.0 4650
Processing waveform 140 of 180
75.0 2475
Processing waveform 141 of 180
75.0 1875
Processing waveform 142 of 180
75.0 3450
Processing waveform 143 of 180
75.0 1500
Processing waveform 144 of 180
75.0 2625
Processing waveform 145 of 180
75.0 9075
Processing waveform 146 of 180
75.0 3525
Processing waveform 147 of 180
75.0 5625
Processing waveform 148 of 180
75.0 3675
Processing waveform 149 of 180
75.0 9075
Processing waveform 150 of 180
75.0 2700
Processing waveform 151 of 180
75.0 3000
Processing waveform 152 of 180
75.0 3525
Processing waveform 153 of 180
75.0 7200
Processing waveform 154 of 180
75.0 4875
Processing waveform 155 of 180
75.0 6075
Processing waveform 156 of 180
75.0 2700
Processing waveform 157 of 180
75.0 5250
Processing waveform 158 of 180
75.0 12750
Processing waveform 159 of 180
75.0 5100
Processing waveform 160 of 180
75.0 2475
Processing waveform 161 of 180
75.0 6525
Processing waveform 162 of 180
75.0 7200
Processing waveform 163 of 180
75.0 4950
Processing waveform 164 of 180
75.0 8250
Processing waveform 165 of 180
75.0 3150
Processing waveform 166 of 180
75.0 2325
Processing waveform 167 of 180
75.0 4200
Processing waveform 168 of 180
75.0 3300
Processing waveform 169 of 180
75.0 2325
Processing waveform 170 of 180
75.0 13500
Processing waveform 171 of 180
75.0 3525
Processing waveform 172 of 180
75.0 3075
Processing waveform 173 of 180
75.0 6600
Processing waveform 174 of 180
75.0 6225
Processing waveform 175 of 180
75.0 6975
Processing waveform 176 of 180
75.0 4425
Processing waveform 177 of 180
75.0 13350
Processing waveform 178 of 180
75.0 9525
Processing waveform 179 of 180
75.0 2325
Training data have been read and features have been extracted  (180, 120)
Computation time:  264.5363698005676
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.15197348594665527
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    87       
        r          93 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  52.2962962963  +/-  5.48048038906  %
          Predicted class
              e     r 
        e    21    23 
        r    20    26 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't'] 2
class
h    104
l     97
t    100
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 301
75.0 3975
Processing waveform 1 of 301
75.0 9075
Processing waveform 2 of 301
75.0 2175
Processing waveform 3 of 301
75.0 3300
Processing waveform 4 of 301
75.0 4275
Processing waveform 5 of 301
75.0 2850
Processing waveform 6 of 301
75.0 2850
Processing waveform 7 of 301
75.0 2700
Processing waveform 8 of 301
75.0 5250
Processing waveform 9 of 301
75.0 3000
Processing waveform 10 of 301
75.0 2775
Processing waveform 11 of 301
75.0 6450
Processing waveform 12 of 301
75.0 3150
Processing waveform 13 of 301
75.0 1650
Processing waveform 14 of 301

75.0 4500
Processing waveform 15 of 301
75.0 2250
Processing waveform 16 of 301
75.0 2175
Processing waveform 17 of 301
75.0 3225
Processing waveform 18 of 301
75.0 4050
Processing waveform 19 of 301
75.0 3300
Processing waveform 20 of 301
75.0 2850
Processing waveform 21 of 301
75.0 2025
Processing waveform 22 of 301
75.0 4800
Processing waveform 23 of 301
75.0 2325
Processing waveform 24 of 301
75.0 6150
Processing waveform 25 of 301
75.0 2775
Processing waveform 26 of 301
75.0 2850
Processing waveform 27 of 301
75.0 4425
Processing waveform 28 of 301
75.0 2250
Processing waveform 29 of 301
75.0 3075
Processing waveform 30 of 301
75.0 2775
Processing waveform 31 of 301
75.0 1875
Processing waveform 32 of 301
75.0 3300
Processing waveform 33 of 301
75.0 1875
Processing waveform 34 of 301
75.0 5550
Processing waveform 35 of 301
75.0 9075
Processing waveform 36 of 301
75.0 4650
Processing waveform 37 of 301
75.0 6525
Processing waveform 38 of 301
75.0 9075
Processing waveform 39 of 301
75.0 3225
Processing waveform 40 of 301
75.0 3150
Processing waveform 41 of 301
75.0 4275
Processing waveform 42 of 301
75.0 4050
Processing waveform 43 of 301
75.0 5400
Processing waveform 44 of 301
75.0 4725
Processing waveform 45 of 301
75.0 3450
Processing waveform 46 of 301
75.0 6150
Processing waveform 47 of 301
75.0 2250
Processing waveform 48 of 301
75.0 9000
Processing waveform 49 of 301
75.0 3150
Processing waveform 50 of 301
75.0 4950
Processing waveform 51 of 301
75.0 6750
Processing waveform 52 of 301
75.0 3150
Processing waveform 53 of 301
75.0 7050
Processing waveform 54 of 301
75.0 3675
Processing waveform 55 of 301
75.0 3225
Processing waveform 56 of 301
75.0 2325
Processing waveform 57 of 301
75.0 7500
Processing waveform 58 of 301
75.0 3975
Processing waveform 59 of 301
75.0 6150
Processing waveform 60 of 301
75.0 7875
Processing waveform 61 of 301
75.0 7275
Processing waveform 62 of 301
75.0 6075
Processing waveform 63 of 301
75.0 3900
Processing waveform 64 of 301
75.0 3000
Processing waveform 65 of 301
75.0 3900
Processing waveform 66 of 301
75.0 6525
Processing waveform 67 of 301
75.0 2925
Processing waveform 68 of 301
75.0 2700
Processing waveform 69 of 301
75.0 2250
Processing waveform 70 of 301
75.0 4425
Processing waveform 71 of 301
75.0 2250
Processing waveform 72 of 301
75.0 9600
Processing waveform 73 of 301
75.0 4650
Processing waveform 74 of 301
75.0 6750
Processing waveform 75 of 301
75.0 7125
Processing waveform 76 of 301
75.0 3000
Processing waveform 77 of 301
75.0 10200
Processing waveform 78 of 301
75.0 1800
Processing waveform 79 of 301
75.0 4425
Processing waveform 80 of 301
75.0 2250
Processing waveform 81 of 301
75.0 6150
Processing waveform 82 of 301
75.0 7125
Processing waveform 83 of 301
75.0 3600
Processing waveform 84 of 301
75.0 6000
Processing waveform 85 of 301
75.0 4800
Processing waveform 86 of 301
75.0 7200
Processing waveform 87 of 301
75.0 3000
Processing waveform 88 of 301
75.0 4500
Processing waveform 89 of 301
75.0 3600
Processing waveform 90 of 301
75.0 4875
Processing waveform 91 of 301
75.0 5625
Processing waveform 92 of 301
75.0 3150
Processing waveform 93 of 301
75.0 3975
Processing waveform 94 of 301
75.0 5325
Processing waveform 95 of 301
75.0 2850
Processing waveform 96 of 301
75.0 9450
Processing waveform 97 of 301
75.0 3225
Processing waveform 98 of 301
75.0 2100
Processing waveform 99 of 301
75.0 2700
Processing waveform 100 of 301
75.0 2325
Processing waveform 101 of 301
75.0 2775
Processing waveform 102 of 301
75.0 2625
Processing waveform 103 of 301
75.0 5850
Processing waveform 104 of 301
75.0 3825
Processing waveform 105 of 301
75.0 6000
Processing waveform 106 of 301
75.0 2025
Processing waveform 107 of 301
75.0 1725
Processing waveform 108 of 301
75.0 2175
Processing waveform 109 of 301
75.0 5400
Processing waveform 110 of 301
75.0 5325
Processing waveform 111 of 301
75.0 3750
Processing waveform 112 of 301
75.0 3075
Processing waveform 113 of 301
75.0 5550
Processing waveform 114 of 301
75.0 7350
Processing waveform 115 of 301
75.0 4650
Processing waveform 116 of 301
75.0 2775
Processing waveform 117 of 301
75.0 2325
Processing waveform 118 of 301
75.0 2925
Processing waveform 119 of 301
75.0 3075
Processing waveform 120 of 301
75.0 6375
Processing waveform 121 of 301
75.0 2475
Processing waveform 122 of 301
75.0 3975
Processing waveform 123 of 301
75.0 3150
Processing waveform 124 of 301
75.0 5925
Processing waveform 125 of 301
75.0 5400
Processing waveform 126 of 301
75.0 1875
Processing waveform 127 of 301
75.0 5850
Processing waveform 128 of 301
75.0 1725
Processing waveform 129 of 301
75.0 7650
Processing waveform 130 of 301
75.0 2700
Processing waveform 131 of 301
75.0 9075
Processing waveform 132 of 301
75.0 6975
Processing waveform 133 of 301
75.0 4050
Processing waveform 134 of 301
75.0 3450
Processing waveform 135 of 301
75.0 5925
Processing waveform 136 of 301
75.0 2250
Processing waveform 137 of 301
75.0 3675
Processing waveform 138 of 301
75.0 2625
Processing waveform 139 of 301
75.0 4575
Processing waveform 140 of 301
75.0 2025
Processing waveform 141 of 301
75.0 5175
Processing waveform 142 of 301
75.0 3225
Processing waveform 143 of 301
75.0 4950
Processing waveform 144 of 301
75.0 2925
Processing waveform 145 of 301
75.0 1800
Processing waveform 146 of 301
75.0 8925
Processing waveform 147 of 301
75.0 7350
Processing waveform 148 of 301
75.0 4125
Processing waveform 149 of 301
75.0 3225
Processing waveform 150 of 301
75.0 9075
Processing waveform 151 of 301
75.0 5025
Processing waveform 152 of 301
75.0 8925
Processing waveform 153 of 301
75.0 4800
Processing waveform 154 of 301
75.0 7800
Processing waveform 155 of 301
75.0 2775
Processing waveform 156 of 301
75.0 6600
Processing waveform 157 of 301
75.0 1800
Processing waveform 158 of 301
75.0 2625
Processing waveform 159 of 301
75.0 9075
Processing waveform 160 of 301
75.0 7200
Processing waveform 161 of 301
75.0 5550
Processing waveform 162 of 301
75.0 2775
Processing waveform 163 of 301
75.0 5250
Processing waveform 164 of 301
75.0 1875
Processing waveform 165 of 301
75.0 1875
Processing waveform 166 of 301
75.0 5025
Processing waveform 167 of 301
75.0 3600
Processing waveform 168 of 301
75.0 3375
Processing waveform 169 of 301
75.0 4050
Processing waveform 170 of 301
75.0 3375
Processing waveform 171 of 301
75.0 2925
Processing waveform 172 of 301
75.0 6675
Processing waveform 173 of 301
75.0 3750
Processing waveform 174 of 301
75.0 4575
Processing waveform 175 of 301
75.0 9075
Processing waveform 176 of 301
75.0 3900
Processing waveform 177 of 301
75.0 9075
Processing waveform 178 of 301
75.0 4875
Processing waveform 179 of 301
75.0 3075
Processing waveform 180 of 301
75.0 3150
Processing waveform 181 of 301
75.0 9075
Processing waveform 182 of 301
75.0 7425
Processing waveform 183 of 301
75.0 5025
Processing waveform 184 of 301
75.0 2850
Processing waveform 185 of 301
75.0 1875
Processing waveform 186 of 301
75.0 7125
Processing waveform 187 of 301
75.0 4500
Processing waveform 188 of 301
75.0 6075
Processing waveform 189 of 301
75.0 2475
Processing waveform 190 of 301
75.0 4125
Processing waveform 191 of 301
75.0 2100
Processing waveform 192 of 301
75.0 3975
Processing waveform 193 of 301
75.0 3225
Processing waveform 194 of 301
75.0 2550
Processing waveform 195 of 301
75.0 6975
Processing waveform 196 of 301
75.0 3075
Processing waveform 197 of 301
75.0 3675
Processing waveform 198 of 301
75.0 4800
Processing waveform 199 of 301
75.0 5850
Processing waveform 200 of 301
75.0 3675
Processing waveform 201 of 301
75.0 2250
Processing waveform 202 of 301
75.0 3600
Processing waveform 203 of 301
75.0 5775
Processing waveform 204 of 301
75.0 2325
Processing waveform 205 of 301
75.0 2325
Processing waveform 206 of 301
75.0 3525
Processing waveform 207 of 301
75.0 4500
Processing waveform 208 of 301
75.0 4275
Processing waveform 209 of 301
75.0 3450
Processing waveform 210 of 301
75.0 2550
Processing waveform 211 of 301
75.0 3300
Processing waveform 212 of 301
75.0 9075
Processing waveform 213 of 301
75.0 9075
Processing waveform 214 of 301
75.0 6525
Processing waveform 215 of 301
75.0 6975
Processing waveform 216 of 301
75.0 6750

Processing waveform 217 of 301
75.0 7275
Processing waveform 218 of 301
75.0 2400
Processing waveform 219 of 301
75.0 4125
Processing waveform 220 of 301
75.0 3525
Processing waveform 221 of 301
75.0 5175
Processing waveform 222 of 301
75.0 3750
Processing waveform 223 of 301
75.0 4875
Processing waveform 224 of 301
75.0 2250
Processing waveform 225 of 301
75.0 4125
Processing waveform 226 of 301
75.0 4425
Processing waveform 227 of 301
75.0 5250
Processing waveform 228 of 301
75.0 9075
Processing waveform 229 of 301
75.0 4725
Processing waveform 230 of 301
75.0 2475
Processing waveform 231 of 301
75.0 2925
Processing waveform 232 of 301
75.0 7350
Processing waveform 233 of 301
75.0 2550
Processing waveform 234 of 301
75.0 2850
Processing waveform 235 of 301
75.0 2550
Processing waveform 236 of 301
75.0 4425
Processing waveform 237 of 301
75.0 5775
Processing waveform 238 of 301
75.0 4200
Processing waveform 239 of 301
75.0 2175
Processing waveform 240 of 301
75.0 2925
Processing waveform 241 of 301
75.0 10350
Processing waveform 242 of 301
75.0 15600
Processing waveform 243 of 301
75.0 6450
Processing waveform 244 of 301
75.0 3450
Processing waveform 245 of 301
75.0 11175
Processing waveform 246 of 301
75.0 4725
Processing waveform 247 of 301
75.0 4200
Processing waveform 248 of 301
75.0 3750
Processing waveform 249 of 301
75.0 3975
Processing waveform 250 of 301
75.0 4575
Processing waveform 251 of 301
75.0 3225
Processing waveform 252 of 301
75.0 13500
Processing waveform 253 of 301
75.0 4050
Processing waveform 254 of 301
75.0 5250
Processing waveform 255 of 301
75.0 7050
Processing waveform 256 of 301
75.0 3450
Processing waveform 257 of 301
75.0 6675
Processing waveform 258 of 301
75.0 2850
Processing waveform 259 of 301
75.0 6750
Processing waveform 260 of 301
75.0 2475
Processing waveform 261 of 301
75.0 7650
Processing waveform 262 of 301
75.0 6750
Processing waveform 263 of 301
75.0 4125
Processing waveform 264 of 301
75.0 7425
Processing waveform 265 of 301
75.0 5400
Processing waveform 266 of 301
75.0 4800
Processing waveform 267 of 301
75.0 10200
Processing waveform 268 of 301
75.0 18225
Processing waveform 269 of 301
75.0 5925
Processing waveform 270 of 301
75.0 3525
Processing waveform 271 of 301
75.0 3300
Processing waveform 272 of 301
75.0 3600
Processing waveform 273 of 301
75.0 4650
Processing waveform 274 of 301
75.0 7875
Processing waveform 275 of 301
75.0 6300
Processing waveform 276 of 301
75.0 6750
Processing waveform 277 of 301
75.0 2925
Processing waveform 278 of 301
75.0 3525
Processing waveform 279 of 301
75.0 7875
Processing waveform 280 of 301
75.0 8925
Processing waveform 281 of 301
75.0 2850
Processing waveform 282 of 301
75.0 2475
Processing waveform 283 of 301
75.0 2325
Processing waveform 284 of 301
75.0 2250
Processing waveform 285 of 301
75.0 2700
Processing waveform 286 of 301
75.0 3000
Processing waveform 287 of 301
75.0 3300
Processing waveform 288 of 301
75.0 4425
Processing waveform 289 of 301
75.0 4500
Processing waveform 290 of 301
75.0 7500
Processing waveform 291 of 301
75.0 4200
Processing waveform 292 of 301
75.0 4800
Processing waveform 293 of 301
75.0 3000
Processing waveform 294 of 301
75.0 1950
Processing waveform 295 of 301
75.0 2625
Processing waveform 296 of 301
75.0 4650
Processing waveform 297 of 301
75.0 5475
Processing waveform 298 of 301
75.0 4950
Processing waveform 299 of 301
75.0 6375
Processing waveform 300 of 301
75.0 7500
Training data have been read and features have been extracted  (301, 120)
Computation time:  389.87127685546875
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.31726622581481934
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h   104             
        l          97       
        t               100 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  39.4481236203  +/-  4.20823742965  %
          Predicted class
              h     l     t 
        h    27    12    13 
        l    19    14    15 
        t    17    15    18 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't', 'r'] 2
class
h    104
l     97
r     93
t    100
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 394
75.0 5325
Processing waveform 1 of 394
75.0 2700
Processing waveform 2 of 394
75.0 4350
Processing waveform 3 of 394
75.0 2775
Processing waveform 4 of 394
75.0 2250
Processing waveform 5 of 394
75.0 2250
Processing waveform 6 of 394
75.0 2475
Processing waveform 7 of 394
75.0 3300
Processing waveform 8 of 394
75.0 9075
Processing waveform 9 of 394
75.0 3225
Processing waveform 10 of 394
75.0 4125
Processing waveform 11 of 394
75.0 3300
Processing waveform 12 of 394
75.0 3225
Processing waveform 13 of 394
75.0 2850
Processing waveform 14 of 394
75.0 2400
Processing waveform 15 of 394
75.0 3900
Processing waveform 16 of 394
75.0 2775
Processing waveform 17 of 394
75.0 2325
Processing waveform 18 of 394
75.0 4425
Processing waveform 19 of 394
75.0 3525
Processing waveform 20 of 394
75.0 6300
Processing waveform 21 of 394
75.0 2625
Processing waveform 22 of 394
75.0 4050
Processing waveform 23 of 394
75.0 2775
Processing waveform 24 of 394
75.0 2400
Processing waveform 25 of 394
75.0 2850
Processing waveform 26 of 394
75.0 2250
Processing waveform 27 of 394
75.0 2325
Processing waveform 28 of 394
75.0 3750
Processing waveform 29 of 394
75.0 3525
Processing waveform 30 of 394
75.0 5700
Processing waveform 31 of 394
75.0 2700
Processing waveform 32 of 394
75.0 3000
Processing waveform 33 of 394
75.0 4350
Processing waveform 34 of 394
75.0 4950
Processing waveform 35 of 394
75.0 2925
Processing waveform 36 of 394
75.0 2775
Processing waveform 37 of 394
75.0 3450
Processing waveform 38 of 394
75.0 4875
Processing waveform 39 of 394
75.0 9075
Processing waveform 40 of 394
75.0 5175
Processing waveform 41 of 394
75.0 3075
Processing waveform 42 of 394
75.0 4125
Processing waveform 43 of 394
75.0 9075
Processing waveform 44 of 394
75.0 2775
Processing waveform 45 of 394
75.0 4575
Processing waveform 46 of 394
75.0 2700
Processing waveform 47 of 394
75.0 1950
Processing waveform 48 of 394
75.0 2100
Processing waveform 49 of 394
75.0 4575
Processing waveform 50 of 394
75.0 5850
Processing waveform 51 of 394
75.0 3150
Processing waveform 52 of 394
75.0 4275
Processing waveform 53 of 394
75.0 9075
Processing waveform 54 of 394
75.0 3600
Processing waveform 55 of 394
75.0 2625
Processing waveform 56 of 394
75.0 2700
Processing waveform 57 of 394
75.0 2700
Processing waveform 58 of 394
75.0 3525
Processing waveform 59 of 394
75.0 4200
Processing waveform 60 of 394
75.0 4200
Processing waveform 61 of 394
75.0 1950
Processing waveform 62 of 394
75.0 5775
Processing waveform 63 of 394
75.0 9075
Processing waveform 64 of 394
75.0 4875
Processing waveform 65 of 394
75.0 14625
Processing waveform 66 of 394
75.0 11025
Processing waveform 67 of 394
75.0 1950
Processing waveform 68 of 394
75.0 3975
Processing waveform 69 of 394
75.0 2550
Processing waveform 70 of 394
75.0 7500
Processing waveform 71 of 394
75.0 14925
Processing waveform 72 of 394
75.0 13275
Processing waveform 73 of 394
75.0 1725
Processing waveform 74 of 394
75.0 2400
Processing waveform 75 of 394
75.0 6375
Processing waveform 76 of 394
75.0 6450
Processing waveform 77 of 394
75.0 4125
Processing waveform 78 of 394
75.0 4800
Processing waveform 79 of 394
75.0 7125
Processing waveform 80 of 394
75.0 5325
Processing waveform 81 of 394
75.0 5400
Processing waveform 82 of 394
75.0 6825
Processing waveform 83 of 394
75.0 7050
Processing waveform 84 of 394
75.0 4500
Processing waveform 85 of 394
75.0 5625
Processing waveform 86 of 394
75.0 4275
Processing waveform 87 of 394
75.0 3150
Processing waveform 88 of 394
75.0 3150
Processing waveform 89 of 394
75.0 12600
Processing waveform 90 of 394
75.0 7425
Processing waveform 91 of 394
75.0 16725
Processing waveform 92 of 394
75.0 6075

Processing waveform 93 of 394
75.0 3975
Processing waveform 94 of 394
75.0 9075
Processing waveform 95 of 394
75.0 2175
Processing waveform 96 of 394
75.0 3300
Processing waveform 97 of 394
75.0 4275
Processing waveform 98 of 394
75.0 2850
Processing waveform 99 of 394
75.0 2850
Processing waveform 100 of 394
75.0 2700
Processing waveform 101 of 394
75.0 5250
Processing waveform 102 of 394
75.0 3000
Processing waveform 103 of 394
75.0 2775
Processing waveform 104 of 394
75.0 6450
Processing waveform 105 of 394
75.0 3150
Processing waveform 106 of 394
75.0 1650
Processing waveform 107 of 394
75.0 4500
Processing waveform 108 of 394
75.0 2250
Processing waveform 109 of 394
75.0 2175
Processing waveform 110 of 394
75.0 3225
Processing waveform 111 of 394
75.0 4050
Processing waveform 112 of 394
75.0 3300
Processing waveform 113 of 394
75.0 2850
Processing waveform 114 of 394
75.0 2025
Processing waveform 115 of 394
75.0 4800
Processing waveform 116 of 394
75.0 2325
Processing waveform 117 of 394
75.0 6150
Processing waveform 118 of 394
75.0 2775
Processing waveform 119 of 394
75.0 2850
Processing waveform 120 of 394
75.0 4425
Processing waveform 121 of 394
75.0 2250
Processing waveform 122 of 394
75.0 3075
Processing waveform 123 of 394
75.0 2775
Processing waveform 124 of 394
75.0 1875
Processing waveform 125 of 394
75.0 3300
Processing waveform 126 of 394
75.0 1875
Processing waveform 127 of 394
75.0 5550
Processing waveform 128 of 394
75.0 9075
Processing waveform 129 of 394
75.0 4650
Processing waveform 130 of 394
75.0 6525
Processing waveform 131 of 394
75.0 9075
Processing waveform 132 of 394
75.0 3225
Processing waveform 133 of 394
75.0 3150
Processing waveform 134 of 394
75.0 4275
Processing waveform 135 of 394
75.0 4050
Processing waveform 136 of 394
75.0 5400
Processing waveform 137 of 394
75.0 4725
Processing waveform 138 of 394
75.0 3450
Processing waveform 139 of 394
75.0 6150
Processing waveform 140 of 394
75.0 2250
Processing waveform 141 of 394
75.0 9000
Processing waveform 142 of 394
75.0 3150
Processing waveform 143 of 394
75.0 4950
Processing waveform 144 of 394
75.0 6750
Processing waveform 145 of 394
75.0 3150
Processing waveform 146 of 394
75.0 7050
Processing waveform 147 of 394
75.0 3675
Processing waveform 148 of 394
75.0 3225
Processing waveform 149 of 394
75.0 2325
Processing waveform 150 of 394
75.0 7500
Processing waveform 151 of 394
75.0 3975
Processing waveform 152 of 394
75.0 6150
Processing waveform 153 of 394
75.0 7875
Processing waveform 154 of 394
75.0 7275
Processing waveform 155 of 394
75.0 6075
Processing waveform 156 of 394
75.0 3900
Processing waveform 157 of 394
75.0 3000
Processing waveform 158 of 394
75.0 3900
Processing waveform 159 of 394
75.0 6525
Processing waveform 160 of 394
75.0 2925
Processing waveform 161 of 394
75.0 2700
Processing waveform 162 of 394
75.0 2250
Processing waveform 163 of 394
75.0 4425
Processing waveform 164 of 394
75.0 2250
Processing waveform 165 of 394
75.0 9600
Processing waveform 166 of 394
75.0 4650
Processing waveform 167 of 394
75.0 6750
Processing waveform 168 of 394
75.0 7125
Processing waveform 169 of 394
75.0 3000
Processing waveform 170 of 394
75.0 10200
Processing waveform 171 of 394
75.0 1800
Processing waveform 172 of 394
75.0 4425
Processing waveform 173 of 394
75.0 2250
Processing waveform 174 of 394
75.0 6150
Processing waveform 175 of 394
75.0 7125
Processing waveform 176 of 394
75.0 3600
Processing waveform 177 of 394
75.0 6000
Processing waveform 178 of 394
75.0 4800
Processing waveform 179 of 394
75.0 7200
Processing waveform 180 of 394
75.0 3000
Processing waveform 181 of 394
75.0 4500
Processing waveform 182 of 394
75.0 3600
Processing waveform 183 of 394
75.0 4875
Processing waveform 184 of 394
75.0 5625
Processing waveform 185 of 394
75.0 3150
Processing waveform 186 of 394
75.0 3975
Processing waveform 187 of 394
75.0 5325
Processing waveform 188 of 394
75.0 2850
Processing waveform 189 of 394
75.0 9450
Processing waveform 190 of 394
75.0 3225
Processing waveform 191 of 394
75.0 2100
Processing waveform 192 of 394
75.0 2700
Processing waveform 193 of 394
75.0 2325
Processing waveform 194 of 394
75.0 2775
Processing waveform 195 of 394
75.0 2625
Processing waveform 196 of 394
75.0 5850
Processing waveform 197 of 394
75.0 3825
Processing waveform 198 of 394
75.0 6000
Processing waveform 199 of 394
75.0 2025
Processing waveform 200 of 394
75.0 1725
Processing waveform 201 of 394
75.0 2175
Processing waveform 202 of 394
75.0 5400
Processing waveform 203 of 394
75.0 5325
Processing waveform 204 of 394
75.0 3750
Processing waveform 205 of 394
75.0 3075
Processing waveform 206 of 394
75.0 5550
Processing waveform 207 of 394
75.0 7350
Processing waveform 208 of 394
75.0 4650
Processing waveform 209 of 394
75.0 2775
Processing waveform 210 of 394
75.0 2325
Processing waveform 211 of 394
75.0 2925
Processing waveform 212 of 394
75.0 3075
Processing waveform 213 of 394
75.0 6375
Processing waveform 214 of 394
75.0 2475
Processing waveform 215 of 394
75.0 3975
Processing waveform 216 of 394
75.0 3150
Processing waveform 217 of 394
75.0 5925
Processing waveform 218 of 394
75.0 5400
Processing waveform 219 of 394
75.0 1875
Processing waveform 220 of 394
75.0 5850
Processing waveform 221 of 394
75.0 1725
Processing waveform 222 of 394
75.0 7650
Processing waveform 223 of 394
75.0 2700
Processing waveform 224 of 394
75.0 9075
Processing waveform 225 of 394
75.0 6975
Processing waveform 226 of 394
75.0 4050
Processing waveform 227 of 394
75.0 3450
Processing waveform 228 of 394
75.0 5925
Processing waveform 229 of 394
75.0 2250
Processing waveform 230 of 394
75.0 3675
Processing waveform 231 of 394
75.0 2625
Processing waveform 232 of 394
75.0 4575
Processing waveform 233 of 394
75.0 2025
Processing waveform 234 of 394
75.0 5175
Processing waveform 235 of 394
75.0 3225
Processing waveform 236 of 394
75.0 4950
Processing waveform 237 of 394
75.0 2925
Processing waveform 238 of 394
75.0 1800
Processing waveform 239 of 394
75.0 8925
Processing waveform 240 of 394
75.0 7350
Processing waveform 241 of 394
75.0 4125
Processing waveform 242 of 394
75.0 3225
Processing waveform 243 of 394
75.0 9075
Processing waveform 244 of 394
75.0 5025
Processing waveform 245 of 394
75.0 8925
Processing waveform 246 of 394
75.0 4800
Processing waveform 247 of 394
75.0 7800
Processing waveform 248 of 394
75.0 2775
Processing waveform 249 of 394
75.0 6600
Processing waveform 250 of 394
75.0 1800
Processing waveform 251 of 394
75.0 2625
Processing waveform 252 of 394
75.0 9075
Processing waveform 253 of 394
75.0 7200
Processing waveform 254 of 394
75.0 5550
Processing waveform 255 of 394
75.0 2775
Processing waveform 256 of 394
75.0 5250
Processing waveform 257 of 394
75.0 1875
Processing waveform 258 of 394
75.0 1875
Processing waveform 259 of 394
75.0 5025
Processing waveform 260 of 394
75.0 3600
Processing waveform 261 of 394
75.0 3375
Processing waveform 262 of 394
75.0 4050
Processing waveform 263 of 394
75.0 3375
Processing waveform 264 of 394
75.0 2925
Processing waveform 265 of 394
75.0 6675
Processing waveform 266 of 394
75.0 3750
Processing waveform 267 of 394
75.0 4575
Processing waveform 268 of 394
75.0 9075
Processing waveform 269 of 394
75.0 3900
Processing waveform 270 of 394
75.0 9075
Processing waveform 271 of 394
75.0 4875
Processing waveform 272 of 394
75.0 3075
Processing waveform 273 of 394
75.0 3150
Processing waveform 274 of 394
75.0 9075
Processing waveform 275 of 394
75.0 7425
Processing waveform 276 of 394
75.0 5025
Processing waveform 277 of 394
75.0 2850
Processing waveform 278 of 394
75.0 1875
Processing waveform 279 of 394
75.0 7125
Processing waveform 280 of 394
75.0 4500
Processing waveform 281 of 394
75.0 6075
Processing waveform 282 of 394
75.0 2475
Processing waveform 283 of 394
75.0 4125
Processing waveform 284 of 394
75.0 2100
Processing waveform 285 of 394
75.0 3975
Processing waveform 286 of 394
75.0 3225
Processing waveform 287 of 394
75.0 2550
Processing waveform 288 of 394
75.0 6975
Processing waveform 289 of 394
75.0 3075
Processing waveform 290 of 394
75.0 3675
Processing waveform 291 of 394
75.0 4800
Processing waveform 292 of 394
75.0 5850

Processing waveform 293 of 394
75.0 3675
Processing waveform 294 of 394
75.0 2250
Processing waveform 295 of 394
75.0 3600
Processing waveform 296 of 394
75.0 5775
Processing waveform 297 of 394
75.0 2325
Processing waveform 298 of 394
75.0 2325
Processing waveform 299 of 394
75.0 3525
Processing waveform 300 of 394
75.0 4500
Processing waveform 301 of 394
75.0 4275
Processing waveform 302 of 394
75.0 3450
Processing waveform 303 of 394
75.0 2550
Processing waveform 304 of 394
75.0 3300
Processing waveform 305 of 394
75.0 9075
Processing waveform 306 of 394
75.0 9075
Processing waveform 307 of 394
75.0 6525
Processing waveform 308 of 394
75.0 6975
Processing waveform 309 of 394
75.0 6750
Processing waveform 310 of 394
75.0 7275
Processing waveform 311 of 394
75.0 2400
Processing waveform 312 of 394
75.0 4125
Processing waveform 313 of 394
75.0 3525
Processing waveform 314 of 394
75.0 5175
Processing waveform 315 of 394
75.0 3750
Processing waveform 316 of 394
75.0 4875
Processing waveform 317 of 394
75.0 2250
Processing waveform 318 of 394
75.0 4125
Processing waveform 319 of 394
75.0 4425
Processing waveform 320 of 394
75.0 5250
Processing waveform 321 of 394
75.0 9075
Processing waveform 322 of 394
75.0 4725
Processing waveform 323 of 394
75.0 2475
Processing waveform 324 of 394
75.0 2925
Processing waveform 325 of 394
75.0 7350
Processing waveform 326 of 394
75.0 2550
Processing waveform 327 of 394
75.0 2850
Processing waveform 328 of 394
75.0 2550
Processing waveform 329 of 394
75.0 4425
Processing waveform 330 of 394
75.0 5775
Processing waveform 331 of 394
75.0 4200
Processing waveform 332 of 394
75.0 2175
Processing waveform 333 of 394
75.0 2925
Processing waveform 334 of 394
75.0 10350
Processing waveform 335 of 394
75.0 15600
Processing waveform 336 of 394
75.0 6450
Processing waveform 337 of 394
75.0 3450
Processing waveform 338 of 394
75.0 11175
Processing waveform 339 of 394
75.0 4725
Processing waveform 340 of 394
75.0 4200
Processing waveform 341 of 394
75.0 3750
Processing waveform 342 of 394
75.0 3975
Processing waveform 343 of 394
75.0 4575
Processing waveform 344 of 394
75.0 3225
Processing waveform 345 of 394
75.0 13500
Processing waveform 346 of 394
75.0 4050
Processing waveform 347 of 394
75.0 5250
Processing waveform 348 of 394
75.0 7050
Processing waveform 349 of 394
75.0 3450
Processing waveform 350 of 394
75.0 6675
Processing waveform 351 of 394
75.0 2850
Processing waveform 352 of 394
75.0 6750
Processing waveform 353 of 394
75.0 2475
Processing waveform 354 of 394
75.0 7650
Processing waveform 355 of 394
75.0 6750
Processing waveform 356 of 394
75.0 4125
Processing waveform 357 of 394
75.0 7425
Processing waveform 358 of 394
75.0 5400
Processing waveform 359 of 394
75.0 4800
Processing waveform 360 of 394
75.0 10200
Processing waveform 361 of 394
75.0 18225
Processing waveform 362 of 394
75.0 5925
Processing waveform 363 of 394
75.0 3525
Processing waveform 364 of 394
75.0 3300
Processing waveform 365 of 394
75.0 3600
Processing waveform 366 of 394
75.0 4650
Processing waveform 367 of 394
75.0 7875
Processing waveform 368 of 394
75.0 6300
Processing waveform 369 of 394
75.0 6750
Processing waveform 370 of 394
75.0 2925
Processing waveform 371 of 394
75.0 3525
Processing waveform 372 of 394
75.0 7875
Processing waveform 373 of 394
75.0 8925
Processing waveform 374 of 394
75.0 2850
Processing waveform 375 of 394
75.0 2475
Processing waveform 376 of 394
75.0 2325
Processing waveform 377 of 394
75.0 2250
Processing waveform 378 of 394
75.0 2700
Processing waveform 379 of 394
75.0 3000
Processing waveform 380 of 394
75.0 3300
Processing waveform 381 of 394
75.0 4425
Processing waveform 382 of 394
75.0 4500
Processing waveform 383 of 394
75.0 7500
Processing waveform 384 of 394
75.0 4200
Processing waveform 385 of 394
75.0 4800
Processing waveform 386 of 394
75.0 3000
Processing waveform 387 of 394
75.0 1950
Processing waveform 388 of 394
75.0 2625
Processing waveform 389 of 394
75.0 4650
Processing waveform 390 of 394
75.0 5475
Processing waveform 391 of 394
75.0 4950
Processing waveform 392 of 394
75.0 6375
Processing waveform 393 of 394
75.0 7500
Training data have been read and features have been extracted  (394, 120)
Computation time:  529.0536880493164
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.4955759048461914
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h   104                   
        l          97             
        r                93       
        t                     100 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  29.9323181049  +/-  3.29879813947  %
          Predicted class
              h     l     r     t 
        h    20    10    11    11 
        l    14    12     9    13 
        r    17     9    10    10 
        t    12    13     9    16 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'h', 'l', 't', 'r'] 2
class
e     87
h    104
l     97
r     93
t    100
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 481
75.0 5325
Processing waveform 1 of 481
75.0 2700
Processing waveform 2 of 481
75.0 4350
Processing waveform 3 of 481
75.0 2775
Processing waveform 4 of 481
75.0 2250
Processing waveform 5 of 481
75.0 2250
Processing waveform 6 of 481
75.0 2475
Processing waveform 7 of 481
75.0 3300
Processing waveform 8 of 481
75.0 9075
Processing waveform 9 of 481
75.0 3225
Processing waveform 10 of 481
75.0 4125
Processing waveform 11 of 481
75.0 3300
Processing waveform 12 of 481
75.0 3225
Processing waveform 13 of 481
75.0 2850
Processing waveform 14 of 481
75.0 2400
Processing waveform 15 of 481
75.0 3900
Processing waveform 16 of 481
75.0 2775
Processing waveform 17 of 481
75.0 2325
Processing waveform 18 of 481
75.0 4425
Processing waveform 19 of 481
75.0 3525
Processing waveform 20 of 481
75.0 6300
Processing waveform 21 of 481
75.0 2625
Processing waveform 22 of 481
75.0 4050
Processing waveform 23 of 481
75.0 2775
Processing waveform 24 of 481
75.0 2400
Processing waveform 25 of 481
75.0 2850
Processing waveform 26 of 481
75.0 2250
Processing waveform 27 of 481
75.0 2325
Processing waveform 28 of 481
75.0 3750
Processing waveform 29 of 481
75.0 3525
Processing waveform 30 of 481
75.0 5700
Processing waveform 31 of 481
75.0 2700
Processing waveform 32 of 481
75.0 3000
Processing waveform 33 of 481
75.0 4350
Processing waveform 34 of 481
75.0 4950
Processing waveform 35 of 481
75.0 2925
Processing waveform 36 of 481
75.0 2775
Processing waveform 37 of 481
75.0 3450
Processing waveform 38 of 481
75.0 4875
Processing waveform 39 of 481
75.0 9075
Processing waveform 40 of 481
75.0 5175
Processing waveform 41 of 481
75.0 3075
Processing waveform 42 of 481
75.0 4125
Processing waveform 43 of 481
75.0 9075
Processing waveform 44 of 481
75.0 2775
Processing waveform 45 of 481
75.0 4575
Processing waveform 46 of 481
75.0 2700
Processing waveform 47 of 481
75.0 1950
Processing waveform 48 of 481
75.0 2100
Processing waveform 49 of 481
75.0 4575
Processing waveform 50 of 481
75.0 5850
Processing waveform 51 of 481
75.0 3150
Processing waveform 52 of 481
75.0 4275
Processing waveform 53 of 481
75.0 9075
Processing waveform 54 of 481
75.0 3600
Processing waveform 55 of 481
75.0 2625
Processing waveform 56 of 481
75.0 2700
Processing waveform 57 of 481
75.0 2700
Processing waveform 58 of 481
75.0 3525
Processing waveform 59 of 481
75.0 4200
Processing waveform 60 of 481
75.0 4200
Processing waveform 61 of 481
75.0 1950
Processing waveform 62 of 481
75.0 5775
Processing waveform 63 of 481
75.0 9075
Processing waveform 64 of 481
75.0 4875
Processing waveform 65 of 481
75.0 14625
Processing waveform 66 of 481
75.0 11025
Processing waveform 67 of 481
75.0 1950
Processing waveform 68 of 481
75.0 3975
Processing waveform 69 of 481
75.0 2550
Processing waveform 70 of 481
75.0 7500
Processing waveform 71 of 481
75.0 14925

Processing waveform 72 of 481
75.0 13275
Processing waveform 73 of 481
75.0 1725
Processing waveform 74 of 481
75.0 2400
Processing waveform 75 of 481
75.0 6375
Processing waveform 76 of 481
75.0 6450
Processing waveform 77 of 481
75.0 4125
Processing waveform 78 of 481
75.0 4800
Processing waveform 79 of 481
75.0 7125
Processing waveform 80 of 481
75.0 5325
Processing waveform 81 of 481
75.0 5400
Processing waveform 82 of 481
75.0 6825
Processing waveform 83 of 481
75.0 7050
Processing waveform 84 of 481
75.0 4500
Processing waveform 85 of 481
75.0 5625
Processing waveform 86 of 481
75.0 4275
Processing waveform 87 of 481
75.0 3150
Processing waveform 88 of 481
75.0 3150
Processing waveform 89 of 481
75.0 12600
Processing waveform 90 of 481
75.0 7425
Processing waveform 91 of 481
75.0 16725
Processing waveform 92 of 481
75.0 6075
Processing waveform 93 of 481
75.0 3450
Processing waveform 94 of 481
75.0 2325
Processing waveform 95 of 481
75.0 2925
Processing waveform 96 of 481
75.0 4575
Processing waveform 97 of 481
75.0 7050
Processing waveform 98 of 481
75.0 2025
Processing waveform 99 of 481
75.0 2925
Processing waveform 100 of 481
75.0 2850
Processing waveform 101 of 481
75.0 3600
Processing waveform 102 of 481
75.0 3750
Processing waveform 103 of 481
75.0 2100
Processing waveform 104 of 481
75.0 1650
Processing waveform 105 of 481
75.0 4200
Processing waveform 106 of 481
75.0 9075
Processing waveform 107 of 481
75.0 4125
Processing waveform 108 of 481
75.0 2925
Processing waveform 109 of 481
75.0 2250
Processing waveform 110 of 481
75.0 5475
Processing waveform 111 of 481
75.0 2625
Processing waveform 112 of 481
75.0 2400
Processing waveform 113 of 481
75.0 2700
Processing waveform 114 of 481
75.0 9075
Processing waveform 115 of 481
75.0 3300
Processing waveform 116 of 481
75.0 9075
Processing waveform 117 of 481
75.0 6225
Processing waveform 118 of 481
75.0 7125
Processing waveform 119 of 481
75.0 5850
Processing waveform 120 of 481
75.0 9075
Processing waveform 121 of 481
75.0 3225
Processing waveform 122 of 481
75.0 8100
Processing waveform 123 of 481
75.0 4575
Processing waveform 124 of 481
75.0 2850
Processing waveform 125 of 481
75.0 3375
Processing waveform 126 of 481
75.0 2100
Processing waveform 127 of 481
75.0 9075
Processing waveform 128 of 481
75.0 5850
Processing waveform 129 of 481
75.0 2775
Processing waveform 130 of 481
75.0 2700
Processing waveform 131 of 481
75.0 2100
Processing waveform 132 of 481
75.0 9075
Processing waveform 133 of 481
75.0 3225
Processing waveform 134 of 481
75.0 3300
Processing waveform 135 of 481
75.0 2475
Processing waveform 136 of 481
75.0 3075
Processing waveform 137 of 481
75.0 6450
Processing waveform 138 of 481
75.0 3600
Processing waveform 139 of 481
75.0 4650
Processing waveform 140 of 481
75.0 2475
Processing waveform 141 of 481
75.0 1875
Processing waveform 142 of 481
75.0 3450
Processing waveform 143 of 481
75.0 1500
Processing waveform 144 of 481
75.0 2625
Processing waveform 145 of 481
75.0 9075
Processing waveform 146 of 481
75.0 3525
Processing waveform 147 of 481
75.0 5625
Processing waveform 148 of 481
75.0 3675
Processing waveform 149 of 481
75.0 9075
Processing waveform 150 of 481
75.0 2700
Processing waveform 151 of 481
75.0 3000
Processing waveform 152 of 481
75.0 3525
Processing waveform 153 of 481
75.0 7200
Processing waveform 154 of 481
75.0 4875
Processing waveform 155 of 481
75.0 6075
Processing waveform 156 of 481
75.0 2700
Processing waveform 157 of 481
75.0 5250
Processing waveform 158 of 481
75.0 12750
Processing waveform 159 of 481
75.0 5100
Processing waveform 160 of 481
75.0 2475
Processing waveform 161 of 481
75.0 6525
Processing waveform 162 of 481
75.0 7200
Processing waveform 163 of 481
75.0 4950
Processing waveform 164 of 481
75.0 8250
Processing waveform 165 of 481
75.0 3150
Processing waveform 166 of 481
75.0 2325
Processing waveform 167 of 481
75.0 4200
Processing waveform 168 of 481
75.0 3300
Processing waveform 169 of 481
75.0 2325
Processing waveform 170 of 481
75.0 13500
Processing waveform 171 of 481
75.0 3525
Processing waveform 172 of 481
75.0 3075
Processing waveform 173 of 481
75.0 6600
Processing waveform 174 of 481
75.0 6225
Processing waveform 175 of 481
75.0 6975
Processing waveform 176 of 481
75.0 4425
Processing waveform 177 of 481
75.0 13350
Processing waveform 178 of 481
75.0 9525
Processing waveform 179 of 481
75.0 2325
Processing waveform 180 of 481
75.0 3975
Processing waveform 181 of 481
75.0 9075
Processing waveform 182 of 481
75.0 2175
Processing waveform 183 of 481
75.0 3300
Processing waveform 184 of 481
75.0 4275
Processing waveform 185 of 481
75.0 2850
Processing waveform 186 of 481
75.0 2850
Processing waveform 187 of 481
75.0 2700
Processing waveform 188 of 481
75.0 5250
Processing waveform 189 of 481
75.0 3000
Processing waveform 190 of 481
75.0 2775
Processing waveform 191 of 481
75.0 6450
Processing waveform 192 of 481
75.0 3150
Processing waveform 193 of 481
75.0 1650
Processing waveform 194 of 481
75.0 4500
Processing waveform 195 of 481
75.0 2250
Processing waveform 196 of 481
75.0 2175
Processing waveform 197 of 481
75.0 3225
Processing waveform 198 of 481
75.0 4050
Processing waveform 199 of 481
75.0 3300
Processing waveform 200 of 481
75.0 2850
Processing waveform 201 of 481
75.0 2025
Processing waveform 202 of 481
75.0 4800
Processing waveform 203 of 481
75.0 2325
Processing waveform 204 of 481
75.0 6150
Processing waveform 205 of 481
75.0 2775
Processing waveform 206 of 481
75.0 2850
Processing waveform 207 of 481
75.0 4425
Processing waveform 208 of 481
75.0 2250
Processing waveform 209 of 481
75.0 3075
Processing waveform 210 of 481
75.0 2775
Processing waveform 211 of 481
75.0 1875
Processing waveform 212 of 481
75.0 3300
Processing waveform 213 of 481
75.0 1875
Processing waveform 214 of 481
75.0 5550
Processing waveform 215 of 481
75.0 9075
Processing waveform 216 of 481
75.0 4650
Processing waveform 217 of 481
75.0 6525
Processing waveform 218 of 481
75.0 9075
Processing waveform 219 of 481
75.0 3225
Processing waveform 220 of 481
75.0 3150
Processing waveform 221 of 481
75.0 4275
Processing waveform 222 of 481
75.0 4050
Processing waveform 223 of 481
75.0 5400
Processing waveform 224 of 481
75.0 4725
Processing waveform 225 of 481
75.0 3450
Processing waveform 226 of 481
75.0 6150
Processing waveform 227 of 481
75.0 2250
Processing waveform 228 of 481
75.0 9000
Processing waveform 229 of 481
75.0 3150
Processing waveform 230 of 481
75.0 4950
Processing waveform 231 of 481
75.0 6750
Processing waveform 232 of 481
75.0 3150
Processing waveform 233 of 481
75.0 7050
Processing waveform 234 of 481
75.0 3675
Processing waveform 235 of 481
75.0 3225
Processing waveform 236 of 481
75.0 2325
Processing waveform 237 of 481
75.0 7500
Processing waveform 238 of 481
75.0 3975
Processing waveform 239 of 481
75.0 6150
Processing waveform 240 of 481
75.0 7875
Processing waveform 241 of 481
75.0 7275
Processing waveform 242 of 481
75.0 6075
Processing waveform 243 of 481
75.0 3900
Processing waveform 244 of 481
75.0 3000
Processing waveform 245 of 481
75.0 3900
Processing waveform 246 of 481
75.0 6525
Processing waveform 247 of 481
75.0 2925
Processing waveform 248 of 481
75.0 2700
Processing waveform 249 of 481
75.0 2250
Processing waveform 250 of 481
75.0 4425
Processing waveform 251 of 481
75.0 2250
Processing waveform 252 of 481
75.0 9600
Processing waveform 253 of 481
75.0 4650
Processing waveform 254 of 481
75.0 6750
Processing waveform 255 of 481
75.0 7125
Processing waveform 256 of 481
75.0 3000
Processing waveform 257 of 481
75.0 10200
Processing waveform 258 of 481
75.0 1800
Processing waveform 259 of 481
75.0 4425
Processing waveform 260 of 481
75.0 2250
Processing waveform 261 of 481
75.0 6150
Processing waveform 262 of 481
75.0 7125
Processing waveform 263 of 481
75.0 3600
Processing waveform 264 of 481
75.0 6000
Processing waveform 265 of 481
75.0 4800
Processing waveform 266 of 481
75.0 7200
Processing waveform 267 of 481
75.0 3000
Processing waveform 268 of 481
75.0 4500
Processing waveform 269 of 481
75.0 3600
Processing waveform 270 of 481
75.0 4875
Processing waveform 271 of 481
75.0 5625
Processing waveform 272 of 481
75.0 3150

Processing waveform 273 of 481
75.0 3975
Processing waveform 274 of 481
75.0 5325
Processing waveform 275 of 481
75.0 2850
Processing waveform 276 of 481
75.0 9450
Processing waveform 277 of 481
75.0 3225
Processing waveform 278 of 481
75.0 2100
Processing waveform 279 of 481
75.0 2700
Processing waveform 280 of 481
75.0 2325
Processing waveform 281 of 481
75.0 2775
Processing waveform 282 of 481
75.0 2625
Processing waveform 283 of 481
75.0 5850
Processing waveform 284 of 481
75.0 3825
Processing waveform 285 of 481
75.0 6000
Processing waveform 286 of 481
75.0 2025
Processing waveform 287 of 481
75.0 1725
Processing waveform 288 of 481
75.0 2175
Processing waveform 289 of 481
75.0 5400
Processing waveform 290 of 481
75.0 5325
Processing waveform 291 of 481
75.0 3750
Processing waveform 292 of 481
75.0 3075
Processing waveform 293 of 481
75.0 5550
Processing waveform 294 of 481
75.0 7350
Processing waveform 295 of 481
75.0 4650
Processing waveform 296 of 481
75.0 2775
Processing waveform 297 of 481
75.0 2325
Processing waveform 298 of 481
75.0 2925
Processing waveform 299 of 481
75.0 3075
Processing waveform 300 of 481
75.0 6375
Processing waveform 301 of 481
75.0 2475
Processing waveform 302 of 481
75.0 3975
Processing waveform 303 of 481
75.0 3150
Processing waveform 304 of 481
75.0 5925
Processing waveform 305 of 481
75.0 5400
Processing waveform 306 of 481
75.0 1875
Processing waveform 307 of 481
75.0 5850
Processing waveform 308 of 481
75.0 1725
Processing waveform 309 of 481
75.0 7650
Processing waveform 310 of 481
75.0 2700
Processing waveform 311 of 481
75.0 9075
Processing waveform 312 of 481
75.0 6975
Processing waveform 313 of 481
75.0 4050
Processing waveform 314 of 481
75.0 3450
Processing waveform 315 of 481
75.0 5925
Processing waveform 316 of 481
75.0 2250
Processing waveform 317 of 481
75.0 3675
Processing waveform 318 of 481
75.0 2625
Processing waveform 319 of 481
75.0 4575
Processing waveform 320 of 481
75.0 2025
Processing waveform 321 of 481
75.0 5175
Processing waveform 322 of 481
75.0 3225
Processing waveform 323 of 481
75.0 4950
Processing waveform 324 of 481
75.0 2925
Processing waveform 325 of 481
75.0 1800
Processing waveform 326 of 481
75.0 8925
Processing waveform 327 of 481
75.0 7350
Processing waveform 328 of 481
75.0 4125
Processing waveform 329 of 481
75.0 3225
Processing waveform 330 of 481
75.0 9075
Processing waveform 331 of 481
75.0 5025
Processing waveform 332 of 481
75.0 8925
Processing waveform 333 of 481
75.0 4800
Processing waveform 334 of 481
75.0 7800
Processing waveform 335 of 481
75.0 2775
Processing waveform 336 of 481
75.0 6600
Processing waveform 337 of 481
75.0 1800
Processing waveform 338 of 481
75.0 2625
Processing waveform 339 of 481
75.0 9075
Processing waveform 340 of 481
75.0 7200
Processing waveform 341 of 481
75.0 5550
Processing waveform 342 of 481
75.0 2775
Processing waveform 343 of 481
75.0 5250
Processing waveform 344 of 481
75.0 1875
Processing waveform 345 of 481
75.0 1875
Processing waveform 346 of 481
75.0 5025
Processing waveform 347 of 481
75.0 3600
Processing waveform 348 of 481
75.0 3375
Processing waveform 349 of 481
75.0 4050
Processing waveform 350 of 481
75.0 3375
Processing waveform 351 of 481
75.0 2925
Processing waveform 352 of 481
75.0 6675
Processing waveform 353 of 481
75.0 3750
Processing waveform 354 of 481
75.0 4575
Processing waveform 355 of 481
75.0 9075
Processing waveform 356 of 481
75.0 3900
Processing waveform 357 of 481
75.0 9075
Processing waveform 358 of 481
75.0 4875
Processing waveform 359 of 481
75.0 3075
Processing waveform 360 of 481
75.0 3150
Processing waveform 361 of 481
75.0 9075
Processing waveform 362 of 481
75.0 7425
Processing waveform 363 of 481
75.0 5025
Processing waveform 364 of 481
75.0 2850
Processing waveform 365 of 481
75.0 1875
Processing waveform 366 of 481
75.0 7125
Processing waveform 367 of 481
75.0 4500
Processing waveform 368 of 481
75.0 6075
Processing waveform 369 of 481
75.0 2475
Processing waveform 370 of 481
75.0 4125
Processing waveform 371 of 481
75.0 2100
Processing waveform 372 of 481
75.0 3975
Processing waveform 373 of 481
75.0 3225
Processing waveform 374 of 481
75.0 2550
Processing waveform 375 of 481
75.0 6975
Processing waveform 376 of 481
75.0 3075
Processing waveform 377 of 481
75.0 3675
Processing waveform 378 of 481
75.0 4800
Processing waveform 379 of 481
75.0 5850
Processing waveform 380 of 481
75.0 3675
Processing waveform 381 of 481
75.0 2250
Processing waveform 382 of 481
75.0 3600
Processing waveform 383 of 481
75.0 5775
Processing waveform 384 of 481
75.0 2325
Processing waveform 385 of 481
75.0 2325
Processing waveform 386 of 481
75.0 3525
Processing waveform 387 of 481
75.0 4500
Processing waveform 388 of 481
75.0 4275
Processing waveform 389 of 481
75.0 3450
Processing waveform 390 of 481
75.0 2550
Processing waveform 391 of 481
75.0 3300
Processing waveform 392 of 481
75.0 9075
Processing waveform 393 of 481
75.0 9075
Processing waveform 394 of 481
75.0 6525
Processing waveform 395 of 481
75.0 6975
Processing waveform 396 of 481
75.0 6750
Processing waveform 397 of 481
75.0 7275
Processing waveform 398 of 481
75.0 2400
Processing waveform 399 of 481
75.0 4125
Processing waveform 400 of 481
75.0 3525
Processing waveform 401 of 481
75.0 5175
Processing waveform 402 of 481
75.0 3750
Processing waveform 403 of 481
75.0 4875
Processing waveform 404 of 481
75.0 2250
Processing waveform 405 of 481
75.0 4125
Processing waveform 406 of 481
75.0 4425
Processing waveform 407 of 481
75.0 5250
Processing waveform 408 of 481
75.0 9075
Processing waveform 409 of 481
75.0 4725
Processing waveform 410 of 481
75.0 2475
Processing waveform 411 of 481
75.0 2925
Processing waveform 412 of 481
75.0 7350
Processing waveform 413 of 481
75.0 2550
Processing waveform 414 of 481
75.0 2850
Processing waveform 415 of 481
75.0 2550
Processing waveform 416 of 481
75.0 4425
Processing waveform 417 of 481
75.0 5775
Processing waveform 418 of 481
75.0 4200
Processing waveform 419 of 481
75.0 2175
Processing waveform 420 of 481
75.0 2925
Processing waveform 421 of 481
75.0 10350
Processing waveform 422 of 481
75.0 15600
Processing waveform 423 of 481
75.0 6450
Processing waveform 424 of 481
75.0 3450
Processing waveform 425 of 481
75.0 11175
Processing waveform 426 of 481
75.0 4725
Processing waveform 427 of 481
75.0 4200
Processing waveform 428 of 481
75.0 3750
Processing waveform 429 of 481
75.0 3975
Processing waveform 430 of 481
75.0 4575
Processing waveform 431 of 481
75.0 3225
Processing waveform 432 of 481
75.0 13500
Processing waveform 433 of 481
75.0 4050
Processing waveform 434 of 481
75.0 5250
Processing waveform 435 of 481
75.0 7050
Processing waveform 436 of 481
75.0 3450
Processing waveform 437 of 481
75.0 6675
Processing waveform 438 of 481
75.0 2850
Processing waveform 439 of 481
75.0 6750
Processing waveform 440 of 481
75.0 2475
Processing waveform 441 of 481
75.0 7650
Processing waveform 442 of 481
75.0 6750
Processing waveform 443 of 481
75.0 4125
Processing waveform 444 of 481
75.0 7425
Processing waveform 445 of 481
75.0 5400
Processing waveform 446 of 481
75.0 4800
Processing waveform 447 of 481
75.0 10200
Processing waveform 448 of 481
75.0 18225
Processing waveform 449 of 481
75.0 5925
Processing waveform 450 of 481
75.0 3525
Processing waveform 451 of 481
75.0 3300
Processing waveform 452 of 481
75.0 3600
Processing waveform 453 of 481
75.0 4650
Processing waveform 454 of 481
75.0 7875
Processing waveform 455 of 481
75.0 6300
Processing waveform 456 of 481
75.0 6750
Processing waveform 457 of 481
75.0 2925
Processing waveform 458 of 481
75.0 3525
Processing waveform 459 of 481
75.0 7875
Processing waveform 460 of 481
75.0 8925
Processing waveform 461 of 481
75.0 2850
Processing waveform 462 of 481
75.0 2475
Processing waveform 463 of 481
75.0 2325
Processing waveform 464 of 481
75.0 2250
Processing waveform 465 of 481
75.0 2700
Processing waveform 466 of 481
75.0 3000
Processing waveform 467 of 481
75.0 3300
Processing waveform 468 of 481
75.0 4425
Processing waveform 469 of 481
75.0 4500
Processing waveform 470 of 481
75.0 7500
Processing waveform 471 of 481
75.0 4200
Processing waveform 472 of 481
75.0 4800

Processing waveform 473 of 481
75.0 3000
Processing waveform 474 of 481
75.0 1950
Processing waveform 475 of 481
75.0 2625
Processing waveform 476 of 481
75.0 4650
Processing waveform 477 of 481
75.0 5475
Processing waveform 478 of 481
75.0 4950
Processing waveform 479 of 481
75.0 6375
Processing waveform 480 of 481
75.0 7500
Training data have been read and features have been extracted  (481, 120)
Computation time:  652.2428295612335
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.6890015602111816
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    87                         
        h         104                   
        l                97             
        r                      93       
        t                           100 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  24.1217150761  +/-  2.59069480282  %
          Predicted class
              e     h     l     r     t 
        e     8    11     8     7    10 
        h     8    18     8     9     9 
        l     8    13    11     7    11 
        r     7    14     8     9    10 
        t     8    11    11     7    13 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['l', 't'] 3
class
l    90
t    98
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 188
75.0 9075
Processing waveform 1 of 188
75.0 2175
Processing waveform 2 of 188
75.0 3300
Processing waveform 3 of 188
75.0 2700
Processing waveform 4 of 188
75.0 5250
Processing waveform 5 of 188
75.0 3000
Processing waveform 6 of 188
75.0 2775
Processing waveform 7 of 188
75.0 6450
Processing waveform 8 of 188
75.0 3150
Processing waveform 9 of 188
75.0 1650
Processing waveform 10 of 188
75.0 4500
Processing waveform 11 of 188
75.0 2250
Processing waveform 12 of 188
75.0 2175
Processing waveform 13 of 188
75.0 3225
Processing waveform 14 of 188
75.0 4050
Processing waveform 15 of 188
75.0 3300
Processing waveform 16 of 188
75.0 2850
Processing waveform 17 of 188
75.0 2025
Processing waveform 18 of 188
75.0 4800
Processing waveform 19 of 188
75.0 2325
Processing waveform 20 of 188
75.0 6150
Processing waveform 21 of 188
75.0 2775
Processing waveform 22 of 188
75.0 2850
Processing waveform 23 of 188
75.0 4425
Processing waveform 24 of 188
75.0 2250
Processing waveform 25 of 188
75.0 3075
Processing waveform 26 of 188
75.0 2775
Processing waveform 27 of 188
75.0 1875
Processing waveform 28 of 188
75.0 3300
Processing waveform 29 of 188
75.0 1875
Processing waveform 30 of 188
75.0 5550
Processing waveform 31 of 188
75.0 9075
Processing waveform 32 of 188
75.0 6525
Processing waveform 33 of 188
75.0 9075
Processing waveform 34 of 188
75.0 3225
Processing waveform 35 of 188
75.0 3150
Processing waveform 36 of 188
75.0 4275
Processing waveform 37 of 188
75.0 4050
Processing waveform 38 of 188
75.0 5400
Processing waveform 39 of 188
75.0 4725
Processing waveform 40 of 188
75.0 3450
Processing waveform 41 of 188
75.0 6150
Processing waveform 42 of 188
75.0 2250
Processing waveform 43 of 188
75.0 9000
Processing waveform 44 of 188
75.0 3150
Processing waveform 45 of 188
75.0 4950
Processing waveform 46 of 188
75.0 6750
Processing waveform 47 of 188
75.0 3150
Processing waveform 48 of 188
75.0 7050
Processing waveform 49 of 188
75.0 3675
Processing waveform 50 of 188
75.0 3225
Processing waveform 51 of 188
75.0 2325
Processing waveform 52 of 188
75.0 3975
Processing waveform 53 of 188
75.0 6150
Processing waveform 54 of 188
75.0 7875
Processing waveform 55 of 188
75.0 7275
Processing waveform 56 of 188
75.0 6075
Processing waveform 57 of 188
75.0 3900
Processing waveform 58 of 188
75.0 3000
Processing waveform 59 of 188
75.0 3900
Processing waveform 60 of 188
75.0 6525
Processing waveform 61 of 188
75.0 2925
Processing waveform 62 of 188
75.0 2700
Processing waveform 63 of 188
75.0 2250
Processing waveform 64 of 188
75.0 2250
Processing waveform 65 of 188
75.0 9600
Processing waveform 66 of 188
75.0 4650
Processing waveform 67 of 188
75.0 6750
Processing waveform 68 of 188
75.0 7125
Processing waveform 69 of 188
75.0 3000
Processing waveform 70 of 188
75.0 10200
Processing waveform 71 of 188
75.0 1800
Processing waveform 72 of 188
75.0 4425
Processing waveform 73 of 188
75.0 2250
Processing waveform 74 of 188
75.0 6150
Processing waveform 75 of 188
75.0 7125
Processing waveform 76 of 188
75.0 3600
Processing waveform 77 of 188
75.0 6000
Processing waveform 78 of 188
75.0 4800
Processing waveform 79 of 188
75.0 7200
Processing waveform 80 of 188
75.0 3000
Processing waveform 81 of 188
75.0 4500
Processing waveform 82 of 188
75.0 3600
Processing waveform 83 of 188
75.0 4875
Processing waveform 84 of 188
75.0 5625
Processing waveform 85 of 188
75.0 3150
Processing waveform 86 of 188
75.0 3975
Processing waveform 87 of 188
75.0 5325
Processing waveform 88 of 188
75.0 2850
Processing waveform 89 of 188
75.0 9450
Processing waveform 90 of 188
75.0 2250
Processing waveform 91 of 188
75.0 3600
Processing waveform 92 of 188
75.0 5775
Processing waveform 93 of 188
75.0 2325
Processing waveform 94 of 188
75.0 2325
Processing waveform 95 of 188
75.0 3525
Processing waveform 96 of 188
75.0 4500
Processing waveform 97 of 188
75.0 4275
Processing waveform 98 of 188
75.0 3450
Processing waveform 99 of 188
75.0 2550
Processing waveform 100 of 188
75.0 3300
Processing waveform 101 of 188
75.0 9075
Processing waveform 102 of 188
75.0 9075
Processing waveform 103 of 188
75.0 6525
Processing waveform 104 of 188
75.0 6975
Processing waveform 105 of 188
75.0 6750
Processing waveform 106 of 188
75.0 7275
Processing waveform 107 of 188
75.0 2400
Processing waveform 108 of 188
75.0 4125
Processing waveform 109 of 188
75.0 3525
Processing waveform 110 of 188
75.0 5175
Processing waveform 111 of 188
75.0 3750
Processing waveform 112 of 188
75.0 4875
Processing waveform 113 of 188
75.0 2250
Processing waveform 114 of 188
75.0 4125
Processing waveform 115 of 188
75.0 4425
Processing waveform 116 of 188
75.0 5250
Processing waveform 117 of 188
75.0 9075
Processing waveform 118 of 188
75.0 4725
Processing waveform 119 of 188
75.0 2925
Processing waveform 120 of 188
75.0 7350
Processing waveform 121 of 188
75.0 2550
Processing waveform 122 of 188
75.0 2850
Processing waveform 123 of 188
75.0 2550
Processing waveform 124 of 188
75.0 4425
Processing waveform 125 of 188
75.0 5775
Processing waveform 126 of 188
75.0 4200
Processing waveform 127 of 188
75.0 2175
Processing waveform 128 of 188
75.0 10350
Processing waveform 129 of 188
75.0 15600
Processing waveform 130 of 188
75.0 6450
Processing waveform 131 of 188
75.0 3450
Processing waveform 132 of 188
75.0 11175
Processing waveform 133 of 188
75.0 4725
Processing waveform 134 of 188
75.0 4200
Processing waveform 135 of 188
75.0 3750
Processing waveform 136 of 188
75.0 3975
Processing waveform 137 of 188
75.0 4575
Processing waveform 138 of 188
75.0 3225
Processing waveform 139 of 188
75.0 13500
Processing waveform 140 of 188
75.0 4050
Processing waveform 141 of 188
75.0 5250
Processing waveform 142 of 188
75.0 7050
Processing waveform 143 of 188
75.0 3450
Processing waveform 144 of 188
75.0 6675
Processing waveform 145 of 188
75.0 2850
Processing waveform 146 of 188
75.0 6750
Processing waveform 147 of 188
75.0 2475
Processing waveform 148 of 188
75.0 7650
Processing waveform 149 of 188
75.0 6750
Processing waveform 150 of 188
75.0 4125
Processing waveform 151 of 188
75.0 7425
Processing waveform 152 of 188
75.0 5400
Processing waveform 153 of 188
75.0 4800
Processing waveform 154 of 188
75.0 10200
Processing waveform 155 of 188
75.0 18225
Processing waveform 156 of 188
75.0 5925
Processing waveform 157 of 188
75.0 3525
Processing waveform 158 of 188
75.0 3300
Processing waveform 159 of 188
75.0 3600
Processing waveform 160 of 188
75.0 4650
Processing waveform 161 of 188
75.0 7875
Processing waveform 162 of 188
75.0 6300

Processing waveform 163 of 188
75.0 6750
Processing waveform 164 of 188
75.0 2925
Processing waveform 165 of 188
75.0 3525
Processing waveform 166 of 188
75.0 7875
Processing waveform 167 of 188
75.0 8925
Processing waveform 168 of 188
75.0 2850
Processing waveform 169 of 188
75.0 2475
Processing waveform 170 of 188
75.0 2325
Processing waveform 171 of 188
75.0 2250
Processing waveform 172 of 188
75.0 2700
Processing waveform 173 of 188
75.0 3000
Processing waveform 174 of 188
75.0 3300
Processing waveform 175 of 188
75.0 4425
Processing waveform 176 of 188
75.0 4500
Processing waveform 177 of 188
75.0 7500
Processing waveform 178 of 188
75.0 4200
Processing waveform 179 of 188
75.0 4800
Processing waveform 180 of 188
75.0 3000
Processing waveform 181 of 188
75.0 1950
Processing waveform 182 of 188
75.0 2625
Processing waveform 183 of 188
75.0 4650
Processing waveform 184 of 188
75.0 5475
Processing waveform 185 of 188
75.0 4950
Processing waveform 186 of 188
75.0 6375
Processing waveform 187 of 188
75.0 7500
Training data have been read and features have been extracted  (188, 120)
Computation time:  263.4754798412323
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.1524205207824707
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l    90       
        t          98 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  53.1914893617  +/-  4.57158081404  %
          Predicted class
              l     t 
        l    21    24 
        t    20    29 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'r'] 3
class
e    83
r    82
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 165
75.0 5325
Processing waveform 1 of 165
75.0 2700
Processing waveform 2 of 165
75.0 4350
Processing waveform 3 of 165
75.0 2775
Processing waveform 4 of 165
75.0 2250
Processing waveform 5 of 165
75.0 2475
Processing waveform 6 of 165
75.0 3300
Processing waveform 7 of 165
75.0 9075
Processing waveform 8 of 165
75.0 4125
Processing waveform 9 of 165
75.0 3300
Processing waveform 10 of 165
75.0 3225
Processing waveform 11 of 165
75.0 2850
Processing waveform 12 of 165
75.0 2400
Processing waveform 13 of 165
75.0 3900
Processing waveform 14 of 165
75.0 2775
Processing waveform 15 of 165
75.0 2325
Processing waveform 16 of 165
75.0 4425
Processing waveform 17 of 165
75.0 3525
Processing waveform 18 of 165
75.0 6300
Processing waveform 19 of 165
75.0 2625
Processing waveform 20 of 165
75.0 2400
Processing waveform 21 of 165
75.0 2250
Processing waveform 22 of 165
75.0 3750
Processing waveform 23 of 165
75.0 3525
Processing waveform 24 of 165
75.0 5700
Processing waveform 25 of 165
75.0 2700
Processing waveform 26 of 165
75.0 4350
Processing waveform 27 of 165
75.0 4950
Processing waveform 28 of 165
75.0 2925
Processing waveform 29 of 165
75.0 2775
Processing waveform 30 of 165
75.0 3450
Processing waveform 31 of 165
75.0 4875
Processing waveform 32 of 165
75.0 9075
Processing waveform 33 of 165
75.0 5175
Processing waveform 34 of 165
75.0 3075
Processing waveform 35 of 165
75.0 4125
Processing waveform 36 of 165
75.0 9075
Processing waveform 37 of 165
75.0 2775
Processing waveform 38 of 165
75.0 4575
Processing waveform 39 of 165
75.0 2700
Processing waveform 40 of 165
75.0 1950
Processing waveform 41 of 165
75.0 2100
Processing waveform 42 of 165
75.0 4575
Processing waveform 43 of 165
75.0 5850
Processing waveform 44 of 165
75.0 3150
Processing waveform 45 of 165
75.0 4275
Processing waveform 46 of 165
75.0 9075
Processing waveform 47 of 165
75.0 3600
Processing waveform 48 of 165
75.0 2625
Processing waveform 49 of 165
75.0 2700
Processing waveform 50 of 165
75.0 2700
Processing waveform 51 of 165
75.0 3525
Processing waveform 52 of 165
75.0 4200
Processing waveform 53 of 165
75.0 4200
Processing waveform 54 of 165
75.0 1950
Processing waveform 55 of 165
75.0 5775
Processing waveform 56 of 165
75.0 9075
Processing waveform 57 of 165
75.0 4875
Processing waveform 58 of 165
75.0 14625
Processing waveform 59 of 165
75.0 1950
Processing waveform 60 of 165
75.0 3975
Processing waveform 61 of 165
75.0 2550
Processing waveform 62 of 165
75.0 7500
Processing waveform 63 of 165
75.0 14925
Processing waveform 64 of 165
75.0 13275
Processing waveform 65 of 165
75.0 1725
Processing waveform 66 of 165
75.0 2400
Processing waveform 67 of 165
75.0 6375
Processing waveform 68 of 165
75.0 4125
Processing waveform 69 of 165
75.0 4800
Processing waveform 70 of 165
75.0 7125
Processing waveform 71 of 165
75.0 5400
Processing waveform 72 of 165
75.0 6825
Processing waveform 73 of 165
75.0 7050
Processing waveform 74 of 165
75.0 4500
Processing waveform 75 of 165
75.0 5625
Processing waveform 76 of 165
75.0 4275
Processing waveform 77 of 165
75.0 3150
Processing waveform 78 of 165
75.0 3150
Processing waveform 79 of 165
75.0 12600
Processing waveform 80 of 165
75.0 16725
Processing waveform 81 of 165
75.0 6075
Processing waveform 82 of 165
75.0 3450
Processing waveform 83 of 165
75.0 2925
Processing waveform 84 of 165
75.0 4575
Processing waveform 85 of 165
75.0 7050
Processing waveform 86 of 165
75.0 2025
Processing waveform 87 of 165
75.0 2925
Processing waveform 88 of 165
75.0 2850
Processing waveform 89 of 165
75.0 3600
Processing waveform 90 of 165
75.0 3750
Processing waveform 91 of 165
75.0 2100
Processing waveform 92 of 165
75.0 1650
Processing waveform 93 of 165
75.0 4200
Processing waveform 94 of 165
75.0 9075
Processing waveform 95 of 165
75.0 4125
Processing waveform 96 of 165
75.0 2925
Processing waveform 97 of 165
75.0 2250
Processing waveform 98 of 165
75.0 5475
Processing waveform 99 of 165
75.0 2625
Processing waveform 100 of 165
75.0 2400
Processing waveform 101 of 165
75.0 2700
Processing waveform 102 of 165
75.0 9075
Processing waveform 103 of 165
75.0 3300
Processing waveform 104 of 165
75.0 9075
Processing waveform 105 of 165
75.0 6225
Processing waveform 106 of 165
75.0 7125
Processing waveform 107 of 165
75.0 5850
Processing waveform 108 of 165
75.0 9075
Processing waveform 109 of 165
75.0 3225
Processing waveform 110 of 165
75.0 8100
Processing waveform 111 of 165
75.0 4575
Processing waveform 112 of 165
75.0 2850
Processing waveform 113 of 165
75.0 3375
Processing waveform 114 of 165
75.0 2100
Processing waveform 115 of 165
75.0 9075
Processing waveform 116 of 165
75.0 5850
Processing waveform 117 of 165
75.0 2775
Processing waveform 118 of 165
75.0 2700
Processing waveform 119 of 165
75.0 2100
Processing waveform 120 of 165
75.0 9075
Processing waveform 121 of 165
75.0 3225
Processing waveform 122 of 165
75.0 3300
Processing waveform 123 of 165
75.0 2475
Processing waveform 124 of 165
75.0 3075
Processing waveform 125 of 165
75.0 6450
Processing waveform 126 of 165
75.0 3600
Processing waveform 127 of 165
75.0 4650
Processing waveform 128 of 165
75.0 2475
Processing waveform 129 of 165
75.0 1875
Processing waveform 130 of 165
75.0 3450
Processing waveform 131 of 165
75.0 1500
Processing waveform 132 of 165
75.0 2625
Processing waveform 133 of 165
75.0 9075
Processing waveform 134 of 165
75.0 3525
Processing waveform 135 of 165
75.0 5625
Processing waveform 136 of 165
75.0 3675
Processing waveform 137 of 165
75.0 9075
Processing waveform 138 of 165
75.0 2700
Processing waveform 139 of 165
75.0 3000
Processing waveform 140 of 165
75.0 3525
Processing waveform 141 of 165
75.0 7200
Processing waveform 142 of 165
75.0 4875
Processing waveform 143 of 165
75.0 6075
Processing waveform 144 of 165
75.0 2700
Processing waveform 145 of 165
75.0 5250
Processing waveform 146 of 165
75.0 12750
Processing waveform 147 of 165
75.0 5100
Processing waveform 148 of 165
75.0 2475
Processing waveform 149 of 165
75.0 6525
Processing waveform 150 of 165
75.0 7200
Processing waveform 151 of 165
75.0 4950
Processing waveform 152 of 165
75.0 8250
Processing waveform 153 of 165
75.0 3300
Processing waveform 154 of 165
75.0 2325

Processing waveform 155 of 165
75.0 13500
Processing waveform 156 of 165
75.0 3525
Processing waveform 157 of 165
75.0 3075
Processing waveform 158 of 165
75.0 6600
Processing waveform 159 of 165
75.0 6225
Processing waveform 160 of 165
75.0 6975
Processing waveform 161 of 165
75.0 4425
Processing waveform 162 of 165
75.0 13350
Processing waveform 163 of 165
75.0 9525
Processing waveform 164 of 165
75.0 2325
Training data have been read and features have been extracted  (165, 120)
Computation time:  248.4623212814331
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.1412677764892578
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    83       
        r          82 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  51.8072289157  +/-  4.63503230402  %
          Predicted class
              e     r 
        e    21    21 
        r    19    22 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't'] 3
class
h    104
l     90
t     98
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 292
75.0 9075
Processing waveform 1 of 292
75.0 2175
Processing waveform 2 of 292
75.0 3300
Processing waveform 3 of 292
75.0 2700
Processing waveform 4 of 292
75.0 5250
Processing waveform 5 of 292
75.0 3000
Processing waveform 6 of 292
75.0 2775
Processing waveform 7 of 292
75.0 6450
Processing waveform 8 of 292
75.0 3150
Processing waveform 9 of 292
75.0 1650
Processing waveform 10 of 292
75.0 4500
Processing waveform 11 of 292
75.0 2250
Processing waveform 12 of 292
75.0 2175
Processing waveform 13 of 292
75.0 3225
Processing waveform 14 of 292
75.0 4050
Processing waveform 15 of 292
75.0 3300
Processing waveform 16 of 292
75.0 2850
Processing waveform 17 of 292
75.0 2025
Processing waveform 18 of 292
75.0 4800
Processing waveform 19 of 292
75.0 2325
Processing waveform 20 of 292
75.0 6150
Processing waveform 21 of 292
75.0 2775
Processing waveform 22 of 292
75.0 2850
Processing waveform 23 of 292
75.0 4425
Processing waveform 24 of 292
75.0 2250
Processing waveform 25 of 292
75.0 3075
Processing waveform 26 of 292
75.0 2775
Processing waveform 27 of 292
75.0 1875
Processing waveform 28 of 292
75.0 3300
Processing waveform 29 of 292
75.0 1875
Processing waveform 30 of 292
75.0 5550
Processing waveform 31 of 292
75.0 9075
Processing waveform 32 of 292
75.0 6525
Processing waveform 33 of 292
75.0 9075
Processing waveform 34 of 292
75.0 3225
Processing waveform 35 of 292
75.0 3150
Processing waveform 36 of 292
75.0 4275
Processing waveform 37 of 292
75.0 4050
Processing waveform 38 of 292
75.0 5400
Processing waveform 39 of 292
75.0 4725
Processing waveform 40 of 292
75.0 3450
Processing waveform 41 of 292
75.0 6150
Processing waveform 42 of 292
75.0 2250
Processing waveform 43 of 292
75.0 9000
Processing waveform 44 of 292
75.0 3150
Processing waveform 45 of 292
75.0 4950
Processing waveform 46 of 292
75.0 6750
Processing waveform 47 of 292
75.0 3150
Processing waveform 48 of 292
75.0 7050
Processing waveform 49 of 292
75.0 3675
Processing waveform 50 of 292
75.0 3225
Processing waveform 51 of 292
75.0 2325
Processing waveform 52 of 292
75.0 3975
Processing waveform 53 of 292
75.0 6150
Processing waveform 54 of 292
75.0 7875
Processing waveform 55 of 292
75.0 7275
Processing waveform 56 of 292
75.0 6075
Processing waveform 57 of 292
75.0 3900
Processing waveform 58 of 292
75.0 3000
Processing waveform 59 of 292
75.0 3900
Processing waveform 60 of 292
75.0 6525
Processing waveform 61 of 292
75.0 2925
Processing waveform 62 of 292
75.0 2700
Processing waveform 63 of 292
75.0 2250
Processing waveform 64 of 292
75.0 2250
Processing waveform 65 of 292
75.0 9600
Processing waveform 66 of 292
75.0 4650
Processing waveform 67 of 292
75.0 6750
Processing waveform 68 of 292
75.0 7125
Processing waveform 69 of 292
75.0 3000
Processing waveform 70 of 292
75.0 10200
Processing waveform 71 of 292
75.0 1800
Processing waveform 72 of 292
75.0 4425
Processing waveform 73 of 292
75.0 2250
Processing waveform 74 of 292
75.0 6150
Processing waveform 75 of 292
75.0 7125
Processing waveform 76 of 292
75.0 3600
Processing waveform 77 of 292
75.0 6000
Processing waveform 78 of 292
75.0 4800
Processing waveform 79 of 292
75.0 7200
Processing waveform 80 of 292
75.0 3000
Processing waveform 81 of 292
75.0 4500
Processing waveform 82 of 292
75.0 3600
Processing waveform 83 of 292
75.0 4875
Processing waveform 84 of 292
75.0 5625
Processing waveform 85 of 292
75.0 3150
Processing waveform 86 of 292
75.0 3975
Processing waveform 87 of 292
75.0 5325
Processing waveform 88 of 292
75.0 2850
Processing waveform 89 of 292
75.0 9450
Processing waveform 90 of 292
75.0 3225
Processing waveform 91 of 292
75.0 2100
Processing waveform 92 of 292
75.0 2700
Processing waveform 93 of 292
75.0 2325
Processing waveform 94 of 292
75.0 2775
Processing waveform 95 of 292
75.0 2625
Processing waveform 96 of 292
75.0 5850
Processing waveform 97 of 292
75.0 3825
Processing waveform 98 of 292
75.0 6000
Processing waveform 99 of 292
75.0 2025
Processing waveform 100 of 292
75.0 1725
Processing waveform 101 of 292
75.0 2175
Processing waveform 102 of 292
75.0 5400
Processing waveform 103 of 292
75.0 5325
Processing waveform 104 of 292
75.0 3750
Processing waveform 105 of 292
75.0 3075
Processing waveform 106 of 292
75.0 5550
Processing waveform 107 of 292
75.0 7350
Processing waveform 108 of 292
75.0 4650
Processing waveform 109 of 292
75.0 2775
Processing waveform 110 of 292
75.0 2325
Processing waveform 111 of 292
75.0 2925
Processing waveform 112 of 292
75.0 3075
Processing waveform 113 of 292
75.0 6375
Processing waveform 114 of 292
75.0 2475
Processing waveform 115 of 292
75.0 3975
Processing waveform 116 of 292
75.0 3150
Processing waveform 117 of 292
75.0 5925
Processing waveform 118 of 292
75.0 5400
Processing waveform 119 of 292
75.0 1875
Processing waveform 120 of 292
75.0 5850
Processing waveform 121 of 292
75.0 1725
Processing waveform 122 of 292
75.0 7650
Processing waveform 123 of 292
75.0 2700
Processing waveform 124 of 292
75.0 9075
Processing waveform 125 of 292
75.0 6975
Processing waveform 126 of 292
75.0 4050
Processing waveform 127 of 292
75.0 3450
Processing waveform 128 of 292
75.0 5925
Processing waveform 129 of 292
75.0 2250
Processing waveform 130 of 292
75.0 3675
Processing waveform 131 of 292
75.0 2625
Processing waveform 132 of 292
75.0 4575
Processing waveform 133 of 292
75.0 2025
Processing waveform 134 of 292
75.0 5175
Processing waveform 135 of 292
75.0 3225
Processing waveform 136 of 292
75.0 4950
Processing waveform 137 of 292
75.0 2925
Processing waveform 138 of 292
75.0 1800
Processing waveform 139 of 292
75.0 8925
Processing waveform 140 of 292
75.0 7350
Processing waveform 141 of 292
75.0 4125
Processing waveform 142 of 292
75.0 3225
Processing waveform 143 of 292
75.0 9075
Processing waveform 144 of 292
75.0 5025
Processing waveform 145 of 292
75.0 8925
Processing waveform 146 of 292
75.0 4800
Processing waveform 147 of 292
75.0 7800
Processing waveform 148 of 292
75.0 2775
Processing waveform 149 of 292
75.0 6600
Processing waveform 150 of 292
75.0 1800
Processing waveform 151 of 292
75.0 2625
Processing waveform 152 of 292
75.0 9075
Processing waveform 153 of 292
75.0 7200
Processing waveform 154 of 292
75.0 5550
Processing waveform 155 of 292
75.0 2775
Processing waveform 156 of 292
75.0 5250
Processing waveform 157 of 292
75.0 1875
Processing waveform 158 of 292
75.0 1875
Processing waveform 159 of 292
75.0 5025
Processing waveform 160 of 292
75.0 3600
Processing waveform 161 of 292
75.0 3375
Processing waveform 162 of 292
75.0 4050
Processing waveform 163 of 292
75.0 3375
Processing waveform 164 of 292
75.0 2925
Processing waveform 165 of 292
75.0 6675
Processing waveform 166 of 292
75.0 3750
Processing waveform 167 of 292
75.0 4575
Processing waveform 168 of 292
75.0 9075
Processing waveform 169 of 292
75.0 3900

Processing waveform 170 of 292
75.0 9075
Processing waveform 171 of 292
75.0 4875
Processing waveform 172 of 292
75.0 3075
Processing waveform 173 of 292
75.0 3150
Processing waveform 174 of 292
75.0 9075
Processing waveform 175 of 292
75.0 7425
Processing waveform 176 of 292
75.0 5025
Processing waveform 177 of 292
75.0 2850
Processing waveform 178 of 292
75.0 1875
Processing waveform 179 of 292
75.0 7125
Processing waveform 180 of 292
75.0 4500
Processing waveform 181 of 292
75.0 6075
Processing waveform 182 of 292
75.0 2475
Processing waveform 183 of 292
75.0 4125
Processing waveform 184 of 292
75.0 2100
Processing waveform 185 of 292
75.0 3975
Processing waveform 186 of 292
75.0 3225
Processing waveform 187 of 292
75.0 2550
Processing waveform 188 of 292
75.0 6975
Processing waveform 189 of 292
75.0 3075
Processing waveform 190 of 292
75.0 3675
Processing waveform 191 of 292
75.0 4800
Processing waveform 192 of 292
75.0 5850
Processing waveform 193 of 292
75.0 3675
Processing waveform 194 of 292
75.0 2250
Processing waveform 195 of 292
75.0 3600
Processing waveform 196 of 292
75.0 5775
Processing waveform 197 of 292
75.0 2325
Processing waveform 198 of 292
75.0 2325
Processing waveform 199 of 292
75.0 3525
Processing waveform 200 of 292
75.0 4500
Processing waveform 201 of 292
75.0 4275
Processing waveform 202 of 292
75.0 3450
Processing waveform 203 of 292
75.0 2550
Processing waveform 204 of 292
75.0 3300
Processing waveform 205 of 292
75.0 9075
Processing waveform 206 of 292
75.0 9075
Processing waveform 207 of 292
75.0 6525
Processing waveform 208 of 292
75.0 6975
Processing waveform 209 of 292
75.0 6750
Processing waveform 210 of 292
75.0 7275
Processing waveform 211 of 292
75.0 2400
Processing waveform 212 of 292
75.0 4125
Processing waveform 213 of 292
75.0 3525
Processing waveform 214 of 292
75.0 5175
Processing waveform 215 of 292
75.0 3750
Processing waveform 216 of 292
75.0 4875
Processing waveform 217 of 292
75.0 2250
Processing waveform 218 of 292
75.0 4125
Processing waveform 219 of 292
75.0 4425
Processing waveform 220 of 292
75.0 5250
Processing waveform 221 of 292
75.0 9075
Processing waveform 222 of 292
75.0 4725
Processing waveform 223 of 292
75.0 2925
Processing waveform 224 of 292
75.0 7350
Processing waveform 225 of 292
75.0 2550
Processing waveform 226 of 292
75.0 2850
Processing waveform 227 of 292
75.0 2550
Processing waveform 228 of 292
75.0 4425
Processing waveform 229 of 292
75.0 5775
Processing waveform 230 of 292
75.0 4200
Processing waveform 231 of 292
75.0 2175
Processing waveform 232 of 292
75.0 10350
Processing waveform 233 of 292
75.0 15600
Processing waveform 234 of 292
75.0 6450
Processing waveform 235 of 292
75.0 3450
Processing waveform 236 of 292
75.0 11175
Processing waveform 237 of 292
75.0 4725
Processing waveform 238 of 292
75.0 4200
Processing waveform 239 of 292
75.0 3750
Processing waveform 240 of 292
75.0 3975
Processing waveform 241 of 292
75.0 4575
Processing waveform 242 of 292
75.0 3225
Processing waveform 243 of 292
75.0 13500
Processing waveform 244 of 292
75.0 4050
Processing waveform 245 of 292
75.0 5250
Processing waveform 246 of 292
75.0 7050
Processing waveform 247 of 292
75.0 3450
Processing waveform 248 of 292
75.0 6675
Processing waveform 249 of 292
75.0 2850
Processing waveform 250 of 292
75.0 6750
Processing waveform 251 of 292
75.0 2475
Processing waveform 252 of 292
75.0 7650
Processing waveform 253 of 292
75.0 6750
Processing waveform 254 of 292
75.0 4125
Processing waveform 255 of 292
75.0 7425
Processing waveform 256 of 292
75.0 5400
Processing waveform 257 of 292
75.0 4800
Processing waveform 258 of 292
75.0 10200
Processing waveform 259 of 292
75.0 18225
Processing waveform 260 of 292
75.0 5925
Processing waveform 261 of 292
75.0 3525
Processing waveform 262 of 292
75.0 3300
Processing waveform 263 of 292
75.0 3600
Processing waveform 264 of 292
75.0 4650
Processing waveform 265 of 292
75.0 7875
Processing waveform 266 of 292
75.0 6300
Processing waveform 267 of 292
75.0 6750
Processing waveform 268 of 292
75.0 2925
Processing waveform 269 of 292
75.0 3525
Processing waveform 270 of 292
75.0 7875
Processing waveform 271 of 292
75.0 8925
Processing waveform 272 of 292
75.0 2850
Processing waveform 273 of 292
75.0 2475
Processing waveform 274 of 292
75.0 2325
Processing waveform 275 of 292
75.0 2250
Processing waveform 276 of 292
75.0 2700
Processing waveform 277 of 292
75.0 3000
Processing waveform 278 of 292
75.0 3300
Processing waveform 279 of 292
75.0 4425
Processing waveform 280 of 292
75.0 4500
Processing waveform 281 of 292
75.0 7500
Processing waveform 282 of 292
75.0 4200
Processing waveform 283 of 292
75.0 4800
Processing waveform 284 of 292
75.0 3000
Processing waveform 285 of 292
75.0 1950
Processing waveform 286 of 292
75.0 2625
Processing waveform 287 of 292
75.0 4650
Processing waveform 288 of 292
75.0 5475
Processing waveform 289 of 292
75.0 4950
Processing waveform 290 of 292
75.0 6375
Processing waveform 291 of 292
75.0 7500
Training data have been read and features have been extracted  (292, 120)
Computation time:  382.01826643943787
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.3002510070800781
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h   104             
        l          90       
        t                98 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  41.0502283105  +/-  3.16323356386  %
          Predicted class
              h     l     t 
        h    28    10    14 
        l    18    12    15 
        t    16    12    20 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['h', 'l', 't', 'r'] 3
class
h    104
l     90
r     82
t     98
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 374
75.0 5325
Processing waveform 1 of 374
75.0 2700
Processing waveform 2 of 374
75.0 4350
Processing waveform 3 of 374
75.0 2775
Processing waveform 4 of 374
75.0 2250
Processing waveform 5 of 374
75.0 2475
Processing waveform 6 of 374
75.0 3300
Processing waveform 7 of 374
75.0 9075
Processing waveform 8 of 374
75.0 4125
Processing waveform 9 of 374
75.0 3300
Processing waveform 10 of 374
75.0 3225
Processing waveform 11 of 374
75.0 2850
Processing waveform 12 of 374
75.0 2400
Processing waveform 13 of 374
75.0 3900
Processing waveform 14 of 374
75.0 2775
Processing waveform 15 of 374
75.0 2325
Processing waveform 16 of 374
75.0 4425
Processing waveform 17 of 374
75.0 3525
Processing waveform 18 of 374
75.0 6300
Processing waveform 19 of 374
75.0 2625
Processing waveform 20 of 374
75.0 2400
Processing waveform 21 of 374
75.0 2250
Processing waveform 22 of 374
75.0 3750
Processing waveform 23 of 374
75.0 3525
Processing waveform 24 of 374
75.0 5700
Processing waveform 25 of 374
75.0 2700
Processing waveform 26 of 374
75.0 4350
Processing waveform 27 of 374
75.0 4950
Processing waveform 28 of 374
75.0 2925
Processing waveform 29 of 374
75.0 2775
Processing waveform 30 of 374
75.0 3450
Processing waveform 31 of 374
75.0 4875
Processing waveform 32 of 374
75.0 9075
Processing waveform 33 of 374
75.0 5175
Processing waveform 34 of 374
75.0 3075
Processing waveform 35 of 374
75.0 4125
Processing waveform 36 of 374
75.0 9075
Processing waveform 37 of 374
75.0 2775
Processing waveform 38 of 374
75.0 4575
Processing waveform 39 of 374
75.0 2700
Processing waveform 40 of 374
75.0 1950
Processing waveform 41 of 374
75.0 2100
Processing waveform 42 of 374
75.0 4575
Processing waveform 43 of 374
75.0 5850
Processing waveform 44 of 374
75.0 3150
Processing waveform 45 of 374
75.0 4275
Processing waveform 46 of 374
75.0 9075
Processing waveform 47 of 374
75.0 3600
Processing waveform 48 of 374
75.0 2625
Processing waveform 49 of 374
75.0 2700
Processing waveform 50 of 374
75.0 2700
Processing waveform 51 of 374
75.0 3525
Processing waveform 52 of 374
75.0 4200
Processing waveform 53 of 374
75.0 4200

Processing waveform 54 of 374
75.0 1950
Processing waveform 55 of 374
75.0 5775
Processing waveform 56 of 374
75.0 9075
Processing waveform 57 of 374
75.0 4875
Processing waveform 58 of 374
75.0 14625
Processing waveform 59 of 374
75.0 1950
Processing waveform 60 of 374
75.0 3975
Processing waveform 61 of 374
75.0 2550
Processing waveform 62 of 374
75.0 7500
Processing waveform 63 of 374
75.0 14925
Processing waveform 64 of 374
75.0 13275
Processing waveform 65 of 374
75.0 1725
Processing waveform 66 of 374
75.0 2400
Processing waveform 67 of 374
75.0 6375
Processing waveform 68 of 374
75.0 4125
Processing waveform 69 of 374
75.0 4800
Processing waveform 70 of 374
75.0 7125
Processing waveform 71 of 374
75.0 5400
Processing waveform 72 of 374
75.0 6825
Processing waveform 73 of 374
75.0 7050
Processing waveform 74 of 374
75.0 4500
Processing waveform 75 of 374
75.0 5625
Processing waveform 76 of 374
75.0 4275
Processing waveform 77 of 374
75.0 3150
Processing waveform 78 of 374
75.0 3150
Processing waveform 79 of 374
75.0 12600
Processing waveform 80 of 374
75.0 16725
Processing waveform 81 of 374
75.0 6075
Processing waveform 82 of 374
75.0 9075
Processing waveform 83 of 374
75.0 2175
Processing waveform 84 of 374
75.0 3300
Processing waveform 85 of 374
75.0 2700
Processing waveform 86 of 374
75.0 5250
Processing waveform 87 of 374
75.0 3000
Processing waveform 88 of 374
75.0 2775
Processing waveform 89 of 374
75.0 6450
Processing waveform 90 of 374
75.0 3150
Processing waveform 91 of 374
75.0 1650
Processing waveform 92 of 374
75.0 4500
Processing waveform 93 of 374
75.0 2250
Processing waveform 94 of 374
75.0 2175
Processing waveform 95 of 374
75.0 3225
Processing waveform 96 of 374
75.0 4050
Processing waveform 97 of 374
75.0 3300
Processing waveform 98 of 374
75.0 2850
Processing waveform 99 of 374
75.0 2025
Processing waveform 100 of 374
75.0 4800
Processing waveform 101 of 374
75.0 2325
Processing waveform 102 of 374
75.0 6150
Processing waveform 103 of 374
75.0 2775
Processing waveform 104 of 374
75.0 2850
Processing waveform 105 of 374
75.0 4425
Processing waveform 106 of 374
75.0 2250
Processing waveform 107 of 374
75.0 3075
Processing waveform 108 of 374
75.0 2775
Processing waveform 109 of 374
75.0 1875
Processing waveform 110 of 374
75.0 3300
Processing waveform 111 of 374
75.0 1875
Processing waveform 112 of 374
75.0 5550
Processing waveform 113 of 374
75.0 9075
Processing waveform 114 of 374
75.0 6525
Processing waveform 115 of 374
75.0 9075
Processing waveform 116 of 374
75.0 3225
Processing waveform 117 of 374
75.0 3150
Processing waveform 118 of 374
75.0 4275
Processing waveform 119 of 374
75.0 4050
Processing waveform 120 of 374
75.0 5400
Processing waveform 121 of 374
75.0 4725
Processing waveform 122 of 374
75.0 3450
Processing waveform 123 of 374
75.0 6150
Processing waveform 124 of 374
75.0 2250
Processing waveform 125 of 374
75.0 9000
Processing waveform 126 of 374
75.0 3150
Processing waveform 127 of 374
75.0 4950
Processing waveform 128 of 374
75.0 6750
Processing waveform 129 of 374
75.0 3150
Processing waveform 130 of 374
75.0 7050
Processing waveform 131 of 374
75.0 3675
Processing waveform 132 of 374
75.0 3225
Processing waveform 133 of 374
75.0 2325
Processing waveform 134 of 374
75.0 3975
Processing waveform 135 of 374
75.0 6150
Processing waveform 136 of 374
75.0 7875
Processing waveform 137 of 374
75.0 7275
Processing waveform 138 of 374
75.0 6075
Processing waveform 139 of 374
75.0 3900
Processing waveform 140 of 374
75.0 3000
Processing waveform 141 of 374
75.0 3900
Processing waveform 142 of 374
75.0 6525
Processing waveform 143 of 374
75.0 2925
Processing waveform 144 of 374
75.0 2700
Processing waveform 145 of 374
75.0 2250
Processing waveform 146 of 374
75.0 2250
Processing waveform 147 of 374
75.0 9600
Processing waveform 148 of 374
75.0 4650
Processing waveform 149 of 374
75.0 6750
Processing waveform 150 of 374
75.0 7125
Processing waveform 151 of 374
75.0 3000
Processing waveform 152 of 374
75.0 10200
Processing waveform 153 of 374
75.0 1800
Processing waveform 154 of 374
75.0 4425
Processing waveform 155 of 374
75.0 2250
Processing waveform 156 of 374
75.0 6150
Processing waveform 157 of 374
75.0 7125
Processing waveform 158 of 374
75.0 3600
Processing waveform 159 of 374
75.0 6000
Processing waveform 160 of 374
75.0 4800
Processing waveform 161 of 374
75.0 7200
Processing waveform 162 of 374
75.0 3000
Processing waveform 163 of 374
75.0 4500
Processing waveform 164 of 374
75.0 3600
Processing waveform 165 of 374
75.0 4875
Processing waveform 166 of 374
75.0 5625
Processing waveform 167 of 374
75.0 3150
Processing waveform 168 of 374
75.0 3975
Processing waveform 169 of 374
75.0 5325
Processing waveform 170 of 374
75.0 2850
Processing waveform 171 of 374
75.0 9450
Processing waveform 172 of 374
75.0 3225
Processing waveform 173 of 374
75.0 2100
Processing waveform 174 of 374
75.0 2700
Processing waveform 175 of 374
75.0 2325
Processing waveform 176 of 374
75.0 2775
Processing waveform 177 of 374
75.0 2625
Processing waveform 178 of 374
75.0 5850
Processing waveform 179 of 374
75.0 3825
Processing waveform 180 of 374
75.0 6000
Processing waveform 181 of 374
75.0 2025
Processing waveform 182 of 374
75.0 1725
Processing waveform 183 of 374
75.0 2175
Processing waveform 184 of 374
75.0 5400
Processing waveform 185 of 374
75.0 5325
Processing waveform 186 of 374
75.0 3750
Processing waveform 187 of 374
75.0 3075
Processing waveform 188 of 374
75.0 5550
Processing waveform 189 of 374
75.0 7350
Processing waveform 190 of 374
75.0 4650
Processing waveform 191 of 374
75.0 2775
Processing waveform 192 of 374
75.0 2325
Processing waveform 193 of 374
75.0 2925
Processing waveform 194 of 374
75.0 3075
Processing waveform 195 of 374
75.0 6375
Processing waveform 196 of 374
75.0 2475
Processing waveform 197 of 374
75.0 3975
Processing waveform 198 of 374
75.0 3150
Processing waveform 199 of 374
75.0 5925
Processing waveform 200 of 374
75.0 5400
Processing waveform 201 of 374
75.0 1875
Processing waveform 202 of 374
75.0 5850
Processing waveform 203 of 374
75.0 1725
Processing waveform 204 of 374
75.0 7650
Processing waveform 205 of 374
75.0 2700
Processing waveform 206 of 374
75.0 9075
Processing waveform 207 of 374
75.0 6975
Processing waveform 208 of 374
75.0 4050
Processing waveform 209 of 374
75.0 3450
Processing waveform 210 of 374
75.0 5925
Processing waveform 211 of 374
75.0 2250
Processing waveform 212 of 374
75.0 3675
Processing waveform 213 of 374
75.0 2625
Processing waveform 214 of 374
75.0 4575
Processing waveform 215 of 374
75.0 2025
Processing waveform 216 of 374
75.0 5175
Processing waveform 217 of 374
75.0 3225
Processing waveform 218 of 374
75.0 4950
Processing waveform 219 of 374
75.0 2925
Processing waveform 220 of 374
75.0 1800
Processing waveform 221 of 374
75.0 8925
Processing waveform 222 of 374
75.0 7350
Processing waveform 223 of 374
75.0 4125
Processing waveform 224 of 374
75.0 3225
Processing waveform 225 of 374
75.0 9075
Processing waveform 226 of 374
75.0 5025
Processing waveform 227 of 374
75.0 8925
Processing waveform 228 of 374
75.0 4800
Processing waveform 229 of 374
75.0 7800
Processing waveform 230 of 374
75.0 2775
Processing waveform 231 of 374
75.0 6600
Processing waveform 232 of 374
75.0 1800
Processing waveform 233 of 374
75.0 2625
Processing waveform 234 of 374
75.0 9075
Processing waveform 235 of 374
75.0 7200
Processing waveform 236 of 374
75.0 5550
Processing waveform 237 of 374
75.0 2775
Processing waveform 238 of 374
75.0 5250
Processing waveform 239 of 374
75.0 1875
Processing waveform 240 of 374
75.0 1875
Processing waveform 241 of 374
75.0 5025
Processing waveform 242 of 374
75.0 3600
Processing waveform 243 of 374
75.0 3375
Processing waveform 244 of 374
75.0 4050
Processing waveform 245 of 374
75.0 3375
Processing waveform 246 of 374
75.0 2925
Processing waveform 247 of 374
75.0 6675
Processing waveform 248 of 374
75.0 3750
Processing waveform 249 of 374
75.0 4575
Processing waveform 250 of 374
75.0 9075
Processing waveform 251 of 374
75.0 3900
Processing waveform 252 of 374
75.0 9075
Processing waveform 253 of 374
75.0 4875
Processing waveform 254 of 374
75.0 3075

Processing waveform 255 of 374
75.0 3150
Processing waveform 256 of 374
75.0 9075
Processing waveform 257 of 374
75.0 7425
Processing waveform 258 of 374
75.0 5025
Processing waveform 259 of 374
75.0 2850
Processing waveform 260 of 374
75.0 1875
Processing waveform 261 of 374
75.0 7125
Processing waveform 262 of 374
75.0 4500
Processing waveform 263 of 374
75.0 6075
Processing waveform 264 of 374
75.0 2475
Processing waveform 265 of 374
75.0 4125
Processing waveform 266 of 374
75.0 2100
Processing waveform 267 of 374
75.0 3975
Processing waveform 268 of 374
75.0 3225
Processing waveform 269 of 374
75.0 2550
Processing waveform 270 of 374
75.0 6975
Processing waveform 271 of 374
75.0 3075
Processing waveform 272 of 374
75.0 3675
Processing waveform 273 of 374
75.0 4800
Processing waveform 274 of 374
75.0 5850
Processing waveform 275 of 374
75.0 3675
Processing waveform 276 of 374
75.0 2250
Processing waveform 277 of 374
75.0 3600
Processing waveform 278 of 374
75.0 5775
Processing waveform 279 of 374
75.0 2325
Processing waveform 280 of 374
75.0 2325
Processing waveform 281 of 374
75.0 3525
Processing waveform 282 of 374
75.0 4500
Processing waveform 283 of 374
75.0 4275
Processing waveform 284 of 374
75.0 3450
Processing waveform 285 of 374
75.0 2550
Processing waveform 286 of 374
75.0 3300
Processing waveform 287 of 374
75.0 9075
Processing waveform 288 of 374
75.0 9075
Processing waveform 289 of 374
75.0 6525
Processing waveform 290 of 374
75.0 6975
Processing waveform 291 of 374
75.0 6750
Processing waveform 292 of 374
75.0 7275
Processing waveform 293 of 374
75.0 2400
Processing waveform 294 of 374
75.0 4125
Processing waveform 295 of 374
75.0 3525
Processing waveform 296 of 374
75.0 5175
Processing waveform 297 of 374
75.0 3750
Processing waveform 298 of 374
75.0 4875
Processing waveform 299 of 374
75.0 2250
Processing waveform 300 of 374
75.0 4125
Processing waveform 301 of 374
75.0 4425
Processing waveform 302 of 374
75.0 5250
Processing waveform 303 of 374
75.0 9075
Processing waveform 304 of 374
75.0 4725
Processing waveform 305 of 374
75.0 2925
Processing waveform 306 of 374
75.0 7350
Processing waveform 307 of 374
75.0 2550
Processing waveform 308 of 374
75.0 2850
Processing waveform 309 of 374
75.0 2550
Processing waveform 310 of 374
75.0 4425
Processing waveform 311 of 374
75.0 5775
Processing waveform 312 of 374
75.0 4200
Processing waveform 313 of 374
75.0 2175
Processing waveform 314 of 374
75.0 10350
Processing waveform 315 of 374
75.0 15600
Processing waveform 316 of 374
75.0 6450
Processing waveform 317 of 374
75.0 3450
Processing waveform 318 of 374
75.0 11175
Processing waveform 319 of 374
75.0 4725
Processing waveform 320 of 374
75.0 4200
Processing waveform 321 of 374
75.0 3750
Processing waveform 322 of 374
75.0 3975
Processing waveform 323 of 374
75.0 4575
Processing waveform 324 of 374
75.0 3225
Processing waveform 325 of 374
75.0 13500
Processing waveform 326 of 374
75.0 4050
Processing waveform 327 of 374
75.0 5250
Processing waveform 328 of 374
75.0 7050
Processing waveform 329 of 374
75.0 3450
Processing waveform 330 of 374
75.0 6675
Processing waveform 331 of 374
75.0 2850
Processing waveform 332 of 374
75.0 6750
Processing waveform 333 of 374
75.0 2475
Processing waveform 334 of 374
75.0 7650
Processing waveform 335 of 374
75.0 6750
Processing waveform 336 of 374
75.0 4125
Processing waveform 337 of 374
75.0 7425
Processing waveform 338 of 374
75.0 5400
Processing waveform 339 of 374
75.0 4800
Processing waveform 340 of 374
75.0 10200
Processing waveform 341 of 374
75.0 18225
Processing waveform 342 of 374
75.0 5925
Processing waveform 343 of 374
75.0 3525
Processing waveform 344 of 374
75.0 3300
Processing waveform 345 of 374
75.0 3600
Processing waveform 346 of 374
75.0 4650
Processing waveform 347 of 374
75.0 7875
Processing waveform 348 of 374
75.0 6300
Processing waveform 349 of 374
75.0 6750
Processing waveform 350 of 374
75.0 2925
Processing waveform 351 of 374
75.0 3525
Processing waveform 352 of 374
75.0 7875
Processing waveform 353 of 374
75.0 8925
Processing waveform 354 of 374
75.0 2850
Processing waveform 355 of 374
75.0 2475
Processing waveform 356 of 374
75.0 2325
Processing waveform 357 of 374
75.0 2250
Processing waveform 358 of 374
75.0 2700
Processing waveform 359 of 374
75.0 3000
Processing waveform 360 of 374
75.0 3300
Processing waveform 361 of 374
75.0 4425
Processing waveform 362 of 374
75.0 4500
Processing waveform 363 of 374
75.0 7500
Processing waveform 364 of 374
75.0 4200
Processing waveform 365 of 374
75.0 4800
Processing waveform 366 of 374
75.0 3000
Processing waveform 367 of 374
75.0 1950
Processing waveform 368 of 374
75.0 2625
Processing waveform 369 of 374
75.0 4650
Processing waveform 370 of 374
75.0 5475
Processing waveform 371 of 374
75.0 4950
Processing waveform 372 of 374
75.0 6375
Processing waveform 373 of 374
75.0 7500
Training data have been read and features have been extracted  (374, 120)
Computation time:  507.7810752391815
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.45678162574768066
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h   104                   
        l          90             
        r                82       
        t                      98 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  31.0695187166  +/-  3.60146300938  %
          Predicted class
              h     l     r     t 
        h    25     9     8    10 
        l    16    10     7    12 
        r    17     7     6    10 
        t    14    11     7    17 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBWH..SHZ ['e', 'h', 'l', 't', 'r'] 3
class
e     83
h    104
l     90
r     82
t     98
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 457
75.0 5325
Processing waveform 1 of 457
75.0 2700
Processing waveform 2 of 457
75.0 4350
Processing waveform 3 of 457
75.0 2775
Processing waveform 4 of 457
75.0 2250
Processing waveform 5 of 457
75.0 2475
Processing waveform 6 of 457
75.0 3300
Processing waveform 7 of 457
75.0 9075
Processing waveform 8 of 457
75.0 4125
Processing waveform 9 of 457
75.0 3300
Processing waveform 10 of 457
75.0 3225
Processing waveform 11 of 457
75.0 2850
Processing waveform 12 of 457
75.0 2400
Processing waveform 13 of 457
75.0 3900
Processing waveform 14 of 457
75.0 2775
Processing waveform 15 of 457
75.0 2325
Processing waveform 16 of 457
75.0 4425
Processing waveform 17 of 457
75.0 3525
Processing waveform 18 of 457
75.0 6300
Processing waveform 19 of 457
75.0 2625
Processing waveform 20 of 457
75.0 2400
Processing waveform 21 of 457
75.0 2250
Processing waveform 22 of 457
75.0 3750
Processing waveform 23 of 457
75.0 3525
Processing waveform 24 of 457
75.0 5700
Processing waveform 25 of 457
75.0 2700
Processing waveform 26 of 457
75.0 4350
Processing waveform 27 of 457
75.0 4950
Processing waveform 28 of 457
75.0 2925
Processing waveform 29 of 457
75.0 2775
Processing waveform 30 of 457
75.0 3450
Processing waveform 31 of 457
75.0 4875
Processing waveform 32 of 457
75.0 9075
Processing waveform 33 of 457
75.0 5175
Processing waveform 34 of 457
75.0 3075
Processing waveform 35 of 457
75.0 4125
Processing waveform 36 of 457
75.0 9075
Processing waveform 37 of 457
75.0 2775
Processing waveform 38 of 457
75.0 4575
Processing waveform 39 of 457
75.0 2700
Processing waveform 40 of 457
75.0 1950
Processing waveform 41 of 457
75.0 2100
Processing waveform 42 of 457
75.0 4575
Processing waveform 43 of 457
75.0 5850
Processing waveform 44 of 457
75.0 3150
Processing waveform 45 of 457
75.0 4275
Processing waveform 46 of 457
75.0 9075
Processing waveform 47 of 457
75.0 3600
Processing waveform 48 of 457
75.0 2625
Processing waveform 49 of 457
75.0 2700
Processing waveform 50 of 457
75.0 2700
Processing waveform 51 of 457
75.0 3525
Processing waveform 52 of 457
75.0 4200
Processing waveform 53 of 457
75.0 4200

Processing waveform 54 of 457
75.0 1950
Processing waveform 55 of 457
75.0 5775
Processing waveform 56 of 457
75.0 9075
Processing waveform 57 of 457
75.0 4875
Processing waveform 58 of 457
75.0 14625
Processing waveform 59 of 457
75.0 1950
Processing waveform 60 of 457
75.0 3975
Processing waveform 61 of 457
75.0 2550
Processing waveform 62 of 457
75.0 7500
Processing waveform 63 of 457
75.0 14925
Processing waveform 64 of 457
75.0 13275
Processing waveform 65 of 457
75.0 1725
Processing waveform 66 of 457
75.0 2400
Processing waveform 67 of 457
75.0 6375
Processing waveform 68 of 457
75.0 4125
Processing waveform 69 of 457
75.0 4800
Processing waveform 70 of 457
75.0 7125
Processing waveform 71 of 457
75.0 5400
Processing waveform 72 of 457
75.0 6825
Processing waveform 73 of 457
75.0 7050
Processing waveform 74 of 457
75.0 4500
Processing waveform 75 of 457
75.0 5625
Processing waveform 76 of 457
75.0 4275
Processing waveform 77 of 457
75.0 3150
Processing waveform 78 of 457
75.0 3150
Processing waveform 79 of 457
75.0 12600
Processing waveform 80 of 457
75.0 16725
Processing waveform 81 of 457
75.0 6075
Processing waveform 82 of 457
75.0 3450
Processing waveform 83 of 457
75.0 2925
Processing waveform 84 of 457
75.0 4575
Processing waveform 85 of 457
75.0 7050
Processing waveform 86 of 457
75.0 2025
Processing waveform 87 of 457
75.0 2925
Processing waveform 88 of 457
75.0 2850
Processing waveform 89 of 457
75.0 3600
Processing waveform 90 of 457
75.0 3750
Processing waveform 91 of 457
75.0 2100
Processing waveform 92 of 457
75.0 1650
Processing waveform 93 of 457
75.0 4200
Processing waveform 94 of 457
75.0 9075
Processing waveform 95 of 457
75.0 4125
Processing waveform 96 of 457
75.0 2925
Processing waveform 97 of 457
75.0 2250
Processing waveform 98 of 457
75.0 5475
Processing waveform 99 of 457
75.0 2625
Processing waveform 100 of 457
75.0 2400
Processing waveform 101 of 457
75.0 2700
Processing waveform 102 of 457
75.0 9075
Processing waveform 103 of 457
75.0 3300
Processing waveform 104 of 457
75.0 9075
Processing waveform 105 of 457
75.0 6225
Processing waveform 106 of 457
75.0 7125
Processing waveform 107 of 457
75.0 5850
Processing waveform 108 of 457
75.0 9075
Processing waveform 109 of 457
75.0 3225
Processing waveform 110 of 457
75.0 8100
Processing waveform 111 of 457
75.0 4575
Processing waveform 112 of 457
75.0 2850
Processing waveform 113 of 457
75.0 3375
Processing waveform 114 of 457
75.0 2100
Processing waveform 115 of 457
75.0 9075
Processing waveform 116 of 457
75.0 5850
Processing waveform 117 of 457
75.0 2775
Processing waveform 118 of 457
75.0 2700
Processing waveform 119 of 457
75.0 2100
Processing waveform 120 of 457
75.0 9075
Processing waveform 121 of 457
75.0 3225
Processing waveform 122 of 457
75.0 3300
Processing waveform 123 of 457
75.0 2475
Processing waveform 124 of 457
75.0 3075
Processing waveform 125 of 457
75.0 6450
Processing waveform 126 of 457
75.0 3600
Processing waveform 127 of 457
75.0 4650
Processing waveform 128 of 457
75.0 2475
Processing waveform 129 of 457
75.0 1875
Processing waveform 130 of 457
75.0 3450
Processing waveform 131 of 457
75.0 1500
Processing waveform 132 of 457
75.0 2625
Processing waveform 133 of 457
75.0 9075
Processing waveform 134 of 457
75.0 3525
Processing waveform 135 of 457
75.0 5625
Processing waveform 136 of 457
75.0 3675
Processing waveform 137 of 457
75.0 9075
Processing waveform 138 of 457
75.0 2700
Processing waveform 139 of 457
75.0 3000
Processing waveform 140 of 457
75.0 3525
Processing waveform 141 of 457
75.0 7200
Processing waveform 142 of 457
75.0 4875
Processing waveform 143 of 457
75.0 6075
Processing waveform 144 of 457
75.0 2700
Processing waveform 145 of 457
75.0 5250
Processing waveform 146 of 457
75.0 12750
Processing waveform 147 of 457
75.0 5100
Processing waveform 148 of 457
75.0 2475
Processing waveform 149 of 457
75.0 6525
Processing waveform 150 of 457
75.0 7200
Processing waveform 151 of 457
75.0 4950
Processing waveform 152 of 457
75.0 8250
Processing waveform 153 of 457
75.0 3300
Processing waveform 154 of 457
75.0 2325
Processing waveform 155 of 457
75.0 13500
Processing waveform 156 of 457
75.0 3525
Processing waveform 157 of 457
75.0 3075
Processing waveform 158 of 457
75.0 6600
Processing waveform 159 of 457
75.0 6225
Processing waveform 160 of 457
75.0 6975
Processing waveform 161 of 457
75.0 4425
Processing waveform 162 of 457
75.0 13350
Processing waveform 163 of 457
75.0 9525
Processing waveform 164 of 457
75.0 2325
Processing waveform 165 of 457
75.0 9075
Processing waveform 166 of 457
75.0 2175
Processing waveform 167 of 457
75.0 3300
Processing waveform 168 of 457
75.0 2700
Processing waveform 169 of 457
75.0 5250
Processing waveform 170 of 457
75.0 3000
Processing waveform 171 of 457
75.0 2775
Processing waveform 172 of 457
75.0 6450
Processing waveform 173 of 457
75.0 3150
Processing waveform 174 of 457
75.0 1650
Processing waveform 175 of 457
75.0 4500
Processing waveform 176 of 457
75.0 2250
Processing waveform 177 of 457
75.0 2175
Processing waveform 178 of 457
75.0 3225
Processing waveform 179 of 457
75.0 4050
Processing waveform 180 of 457
75.0 3300
Processing waveform 181 of 457
75.0 2850
Processing waveform 182 of 457
75.0 2025
Processing waveform 183 of 457
75.0 4800
Processing waveform 184 of 457
75.0 2325
Processing waveform 185 of 457
75.0 6150
Processing waveform 186 of 457
75.0 2775
Processing waveform 187 of 457
75.0 2850
Processing waveform 188 of 457
75.0 4425
Processing waveform 189 of 457
75.0 2250
Processing waveform 190 of 457
75.0 3075
Processing waveform 191 of 457
75.0 2775
Processing waveform 192 of 457
75.0 1875
Processing waveform 193 of 457
75.0 3300
Processing waveform 194 of 457
75.0 1875
Processing waveform 195 of 457
75.0 5550
Processing waveform 196 of 457
75.0 9075
Processing waveform 197 of 457
75.0 6525
Processing waveform 198 of 457
75.0 9075
Processing waveform 199 of 457
75.0 3225
Processing waveform 200 of 457
75.0 3150
Processing waveform 201 of 457
75.0 4275
Processing waveform 202 of 457
75.0 4050
Processing waveform 203 of 457
75.0 5400
Processing waveform 204 of 457
75.0 4725
Processing waveform 205 of 457
75.0 3450
Processing waveform 206 of 457
75.0 6150
Processing waveform 207 of 457
75.0 2250
Processing waveform 208 of 457
75.0 9000
Processing waveform 209 of 457
75.0 3150
Processing waveform 210 of 457
75.0 4950
Processing waveform 211 of 457
75.0 6750
Processing waveform 212 of 457
75.0 3150
Processing waveform 213 of 457
75.0 7050
Processing waveform 214 of 457
75.0 3675
Processing waveform 215 of 457
75.0 3225
Processing waveform 216 of 457
75.0 2325
Processing waveform 217 of 457
75.0 3975
Processing waveform 218 of 457
75.0 6150
Processing waveform 219 of 457
75.0 7875
Processing waveform 220 of 457
75.0 7275
Processing waveform 221 of 457
75.0 6075
Processing waveform 222 of 457
75.0 3900
Processing waveform 223 of 457
75.0 3000
Processing waveform 224 of 457
75.0 3900
Processing waveform 225 of 457
75.0 6525
Processing waveform 226 of 457
75.0 2925
Processing waveform 227 of 457
75.0 2700
Processing waveform 228 of 457
75.0 2250
Processing waveform 229 of 457
75.0 2250
Processing waveform 230 of 457
75.0 9600
Processing waveform 231 of 457
75.0 4650
Processing waveform 232 of 457
75.0 6750
Processing waveform 233 of 457
75.0 7125
Processing waveform 234 of 457
75.0 3000
Processing waveform 235 of 457
75.0 10200
Processing waveform 236 of 457
75.0 1800
Processing waveform 237 of 457
75.0 4425
Processing waveform 238 of 457
75.0 2250
Processing waveform 239 of 457
75.0 6150
Processing waveform 240 of 457
75.0 7125
Processing waveform 241 of 457
75.0 3600
Processing waveform 242 of 457
75.0 6000
Processing waveform 243 of 457
75.0 4800
Processing waveform 244 of 457
75.0 7200
Processing waveform 245 of 457
75.0 3000
Processing waveform 246 of 457
75.0 4500
Processing waveform 247 of 457
75.0 3600
Processing waveform 248 of 457
75.0 4875
Processing waveform 249 of 457
75.0 5625
Processing waveform 250 of 457
75.0 3150
Processing waveform 251 of 457
75.0 3975
Processing waveform 252 of 457
75.0 5325
Processing waveform 253 of 457
75.0 2850
Processing waveform 254 of 457
75.0 9450

Processing waveform 255 of 457
75.0 3225
Processing waveform 256 of 457
75.0 2100
Processing waveform 257 of 457
75.0 2700
Processing waveform 258 of 457
75.0 2325
Processing waveform 259 of 457
75.0 2775
Processing waveform 260 of 457
75.0 2625
Processing waveform 261 of 457
75.0 5850
Processing waveform 262 of 457
75.0 3825
Processing waveform 263 of 457
75.0 6000
Processing waveform 264 of 457
75.0 2025
Processing waveform 265 of 457
75.0 1725
Processing waveform 266 of 457
75.0 2175
Processing waveform 267 of 457
75.0 5400
Processing waveform 268 of 457
75.0 5325
Processing waveform 269 of 457
75.0 3750
Processing waveform 270 of 457
75.0 3075
Processing waveform 271 of 457
75.0 5550
Processing waveform 272 of 457
75.0 7350
Processing waveform 273 of 457
75.0 4650
Processing waveform 274 of 457
75.0 2775
Processing waveform 275 of 457
75.0 2325
Processing waveform 276 of 457
75.0 2925
Processing waveform 277 of 457
75.0 3075
Processing waveform 278 of 457
75.0 6375
Processing waveform 279 of 457
75.0 2475
Processing waveform 280 of 457
75.0 3975
Processing waveform 281 of 457
75.0 3150
Processing waveform 282 of 457
75.0 5925
Processing waveform 283 of 457
75.0 5400
Processing waveform 284 of 457
75.0 1875
Processing waveform 285 of 457
75.0 5850
Processing waveform 286 of 457
75.0 1725
Processing waveform 287 of 457
75.0 7650
Processing waveform 288 of 457
75.0 2700
Processing waveform 289 of 457
75.0 9075
Processing waveform 290 of 457
75.0 6975
Processing waveform 291 of 457
75.0 4050
Processing waveform 292 of 457
75.0 3450
Processing waveform 293 of 457
75.0 5925
Processing waveform 294 of 457
75.0 2250
Processing waveform 295 of 457
75.0 3675
Processing waveform 296 of 457
75.0 2625
Processing waveform 297 of 457
75.0 4575
Processing waveform 298 of 457
75.0 2025
Processing waveform 299 of 457
75.0 5175
Processing waveform 300 of 457
75.0 3225
Processing waveform 301 of 457
75.0 4950
Processing waveform 302 of 457
75.0 2925
Processing waveform 303 of 457
75.0 1800
Processing waveform 304 of 457
75.0 8925
Processing waveform 305 of 457
75.0 7350
Processing waveform 306 of 457
75.0 4125
Processing waveform 307 of 457
75.0 3225
Processing waveform 308 of 457
75.0 9075
Processing waveform 309 of 457
75.0 5025
Processing waveform 310 of 457
75.0 8925
Processing waveform 311 of 457
75.0 4800
Processing waveform 312 of 457
75.0 7800
Processing waveform 313 of 457
75.0 2775
Processing waveform 314 of 457
75.0 6600
Processing waveform 315 of 457
75.0 1800
Processing waveform 316 of 457
75.0 2625
Processing waveform 317 of 457
75.0 9075
Processing waveform 318 of 457
75.0 7200
Processing waveform 319 of 457
75.0 5550
Processing waveform 320 of 457
75.0 2775
Processing waveform 321 of 457
75.0 5250
Processing waveform 322 of 457
75.0 1875
Processing waveform 323 of 457
75.0 1875
Processing waveform 324 of 457
75.0 5025
Processing waveform 325 of 457
75.0 3600
Processing waveform 326 of 457
75.0 3375
Processing waveform 327 of 457
75.0 4050
Processing waveform 328 of 457
75.0 3375
Processing waveform 329 of 457
75.0 2925
Processing waveform 330 of 457
75.0 6675
Processing waveform 331 of 457
75.0 3750
Processing waveform 332 of 457
75.0 4575
Processing waveform 333 of 457
75.0 9075
Processing waveform 334 of 457
75.0 3900
Processing waveform 335 of 457
75.0 9075
Processing waveform 336 of 457
75.0 4875
Processing waveform 337 of 457
75.0 3075
Processing waveform 338 of 457
75.0 3150
Processing waveform 339 of 457
75.0 9075
Processing waveform 340 of 457
75.0 7425
Processing waveform 341 of 457
75.0 5025
Processing waveform 342 of 457
75.0 2850
Processing waveform 343 of 457
75.0 1875
Processing waveform 344 of 457
75.0 7125
Processing waveform 345 of 457
75.0 4500
Processing waveform 346 of 457
75.0 6075
Processing waveform 347 of 457
75.0 2475
Processing waveform 348 of 457
75.0 4125
Processing waveform 349 of 457
75.0 2100
Processing waveform 350 of 457
75.0 3975
Processing waveform 351 of 457
75.0 3225
Processing waveform 352 of 457
75.0 2550
Processing waveform 353 of 457
75.0 6975
Processing waveform 354 of 457
75.0 3075
Processing waveform 355 of 457
75.0 3675
Processing waveform 356 of 457
75.0 4800
Processing waveform 357 of 457
75.0 5850
Processing waveform 358 of 457
75.0 3675
Processing waveform 359 of 457
75.0 2250
Processing waveform 360 of 457
75.0 3600
Processing waveform 361 of 457
75.0 5775
Processing waveform 362 of 457
75.0 2325
Processing waveform 363 of 457
75.0 2325
Processing waveform 364 of 457
75.0 3525
Processing waveform 365 of 457
75.0 4500
Processing waveform 366 of 457
75.0 4275
Processing waveform 367 of 457
75.0 3450
Processing waveform 368 of 457
75.0 2550
Processing waveform 369 of 457
75.0 3300
Processing waveform 370 of 457
75.0 9075
Processing waveform 371 of 457
75.0 9075
Processing waveform 372 of 457
75.0 6525
Processing waveform 373 of 457
75.0 6975
Processing waveform 374 of 457
75.0 6750
Processing waveform 375 of 457
75.0 7275
Processing waveform 376 of 457
75.0 2400
Processing waveform 377 of 457
75.0 4125
Processing waveform 378 of 457
75.0 3525
Processing waveform 379 of 457
75.0 5175
Processing waveform 380 of 457
75.0 3750
Processing waveform 381 of 457
75.0 4875
Processing waveform 382 of 457
75.0 2250
Processing waveform 383 of 457
75.0 4125
Processing waveform 384 of 457
75.0 4425
Processing waveform 385 of 457
75.0 5250
Processing waveform 386 of 457
75.0 9075
Processing waveform 387 of 457
75.0 4725
Processing waveform 388 of 457
75.0 2925
Processing waveform 389 of 457
75.0 7350
Processing waveform 390 of 457
75.0 2550
Processing waveform 391 of 457
75.0 2850
Processing waveform 392 of 457
75.0 2550
Processing waveform 393 of 457
75.0 4425
Processing waveform 394 of 457
75.0 5775
Processing waveform 395 of 457
75.0 4200
Processing waveform 396 of 457
75.0 2175
Processing waveform 397 of 457
75.0 10350
Processing waveform 398 of 457
75.0 15600
Processing waveform 399 of 457
75.0 6450
Processing waveform 400 of 457
75.0 3450
Processing waveform 401 of 457
75.0 11175
Processing waveform 402 of 457
75.0 4725
Processing waveform 403 of 457
75.0 4200
Processing waveform 404 of 457
75.0 3750
Processing waveform 405 of 457
75.0 3975
Processing waveform 406 of 457
75.0 4575
Processing waveform 407 of 457
75.0 3225
Processing waveform 408 of 457
75.0 13500
Processing waveform 409 of 457
75.0 4050
Processing waveform 410 of 457
75.0 5250
Processing waveform 411 of 457
75.0 7050
Processing waveform 412 of 457
75.0 3450
Processing waveform 413 of 457
75.0 6675
Processing waveform 414 of 457
75.0 2850
Processing waveform 415 of 457
75.0 6750
Processing waveform 416 of 457
75.0 2475
Processing waveform 417 of 457
75.0 7650
Processing waveform 418 of 457
75.0 6750
Processing waveform 419 of 457
75.0 4125
Processing waveform 420 of 457
75.0 7425
Processing waveform 421 of 457
75.0 5400
Processing waveform 422 of 457
75.0 4800
Processing waveform 423 of 457
75.0 10200
Processing waveform 424 of 457
75.0 18225
Processing waveform 425 of 457
75.0 5925
Processing waveform 426 of 457
75.0 3525
Processing waveform 427 of 457
75.0 3300
Processing waveform 428 of 457
75.0 3600
Processing waveform 429 of 457
75.0 4650
Processing waveform 430 of 457
75.0 7875
Processing waveform 431 of 457
75.0 6300
Processing waveform 432 of 457
75.0 6750
Processing waveform 433 of 457
75.0 2925
Processing waveform 434 of 457
75.0 3525
Processing waveform 435 of 457
75.0 7875
Processing waveform 436 of 457
75.0 8925
Processing waveform 437 of 457
75.0 2850
Processing waveform 438 of 457
75.0 2475
Processing waveform 439 of 457
75.0 2325
Processing waveform 440 of 457
75.0 2250
Processing waveform 441 of 457
75.0 2700
Processing waveform 442 of 457
75.0 3000
Processing waveform 443 of 457
75.0 3300
Processing waveform 444 of 457
75.0 4425
Processing waveform 445 of 457
75.0 4500
Processing waveform 446 of 457
75.0 7500
Processing waveform 447 of 457
75.0 4200
Processing waveform 448 of 457
75.0 4800
Processing waveform 449 of 457
75.0 3000
Processing waveform 450 of 457
75.0 1950
Processing waveform 451 of 457
75.0 2625
Processing waveform 452 of 457
75.0 4650
Processing waveform 453 of 457
75.0 5475
Processing waveform 454 of 457
75.0 4950

Processing waveform 455 of 457
75.0 6375
Processing waveform 456 of 457
75.0 7500
Training data have been read and features have been extracted  (457, 120)
Computation time:  628.9882671833038
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.6426792144775391
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    83                         
        h         104                   
        l                90             
        r                      82       
        t                            98 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  23.056768559  +/-  2.6667365856  %
          Predicted class
              e     h     l     r     t 
        e     6    13     6     5    11 
        h     6    19     7     8    12 
        l     7    13     7     6    12 
        r     5    14     7     5    10 
        t     6    12     9     7    15 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['l', 't'] 0
class
l    88
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 179
75.0 3975
Processing waveform 1 of 179
75.0 2400
Processing waveform 2 of 179
75.0 5025
Processing waveform 3 of 179
75.0 6825
Processing waveform 4 of 179
75.0 3750
Processing waveform 5 of 179
75.0 9075
Processing waveform 6 of 179
75.0 4500
Processing waveform 7 of 179
75.0 2175
Processing waveform 8 of 179
75.0 3300
Processing waveform 9 of 179
75.0 4275
Processing waveform 10 of 179
75.0 2850
Processing waveform 11 of 179
75.0 2850
Processing waveform 12 of 179
75.0 2700
Processing waveform 13 of 179
75.0 5250
Processing waveform 14 of 179
75.0 3000
Processing waveform 15 of 179
75.0 2775
Processing waveform 16 of 179
75.0 6450
Processing waveform 17 of 179
75.0 3150
Processing waveform 18 of 179
75.0 1650
Processing waveform 19 of 179
75.0 4500
Processing waveform 20 of 179
75.0 2250
Processing waveform 21 of 179
75.0 2175
Processing waveform 22 of 179
75.0 3225
Processing waveform 23 of 179
75.0 4050
Processing waveform 24 of 179
75.0 3300
Processing waveform 25 of 179
75.0 2850
Processing waveform 26 of 179
75.0 2025
Processing waveform 27 of 179
75.0 4800
Processing waveform 28 of 179
75.0 2325
Processing waveform 29 of 179
75.0 6150
Processing waveform 30 of 179
75.0 2775
Processing waveform 31 of 179
75.0 2850
Processing waveform 32 of 179
75.0 4425
Processing waveform 33 of 179
75.0 2250
Processing waveform 34 of 179
75.0 3075
Processing waveform 35 of 179
75.0 2775
Processing waveform 36 of 179
75.0 1875
Processing waveform 37 of 179
75.0 3300
Processing waveform 38 of 179
75.0 1875
Processing waveform 39 of 179
75.0 5550
Processing waveform 40 of 179
75.0 9075
Processing waveform 41 of 179
75.0 4650
Processing waveform 42 of 179
75.0 6525
Processing waveform 43 of 179
75.0 9075
Processing waveform 44 of 179
75.0 3225
Processing waveform 45 of 179
75.0 3150
Processing waveform 46 of 179
75.0 4275
Processing waveform 47 of 179
75.0 4050
Processing waveform 48 of 179
75.0 5400
Processing waveform 49 of 179
75.0 4725
Processing waveform 50 of 179
75.0 3450
Processing waveform 51 of 179
75.0 6150
Processing waveform 52 of 179
75.0 2250
Processing waveform 53 of 179
75.0 9000
Processing waveform 54 of 179
75.0 3150
Processing waveform 55 of 179
75.0 4950
Processing waveform 56 of 179
75.0 6750
Processing waveform 57 of 179
75.0 3150
Processing waveform 58 of 179
75.0 7050
Processing waveform 59 of 179
75.0 3675
Processing waveform 60 of 179
75.0 3225
Processing waveform 61 of 179
75.0 2325
Processing waveform 62 of 179
75.0 7500
Processing waveform 63 of 179
75.0 3975
Processing waveform 64 of 179
75.0 6150
Processing waveform 65 of 179
75.0 7875
Processing waveform 66 of 179
75.0 7275
Processing waveform 67 of 179
75.0 6075
Processing waveform 68 of 179
75.0 3900
Processing waveform 69 of 179
75.0 3000
Processing waveform 70 of 179
75.0 3900
Processing waveform 71 of 179
75.0 6525
Processing waveform 72 of 179
75.0 2925
Processing waveform 73 of 179
75.0 2700
Processing waveform 74 of 179
75.0 2250
Processing waveform 75 of 179
75.0 4425
Processing waveform 76 of 179
75.0 7500
Processing waveform 77 of 179
75.0 2250
Processing waveform 78 of 179
75.0 9600
Processing waveform 79 of 179
75.0 4650
Processing waveform 80 of 179
75.0 6750
Processing waveform 81 of 179
75.0 8925
Processing waveform 82 of 179
75.0 7125
Processing waveform 83 of 179
75.0 3000
Processing waveform 84 of 179
75.0 1800
Processing waveform 85 of 179
75.0 4425
Processing waveform 86 of 179
75.0 9450
Processing waveform 87 of 179
75.0 6525
Processing waveform 88 of 179
75.0 5550
Processing waveform 89 of 179
75.0 2250
Processing waveform 90 of 179
75.0 3600
Processing waveform 91 of 179
75.0 5775
Processing waveform 92 of 179
75.0 2325
Processing waveform 93 of 179
75.0 2325
Processing waveform 94 of 179
75.0 3525
Processing waveform 95 of 179
75.0 4500
Processing waveform 96 of 179
75.0 4275
Processing waveform 97 of 179
75.0 3450
Processing waveform 98 of 179
75.0 2550
Processing waveform 99 of 179
75.0 3300
Processing waveform 100 of 179
75.0 9075
Processing waveform 101 of 179
75.0 9075
Processing waveform 102 of 179
75.0 6525
Processing waveform 103 of 179
75.0 6975
Processing waveform 104 of 179
75.0 6750
Processing waveform 105 of 179
75.0 7275
Processing waveform 106 of 179
75.0 2400
Processing waveform 107 of 179
75.0 4125
Processing waveform 108 of 179
75.0 3525
Processing waveform 109 of 179
75.0 5175
Processing waveform 110 of 179
75.0 3750
Processing waveform 111 of 179
75.0 4875
Processing waveform 112 of 179
75.0 2250
Processing waveform 113 of 179
75.0 4125
Processing waveform 114 of 179
75.0 4425
Processing waveform 115 of 179
75.0 5250
Processing waveform 116 of 179
75.0 9075
Processing waveform 117 of 179
75.0 4725
Processing waveform 118 of 179
75.0 2475
Processing waveform 119 of 179
75.0 2925
Processing waveform 120 of 179
75.0 7350
Processing waveform 121 of 179
75.0 2550
Processing waveform 122 of 179
75.0 2850
Processing waveform 123 of 179
75.0 2550
Processing waveform 124 of 179
75.0 5775
Processing waveform 125 of 179
75.0 4200
Processing waveform 126 of 179
75.0 2175
Processing waveform 127 of 179
75.0 2925
Processing waveform 128 of 179
75.0 10350
Processing waveform 129 of 179
75.0 15600
Processing waveform 130 of 179
75.0 6450
Processing waveform 131 of 179
75.0 3450
Processing waveform 132 of 179
75.0 11175
Processing waveform 133 of 179
75.0 4725
Processing waveform 134 of 179
75.0 4200
Processing waveform 135 of 179
75.0 3750
Processing waveform 136 of 179
75.0 3975
Processing waveform 137 of 179
75.0 4575
Processing waveform 138 of 179
75.0 3225
Processing waveform 139 of 179
75.0 13500
Processing waveform 140 of 179
75.0 4050
Processing waveform 141 of 179
75.0 5250
Processing waveform 142 of 179
75.0 7050
Processing waveform 143 of 179
75.0 3450
Processing waveform 144 of 179
75.0 6675
Processing waveform 145 of 179
75.0 6750
Processing waveform 146 of 179
75.0 2475
Processing waveform 147 of 179
75.0 7650
Processing waveform 148 of 179
75.0 6750
Processing waveform 149 of 179
75.0 4125
Processing waveform 150 of 179
75.0 7425
Processing waveform 151 of 179
75.0 5400
Processing waveform 152 of 179
75.0 4800
Processing waveform 153 of 179
75.0 10200
Processing waveform 154 of 179
75.0 18225
Processing waveform 155 of 179
75.0 5925
Processing waveform 156 of 179
75.0 3525
Processing waveform 157 of 179
75.0 3300
Processing waveform 158 of 179
75.0 3600
Processing waveform 159 of 179
75.0 4650
Processing waveform 160 of 179
75.0 7875
Processing waveform 161 of 179
75.0 6300
Processing waveform 162 of 179
75.0 6750
Processing waveform 163 of 179
75.0 2925
Processing waveform 164 of 179
75.0 3525
Processing waveform 165 of 179
75.0 7875
Processing waveform 166 of 179
75.0 8925
Processing waveform 167 of 179
75.0 2250
Processing waveform 168 of 179
75.0 2700

Processing waveform 169 of 179
75.0 3000
Processing waveform 170 of 179
75.0 3300
Processing waveform 171 of 179
75.0 4425
Processing waveform 172 of 179
75.0 4500
Processing waveform 173 of 179
75.0 7500
Processing waveform 174 of 179
75.0 4200
Processing waveform 175 of 179
75.0 4800
Processing waveform 176 of 179
75.0 1950
Processing waveform 177 of 179
75.0 4650
Processing waveform 178 of 179
75.0 5475
Training data have been read and features have been extracted  (179, 120)
Computation time:  255.27950501441956
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.14478850364685059
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l    88       
        t          91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  58.4444444444  +/-  3.05774332453  %
          Predicted class
              l     t 
        l    26    18 
        t    20    26 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'r'] 0
class
e    78
r    93
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 171
75.0 3450
Processing waveform 1 of 171
75.0 4125
Processing waveform 2 of 171
75.0 2400
Processing waveform 3 of 171
75.0 2700
Processing waveform 4 of 171
75.0 2700
Processing waveform 5 of 171
75.0 4350
Processing waveform 6 of 171
75.0 2775
Processing waveform 7 of 171
75.0 1950
Processing waveform 8 of 171
75.0 2250
Processing waveform 9 of 171
75.0 2250
Processing waveform 10 of 171
75.0 2250
Processing waveform 11 of 171
75.0 5100
Processing waveform 12 of 171
75.0 2475
Processing waveform 13 of 171
75.0 3300
Processing waveform 14 of 171
75.0 6300
Processing waveform 15 of 171
75.0 5100
Processing waveform 16 of 171
75.0 9075
Processing waveform 17 of 171
75.0 3225
Processing waveform 18 of 171
75.0 4125
Processing waveform 19 of 171
75.0 3300
Processing waveform 20 of 171
75.0 3225
Processing waveform 21 of 171
75.0 2850
Processing waveform 22 of 171
75.0 5400
Processing waveform 23 of 171
75.0 2400
Processing waveform 24 of 171
75.0 7725
Processing waveform 25 of 171
75.0 3900
Processing waveform 26 of 171
75.0 2775
Processing waveform 27 of 171
75.0 2325
Processing waveform 28 of 171
75.0 4425
Processing waveform 29 of 171
75.0 3525
Processing waveform 30 of 171
75.0 6300
Processing waveform 31 of 171
75.0 2625
Processing waveform 32 of 171
75.0 4050
Processing waveform 33 of 171
75.0 2775
Processing waveform 34 of 171
75.0 2400
Processing waveform 35 of 171
75.0 2850
Processing waveform 36 of 171
75.0 2250
Processing waveform 37 of 171
75.0 2325
Processing waveform 38 of 171
75.0 3750
Processing waveform 39 of 171
75.0 3525
Processing waveform 40 of 171
75.0 5700
Processing waveform 41 of 171
75.0 2700
Processing waveform 42 of 171
75.0 3000
Processing waveform 43 of 171
75.0 4350
Processing waveform 44 of 171
75.0 4950
Processing waveform 45 of 171
75.0 2925
Processing waveform 46 of 171
75.0 2775
Processing waveform 47 of 171
75.0 3450
Processing waveform 48 of 171
75.0 4875
Processing waveform 49 of 171
75.0 9075
Processing waveform 50 of 171
75.0 5175
Processing waveform 51 of 171
75.0 3075
Processing waveform 52 of 171
75.0 4125
Processing waveform 53 of 171
75.0 9075
Processing waveform 54 of 171
75.0 3225
Processing waveform 55 of 171
75.0 2775
Processing waveform 56 of 171
75.0 4575
Processing waveform 57 of 171
75.0 2700
Processing waveform 58 of 171
75.0 1950
Processing waveform 59 of 171
75.0 2100
Processing waveform 60 of 171
75.0 4575
Processing waveform 61 of 171
75.0 5850
Processing waveform 62 of 171
75.0 3150
Processing waveform 63 of 171
75.0 4275
Processing waveform 64 of 171
75.0 9075
Processing waveform 65 of 171
75.0 3600
Processing waveform 66 of 171
75.0 2625
Processing waveform 67 of 171
75.0 2700
Processing waveform 68 of 171
75.0 2700
Processing waveform 69 of 171
75.0 3525
Processing waveform 70 of 171
75.0 4200
Processing waveform 71 of 171
75.0 4200
Processing waveform 72 of 171
75.0 1950
Processing waveform 73 of 171
75.0 5775
Processing waveform 74 of 171
75.0 9075
Processing waveform 75 of 171
75.0 4200
Processing waveform 76 of 171
75.0 2625
Processing waveform 77 of 171
75.0 4875
Processing waveform 78 of 171
75.0 5025
Processing waveform 79 of 171
75.0 14625
Processing waveform 80 of 171
75.0 11025
Processing waveform 81 of 171
75.0 1950
Processing waveform 82 of 171
75.0 3975
Processing waveform 83 of 171
75.0 2550
Processing waveform 84 of 171
75.0 7500
Processing waveform 85 of 171
75.0 14925
Processing waveform 86 of 171
75.0 13275
Processing waveform 87 of 171
75.0 1725
Processing waveform 88 of 171
75.0 2400
Processing waveform 89 of 171
75.0 6375
Processing waveform 90 of 171
75.0 6450
Processing waveform 91 of 171
75.0 16725
Processing waveform 92 of 171
75.0 6075
Processing waveform 93 of 171
75.0 3450
Processing waveform 94 of 171
75.0 2325
Processing waveform 95 of 171
75.0 2925
Processing waveform 96 of 171
75.0 4575
Processing waveform 97 of 171
75.0 7050
Processing waveform 98 of 171
75.0 2025
Processing waveform 99 of 171
75.0 2925
Processing waveform 100 of 171
75.0 2850
Processing waveform 101 of 171
75.0 3600
Processing waveform 102 of 171
75.0 3750
Processing waveform 103 of 171
75.0 2100
Processing waveform 104 of 171
75.0 1650
Processing waveform 105 of 171
75.0 4200
Processing waveform 106 of 171
75.0 9075
Processing waveform 107 of 171
75.0 4125
Processing waveform 108 of 171
75.0 2925
Processing waveform 109 of 171
75.0 2250
Processing waveform 110 of 171
75.0 5475
Processing waveform 111 of 171
75.0 2625
Processing waveform 112 of 171
75.0 2400
Processing waveform 113 of 171
75.0 2700
Processing waveform 114 of 171
75.0 9075
Processing waveform 115 of 171
75.0 9075
Processing waveform 116 of 171
75.0 6225
Processing waveform 117 of 171
75.0 7125
Processing waveform 118 of 171
75.0 5850
Processing waveform 119 of 171
75.0 9075
Processing waveform 120 of 171
75.0 3225
Processing waveform 121 of 171
75.0 8100
Processing waveform 122 of 171
75.0 4575
Processing waveform 123 of 171
75.0 2850
Processing waveform 124 of 171
75.0 3375
Processing waveform 125 of 171
75.0 2100
Processing waveform 126 of 171
75.0 9075
Processing waveform 127 of 171
75.0 5850
Processing waveform 128 of 171
75.0 2775
Processing waveform 129 of 171
75.0 2700
Processing waveform 130 of 171
75.0 2100
Processing waveform 131 of 171
75.0 9075
Processing waveform 132 of 171
75.0 3225
Processing waveform 133 of 171
75.0 3300
Processing waveform 134 of 171
75.0 2475
Processing waveform 135 of 171
75.0 3075
Processing waveform 136 of 171
75.0 6450
Processing waveform 137 of 171
75.0 3600
Processing waveform 138 of 171
75.0 4650
Processing waveform 139 of 171
75.0 2475
Processing waveform 140 of 171
75.0 1875
Processing waveform 141 of 171
75.0 3450
Processing waveform 142 of 171
75.0 1500
Processing waveform 143 of 171
75.0 2625
Processing waveform 144 of 171
75.0 3525
Processing waveform 145 of 171
75.0 5625
Processing waveform 146 of 171
75.0 3675
Processing waveform 147 of 171
75.0 9075
Processing waveform 148 of 171
75.0 2700
Processing waveform 149 of 171
75.0 9075
Processing waveform 150 of 171
75.0 3000
Processing waveform 151 of 171
75.0 3525
Processing waveform 152 of 171
75.0 7200
Processing waveform 153 of 171
75.0 4875
Processing waveform 154 of 171
75.0 6075
Processing waveform 155 of 171
75.0 2700
Processing waveform 156 of 171
75.0 5250
Processing waveform 157 of 171
75.0 12750
Processing waveform 158 of 171
75.0 5100
Processing waveform 159 of 171
75.0 2475
Processing waveform 160 of 171
75.0 6525
Processing waveform 161 of 171
75.0 7200
Processing waveform 162 of 171
75.0 4500
Processing waveform 163 of 171
75.0 4950
Processing waveform 164 of 171
75.0 3150
Processing waveform 165 of 171
75.0 5850
Processing waveform 166 of 171
75.0 2325
Processing waveform 167 of 171
75.0 4200
Processing waveform 168 of 171
75.0 13350
Processing waveform 169 of 171
75.0 9525

Processing waveform 170 of 171
75.0 2325
Training data have been read and features have been extracted  (171, 120)
Computation time:  233.7326591014862
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.14520025253295898
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    78       
        r          93 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  49.8449612403  +/-  6.06982189329  %
          Predicted class
              e     r 
        e    14    25 
        r    18    28 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't'] 0
class
h    98
l    88
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 277
75.0 3975
Processing waveform 1 of 277
75.0 2400
Processing waveform 2 of 277
75.0 5025
Processing waveform 3 of 277
75.0 6825
Processing waveform 4 of 277
75.0 3750
Processing waveform 5 of 277
75.0 9075
Processing waveform 6 of 277
75.0 4500
Processing waveform 7 of 277
75.0 2175
Processing waveform 8 of 277
75.0 3300
Processing waveform 9 of 277
75.0 4275
Processing waveform 10 of 277
75.0 2850
Processing waveform 11 of 277
75.0 2850
Processing waveform 12 of 277
75.0 2700
Processing waveform 13 of 277
75.0 5250
Processing waveform 14 of 277
75.0 3000
Processing waveform 15 of 277
75.0 2775
Processing waveform 16 of 277
75.0 6450
Processing waveform 17 of 277
75.0 3150
Processing waveform 18 of 277
75.0 1650
Processing waveform 19 of 277
75.0 4500
Processing waveform 20 of 277
75.0 2250
Processing waveform 21 of 277
75.0 2175
Processing waveform 22 of 277
75.0 3225
Processing waveform 23 of 277
75.0 4050
Processing waveform 24 of 277
75.0 3300
Processing waveform 25 of 277
75.0 2850
Processing waveform 26 of 277
75.0 2025
Processing waveform 27 of 277
75.0 4800
Processing waveform 28 of 277
75.0 2325
Processing waveform 29 of 277
75.0 6150
Processing waveform 30 of 277
75.0 2775
Processing waveform 31 of 277
75.0 2850
Processing waveform 32 of 277
75.0 4425
Processing waveform 33 of 277
75.0 2250
Processing waveform 34 of 277
75.0 3075
Processing waveform 35 of 277
75.0 2775
Processing waveform 36 of 277
75.0 1875
Processing waveform 37 of 277
75.0 3300
Processing waveform 38 of 277
75.0 1875
Processing waveform 39 of 277
75.0 5550
Processing waveform 40 of 277
75.0 9075
Processing waveform 41 of 277
75.0 4650
Processing waveform 42 of 277
75.0 6525
Processing waveform 43 of 277
75.0 9075
Processing waveform 44 of 277
75.0 3225
Processing waveform 45 of 277
75.0 3150
Processing waveform 46 of 277
75.0 4275
Processing waveform 47 of 277
75.0 4050
Processing waveform 48 of 277
75.0 5400
Processing waveform 49 of 277
75.0 4725
Processing waveform 50 of 277
75.0 3450
Processing waveform 51 of 277
75.0 6150
Processing waveform 52 of 277
75.0 2250
Processing waveform 53 of 277
75.0 9000
Processing waveform 54 of 277
75.0 3150
Processing waveform 55 of 277
75.0 4950
Processing waveform 56 of 277
75.0 6750
Processing waveform 57 of 277
75.0 3150
Processing waveform 58 of 277
75.0 7050
Processing waveform 59 of 277
75.0 3675
Processing waveform 60 of 277
75.0 3225
Processing waveform 61 of 277
75.0 2325
Processing waveform 62 of 277
75.0 7500
Processing waveform 63 of 277
75.0 3975
Processing waveform 64 of 277
75.0 6150
Processing waveform 65 of 277
75.0 7875
Processing waveform 66 of 277
75.0 7275
Processing waveform 67 of 277
75.0 6075
Processing waveform 68 of 277
75.0 3900
Processing waveform 69 of 277
75.0 3000
Processing waveform 70 of 277
75.0 3900
Processing waveform 71 of 277
75.0 6525
Processing waveform 72 of 277
75.0 2925
Processing waveform 73 of 277
75.0 2700
Processing waveform 74 of 277
75.0 2250
Processing waveform 75 of 277
75.0 4425
Processing waveform 76 of 277
75.0 7500
Processing waveform 77 of 277
75.0 2250
Processing waveform 78 of 277
75.0 9600
Processing waveform 79 of 277
75.0 4650
Processing waveform 80 of 277
75.0 6750
Processing waveform 81 of 277
75.0 8925
Processing waveform 82 of 277
75.0 7125
Processing waveform 83 of 277
75.0 3000
Processing waveform 84 of 277
75.0 1800
Processing waveform 85 of 277
75.0 4425
Processing waveform 86 of 277
75.0 9450
Processing waveform 87 of 277
75.0 6525
Processing waveform 88 of 277
75.0 3225
Processing waveform 89 of 277
75.0 2100
Processing waveform 90 of 277
75.0 2700
Processing waveform 91 of 277
75.0 2325
Processing waveform 92 of 277
75.0 2775
Processing waveform 93 of 277
75.0 5850
Processing waveform 94 of 277
75.0 3825
Processing waveform 95 of 277
75.0 6000
Processing waveform 96 of 277
75.0 2025
Processing waveform 97 of 277
75.0 1725
Processing waveform 98 of 277
75.0 2175
Processing waveform 99 of 277
75.0 5400
Processing waveform 100 of 277
75.0 3000
Processing waveform 101 of 277
75.0 5325
Processing waveform 102 of 277
75.0 3750
Processing waveform 103 of 277
75.0 3075
Processing waveform 104 of 277
75.0 5550
Processing waveform 105 of 277
75.0 7350
Processing waveform 106 of 277
75.0 4650
Processing waveform 107 of 277
75.0 2775
Processing waveform 108 of 277
75.0 2325
Processing waveform 109 of 277
75.0 2925
Processing waveform 110 of 277
75.0 3075
Processing waveform 111 of 277
75.0 6375
Processing waveform 112 of 277
75.0 2475
Processing waveform 113 of 277
75.0 3975
Processing waveform 114 of 277
75.0 3150
Processing waveform 115 of 277
75.0 5925
Processing waveform 116 of 277
75.0 5400
Processing waveform 117 of 277
75.0 1875
Processing waveform 118 of 277
75.0 5850
Processing waveform 119 of 277
75.0 1725
Processing waveform 120 of 277
75.0 7650
Processing waveform 121 of 277
75.0 2700
Processing waveform 122 of 277
75.0 9075
Processing waveform 123 of 277
75.0 6975
Processing waveform 124 of 277
75.0 4050
Processing waveform 125 of 277
75.0 3450
Processing waveform 126 of 277
75.0 5925
Processing waveform 127 of 277
75.0 2250
Processing waveform 128 of 277
75.0 3675
Processing waveform 129 of 277
75.0 2625
Processing waveform 130 of 277
75.0 4575
Processing waveform 131 of 277
75.0 2025
Processing waveform 132 of 277
75.0 5175
Processing waveform 133 of 277
75.0 3225
Processing waveform 134 of 277
75.0 4950
Processing waveform 135 of 277
75.0 2925
Processing waveform 136 of 277
75.0 1800
Processing waveform 137 of 277
75.0 8925
Processing waveform 138 of 277
75.0 7350
Processing waveform 139 of 277
75.0 4125
Processing waveform 140 of 277
75.0 3225
Processing waveform 141 of 277
75.0 9075
Processing waveform 142 of 277
75.0 5025
Processing waveform 143 of 277
75.0 8925
Processing waveform 144 of 277
75.0 4800
Processing waveform 145 of 277
75.0 7800
Processing waveform 146 of 277
75.0 2775
Processing waveform 147 of 277
75.0 6600
Processing waveform 148 of 277
75.0 1800
Processing waveform 149 of 277
75.0 2625
Processing waveform 150 of 277
75.0 9075
Processing waveform 151 of 277
75.0 7200
Processing waveform 152 of 277
75.0 5550
Processing waveform 153 of 277
75.0 2775
Processing waveform 154 of 277
75.0 5250
Processing waveform 155 of 277
75.0 1875
Processing waveform 156 of 277
75.0 1875
Processing waveform 157 of 277
75.0 5025
Processing waveform 158 of 277
75.0 3600
Processing waveform 159 of 277
75.0 3375
Processing waveform 160 of 277
75.0 4050
Processing waveform 161 of 277
75.0 3375
Processing waveform 162 of 277
75.0 2925
Processing waveform 163 of 277
75.0 6675
Processing waveform 164 of 277
75.0 3750
Processing waveform 165 of 277
75.0 4575
Processing waveform 166 of 277
75.0 9075
Processing waveform 167 of 277
75.0 3900
Processing waveform 168 of 277
75.0 9075
Processing waveform 169 of 277
75.0 4875
Processing waveform 170 of 277
75.0 3075
Processing waveform 171 of 277
75.0 3150
Processing waveform 172 of 277
75.0 2550
Processing waveform 173 of 277
75.0 9075
Processing waveform 174 of 277
75.0 7425
Processing waveform 175 of 277
75.0 5025
Processing waveform 176 of 277
75.0 2850
Processing waveform 177 of 277
75.0 1875
Processing waveform 178 of 277
75.0 7125

Processing waveform 179 of 277
75.0 4500
Processing waveform 180 of 277
75.0 6075
Processing waveform 181 of 277
75.0 2475
Processing waveform 182 of 277
75.0 4125
Processing waveform 183 of 277
75.0 2100
Processing waveform 184 of 277
75.0 3975
Processing waveform 185 of 277
75.0 3075
Processing waveform 186 of 277
75.0 5550
Processing waveform 187 of 277
75.0 2250
Processing waveform 188 of 277
75.0 3600
Processing waveform 189 of 277
75.0 5775
Processing waveform 190 of 277
75.0 2325
Processing waveform 191 of 277
75.0 2325
Processing waveform 192 of 277
75.0 3525
Processing waveform 193 of 277
75.0 4500
Processing waveform 194 of 277
75.0 4275
Processing waveform 195 of 277
75.0 3450
Processing waveform 196 of 277
75.0 2550
Processing waveform 197 of 277
75.0 3300
Processing waveform 198 of 277
75.0 9075
Processing waveform 199 of 277
75.0 9075
Processing waveform 200 of 277
75.0 6525
Processing waveform 201 of 277
75.0 6975
Processing waveform 202 of 277
75.0 6750
Processing waveform 203 of 277
75.0 7275
Processing waveform 204 of 277
75.0 2400
Processing waveform 205 of 277
75.0 4125
Processing waveform 206 of 277
75.0 3525
Processing waveform 207 of 277
75.0 5175
Processing waveform 208 of 277
75.0 3750
Processing waveform 209 of 277
75.0 4875
Processing waveform 210 of 277
75.0 2250
Processing waveform 211 of 277
75.0 4125
Processing waveform 212 of 277
75.0 4425
Processing waveform 213 of 277
75.0 5250
Processing waveform 214 of 277
75.0 9075
Processing waveform 215 of 277
75.0 4725
Processing waveform 216 of 277
75.0 2475
Processing waveform 217 of 277
75.0 2925
Processing waveform 218 of 277
75.0 7350
Processing waveform 219 of 277
75.0 2550
Processing waveform 220 of 277
75.0 2850
Processing waveform 221 of 277
75.0 2550
Processing waveform 222 of 277
75.0 5775
Processing waveform 223 of 277
75.0 4200
Processing waveform 224 of 277
75.0 2175
Processing waveform 225 of 277
75.0 2925
Processing waveform 226 of 277
75.0 10350
Processing waveform 227 of 277
75.0 15600
Processing waveform 228 of 277
75.0 6450
Processing waveform 229 of 277
75.0 3450
Processing waveform 230 of 277
75.0 11175
Processing waveform 231 of 277
75.0 4725
Processing waveform 232 of 277
75.0 4200
Processing waveform 233 of 277
75.0 3750
Processing waveform 234 of 277
75.0 3975
Processing waveform 235 of 277
75.0 4575
Processing waveform 236 of 277
75.0 3225
Processing waveform 237 of 277
75.0 13500
Processing waveform 238 of 277
75.0 4050
Processing waveform 239 of 277
75.0 5250
Processing waveform 240 of 277
75.0 7050
Processing waveform 241 of 277
75.0 3450
Processing waveform 242 of 277
75.0 6675
Processing waveform 243 of 277
75.0 6750
Processing waveform 244 of 277
75.0 2475
Processing waveform 245 of 277
75.0 7650
Processing waveform 246 of 277
75.0 6750
Processing waveform 247 of 277
75.0 4125
Processing waveform 248 of 277
75.0 7425
Processing waveform 249 of 277
75.0 5400
Processing waveform 250 of 277
75.0 4800
Processing waveform 251 of 277
75.0 10200
Processing waveform 252 of 277
75.0 18225
Processing waveform 253 of 277
75.0 5925
Processing waveform 254 of 277
75.0 3525
Processing waveform 255 of 277
75.0 3300
Processing waveform 256 of 277
75.0 3600
Processing waveform 257 of 277
75.0 4650
Processing waveform 258 of 277
75.0 7875
Processing waveform 259 of 277
75.0 6300
Processing waveform 260 of 277
75.0 6750
Processing waveform 261 of 277
75.0 2925
Processing waveform 262 of 277
75.0 3525
Processing waveform 263 of 277
75.0 7875
Processing waveform 264 of 277
75.0 8925
Processing waveform 265 of 277
75.0 2250
Processing waveform 266 of 277
75.0 2700
Processing waveform 267 of 277
75.0 3000
Processing waveform 268 of 277
75.0 3300
Processing waveform 269 of 277
75.0 4425
Processing waveform 270 of 277
75.0 4500
Processing waveform 271 of 277
75.0 7500
Processing waveform 272 of 277
75.0 4200
Processing waveform 273 of 277
75.0 4800
Processing waveform 274 of 277
75.0 1950
Processing waveform 275 of 277
75.0 4650
Processing waveform 276 of 277
75.0 5475
Training data have been read and features have been extracted  (277, 120)
Computation time:  366.30387139320374
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.2888970375061035
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h    98             
        l          88       
        t                91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  35.1558752998  +/-  4.03448265608  %
          Predicted class
              h     l     t 
        h    18    15    16 
        l    19    14    11 
        t    17    11    17 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't', 'r'] 0
class
h    98
l    88
r    93
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 370
75.0 3450
Processing waveform 1 of 370
75.0 4125
Processing waveform 2 of 370
75.0 2400
Processing waveform 3 of 370
75.0 2700
Processing waveform 4 of 370
75.0 2700
Processing waveform 5 of 370
75.0 4350
Processing waveform 6 of 370
75.0 2775
Processing waveform 7 of 370
75.0 1950
Processing waveform 8 of 370
75.0 2250
Processing waveform 9 of 370
75.0 2250
Processing waveform 10 of 370
75.0 2250
Processing waveform 11 of 370
75.0 5100
Processing waveform 12 of 370
75.0 2475
Processing waveform 13 of 370
75.0 3300
Processing waveform 14 of 370
75.0 6300
Processing waveform 15 of 370
75.0 5100
Processing waveform 16 of 370
75.0 9075
Processing waveform 17 of 370
75.0 3225
Processing waveform 18 of 370
75.0 4125
Processing waveform 19 of 370
75.0 3300
Processing waveform 20 of 370
75.0 3225
Processing waveform 21 of 370
75.0 2850
Processing waveform 22 of 370
75.0 5400
Processing waveform 23 of 370
75.0 2400
Processing waveform 24 of 370
75.0 7725
Processing waveform 25 of 370
75.0 3900
Processing waveform 26 of 370
75.0 2775
Processing waveform 27 of 370
75.0 2325
Processing waveform 28 of 370
75.0 4425
Processing waveform 29 of 370
75.0 3525
Processing waveform 30 of 370
75.0 6300
Processing waveform 31 of 370
75.0 2625
Processing waveform 32 of 370
75.0 4050
Processing waveform 33 of 370
75.0 2775
Processing waveform 34 of 370
75.0 2400
Processing waveform 35 of 370
75.0 2850
Processing waveform 36 of 370
75.0 2250
Processing waveform 37 of 370
75.0 2325
Processing waveform 38 of 370
75.0 3750
Processing waveform 39 of 370
75.0 3525
Processing waveform 40 of 370
75.0 5700
Processing waveform 41 of 370
75.0 2700
Processing waveform 42 of 370
75.0 3000
Processing waveform 43 of 370
75.0 4350
Processing waveform 44 of 370
75.0 4950
Processing waveform 45 of 370
75.0 2925
Processing waveform 46 of 370
75.0 2775
Processing waveform 47 of 370
75.0 3450
Processing waveform 48 of 370
75.0 4875
Processing waveform 49 of 370
75.0 9075
Processing waveform 50 of 370
75.0 5175
Processing waveform 51 of 370
75.0 3075
Processing waveform 52 of 370
75.0 4125
Processing waveform 53 of 370
75.0 9075
Processing waveform 54 of 370
75.0 3225
Processing waveform 55 of 370
75.0 2775
Processing waveform 56 of 370
75.0 4575
Processing waveform 57 of 370
75.0 2700
Processing waveform 58 of 370
75.0 1950
Processing waveform 59 of 370
75.0 2100
Processing waveform 60 of 370
75.0 4575
Processing waveform 61 of 370
75.0 5850
Processing waveform 62 of 370
75.0 3150
Processing waveform 63 of 370
75.0 4275
Processing waveform 64 of 370
75.0 9075
Processing waveform 65 of 370
75.0 3600
Processing waveform 66 of 370
75.0 2625
Processing waveform 67 of 370
75.0 2700
Processing waveform 68 of 370
75.0 2700
Processing waveform 69 of 370
75.0 3525
Processing waveform 70 of 370
75.0 4200
Processing waveform 71 of 370
75.0 4200
Processing waveform 72 of 370
75.0 1950
Processing waveform 73 of 370
75.0 5775
Processing waveform 74 of 370
75.0 9075
Processing waveform 75 of 370
75.0 4200
Processing waveform 76 of 370
75.0 2625
Processing waveform 77 of 370
75.0 4875
Processing waveform 78 of 370
75.0 5025

Processing waveform 79 of 370
75.0 14625
Processing waveform 80 of 370
75.0 11025
Processing waveform 81 of 370
75.0 1950
Processing waveform 82 of 370
75.0 3975
Processing waveform 83 of 370
75.0 2550
Processing waveform 84 of 370
75.0 7500
Processing waveform 85 of 370
75.0 14925
Processing waveform 86 of 370
75.0 13275
Processing waveform 87 of 370
75.0 1725
Processing waveform 88 of 370
75.0 2400
Processing waveform 89 of 370
75.0 6375
Processing waveform 90 of 370
75.0 6450
Processing waveform 91 of 370
75.0 16725
Processing waveform 92 of 370
75.0 6075
Processing waveform 93 of 370
75.0 3975
Processing waveform 94 of 370
75.0 2400
Processing waveform 95 of 370
75.0 5025
Processing waveform 96 of 370
75.0 6825
Processing waveform 97 of 370
75.0 3750
Processing waveform 98 of 370
75.0 9075
Processing waveform 99 of 370
75.0 4500
Processing waveform 100 of 370
75.0 2175
Processing waveform 101 of 370
75.0 3300
Processing waveform 102 of 370
75.0 4275
Processing waveform 103 of 370
75.0 2850
Processing waveform 104 of 370
75.0 2850
Processing waveform 105 of 370
75.0 2700
Processing waveform 106 of 370
75.0 5250
Processing waveform 107 of 370
75.0 3000
Processing waveform 108 of 370
75.0 2775
Processing waveform 109 of 370
75.0 6450
Processing waveform 110 of 370
75.0 3150
Processing waveform 111 of 370
75.0 1650
Processing waveform 112 of 370
75.0 4500
Processing waveform 113 of 370
75.0 2250
Processing waveform 114 of 370
75.0 2175
Processing waveform 115 of 370
75.0 3225
Processing waveform 116 of 370
75.0 4050
Processing waveform 117 of 370
75.0 3300
Processing waveform 118 of 370
75.0 2850
Processing waveform 119 of 370
75.0 2025
Processing waveform 120 of 370
75.0 4800
Processing waveform 121 of 370
75.0 2325
Processing waveform 122 of 370
75.0 6150
Processing waveform 123 of 370
75.0 2775
Processing waveform 124 of 370
75.0 2850
Processing waveform 125 of 370
75.0 4425
Processing waveform 126 of 370
75.0 2250
Processing waveform 127 of 370
75.0 3075
Processing waveform 128 of 370
75.0 2775
Processing waveform 129 of 370
75.0 1875
Processing waveform 130 of 370
75.0 3300
Processing waveform 131 of 370
75.0 1875
Processing waveform 132 of 370
75.0 5550
Processing waveform 133 of 370
75.0 9075
Processing waveform 134 of 370
75.0 4650
Processing waveform 135 of 370
75.0 6525
Processing waveform 136 of 370
75.0 9075
Processing waveform 137 of 370
75.0 3225
Processing waveform 138 of 370
75.0 3150
Processing waveform 139 of 370
75.0 4275
Processing waveform 140 of 370
75.0 4050
Processing waveform 141 of 370
75.0 5400
Processing waveform 142 of 370
75.0 4725
Processing waveform 143 of 370
75.0 3450
Processing waveform 144 of 370
75.0 6150
Processing waveform 145 of 370
75.0 2250
Processing waveform 146 of 370
75.0 9000
Processing waveform 147 of 370
75.0 3150
Processing waveform 148 of 370
75.0 4950
Processing waveform 149 of 370
75.0 6750
Processing waveform 150 of 370
75.0 3150
Processing waveform 151 of 370
75.0 7050
Processing waveform 152 of 370
75.0 3675
Processing waveform 153 of 370
75.0 3225
Processing waveform 154 of 370
75.0 2325
Processing waveform 155 of 370
75.0 7500
Processing waveform 156 of 370
75.0 3975
Processing waveform 157 of 370
75.0 6150
Processing waveform 158 of 370
75.0 7875
Processing waveform 159 of 370
75.0 7275
Processing waveform 160 of 370
75.0 6075
Processing waveform 161 of 370
75.0 3900
Processing waveform 162 of 370
75.0 3000
Processing waveform 163 of 370
75.0 3900
Processing waveform 164 of 370
75.0 6525
Processing waveform 165 of 370
75.0 2925
Processing waveform 166 of 370
75.0 2700
Processing waveform 167 of 370
75.0 2250
Processing waveform 168 of 370
75.0 4425
Processing waveform 169 of 370
75.0 7500
Processing waveform 170 of 370
75.0 2250
Processing waveform 171 of 370
75.0 9600
Processing waveform 172 of 370
75.0 4650
Processing waveform 173 of 370
75.0 6750
Processing waveform 174 of 370
75.0 8925
Processing waveform 175 of 370
75.0 7125
Processing waveform 176 of 370
75.0 3000
Processing waveform 177 of 370
75.0 1800
Processing waveform 178 of 370
75.0 4425
Processing waveform 179 of 370
75.0 9450
Processing waveform 180 of 370
75.0 6525
Processing waveform 181 of 370
75.0 3225
Processing waveform 182 of 370
75.0 2100
Processing waveform 183 of 370
75.0 2700
Processing waveform 184 of 370
75.0 2325
Processing waveform 185 of 370
75.0 2775
Processing waveform 186 of 370
75.0 5850
Processing waveform 187 of 370
75.0 3825
Processing waveform 188 of 370
75.0 6000
Processing waveform 189 of 370
75.0 2025
Processing waveform 190 of 370
75.0 1725
Processing waveform 191 of 370
75.0 2175
Processing waveform 192 of 370
75.0 5400
Processing waveform 193 of 370
75.0 3000
Processing waveform 194 of 370
75.0 5325
Processing waveform 195 of 370
75.0 3750
Processing waveform 196 of 370
75.0 3075
Processing waveform 197 of 370
75.0 5550
Processing waveform 198 of 370
75.0 7350
Processing waveform 199 of 370
75.0 4650
Processing waveform 200 of 370
75.0 2775
Processing waveform 201 of 370
75.0 2325
Processing waveform 202 of 370
75.0 2925
Processing waveform 203 of 370
75.0 3075
Processing waveform 204 of 370
75.0 6375
Processing waveform 205 of 370
75.0 2475
Processing waveform 206 of 370
75.0 3975
Processing waveform 207 of 370
75.0 3150
Processing waveform 208 of 370
75.0 5925
Processing waveform 209 of 370
75.0 5400
Processing waveform 210 of 370
75.0 1875
Processing waveform 211 of 370
75.0 5850
Processing waveform 212 of 370
75.0 1725
Processing waveform 213 of 370
75.0 7650
Processing waveform 214 of 370
75.0 2700
Processing waveform 215 of 370
75.0 9075
Processing waveform 216 of 370
75.0 6975
Processing waveform 217 of 370
75.0 4050
Processing waveform 218 of 370
75.0 3450
Processing waveform 219 of 370
75.0 5925
Processing waveform 220 of 370
75.0 2250
Processing waveform 221 of 370
75.0 3675
Processing waveform 222 of 370
75.0 2625
Processing waveform 223 of 370
75.0 4575
Processing waveform 224 of 370
75.0 2025
Processing waveform 225 of 370
75.0 5175
Processing waveform 226 of 370
75.0 3225
Processing waveform 227 of 370
75.0 4950
Processing waveform 228 of 370
75.0 2925
Processing waveform 229 of 370
75.0 1800
Processing waveform 230 of 370
75.0 8925
Processing waveform 231 of 370
75.0 7350
Processing waveform 232 of 370
75.0 4125
Processing waveform 233 of 370
75.0 3225
Processing waveform 234 of 370
75.0 9075
Processing waveform 235 of 370
75.0 5025
Processing waveform 236 of 370
75.0 8925
Processing waveform 237 of 370
75.0 4800
Processing waveform 238 of 370
75.0 7800
Processing waveform 239 of 370
75.0 2775
Processing waveform 240 of 370
75.0 6600
Processing waveform 241 of 370
75.0 1800
Processing waveform 242 of 370
75.0 2625
Processing waveform 243 of 370
75.0 9075
Processing waveform 244 of 370
75.0 7200
Processing waveform 245 of 370
75.0 5550
Processing waveform 246 of 370
75.0 2775
Processing waveform 247 of 370
75.0 5250
Processing waveform 248 of 370
75.0 1875
Processing waveform 249 of 370
75.0 1875
Processing waveform 250 of 370
75.0 5025
Processing waveform 251 of 370
75.0 3600
Processing waveform 252 of 370
75.0 3375
Processing waveform 253 of 370
75.0 4050
Processing waveform 254 of 370
75.0 3375
Processing waveform 255 of 370
75.0 2925
Processing waveform 256 of 370
75.0 6675
Processing waveform 257 of 370
75.0 3750
Processing waveform 258 of 370
75.0 4575
Processing waveform 259 of 370
75.0 9075
Processing waveform 260 of 370
75.0 3900
Processing waveform 261 of 370
75.0 9075
Processing waveform 262 of 370
75.0 4875
Processing waveform 263 of 370
75.0 3075
Processing waveform 264 of 370
75.0 3150
Processing waveform 265 of 370
75.0 2550
Processing waveform 266 of 370
75.0 9075
Processing waveform 267 of 370
75.0 7425
Processing waveform 268 of 370
75.0 5025
Processing waveform 269 of 370
75.0 2850
Processing waveform 270 of 370
75.0 1875
Processing waveform 271 of 370
75.0 7125
Processing waveform 272 of 370
75.0 4500
Processing waveform 273 of 370
75.0 6075
Processing waveform 274 of 370
75.0 2475
Processing waveform 275 of 370
75.0 4125
Processing waveform 276 of 370
75.0 2100
Processing waveform 277 of 370
75.0 3975
Processing waveform 278 of 370
75.0 3075
Processing waveform 279 of 370
75.0 5550

Processing waveform 280 of 370
75.0 2250
Processing waveform 281 of 370
75.0 3600
Processing waveform 282 of 370
75.0 5775
Processing waveform 283 of 370
75.0 2325
Processing waveform 284 of 370
75.0 2325
Processing waveform 285 of 370
75.0 3525
Processing waveform 286 of 370
75.0 4500
Processing waveform 287 of 370
75.0 4275
Processing waveform 288 of 370
75.0 3450
Processing waveform 289 of 370
75.0 2550
Processing waveform 290 of 370
75.0 3300
Processing waveform 291 of 370
75.0 9075
Processing waveform 292 of 370
75.0 9075
Processing waveform 293 of 370
75.0 6525
Processing waveform 294 of 370
75.0 6975
Processing waveform 295 of 370
75.0 6750
Processing waveform 296 of 370
75.0 7275
Processing waveform 297 of 370
75.0 2400
Processing waveform 298 of 370
75.0 4125
Processing waveform 299 of 370
75.0 3525
Processing waveform 300 of 370
75.0 5175
Processing waveform 301 of 370
75.0 3750
Processing waveform 302 of 370
75.0 4875
Processing waveform 303 of 370
75.0 2250
Processing waveform 304 of 370
75.0 4125
Processing waveform 305 of 370
75.0 4425
Processing waveform 306 of 370
75.0 5250
Processing waveform 307 of 370
75.0 9075
Processing waveform 308 of 370
75.0 4725
Processing waveform 309 of 370
75.0 2475
Processing waveform 310 of 370
75.0 2925
Processing waveform 311 of 370
75.0 7350
Processing waveform 312 of 370
75.0 2550
Processing waveform 313 of 370
75.0 2850
Processing waveform 314 of 370
75.0 2550
Processing waveform 315 of 370
75.0 5775
Processing waveform 316 of 370
75.0 4200
Processing waveform 317 of 370
75.0 2175
Processing waveform 318 of 370
75.0 2925
Processing waveform 319 of 370
75.0 10350
Processing waveform 320 of 370
75.0 15600
Processing waveform 321 of 370
75.0 6450
Processing waveform 322 of 370
75.0 3450
Processing waveform 323 of 370
75.0 11175
Processing waveform 324 of 370
75.0 4725
Processing waveform 325 of 370
75.0 4200
Processing waveform 326 of 370
75.0 3750
Processing waveform 327 of 370
75.0 3975
Processing waveform 328 of 370
75.0 4575
Processing waveform 329 of 370
75.0 3225
Processing waveform 330 of 370
75.0 13500
Processing waveform 331 of 370
75.0 4050
Processing waveform 332 of 370
75.0 5250
Processing waveform 333 of 370
75.0 7050
Processing waveform 334 of 370
75.0 3450
Processing waveform 335 of 370
75.0 6675
Processing waveform 336 of 370
75.0 6750
Processing waveform 337 of 370
75.0 2475
Processing waveform 338 of 370
75.0 7650
Processing waveform 339 of 370
75.0 6750
Processing waveform 340 of 370
75.0 4125
Processing waveform 341 of 370
75.0 7425
Processing waveform 342 of 370
75.0 5400
Processing waveform 343 of 370
75.0 4800
Processing waveform 344 of 370
75.0 10200
Processing waveform 345 of 370
75.0 18225
Processing waveform 346 of 370
75.0 5925
Processing waveform 347 of 370
75.0 3525
Processing waveform 348 of 370
75.0 3300
Processing waveform 349 of 370
75.0 3600
Processing waveform 350 of 370
75.0 4650
Processing waveform 351 of 370
75.0 7875
Processing waveform 352 of 370
75.0 6300
Processing waveform 353 of 370
75.0 6750
Processing waveform 354 of 370
75.0 2925
Processing waveform 355 of 370
75.0 3525
Processing waveform 356 of 370
75.0 7875
Processing waveform 357 of 370
75.0 8925
Processing waveform 358 of 370
75.0 2250
Processing waveform 359 of 370
75.0 2700
Processing waveform 360 of 370
75.0 3000
Processing waveform 361 of 370
75.0 3300
Processing waveform 362 of 370
75.0 4425
Processing waveform 363 of 370
75.0 4500
Processing waveform 364 of 370
75.0 7500
Processing waveform 365 of 370
75.0 4200
Processing waveform 366 of 370
75.0 4800
Processing waveform 367 of 370
75.0 1950
Processing waveform 368 of 370
75.0 4650
Processing waveform 369 of 370
75.0 5475
Training data have been read and features have been extracted  (370, 120)
Computation time:  494.3386580944061
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.461331844329834
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h    98                   
        l          88             
        r                93       
        t                      91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  25.8738738739  +/-  2.61379283687  %
          Predicted class
              h     l     r     t 
        h    11    13    11    13 
        l    13    10    11     9 
        r    14    10    12    10 
        t    14     8     9    14 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'h', 'l', 't', 'r'] 0
class
e    78
h    98
l    88
r    93
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 448
75.0 3450
Processing waveform 1 of 448
75.0 4125
Processing waveform 2 of 448
75.0 2400
Processing waveform 3 of 448
75.0 2700
Processing waveform 4 of 448
75.0 2700
Processing waveform 5 of 448
75.0 4350
Processing waveform 6 of 448
75.0 2775
Processing waveform 7 of 448
75.0 1950
Processing waveform 8 of 448
75.0 2250
Processing waveform 9 of 448
75.0 2250
Processing waveform 10 of 448
75.0 2250
Processing waveform 11 of 448
75.0 5100
Processing waveform 12 of 448
75.0 2475
Processing waveform 13 of 448
75.0 3300
Processing waveform 14 of 448
75.0 6300
Processing waveform 15 of 448
75.0 5100
Processing waveform 16 of 448
75.0 9075
Processing waveform 17 of 448
75.0 3225
Processing waveform 18 of 448
75.0 4125
Processing waveform 19 of 448
75.0 3300
Processing waveform 20 of 448
75.0 3225
Processing waveform 21 of 448
75.0 2850
Processing waveform 22 of 448
75.0 5400
Processing waveform 23 of 448
75.0 2400
Processing waveform 24 of 448
75.0 7725
Processing waveform 25 of 448
75.0 3900
Processing waveform 26 of 448
75.0 2775
Processing waveform 27 of 448
75.0 2325
Processing waveform 28 of 448
75.0 4425
Processing waveform 29 of 448
75.0 3525
Processing waveform 30 of 448
75.0 6300
Processing waveform 31 of 448
75.0 2625
Processing waveform 32 of 448
75.0 4050
Processing waveform 33 of 448
75.0 2775
Processing waveform 34 of 448
75.0 2400
Processing waveform 35 of 448
75.0 2850
Processing waveform 36 of 448
75.0 2250
Processing waveform 37 of 448
75.0 2325
Processing waveform 38 of 448
75.0 3750
Processing waveform 39 of 448
75.0 3525
Processing waveform 40 of 448
75.0 5700
Processing waveform 41 of 448
75.0 2700
Processing waveform 42 of 448
75.0 3000
Processing waveform 43 of 448
75.0 4350
Processing waveform 44 of 448
75.0 4950
Processing waveform 45 of 448
75.0 2925
Processing waveform 46 of 448
75.0 2775
Processing waveform 47 of 448
75.0 3450
Processing waveform 48 of 448
75.0 4875
Processing waveform 49 of 448
75.0 9075
Processing waveform 50 of 448
75.0 5175
Processing waveform 51 of 448
75.0 3075
Processing waveform 52 of 448
75.0 4125
Processing waveform 53 of 448
75.0 9075
Processing waveform 54 of 448
75.0 3225
Processing waveform 55 of 448
75.0 2775
Processing waveform 56 of 448
75.0 4575
Processing waveform 57 of 448
75.0 2700
Processing waveform 58 of 448
75.0 1950
Processing waveform 59 of 448
75.0 2100
Processing waveform 60 of 448
75.0 4575
Processing waveform 61 of 448
75.0 5850
Processing waveform 62 of 448
75.0 3150
Processing waveform 63 of 448
75.0 4275
Processing waveform 64 of 448
75.0 9075
Processing waveform 65 of 448
75.0 3600
Processing waveform 66 of 448
75.0 2625
Processing waveform 67 of 448
75.0 2700
Processing waveform 68 of 448
75.0 2700
Processing waveform 69 of 448
75.0 3525
Processing waveform 70 of 448
75.0 4200
Processing waveform 71 of 448
75.0 4200
Processing waveform 72 of 448
75.0 1950
Processing waveform 73 of 448
75.0 5775
Processing waveform 74 of 448
75.0 9075
Processing waveform 75 of 448
75.0 4200
Processing waveform 76 of 448
75.0 2625
Processing waveform 77 of 448
75.0 4875
Processing waveform 78 of 448
75.0 5025
Processing waveform 79 of 448
75.0 14625
Processing waveform 80 of 448
75.0 11025
Processing waveform 81 of 448
75.0 1950
Processing waveform 82 of 448
75.0 3975

Processing waveform 83 of 448
75.0 2550
Processing waveform 84 of 448
75.0 7500
Processing waveform 85 of 448
75.0 14925
Processing waveform 86 of 448
75.0 13275
Processing waveform 87 of 448
75.0 1725
Processing waveform 88 of 448
75.0 2400
Processing waveform 89 of 448
75.0 6375
Processing waveform 90 of 448
75.0 6450
Processing waveform 91 of 448
75.0 16725
Processing waveform 92 of 448
75.0 6075
Processing waveform 93 of 448
75.0 3450
Processing waveform 94 of 448
75.0 2325
Processing waveform 95 of 448
75.0 2925
Processing waveform 96 of 448
75.0 4575
Processing waveform 97 of 448
75.0 7050
Processing waveform 98 of 448
75.0 2025
Processing waveform 99 of 448
75.0 2925
Processing waveform 100 of 448
75.0 2850
Processing waveform 101 of 448
75.0 3600
Processing waveform 102 of 448
75.0 3750
Processing waveform 103 of 448
75.0 2100
Processing waveform 104 of 448
75.0 1650
Processing waveform 105 of 448
75.0 4200
Processing waveform 106 of 448
75.0 9075
Processing waveform 107 of 448
75.0 4125
Processing waveform 108 of 448
75.0 2925
Processing waveform 109 of 448
75.0 2250
Processing waveform 110 of 448
75.0 5475
Processing waveform 111 of 448
75.0 2625
Processing waveform 112 of 448
75.0 2400
Processing waveform 113 of 448
75.0 2700
Processing waveform 114 of 448
75.0 9075
Processing waveform 115 of 448
75.0 9075
Processing waveform 116 of 448
75.0 6225
Processing waveform 117 of 448
75.0 7125
Processing waveform 118 of 448
75.0 5850
Processing waveform 119 of 448
75.0 9075
Processing waveform 120 of 448
75.0 3225
Processing waveform 121 of 448
75.0 8100
Processing waveform 122 of 448
75.0 4575
Processing waveform 123 of 448
75.0 2850
Processing waveform 124 of 448
75.0 3375
Processing waveform 125 of 448
75.0 2100
Processing waveform 126 of 448
75.0 9075
Processing waveform 127 of 448
75.0 5850
Processing waveform 128 of 448
75.0 2775
Processing waveform 129 of 448
75.0 2700
Processing waveform 130 of 448
75.0 2100
Processing waveform 131 of 448
75.0 9075
Processing waveform 132 of 448
75.0 3225
Processing waveform 133 of 448
75.0 3300
Processing waveform 134 of 448
75.0 2475
Processing waveform 135 of 448
75.0 3075
Processing waveform 136 of 448
75.0 6450
Processing waveform 137 of 448
75.0 3600
Processing waveform 138 of 448
75.0 4650
Processing waveform 139 of 448
75.0 2475
Processing waveform 140 of 448
75.0 1875
Processing waveform 141 of 448
75.0 3450
Processing waveform 142 of 448
75.0 1500
Processing waveform 143 of 448
75.0 2625
Processing waveform 144 of 448
75.0 3525
Processing waveform 145 of 448
75.0 5625
Processing waveform 146 of 448
75.0 3675
Processing waveform 147 of 448
75.0 9075
Processing waveform 148 of 448
75.0 2700
Processing waveform 149 of 448
75.0 9075
Processing waveform 150 of 448
75.0 3000
Processing waveform 151 of 448
75.0 3525
Processing waveform 152 of 448
75.0 7200
Processing waveform 153 of 448
75.0 4875
Processing waveform 154 of 448
75.0 6075
Processing waveform 155 of 448
75.0 2700
Processing waveform 156 of 448
75.0 5250
Processing waveform 157 of 448
75.0 12750
Processing waveform 158 of 448
75.0 5100
Processing waveform 159 of 448
75.0 2475
Processing waveform 160 of 448
75.0 6525
Processing waveform 161 of 448
75.0 7200
Processing waveform 162 of 448
75.0 4500
Processing waveform 163 of 448
75.0 4950
Processing waveform 164 of 448
75.0 3150
Processing waveform 165 of 448
75.0 5850
Processing waveform 166 of 448
75.0 2325
Processing waveform 167 of 448
75.0 4200
Processing waveform 168 of 448
75.0 13350
Processing waveform 169 of 448
75.0 9525
Processing waveform 170 of 448
75.0 2325
Processing waveform 171 of 448
75.0 3975
Processing waveform 172 of 448
75.0 2400
Processing waveform 173 of 448
75.0 5025
Processing waveform 174 of 448
75.0 6825
Processing waveform 175 of 448
75.0 3750
Processing waveform 176 of 448
75.0 9075
Processing waveform 177 of 448
75.0 4500
Processing waveform 178 of 448
75.0 2175
Processing waveform 179 of 448
75.0 3300
Processing waveform 180 of 448
75.0 4275
Processing waveform 181 of 448
75.0 2850
Processing waveform 182 of 448
75.0 2850
Processing waveform 183 of 448
75.0 2700
Processing waveform 184 of 448
75.0 5250
Processing waveform 185 of 448
75.0 3000
Processing waveform 186 of 448
75.0 2775
Processing waveform 187 of 448
75.0 6450
Processing waveform 188 of 448
75.0 3150
Processing waveform 189 of 448
75.0 1650
Processing waveform 190 of 448
75.0 4500
Processing waveform 191 of 448
75.0 2250
Processing waveform 192 of 448
75.0 2175
Processing waveform 193 of 448
75.0 3225
Processing waveform 194 of 448
75.0 4050
Processing waveform 195 of 448
75.0 3300
Processing waveform 196 of 448
75.0 2850
Processing waveform 197 of 448
75.0 2025
Processing waveform 198 of 448
75.0 4800
Processing waveform 199 of 448
75.0 2325
Processing waveform 200 of 448
75.0 6150
Processing waveform 201 of 448
75.0 2775
Processing waveform 202 of 448
75.0 2850
Processing waveform 203 of 448
75.0 4425
Processing waveform 204 of 448
75.0 2250
Processing waveform 205 of 448
75.0 3075
Processing waveform 206 of 448
75.0 2775
Processing waveform 207 of 448
75.0 1875
Processing waveform 208 of 448
75.0 3300
Processing waveform 209 of 448
75.0 1875
Processing waveform 210 of 448
75.0 5550
Processing waveform 211 of 448
75.0 9075
Processing waveform 212 of 448
75.0 4650
Processing waveform 213 of 448
75.0 6525
Processing waveform 214 of 448
75.0 9075
Processing waveform 215 of 448
75.0 3225
Processing waveform 216 of 448
75.0 3150
Processing waveform 217 of 448
75.0 4275
Processing waveform 218 of 448
75.0 4050
Processing waveform 219 of 448
75.0 5400
Processing waveform 220 of 448
75.0 4725
Processing waveform 221 of 448
75.0 3450
Processing waveform 222 of 448
75.0 6150
Processing waveform 223 of 448
75.0 2250
Processing waveform 224 of 448
75.0 9000
Processing waveform 225 of 448
75.0 3150
Processing waveform 226 of 448
75.0 4950
Processing waveform 227 of 448
75.0 6750
Processing waveform 228 of 448
75.0 3150
Processing waveform 229 of 448
75.0 7050
Processing waveform 230 of 448
75.0 3675
Processing waveform 231 of 448
75.0 3225
Processing waveform 232 of 448
75.0 2325
Processing waveform 233 of 448
75.0 7500
Processing waveform 234 of 448
75.0 3975
Processing waveform 235 of 448
75.0 6150
Processing waveform 236 of 448
75.0 7875
Processing waveform 237 of 448
75.0 7275
Processing waveform 238 of 448
75.0 6075
Processing waveform 239 of 448
75.0 3900
Processing waveform 240 of 448
75.0 3000
Processing waveform 241 of 448
75.0 3900
Processing waveform 242 of 448
75.0 6525
Processing waveform 243 of 448
75.0 2925
Processing waveform 244 of 448
75.0 2700
Processing waveform 245 of 448
75.0 2250
Processing waveform 246 of 448
75.0 4425
Processing waveform 247 of 448
75.0 7500
Processing waveform 248 of 448
75.0 2250
Processing waveform 249 of 448
75.0 9600
Processing waveform 250 of 448
75.0 4650
Processing waveform 251 of 448
75.0 6750
Processing waveform 252 of 448
75.0 8925
Processing waveform 253 of 448
75.0 7125
Processing waveform 254 of 448
75.0 3000
Processing waveform 255 of 448
75.0 1800
Processing waveform 256 of 448
75.0 4425
Processing waveform 257 of 448
75.0 9450
Processing waveform 258 of 448
75.0 6525
Processing waveform 259 of 448
75.0 3225
Processing waveform 260 of 448
75.0 2100
Processing waveform 261 of 448
75.0 2700
Processing waveform 262 of 448
75.0 2325
Processing waveform 263 of 448
75.0 2775
Processing waveform 264 of 448
75.0 5850
Processing waveform 265 of 448
75.0 3825
Processing waveform 266 of 448
75.0 6000
Processing waveform 267 of 448
75.0 2025
Processing waveform 268 of 448
75.0 1725
Processing waveform 269 of 448
75.0 2175
Processing waveform 270 of 448
75.0 5400
Processing waveform 271 of 448
75.0 3000
Processing waveform 272 of 448
75.0 5325
Processing waveform 273 of 448
75.0 3750
Processing waveform 274 of 448
75.0 3075
Processing waveform 275 of 448
75.0 5550
Processing waveform 276 of 448
75.0 7350
Processing waveform 277 of 448
75.0 4650
Processing waveform 278 of 448
75.0 2775
Processing waveform 279 of 448
75.0 2325
Processing waveform 280 of 448
75.0 2925
Processing waveform 281 of 448
75.0 3075
Processing waveform 282 of 448
75.0 6375
Processing waveform 283 of 448
75.0 2475

Processing waveform 284 of 448
75.0 3975
Processing waveform 285 of 448
75.0 3150
Processing waveform 286 of 448
75.0 5925
Processing waveform 287 of 448
75.0 5400
Processing waveform 288 of 448
75.0 1875
Processing waveform 289 of 448
75.0 5850
Processing waveform 290 of 448
75.0 1725
Processing waveform 291 of 448
75.0 7650
Processing waveform 292 of 448
75.0 2700
Processing waveform 293 of 448
75.0 9075
Processing waveform 294 of 448
75.0 6975
Processing waveform 295 of 448
75.0 4050
Processing waveform 296 of 448
75.0 3450
Processing waveform 297 of 448
75.0 5925
Processing waveform 298 of 448
75.0 2250
Processing waveform 299 of 448
75.0 3675
Processing waveform 300 of 448
75.0 2625
Processing waveform 301 of 448
75.0 4575
Processing waveform 302 of 448
75.0 2025
Processing waveform 303 of 448
75.0 5175
Processing waveform 304 of 448
75.0 3225
Processing waveform 305 of 448
75.0 4950
Processing waveform 306 of 448
75.0 2925
Processing waveform 307 of 448
75.0 1800
Processing waveform 308 of 448
75.0 8925
Processing waveform 309 of 448
75.0 7350
Processing waveform 310 of 448
75.0 4125
Processing waveform 311 of 448
75.0 3225
Processing waveform 312 of 448
75.0 9075
Processing waveform 313 of 448
75.0 5025
Processing waveform 314 of 448
75.0 8925
Processing waveform 315 of 448
75.0 4800
Processing waveform 316 of 448
75.0 7800
Processing waveform 317 of 448
75.0 2775
Processing waveform 318 of 448
75.0 6600
Processing waveform 319 of 448
75.0 1800
Processing waveform 320 of 448
75.0 2625
Processing waveform 321 of 448
75.0 9075
Processing waveform 322 of 448
75.0 7200
Processing waveform 323 of 448
75.0 5550
Processing waveform 324 of 448
75.0 2775
Processing waveform 325 of 448
75.0 5250
Processing waveform 326 of 448
75.0 1875
Processing waveform 327 of 448
75.0 1875
Processing waveform 328 of 448
75.0 5025
Processing waveform 329 of 448
75.0 3600
Processing waveform 330 of 448
75.0 3375
Processing waveform 331 of 448
75.0 4050
Processing waveform 332 of 448
75.0 3375
Processing waveform 333 of 448
75.0 2925
Processing waveform 334 of 448
75.0 6675
Processing waveform 335 of 448
75.0 3750
Processing waveform 336 of 448
75.0 4575
Processing waveform 337 of 448
75.0 9075
Processing waveform 338 of 448
75.0 3900
Processing waveform 339 of 448
75.0 9075
Processing waveform 340 of 448
75.0 4875
Processing waveform 341 of 448
75.0 3075
Processing waveform 342 of 448
75.0 3150
Processing waveform 343 of 448
75.0 2550
Processing waveform 344 of 448
75.0 9075
Processing waveform 345 of 448
75.0 7425
Processing waveform 346 of 448
75.0 5025
Processing waveform 347 of 448
75.0 2850
Processing waveform 348 of 448
75.0 1875
Processing waveform 349 of 448
75.0 7125
Processing waveform 350 of 448
75.0 4500
Processing waveform 351 of 448
75.0 6075
Processing waveform 352 of 448
75.0 2475
Processing waveform 353 of 448
75.0 4125
Processing waveform 354 of 448
75.0 2100
Processing waveform 355 of 448
75.0 3975
Processing waveform 356 of 448
75.0 3075
Processing waveform 357 of 448
75.0 5550
Processing waveform 358 of 448
75.0 2250
Processing waveform 359 of 448
75.0 3600
Processing waveform 360 of 448
75.0 5775
Processing waveform 361 of 448
75.0 2325
Processing waveform 362 of 448
75.0 2325
Processing waveform 363 of 448
75.0 3525
Processing waveform 364 of 448
75.0 4500
Processing waveform 365 of 448
75.0 4275
Processing waveform 366 of 448
75.0 3450
Processing waveform 367 of 448
75.0 2550
Processing waveform 368 of 448
75.0 3300
Processing waveform 369 of 448
75.0 9075
Processing waveform 370 of 448
75.0 9075
Processing waveform 371 of 448
75.0 6525
Processing waveform 372 of 448
75.0 6975
Processing waveform 373 of 448
75.0 6750
Processing waveform 374 of 448
75.0 7275
Processing waveform 375 of 448
75.0 2400
Processing waveform 376 of 448
75.0 4125
Processing waveform 377 of 448
75.0 3525
Processing waveform 378 of 448
75.0 5175
Processing waveform 379 of 448
75.0 3750
Processing waveform 380 of 448
75.0 4875
Processing waveform 381 of 448
75.0 2250
Processing waveform 382 of 448
75.0 4125
Processing waveform 383 of 448
75.0 4425
Processing waveform 384 of 448
75.0 5250
Processing waveform 385 of 448
75.0 9075
Processing waveform 386 of 448
75.0 4725
Processing waveform 387 of 448
75.0 2475
Processing waveform 388 of 448
75.0 2925
Processing waveform 389 of 448
75.0 7350
Processing waveform 390 of 448
75.0 2550
Processing waveform 391 of 448
75.0 2850
Processing waveform 392 of 448
75.0 2550
Processing waveform 393 of 448
75.0 5775
Processing waveform 394 of 448
75.0 4200
Processing waveform 395 of 448
75.0 2175
Processing waveform 396 of 448
75.0 2925
Processing waveform 397 of 448
75.0 10350
Processing waveform 398 of 448
75.0 15600
Processing waveform 399 of 448
75.0 6450
Processing waveform 400 of 448
75.0 3450
Processing waveform 401 of 448
75.0 11175
Processing waveform 402 of 448
75.0 4725
Processing waveform 403 of 448
75.0 4200
Processing waveform 404 of 448
75.0 3750
Processing waveform 405 of 448
75.0 3975
Processing waveform 406 of 448
75.0 4575
Processing waveform 407 of 448
75.0 3225
Processing waveform 408 of 448
75.0 13500
Processing waveform 409 of 448
75.0 4050
Processing waveform 410 of 448
75.0 5250
Processing waveform 411 of 448
75.0 7050
Processing waveform 412 of 448
75.0 3450
Processing waveform 413 of 448
75.0 6675
Processing waveform 414 of 448
75.0 6750
Processing waveform 415 of 448
75.0 2475
Processing waveform 416 of 448
75.0 7650
Processing waveform 417 of 448
75.0 6750
Processing waveform 418 of 448
75.0 4125
Processing waveform 419 of 448
75.0 7425
Processing waveform 420 of 448
75.0 5400
Processing waveform 421 of 448
75.0 4800
Processing waveform 422 of 448
75.0 10200
Processing waveform 423 of 448
75.0 18225
Processing waveform 424 of 448
75.0 5925
Processing waveform 425 of 448
75.0 3525
Processing waveform 426 of 448
75.0 3300
Processing waveform 427 of 448
75.0 3600
Processing waveform 428 of 448
75.0 4650
Processing waveform 429 of 448
75.0 7875
Processing waveform 430 of 448
75.0 6300
Processing waveform 431 of 448
75.0 6750
Processing waveform 432 of 448
75.0 2925
Processing waveform 433 of 448
75.0 3525
Processing waveform 434 of 448
75.0 7875
Processing waveform 435 of 448
75.0 8925
Processing waveform 436 of 448
75.0 2250
Processing waveform 437 of 448
75.0 2700
Processing waveform 438 of 448
75.0 3000
Processing waveform 439 of 448
75.0 3300
Processing waveform 440 of 448
75.0 4425
Processing waveform 441 of 448
75.0 4500
Processing waveform 442 of 448
75.0 7500
Processing waveform 443 of 448
75.0 4200
Processing waveform 444 of 448
75.0 4800
Processing waveform 445 of 448
75.0 1950
Processing waveform 446 of 448
75.0 4650
Processing waveform 447 of 448
75.0 5475
Training data have been read and features have been extracted  (448, 120)
Computation time:  600.7141726016998
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.6462020874023438
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    78                         
        h          98                   
        l                88             
        r                      93       
        t                            91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  21.6964285714  +/-  2.56350050984  %
          Predicted class
              e     h     l     r     t 
        e     5    10     5     8    10 
        h     5    11    11    12    10 
        l     4    12    11    10     7 
        r     6    12    10    11     8 
        t     7    12     8     8    11 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['l', 't'] 1
class
l    88
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 179
75.0 3975
Processing waveform 1 of 179
75.0 2400
Processing waveform 2 of 179
75.0 5025
Processing waveform 3 of 179
75.0 6825
Processing waveform 4 of 179
75.0 3750

Processing waveform 5 of 179
75.0 9075
Processing waveform 6 of 179
75.0 4500
Processing waveform 7 of 179
75.0 2175
Processing waveform 8 of 179
75.0 3300
Processing waveform 9 of 179
75.0 4275
Processing waveform 10 of 179
75.0 2850
Processing waveform 11 of 179
75.0 2850
Processing waveform 12 of 179
75.0 2700
Processing waveform 13 of 179
75.0 5250
Processing waveform 14 of 179
75.0 3000
Processing waveform 15 of 179
75.0 2775
Processing waveform 16 of 179
75.0 6450
Processing waveform 17 of 179
75.0 3150
Processing waveform 18 of 179
75.0 1650
Processing waveform 19 of 179
75.0 4500
Processing waveform 20 of 179
75.0 2250
Processing waveform 21 of 179
75.0 2175
Processing waveform 22 of 179
75.0 3225
Processing waveform 23 of 179
75.0 4050
Processing waveform 24 of 179
75.0 3300
Processing waveform 25 of 179
75.0 2850
Processing waveform 26 of 179
75.0 2025
Processing waveform 27 of 179
75.0 4800
Processing waveform 28 of 179
75.0 2325
Processing waveform 29 of 179
75.0 6150
Processing waveform 30 of 179
75.0 2775
Processing waveform 31 of 179
75.0 2850
Processing waveform 32 of 179
75.0 4425
Processing waveform 33 of 179
75.0 2250
Processing waveform 34 of 179
75.0 3075
Processing waveform 35 of 179
75.0 2775
Processing waveform 36 of 179
75.0 1875
Processing waveform 37 of 179
75.0 3300
Processing waveform 38 of 179
75.0 1875
Processing waveform 39 of 179
75.0 5550
Processing waveform 40 of 179
75.0 9075
Processing waveform 41 of 179
75.0 4650
Processing waveform 42 of 179
75.0 6525
Processing waveform 43 of 179
75.0 9075
Processing waveform 44 of 179
75.0 3225
Processing waveform 45 of 179
75.0 3150
Processing waveform 46 of 179
75.0 4275
Processing waveform 47 of 179
75.0 4050
Processing waveform 48 of 179
75.0 5400
Processing waveform 49 of 179
75.0 4725
Processing waveform 50 of 179
75.0 3450
Processing waveform 51 of 179
75.0 6150
Processing waveform 52 of 179
75.0 2250
Processing waveform 53 of 179
75.0 9000
Processing waveform 54 of 179
75.0 3150
Processing waveform 55 of 179
75.0 4950
Processing waveform 56 of 179
75.0 6750
Processing waveform 57 of 179
75.0 3150
Processing waveform 58 of 179
75.0 7050
Processing waveform 59 of 179
75.0 3675
Processing waveform 60 of 179
75.0 3225
Processing waveform 61 of 179
75.0 2325
Processing waveform 62 of 179
75.0 7500
Processing waveform 63 of 179
75.0 3975
Processing waveform 64 of 179
75.0 6150
Processing waveform 65 of 179
75.0 7875
Processing waveform 66 of 179
75.0 7275
Processing waveform 67 of 179
75.0 6075
Processing waveform 68 of 179
75.0 3900
Processing waveform 69 of 179
75.0 3000
Processing waveform 70 of 179
75.0 3900
Processing waveform 71 of 179
75.0 6525
Processing waveform 72 of 179
75.0 2925
Processing waveform 73 of 179
75.0 2700
Processing waveform 74 of 179
75.0 2250
Processing waveform 75 of 179
75.0 4425
Processing waveform 76 of 179
75.0 7500
Processing waveform 77 of 179
75.0 2250
Processing waveform 78 of 179
75.0 9600
Processing waveform 79 of 179
75.0 4650
Processing waveform 80 of 179
75.0 6750
Processing waveform 81 of 179
75.0 8925
Processing waveform 82 of 179
75.0 7125
Processing waveform 83 of 179
75.0 3000
Processing waveform 84 of 179
75.0 1800
Processing waveform 85 of 179
75.0 4425
Processing waveform 86 of 179
75.0 9450
Processing waveform 87 of 179
75.0 6525
Processing waveform 88 of 179
75.0 5550
Processing waveform 89 of 179
75.0 2250
Processing waveform 90 of 179
75.0 3600
Processing waveform 91 of 179
75.0 5775
Processing waveform 92 of 179
75.0 2325
Processing waveform 93 of 179
75.0 2325
Processing waveform 94 of 179
75.0 3525
Processing waveform 95 of 179
75.0 4500
Processing waveform 96 of 179
75.0 4275
Processing waveform 97 of 179
75.0 3450
Processing waveform 98 of 179
75.0 2550
Processing waveform 99 of 179
75.0 3300
Processing waveform 100 of 179
75.0 9075
Processing waveform 101 of 179
75.0 9075
Processing waveform 102 of 179
75.0 6525
Processing waveform 103 of 179
75.0 6975
Processing waveform 104 of 179
75.0 6750
Processing waveform 105 of 179
75.0 7275
Processing waveform 106 of 179
75.0 2400
Processing waveform 107 of 179
75.0 4125
Processing waveform 108 of 179
75.0 3525
Processing waveform 109 of 179
75.0 5175
Processing waveform 110 of 179
75.0 3750
Processing waveform 111 of 179
75.0 4875
Processing waveform 112 of 179
75.0 2250
Processing waveform 113 of 179
75.0 4125
Processing waveform 114 of 179
75.0 4425
Processing waveform 115 of 179
75.0 5250
Processing waveform 116 of 179
75.0 9075
Processing waveform 117 of 179
75.0 4725
Processing waveform 118 of 179
75.0 2475
Processing waveform 119 of 179
75.0 2925
Processing waveform 120 of 179
75.0 7350
Processing waveform 121 of 179
75.0 2550
Processing waveform 122 of 179
75.0 2850
Processing waveform 123 of 179
75.0 2550
Processing waveform 124 of 179
75.0 5775
Processing waveform 125 of 179
75.0 4200
Processing waveform 126 of 179
75.0 2175
Processing waveform 127 of 179
75.0 2925
Processing waveform 128 of 179
75.0 10350
Processing waveform 129 of 179
75.0 15600
Processing waveform 130 of 179
75.0 6450
Processing waveform 131 of 179
75.0 3450
Processing waveform 132 of 179
75.0 11175
Processing waveform 133 of 179
75.0 4725
Processing waveform 134 of 179
75.0 4200
Processing waveform 135 of 179
75.0 3750
Processing waveform 136 of 179
75.0 3975
Processing waveform 137 of 179
75.0 4575
Processing waveform 138 of 179
75.0 3225
Processing waveform 139 of 179
75.0 13500
Processing waveform 140 of 179
75.0 4050
Processing waveform 141 of 179
75.0 5250
Processing waveform 142 of 179
75.0 7050
Processing waveform 143 of 179
75.0 3450
Processing waveform 144 of 179
75.0 6675
Processing waveform 145 of 179
75.0 6750
Processing waveform 146 of 179
75.0 2475
Processing waveform 147 of 179
75.0 7650
Processing waveform 148 of 179
75.0 6750
Processing waveform 149 of 179
75.0 4125
Processing waveform 150 of 179
75.0 7425
Processing waveform 151 of 179
75.0 5400
Processing waveform 152 of 179
75.0 4800
Processing waveform 153 of 179
75.0 10200
Processing waveform 154 of 179
75.0 18225
Processing waveform 155 of 179
75.0 5925
Processing waveform 156 of 179
75.0 3525
Processing waveform 157 of 179
75.0 3300
Processing waveform 158 of 179
75.0 3600
Processing waveform 159 of 179
75.0 4650
Processing waveform 160 of 179
75.0 7875
Processing waveform 161 of 179
75.0 6300
Processing waveform 162 of 179
75.0 6750
Processing waveform 163 of 179
75.0 2925
Processing waveform 164 of 179
75.0 3525
Processing waveform 165 of 179
75.0 7875
Processing waveform 166 of 179
75.0 8925
Processing waveform 167 of 179
75.0 2250
Processing waveform 168 of 179
75.0 2700
Processing waveform 169 of 179
75.0 3000
Processing waveform 170 of 179
75.0 3300
Processing waveform 171 of 179
75.0 4425
Processing waveform 172 of 179
75.0 4500
Processing waveform 173 of 179
75.0 7500
Processing waveform 174 of 179
75.0 4200
Processing waveform 175 of 179
75.0 4800
Processing waveform 176 of 179
75.0 1950
Processing waveform 177 of 179
75.0 4650
Processing waveform 178 of 179
75.0 5475
Training data have been read and features have been extracted  (179, 120)
Computation time:  254.76286554336548
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.1509864330291748
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l    88       
        t          91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  58.6296296296  +/-  3.67040587493  %
          Predicted class
              l     t 
        l    28    16 
        t    21    25 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'r'] 1
class
e    78
r    93
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 171
75.0 3450
Processing waveform 1 of 171
75.0 4125
Processing waveform 2 of 171
75.0 2400
Processing waveform 3 of 171
75.0 2700
Processing waveform 4 of 171
75.0 2700
Processing waveform 5 of 171
75.0 4350

Processing waveform 6 of 171
75.0 2775
Processing waveform 7 of 171
75.0 1950
Processing waveform 8 of 171
75.0 2250
Processing waveform 9 of 171
75.0 2250
Processing waveform 10 of 171
75.0 2250
Processing waveform 11 of 171
75.0 5100
Processing waveform 12 of 171
75.0 2475
Processing waveform 13 of 171
75.0 3300
Processing waveform 14 of 171
75.0 6300
Processing waveform 15 of 171
75.0 5100
Processing waveform 16 of 171
75.0 9075
Processing waveform 17 of 171
75.0 3225
Processing waveform 18 of 171
75.0 4125
Processing waveform 19 of 171
75.0 3300
Processing waveform 20 of 171
75.0 3225
Processing waveform 21 of 171
75.0 2850
Processing waveform 22 of 171
75.0 5400
Processing waveform 23 of 171
75.0 2400
Processing waveform 24 of 171
75.0 7725
Processing waveform 25 of 171
75.0 3900
Processing waveform 26 of 171
75.0 2775
Processing waveform 27 of 171
75.0 2325
Processing waveform 28 of 171
75.0 4425
Processing waveform 29 of 171
75.0 3525
Processing waveform 30 of 171
75.0 6300
Processing waveform 31 of 171
75.0 2625
Processing waveform 32 of 171
75.0 4050
Processing waveform 33 of 171
75.0 2775
Processing waveform 34 of 171
75.0 2400
Processing waveform 35 of 171
75.0 2850
Processing waveform 36 of 171
75.0 2250
Processing waveform 37 of 171
75.0 2325
Processing waveform 38 of 171
75.0 3750
Processing waveform 39 of 171
75.0 3525
Processing waveform 40 of 171
75.0 5700
Processing waveform 41 of 171
75.0 2700
Processing waveform 42 of 171
75.0 3000
Processing waveform 43 of 171
75.0 4350
Processing waveform 44 of 171
75.0 4950
Processing waveform 45 of 171
75.0 2925
Processing waveform 46 of 171
75.0 2775
Processing waveform 47 of 171
75.0 3450
Processing waveform 48 of 171
75.0 4875
Processing waveform 49 of 171
75.0 9075
Processing waveform 50 of 171
75.0 5175
Processing waveform 51 of 171
75.0 3075
Processing waveform 52 of 171
75.0 4125
Processing waveform 53 of 171
75.0 9075
Processing waveform 54 of 171
75.0 3225
Processing waveform 55 of 171
75.0 2775
Processing waveform 56 of 171
75.0 4575
Processing waveform 57 of 171
75.0 2700
Processing waveform 58 of 171
75.0 1950
Processing waveform 59 of 171
75.0 2100
Processing waveform 60 of 171
75.0 4575
Processing waveform 61 of 171
75.0 5850
Processing waveform 62 of 171
75.0 3150
Processing waveform 63 of 171
75.0 4275
Processing waveform 64 of 171
75.0 9075
Processing waveform 65 of 171
75.0 3600
Processing waveform 66 of 171
75.0 2625
Processing waveform 67 of 171
75.0 2700
Processing waveform 68 of 171
75.0 2700
Processing waveform 69 of 171
75.0 3525
Processing waveform 70 of 171
75.0 4200
Processing waveform 71 of 171
75.0 4200
Processing waveform 72 of 171
75.0 1950
Processing waveform 73 of 171
75.0 5775
Processing waveform 74 of 171
75.0 9075
Processing waveform 75 of 171
75.0 4200
Processing waveform 76 of 171
75.0 2625
Processing waveform 77 of 171
75.0 4875
Processing waveform 78 of 171
75.0 5025
Processing waveform 79 of 171
75.0 14625
Processing waveform 80 of 171
75.0 11025
Processing waveform 81 of 171
75.0 1950
Processing waveform 82 of 171
75.0 3975
Processing waveform 83 of 171
75.0 2550
Processing waveform 84 of 171
75.0 7500
Processing waveform 85 of 171
75.0 14925
Processing waveform 86 of 171
75.0 13275
Processing waveform 87 of 171
75.0 1725
Processing waveform 88 of 171
75.0 2400
Processing waveform 89 of 171
75.0 6375
Processing waveform 90 of 171
75.0 6450
Processing waveform 91 of 171
75.0 16725
Processing waveform 92 of 171
75.0 6075
Processing waveform 93 of 171
75.0 3450
Processing waveform 94 of 171
75.0 2325
Processing waveform 95 of 171
75.0 2925
Processing waveform 96 of 171
75.0 4575
Processing waveform 97 of 171
75.0 7050
Processing waveform 98 of 171
75.0 2025
Processing waveform 99 of 171
75.0 2925
Processing waveform 100 of 171
75.0 2850
Processing waveform 101 of 171
75.0 3600
Processing waveform 102 of 171
75.0 3750
Processing waveform 103 of 171
75.0 2100
Processing waveform 104 of 171
75.0 1650
Processing waveform 105 of 171
75.0 4200
Processing waveform 106 of 171
75.0 9075
Processing waveform 107 of 171
75.0 4125
Processing waveform 108 of 171
75.0 2925
Processing waveform 109 of 171
75.0 2250
Processing waveform 110 of 171
75.0 5475
Processing waveform 111 of 171
75.0 2625
Processing waveform 112 of 171
75.0 2400
Processing waveform 113 of 171
75.0 2700
Processing waveform 114 of 171
75.0 9075
Processing waveform 115 of 171
75.0 9075
Processing waveform 116 of 171
75.0 6225
Processing waveform 117 of 171
75.0 7125
Processing waveform 118 of 171
75.0 5850
Processing waveform 119 of 171
75.0 9075
Processing waveform 120 of 171
75.0 3225
Processing waveform 121 of 171
75.0 8100
Processing waveform 122 of 171
75.0 4575
Processing waveform 123 of 171
75.0 2850
Processing waveform 124 of 171
75.0 3375
Processing waveform 125 of 171
75.0 2100
Processing waveform 126 of 171
75.0 9075
Processing waveform 127 of 171
75.0 5850
Processing waveform 128 of 171
75.0 2775
Processing waveform 129 of 171
75.0 2700
Processing waveform 130 of 171
75.0 2100
Processing waveform 131 of 171
75.0 9075
Processing waveform 132 of 171
75.0 3225
Processing waveform 133 of 171
75.0 3300
Processing waveform 134 of 171
75.0 2475
Processing waveform 135 of 171
75.0 3075
Processing waveform 136 of 171
75.0 6450
Processing waveform 137 of 171
75.0 3600
Processing waveform 138 of 171
75.0 4650
Processing waveform 139 of 171
75.0 2475
Processing waveform 140 of 171
75.0 1875
Processing waveform 141 of 171
75.0 3450
Processing waveform 142 of 171
75.0 1500
Processing waveform 143 of 171
75.0 2625
Processing waveform 144 of 171
75.0 3525
Processing waveform 145 of 171
75.0 5625
Processing waveform 146 of 171
75.0 3675
Processing waveform 147 of 171
75.0 9075
Processing waveform 148 of 171
75.0 2700
Processing waveform 149 of 171
75.0 9075
Processing waveform 150 of 171
75.0 3000
Processing waveform 151 of 171
75.0 3525
Processing waveform 152 of 171
75.0 7200
Processing waveform 153 of 171
75.0 4875
Processing waveform 154 of 171
75.0 6075
Processing waveform 155 of 171
75.0 2700
Processing waveform 156 of 171
75.0 5250
Processing waveform 157 of 171
75.0 12750
Processing waveform 158 of 171
75.0 5100
Processing waveform 159 of 171
75.0 2475
Processing waveform 160 of 171
75.0 6525
Processing waveform 161 of 171
75.0 7200
Processing waveform 162 of 171
75.0 4500
Processing waveform 163 of 171
75.0 4950
Processing waveform 164 of 171
75.0 3150
Processing waveform 165 of 171
75.0 5850
Processing waveform 166 of 171
75.0 2325
Processing waveform 167 of 171
75.0 4200
Processing waveform 168 of 171
75.0 13350
Processing waveform 169 of 171
75.0 9525
Processing waveform 170 of 171
75.0 2325
Training data have been read and features have been extracted  (171, 120)
Computation time:  235.14406991004944
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.14826703071594238
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    78       
        r          93 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  49.6511627907  +/-  4.97107189607  %
          Predicted class
              e     r 
        e    14    25 
        r    18    28 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't'] 1
class
h    98
l    88
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 277
75.0 3975
Processing waveform 1 of 277
75.0 2400
Processing waveform 2 of 277
75.0 5025
Processing waveform 3 of 277
75.0 6825
Processing waveform 4 of 277
75.0 3750
Processing waveform 5 of 277
75.0 9075
Processing waveform 6 of 277
75.0 4500
Processing waveform 7 of 277
75.0 2175
Processing waveform 8 of 277
75.0 3300
Processing waveform 9 of 277
75.0 4275
Processing waveform 10 of 277
75.0 2850
Processing waveform 11 of 277
75.0 2850
Processing waveform 12 of 277
75.0 2700
Processing waveform 13 of 277
75.0 5250
Processing waveform 14 of 277
75.0 3000

Processing waveform 15 of 277
75.0 2775
Processing waveform 16 of 277
75.0 6450
Processing waveform 17 of 277
75.0 3150
Processing waveform 18 of 277
75.0 1650
Processing waveform 19 of 277
75.0 4500
Processing waveform 20 of 277
75.0 2250
Processing waveform 21 of 277
75.0 2175
Processing waveform 22 of 277
75.0 3225
Processing waveform 23 of 277
75.0 4050
Processing waveform 24 of 277
75.0 3300
Processing waveform 25 of 277
75.0 2850
Processing waveform 26 of 277
75.0 2025
Processing waveform 27 of 277
75.0 4800
Processing waveform 28 of 277
75.0 2325
Processing waveform 29 of 277
75.0 6150
Processing waveform 30 of 277
75.0 2775
Processing waveform 31 of 277
75.0 2850
Processing waveform 32 of 277
75.0 4425
Processing waveform 33 of 277
75.0 2250
Processing waveform 34 of 277
75.0 3075
Processing waveform 35 of 277
75.0 2775
Processing waveform 36 of 277
75.0 1875
Processing waveform 37 of 277
75.0 3300
Processing waveform 38 of 277
75.0 1875
Processing waveform 39 of 277
75.0 5550
Processing waveform 40 of 277
75.0 9075
Processing waveform 41 of 277
75.0 4650
Processing waveform 42 of 277
75.0 6525
Processing waveform 43 of 277
75.0 9075
Processing waveform 44 of 277
75.0 3225
Processing waveform 45 of 277
75.0 3150
Processing waveform 46 of 277
75.0 4275
Processing waveform 47 of 277
75.0 4050
Processing waveform 48 of 277
75.0 5400
Processing waveform 49 of 277
75.0 4725
Processing waveform 50 of 277
75.0 3450
Processing waveform 51 of 277
75.0 6150
Processing waveform 52 of 277
75.0 2250
Processing waveform 53 of 277
75.0 9000
Processing waveform 54 of 277
75.0 3150
Processing waveform 55 of 277
75.0 4950
Processing waveform 56 of 277
75.0 6750
Processing waveform 57 of 277
75.0 3150
Processing waveform 58 of 277
75.0 7050
Processing waveform 59 of 277
75.0 3675
Processing waveform 60 of 277
75.0 3225
Processing waveform 61 of 277
75.0 2325
Processing waveform 62 of 277
75.0 7500
Processing waveform 63 of 277
75.0 3975
Processing waveform 64 of 277
75.0 6150
Processing waveform 65 of 277
75.0 7875
Processing waveform 66 of 277
75.0 7275
Processing waveform 67 of 277
75.0 6075
Processing waveform 68 of 277
75.0 3900
Processing waveform 69 of 277
75.0 3000
Processing waveform 70 of 277
75.0 3900
Processing waveform 71 of 277
75.0 6525
Processing waveform 72 of 277
75.0 2925
Processing waveform 73 of 277
75.0 2700
Processing waveform 74 of 277
75.0 2250
Processing waveform 75 of 277
75.0 4425
Processing waveform 76 of 277
75.0 7500
Processing waveform 77 of 277
75.0 2250
Processing waveform 78 of 277
75.0 9600
Processing waveform 79 of 277
75.0 4650
Processing waveform 80 of 277
75.0 6750
Processing waveform 81 of 277
75.0 8925
Processing waveform 82 of 277
75.0 7125
Processing waveform 83 of 277
75.0 3000
Processing waveform 84 of 277
75.0 1800
Processing waveform 85 of 277
75.0 4425
Processing waveform 86 of 277
75.0 9450
Processing waveform 87 of 277
75.0 6525
Processing waveform 88 of 277
75.0 3225
Processing waveform 89 of 277
75.0 2100
Processing waveform 90 of 277
75.0 2700
Processing waveform 91 of 277
75.0 2325
Processing waveform 92 of 277
75.0 2775
Processing waveform 93 of 277
75.0 5850
Processing waveform 94 of 277
75.0 3825
Processing waveform 95 of 277
75.0 6000
Processing waveform 96 of 277
75.0 2025
Processing waveform 97 of 277
75.0 1725
Processing waveform 98 of 277
75.0 2175
Processing waveform 99 of 277
75.0 5400
Processing waveform 100 of 277
75.0 3000
Processing waveform 101 of 277
75.0 5325
Processing waveform 102 of 277
75.0 3750
Processing waveform 103 of 277
75.0 3075
Processing waveform 104 of 277
75.0 5550
Processing waveform 105 of 277
75.0 7350
Processing waveform 106 of 277
75.0 4650
Processing waveform 107 of 277
75.0 2775
Processing waveform 108 of 277
75.0 2325
Processing waveform 109 of 277
75.0 2925
Processing waveform 110 of 277
75.0 3075
Processing waveform 111 of 277
75.0 6375
Processing waveform 112 of 277
75.0 2475
Processing waveform 113 of 277
75.0 3975
Processing waveform 114 of 277
75.0 3150
Processing waveform 115 of 277
75.0 5925
Processing waveform 116 of 277
75.0 5400
Processing waveform 117 of 277
75.0 1875
Processing waveform 118 of 277
75.0 5850
Processing waveform 119 of 277
75.0 1725
Processing waveform 120 of 277
75.0 7650
Processing waveform 121 of 277
75.0 2700
Processing waveform 122 of 277
75.0 9075
Processing waveform 123 of 277
75.0 6975
Processing waveform 124 of 277
75.0 4050
Processing waveform 125 of 277
75.0 3450
Processing waveform 126 of 277
75.0 5925
Processing waveform 127 of 277
75.0 2250
Processing waveform 128 of 277
75.0 3675
Processing waveform 129 of 277
75.0 2625
Processing waveform 130 of 277
75.0 4575
Processing waveform 131 of 277
75.0 2025
Processing waveform 132 of 277
75.0 5175
Processing waveform 133 of 277
75.0 3225
Processing waveform 134 of 277
75.0 4950
Processing waveform 135 of 277
75.0 2925
Processing waveform 136 of 277
75.0 1800
Processing waveform 137 of 277
75.0 8925
Processing waveform 138 of 277
75.0 7350
Processing waveform 139 of 277
75.0 4125
Processing waveform 140 of 277
75.0 3225
Processing waveform 141 of 277
75.0 9075
Processing waveform 142 of 277
75.0 5025
Processing waveform 143 of 277
75.0 8925
Processing waveform 144 of 277
75.0 4800
Processing waveform 145 of 277
75.0 7800
Processing waveform 146 of 277
75.0 2775
Processing waveform 147 of 277
75.0 6600
Processing waveform 148 of 277
75.0 1800
Processing waveform 149 of 277
75.0 2625
Processing waveform 150 of 277
75.0 9075
Processing waveform 151 of 277
75.0 7200
Processing waveform 152 of 277
75.0 5550
Processing waveform 153 of 277
75.0 2775
Processing waveform 154 of 277
75.0 5250
Processing waveform 155 of 277
75.0 1875
Processing waveform 156 of 277
75.0 1875
Processing waveform 157 of 277
75.0 5025
Processing waveform 158 of 277
75.0 3600
Processing waveform 159 of 277
75.0 3375
Processing waveform 160 of 277
75.0 4050
Processing waveform 161 of 277
75.0 3375
Processing waveform 162 of 277
75.0 2925
Processing waveform 163 of 277
75.0 6675
Processing waveform 164 of 277
75.0 3750
Processing waveform 165 of 277
75.0 4575
Processing waveform 166 of 277
75.0 9075
Processing waveform 167 of 277
75.0 3900
Processing waveform 168 of 277
75.0 9075
Processing waveform 169 of 277
75.0 4875
Processing waveform 170 of 277
75.0 3075
Processing waveform 171 of 277
75.0 3150
Processing waveform 172 of 277
75.0 2550
Processing waveform 173 of 277
75.0 9075
Processing waveform 174 of 277
75.0 7425
Processing waveform 175 of 277
75.0 5025
Processing waveform 176 of 277
75.0 2850
Processing waveform 177 of 277
75.0 1875
Processing waveform 178 of 277
75.0 7125
Processing waveform 179 of 277
75.0 4500
Processing waveform 180 of 277
75.0 6075
Processing waveform 181 of 277
75.0 2475
Processing waveform 182 of 277
75.0 4125
Processing waveform 183 of 277
75.0 2100
Processing waveform 184 of 277
75.0 3975
Processing waveform 185 of 277
75.0 3075
Processing waveform 186 of 277
75.0 5550
Processing waveform 187 of 277
75.0 2250
Processing waveform 188 of 277
75.0 3600
Processing waveform 189 of 277
75.0 5775
Processing waveform 190 of 277
75.0 2325
Processing waveform 191 of 277
75.0 2325
Processing waveform 192 of 277
75.0 3525
Processing waveform 193 of 277
75.0 4500
Processing waveform 194 of 277
75.0 4275
Processing waveform 195 of 277
75.0 3450
Processing waveform 196 of 277
75.0 2550
Processing waveform 197 of 277
75.0 3300
Processing waveform 198 of 277
75.0 9075
Processing waveform 199 of 277
75.0 9075
Processing waveform 200 of 277
75.0 6525
Processing waveform 201 of 277
75.0 6975
Processing waveform 202 of 277
75.0 6750
Processing waveform 203 of 277
75.0 7275
Processing waveform 204 of 277
75.0 2400
Processing waveform 205 of 277
75.0 4125
Processing waveform 206 of 277
75.0 3525
Processing waveform 207 of 277
75.0 5175
Processing waveform 208 of 277
75.0 3750
Processing waveform 209 of 277
75.0 4875
Processing waveform 210 of 277
75.0 2250
Processing waveform 211 of 277
75.0 4125
Processing waveform 212 of 277
75.0 4425
Processing waveform 213 of 277
75.0 5250
Processing waveform 214 of 277
75.0 9075
Processing waveform 215 of 277
75.0 4725
Processing waveform 216 of 277
75.0 2475

Processing waveform 217 of 277
75.0 2925
Processing waveform 218 of 277
75.0 7350
Processing waveform 219 of 277
75.0 2550
Processing waveform 220 of 277
75.0 2850
Processing waveform 221 of 277
75.0 2550
Processing waveform 222 of 277
75.0 5775
Processing waveform 223 of 277
75.0 4200
Processing waveform 224 of 277
75.0 2175
Processing waveform 225 of 277
75.0 2925
Processing waveform 226 of 277
75.0 10350
Processing waveform 227 of 277
75.0 15600
Processing waveform 228 of 277
75.0 6450
Processing waveform 229 of 277
75.0 3450
Processing waveform 230 of 277
75.0 11175
Processing waveform 231 of 277
75.0 4725
Processing waveform 232 of 277
75.0 4200
Processing waveform 233 of 277
75.0 3750
Processing waveform 234 of 277
75.0 3975
Processing waveform 235 of 277
75.0 4575
Processing waveform 236 of 277
75.0 3225
Processing waveform 237 of 277
75.0 13500
Processing waveform 238 of 277
75.0 4050
Processing waveform 239 of 277
75.0 5250
Processing waveform 240 of 277
75.0 7050
Processing waveform 241 of 277
75.0 3450
Processing waveform 242 of 277
75.0 6675
Processing waveform 243 of 277
75.0 6750
Processing waveform 244 of 277
75.0 2475
Processing waveform 245 of 277
75.0 7650
Processing waveform 246 of 277
75.0 6750
Processing waveform 247 of 277
75.0 4125
Processing waveform 248 of 277
75.0 7425
Processing waveform 249 of 277
75.0 5400
Processing waveform 250 of 277
75.0 4800
Processing waveform 251 of 277
75.0 10200
Processing waveform 252 of 277
75.0 18225
Processing waveform 253 of 277
75.0 5925
Processing waveform 254 of 277
75.0 3525
Processing waveform 255 of 277
75.0 3300
Processing waveform 256 of 277
75.0 3600
Processing waveform 257 of 277
75.0 4650
Processing waveform 258 of 277
75.0 7875
Processing waveform 259 of 277
75.0 6300
Processing waveform 260 of 277
75.0 6750
Processing waveform 261 of 277
75.0 2925
Processing waveform 262 of 277
75.0 3525
Processing waveform 263 of 277
75.0 7875
Processing waveform 264 of 277
75.0 8925
Processing waveform 265 of 277
75.0 2250
Processing waveform 266 of 277
75.0 2700
Processing waveform 267 of 277
75.0 3000
Processing waveform 268 of 277
75.0 3300
Processing waveform 269 of 277
75.0 4425
Processing waveform 270 of 277
75.0 4500
Processing waveform 271 of 277
75.0 7500
Processing waveform 272 of 277
75.0 4200
Processing waveform 273 of 277
75.0 4800
Processing waveform 274 of 277
75.0 1950
Processing waveform 275 of 277
75.0 4650
Processing waveform 276 of 277
75.0 5475
Training data have been read and features have been extracted  (277, 120)
Computation time:  366.7886345386505
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.28730034828186035
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h    98             
        l          88       
        t                91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  35.7553956835  +/-  3.09240959194  %
          Predicted class
              h     l     t 
        h    19    15    15 
        l    19    14    11 
        t    19    10    17 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't', 'r'] 1
class
h    98
l    88
r    93
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 370
75.0 3450
Processing waveform 1 of 370
75.0 4125
Processing waveform 2 of 370
75.0 2400
Processing waveform 3 of 370
75.0 2700
Processing waveform 4 of 370
75.0 2700
Processing waveform 5 of 370
75.0 4350
Processing waveform 6 of 370
75.0 2775
Processing waveform 7 of 370
75.0 1950
Processing waveform 8 of 370
75.0 2250
Processing waveform 9 of 370
75.0 2250
Processing waveform 10 of 370
75.0 2250
Processing waveform 11 of 370
75.0 5100
Processing waveform 12 of 370
75.0 2475
Processing waveform 13 of 370
75.0 3300
Processing waveform 14 of 370
75.0 6300
Processing waveform 15 of 370
75.0 5100
Processing waveform 16 of 370
75.0 9075
Processing waveform 17 of 370
75.0 3225
Processing waveform 18 of 370
75.0 4125
Processing waveform 19 of 370
75.0 3300
Processing waveform 20 of 370
75.0 3225
Processing waveform 21 of 370
75.0 2850
Processing waveform 22 of 370
75.0 5400
Processing waveform 23 of 370
75.0 2400
Processing waveform 24 of 370
75.0 7725
Processing waveform 25 of 370
75.0 3900
Processing waveform 26 of 370
75.0 2775
Processing waveform 27 of 370
75.0 2325
Processing waveform 28 of 370
75.0 4425
Processing waveform 29 of 370
75.0 3525
Processing waveform 30 of 370
75.0 6300
Processing waveform 31 of 370
75.0 2625
Processing waveform 32 of 370
75.0 4050
Processing waveform 33 of 370
75.0 2775
Processing waveform 34 of 370
75.0 2400
Processing waveform 35 of 370
75.0 2850
Processing waveform 36 of 370
75.0 2250
Processing waveform 37 of 370
75.0 2325
Processing waveform 38 of 370
75.0 3750
Processing waveform 39 of 370
75.0 3525
Processing waveform 40 of 370
75.0 5700
Processing waveform 41 of 370
75.0 2700
Processing waveform 42 of 370
75.0 3000
Processing waveform 43 of 370
75.0 4350
Processing waveform 44 of 370
75.0 4950
Processing waveform 45 of 370
75.0 2925
Processing waveform 46 of 370
75.0 2775
Processing waveform 47 of 370
75.0 3450
Processing waveform 48 of 370
75.0 4875
Processing waveform 49 of 370
75.0 9075
Processing waveform 50 of 370
75.0 5175
Processing waveform 51 of 370
75.0 3075
Processing waveform 52 of 370
75.0 4125
Processing waveform 53 of 370
75.0 9075
Processing waveform 54 of 370
75.0 3225
Processing waveform 55 of 370
75.0 2775
Processing waveform 56 of 370
75.0 4575
Processing waveform 57 of 370
75.0 2700
Processing waveform 58 of 370
75.0 1950
Processing waveform 59 of 370
75.0 2100
Processing waveform 60 of 370
75.0 4575
Processing waveform 61 of 370
75.0 5850
Processing waveform 62 of 370
75.0 3150
Processing waveform 63 of 370
75.0 4275
Processing waveform 64 of 370
75.0 9075
Processing waveform 65 of 370
75.0 3600
Processing waveform 66 of 370
75.0 2625
Processing waveform 67 of 370
75.0 2700
Processing waveform 68 of 370
75.0 2700
Processing waveform 69 of 370
75.0 3525
Processing waveform 70 of 370
75.0 4200
Processing waveform 71 of 370
75.0 4200
Processing waveform 72 of 370
75.0 1950
Processing waveform 73 of 370
75.0 5775
Processing waveform 74 of 370
75.0 9075
Processing waveform 75 of 370
75.0 4200
Processing waveform 76 of 370
75.0 2625
Processing waveform 77 of 370
75.0 4875
Processing waveform 78 of 370
75.0 5025
Processing waveform 79 of 370
75.0 14625
Processing waveform 80 of 370
75.0 11025
Processing waveform 81 of 370
75.0 1950
Processing waveform 82 of 370
75.0 3975
Processing waveform 83 of 370
75.0 2550
Processing waveform 84 of 370
75.0 7500
Processing waveform 85 of 370
75.0 14925
Processing waveform 86 of 370
75.0 13275
Processing waveform 87 of 370
75.0 1725
Processing waveform 88 of 370
75.0 2400
Processing waveform 89 of 370
75.0 6375
Processing waveform 90 of 370
75.0 6450
Processing waveform 91 of 370
75.0 16725
Processing waveform 92 of 370
75.0 6075
Processing waveform 93 of 370
75.0 3975
Processing waveform 94 of 370
75.0 2400
Processing waveform 95 of 370
75.0 5025
Processing waveform 96 of 370
75.0 6825
Processing waveform 97 of 370
75.0 3750
Processing waveform 98 of 370
75.0 9075
Processing waveform 99 of 370
75.0 4500
Processing waveform 100 of 370
75.0 2175
Processing waveform 101 of 370
75.0 3300
Processing waveform 102 of 370
75.0 4275
Processing waveform 103 of 370
75.0 2850
Processing waveform 104 of 370
75.0 2850
Processing waveform 105 of 370
75.0 2700
Processing waveform 106 of 370
75.0 5250
Processing waveform 107 of 370
75.0 3000
Processing waveform 108 of 370
75.0 2775
Processing waveform 109 of 370
75.0 6450
Processing waveform 110 of 370
75.0 3150
Processing waveform 111 of 370
75.0 1650
Processing waveform 112 of 370
75.0 4500
Processing waveform 113 of 370
75.0 2250
Processing waveform 114 of 370
75.0 2175
Processing waveform 115 of 370
75.0 3225
Processing waveform 116 of 370
75.0 4050

Processing waveform 117 of 370
75.0 3300
Processing waveform 118 of 370
75.0 2850
Processing waveform 119 of 370
75.0 2025
Processing waveform 120 of 370
75.0 4800
Processing waveform 121 of 370
75.0 2325
Processing waveform 122 of 370
75.0 6150
Processing waveform 123 of 370
75.0 2775
Processing waveform 124 of 370
75.0 2850
Processing waveform 125 of 370
75.0 4425
Processing waveform 126 of 370
75.0 2250
Processing waveform 127 of 370
75.0 3075
Processing waveform 128 of 370
75.0 2775
Processing waveform 129 of 370
75.0 1875
Processing waveform 130 of 370
75.0 3300
Processing waveform 131 of 370
75.0 1875
Processing waveform 132 of 370
75.0 5550
Processing waveform 133 of 370
75.0 9075
Processing waveform 134 of 370
75.0 4650
Processing waveform 135 of 370
75.0 6525
Processing waveform 136 of 370
75.0 9075
Processing waveform 137 of 370
75.0 3225
Processing waveform 138 of 370
75.0 3150
Processing waveform 139 of 370
75.0 4275
Processing waveform 140 of 370
75.0 4050
Processing waveform 141 of 370
75.0 5400
Processing waveform 142 of 370
75.0 4725
Processing waveform 143 of 370
75.0 3450
Processing waveform 144 of 370
75.0 6150
Processing waveform 145 of 370
75.0 2250
Processing waveform 146 of 370
75.0 9000
Processing waveform 147 of 370
75.0 3150
Processing waveform 148 of 370
75.0 4950
Processing waveform 149 of 370
75.0 6750
Processing waveform 150 of 370
75.0 3150
Processing waveform 151 of 370
75.0 7050
Processing waveform 152 of 370
75.0 3675
Processing waveform 153 of 370
75.0 3225
Processing waveform 154 of 370
75.0 2325
Processing waveform 155 of 370
75.0 7500
Processing waveform 156 of 370
75.0 3975
Processing waveform 157 of 370
75.0 6150
Processing waveform 158 of 370
75.0 7875
Processing waveform 159 of 370
75.0 7275
Processing waveform 160 of 370
75.0 6075
Processing waveform 161 of 370
75.0 3900
Processing waveform 162 of 370
75.0 3000
Processing waveform 163 of 370
75.0 3900
Processing waveform 164 of 370
75.0 6525
Processing waveform 165 of 370
75.0 2925
Processing waveform 166 of 370
75.0 2700
Processing waveform 167 of 370
75.0 2250
Processing waveform 168 of 370
75.0 4425
Processing waveform 169 of 370
75.0 7500
Processing waveform 170 of 370
75.0 2250
Processing waveform 171 of 370
75.0 9600
Processing waveform 172 of 370
75.0 4650
Processing waveform 173 of 370
75.0 6750
Processing waveform 174 of 370
75.0 8925
Processing waveform 175 of 370
75.0 7125
Processing waveform 176 of 370
75.0 3000
Processing waveform 177 of 370
75.0 1800
Processing waveform 178 of 370
75.0 4425
Processing waveform 179 of 370
75.0 9450
Processing waveform 180 of 370
75.0 6525
Processing waveform 181 of 370
75.0 3225
Processing waveform 182 of 370
75.0 2100
Processing waveform 183 of 370
75.0 2700
Processing waveform 184 of 370
75.0 2325
Processing waveform 185 of 370
75.0 2775
Processing waveform 186 of 370
75.0 5850
Processing waveform 187 of 370
75.0 3825
Processing waveform 188 of 370
75.0 6000
Processing waveform 189 of 370
75.0 2025
Processing waveform 190 of 370
75.0 1725
Processing waveform 191 of 370
75.0 2175
Processing waveform 192 of 370
75.0 5400
Processing waveform 193 of 370
75.0 3000
Processing waveform 194 of 370
75.0 5325
Processing waveform 195 of 370
75.0 3750
Processing waveform 196 of 370
75.0 3075
Processing waveform 197 of 370
75.0 5550
Processing waveform 198 of 370
75.0 7350
Processing waveform 199 of 370
75.0 4650
Processing waveform 200 of 370
75.0 2775
Processing waveform 201 of 370
75.0 2325
Processing waveform 202 of 370
75.0 2925
Processing waveform 203 of 370
75.0 3075
Processing waveform 204 of 370
75.0 6375
Processing waveform 205 of 370
75.0 2475
Processing waveform 206 of 370
75.0 3975
Processing waveform 207 of 370
75.0 3150
Processing waveform 208 of 370
75.0 5925
Processing waveform 209 of 370
75.0 5400
Processing waveform 210 of 370
75.0 1875
Processing waveform 211 of 370
75.0 5850
Processing waveform 212 of 370
75.0 1725
Processing waveform 213 of 370
75.0 7650
Processing waveform 214 of 370
75.0 2700
Processing waveform 215 of 370
75.0 9075
Processing waveform 216 of 370
75.0 6975
Processing waveform 217 of 370
75.0 4050
Processing waveform 218 of 370
75.0 3450
Processing waveform 219 of 370
75.0 5925
Processing waveform 220 of 370
75.0 2250
Processing waveform 221 of 370
75.0 3675
Processing waveform 222 of 370
75.0 2625
Processing waveform 223 of 370
75.0 4575
Processing waveform 224 of 370
75.0 2025
Processing waveform 225 of 370
75.0 5175
Processing waveform 226 of 370
75.0 3225
Processing waveform 227 of 370
75.0 4950
Processing waveform 228 of 370
75.0 2925
Processing waveform 229 of 370
75.0 1800
Processing waveform 230 of 370
75.0 8925
Processing waveform 231 of 370
75.0 7350
Processing waveform 232 of 370
75.0 4125
Processing waveform 233 of 370
75.0 3225
Processing waveform 234 of 370
75.0 9075
Processing waveform 235 of 370
75.0 5025
Processing waveform 236 of 370
75.0 8925
Processing waveform 237 of 370
75.0 4800
Processing waveform 238 of 370
75.0 7800
Processing waveform 239 of 370
75.0 2775
Processing waveform 240 of 370
75.0 6600
Processing waveform 241 of 370
75.0 1800
Processing waveform 242 of 370
75.0 2625
Processing waveform 243 of 370
75.0 9075
Processing waveform 244 of 370
75.0 7200
Processing waveform 245 of 370
75.0 5550
Processing waveform 246 of 370
75.0 2775
Processing waveform 247 of 370
75.0 5250
Processing waveform 248 of 370
75.0 1875
Processing waveform 249 of 370
75.0 1875
Processing waveform 250 of 370
75.0 5025
Processing waveform 251 of 370
75.0 3600
Processing waveform 252 of 370
75.0 3375
Processing waveform 253 of 370
75.0 4050
Processing waveform 254 of 370
75.0 3375
Processing waveform 255 of 370
75.0 2925
Processing waveform 256 of 370
75.0 6675
Processing waveform 257 of 370
75.0 3750
Processing waveform 258 of 370
75.0 4575
Processing waveform 259 of 370
75.0 9075
Processing waveform 260 of 370
75.0 3900
Processing waveform 261 of 370
75.0 9075
Processing waveform 262 of 370
75.0 4875
Processing waveform 263 of 370
75.0 3075
Processing waveform 264 of 370
75.0 3150
Processing waveform 265 of 370
75.0 2550
Processing waveform 266 of 370
75.0 9075
Processing waveform 267 of 370
75.0 7425
Processing waveform 268 of 370
75.0 5025
Processing waveform 269 of 370
75.0 2850
Processing waveform 270 of 370
75.0 1875
Processing waveform 271 of 370
75.0 7125
Processing waveform 272 of 370
75.0 4500
Processing waveform 273 of 370
75.0 6075
Processing waveform 274 of 370
75.0 2475
Processing waveform 275 of 370
75.0 4125
Processing waveform 276 of 370
75.0 2100
Processing waveform 277 of 370
75.0 3975
Processing waveform 278 of 370
75.0 3075
Processing waveform 279 of 370
75.0 5550
Processing waveform 280 of 370
75.0 2250
Processing waveform 281 of 370
75.0 3600
Processing waveform 282 of 370
75.0 5775
Processing waveform 283 of 370
75.0 2325
Processing waveform 284 of 370
75.0 2325
Processing waveform 285 of 370
75.0 3525
Processing waveform 286 of 370
75.0 4500
Processing waveform 287 of 370
75.0 4275
Processing waveform 288 of 370
75.0 3450
Processing waveform 289 of 370
75.0 2550
Processing waveform 290 of 370
75.0 3300
Processing waveform 291 of 370
75.0 9075
Processing waveform 292 of 370
75.0 9075
Processing waveform 293 of 370
75.0 6525
Processing waveform 294 of 370
75.0 6975
Processing waveform 295 of 370
75.0 6750
Processing waveform 296 of 370
75.0 7275
Processing waveform 297 of 370
75.0 2400
Processing waveform 298 of 370
75.0 4125
Processing waveform 299 of 370
75.0 3525
Processing waveform 300 of 370
75.0 5175
Processing waveform 301 of 370
75.0 3750
Processing waveform 302 of 370
75.0 4875
Processing waveform 303 of 370
75.0 2250
Processing waveform 304 of 370
75.0 4125
Processing waveform 305 of 370
75.0 4425
Processing waveform 306 of 370
75.0 5250
Processing waveform 307 of 370
75.0 9075
Processing waveform 308 of 370
75.0 4725
Processing waveform 309 of 370
75.0 2475
Processing waveform 310 of 370
75.0 2925
Processing waveform 311 of 370
75.0 7350
Processing waveform 312 of 370
75.0 2550
Processing waveform 313 of 370
75.0 2850
Processing waveform 314 of 370
75.0 2550
Processing waveform 315 of 370
75.0 5775
Processing waveform 316 of 370
75.0 4200

Processing waveform 317 of 370
75.0 2175
Processing waveform 318 of 370
75.0 2925
Processing waveform 319 of 370
75.0 10350
Processing waveform 320 of 370
75.0 15600
Processing waveform 321 of 370
75.0 6450
Processing waveform 322 of 370
75.0 3450
Processing waveform 323 of 370
75.0 11175
Processing waveform 324 of 370
75.0 4725
Processing waveform 325 of 370
75.0 4200
Processing waveform 326 of 370
75.0 3750
Processing waveform 327 of 370
75.0 3975
Processing waveform 328 of 370
75.0 4575
Processing waveform 329 of 370
75.0 3225
Processing waveform 330 of 370
75.0 13500
Processing waveform 331 of 370
75.0 4050
Processing waveform 332 of 370
75.0 5250
Processing waveform 333 of 370
75.0 7050
Processing waveform 334 of 370
75.0 3450
Processing waveform 335 of 370
75.0 6675
Processing waveform 336 of 370
75.0 6750
Processing waveform 337 of 370
75.0 2475
Processing waveform 338 of 370
75.0 7650
Processing waveform 339 of 370
75.0 6750
Processing waveform 340 of 370
75.0 4125
Processing waveform 341 of 370
75.0 7425
Processing waveform 342 of 370
75.0 5400
Processing waveform 343 of 370
75.0 4800
Processing waveform 344 of 370
75.0 10200
Processing waveform 345 of 370
75.0 18225
Processing waveform 346 of 370
75.0 5925
Processing waveform 347 of 370
75.0 3525
Processing waveform 348 of 370
75.0 3300
Processing waveform 349 of 370
75.0 3600
Processing waveform 350 of 370
75.0 4650
Processing waveform 351 of 370
75.0 7875
Processing waveform 352 of 370
75.0 6300
Processing waveform 353 of 370
75.0 6750
Processing waveform 354 of 370
75.0 2925
Processing waveform 355 of 370
75.0 3525
Processing waveform 356 of 370
75.0 7875
Processing waveform 357 of 370
75.0 8925
Processing waveform 358 of 370
75.0 2250
Processing waveform 359 of 370
75.0 2700
Processing waveform 360 of 370
75.0 3000
Processing waveform 361 of 370
75.0 3300
Processing waveform 362 of 370
75.0 4425
Processing waveform 363 of 370
75.0 4500
Processing waveform 364 of 370
75.0 7500
Processing waveform 365 of 370
75.0 4200
Processing waveform 366 of 370
75.0 4800
Processing waveform 367 of 370
75.0 1950
Processing waveform 368 of 370
75.0 4650
Processing waveform 369 of 370
75.0 5475
Training data have been read and features have been extracted  (370, 120)
Computation time:  501.43119978904724
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.46878862380981445
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h    98                   
        l          88             
        r                93       
        t                      91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  27.1171171171  +/-  3.58848178564  %
          Predicted class
              h     l     r     t 
        h    13    11    11    13 
        l    13    12    10     9 
        r    14    11    11    10 
        t    14     8    10    14 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'h', 'l', 't', 'r'] 1
class
e    78
h    98
l    88
r    93
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 448
75.0 3450
Processing waveform 1 of 448
75.0 4125
Processing waveform 2 of 448
75.0 2400
Processing waveform 3 of 448
75.0 2700
Processing waveform 4 of 448
75.0 2700
Processing waveform 5 of 448
75.0 4350
Processing waveform 6 of 448
75.0 2775
Processing waveform 7 of 448
75.0 1950
Processing waveform 8 of 448
75.0 2250
Processing waveform 9 of 448
75.0 2250
Processing waveform 10 of 448
75.0 2250
Processing waveform 11 of 448
75.0 5100
Processing waveform 12 of 448
75.0 2475
Processing waveform 13 of 448
75.0 3300
Processing waveform 14 of 448
75.0 6300
Processing waveform 15 of 448
75.0 5100
Processing waveform 16 of 448
75.0 9075
Processing waveform 17 of 448
75.0 3225
Processing waveform 18 of 448
75.0 4125
Processing waveform 19 of 448
75.0 3300
Processing waveform 20 of 448
75.0 3225
Processing waveform 21 of 448
75.0 2850
Processing waveform 22 of 448
75.0 5400
Processing waveform 23 of 448
75.0 2400
Processing waveform 24 of 448
75.0 7725
Processing waveform 25 of 448
75.0 3900
Processing waveform 26 of 448
75.0 2775
Processing waveform 27 of 448
75.0 2325
Processing waveform 28 of 448
75.0 4425
Processing waveform 29 of 448
75.0 3525
Processing waveform 30 of 448
75.0 6300
Processing waveform 31 of 448
75.0 2625
Processing waveform 32 of 448
75.0 4050
Processing waveform 33 of 448
75.0 2775
Processing waveform 34 of 448
75.0 2400
Processing waveform 35 of 448
75.0 2850
Processing waveform 36 of 448
75.0 2250
Processing waveform 37 of 448
75.0 2325
Processing waveform 38 of 448
75.0 3750
Processing waveform 39 of 448
75.0 3525
Processing waveform 40 of 448
75.0 5700
Processing waveform 41 of 448
75.0 2700
Processing waveform 42 of 448
75.0 3000
Processing waveform 43 of 448
75.0 4350
Processing waveform 44 of 448
75.0 4950
Processing waveform 45 of 448
75.0 2925
Processing waveform 46 of 448
75.0 2775
Processing waveform 47 of 448
75.0 3450
Processing waveform 48 of 448
75.0 4875
Processing waveform 49 of 448
75.0 9075
Processing waveform 50 of 448
75.0 5175
Processing waveform 51 of 448
75.0 3075
Processing waveform 52 of 448
75.0 4125
Processing waveform 53 of 448
75.0 9075
Processing waveform 54 of 448
75.0 3225
Processing waveform 55 of 448
75.0 2775
Processing waveform 56 of 448
75.0 4575
Processing waveform 57 of 448
75.0 2700
Processing waveform 58 of 448
75.0 1950
Processing waveform 59 of 448
75.0 2100
Processing waveform 60 of 448
75.0 4575
Processing waveform 61 of 448
75.0 5850
Processing waveform 62 of 448
75.0 3150
Processing waveform 63 of 448
75.0 4275
Processing waveform 64 of 448
75.0 9075
Processing waveform 65 of 448
75.0 3600
Processing waveform 66 of 448
75.0 2625
Processing waveform 67 of 448
75.0 2700
Processing waveform 68 of 448
75.0 2700
Processing waveform 69 of 448
75.0 3525
Processing waveform 70 of 448
75.0 4200
Processing waveform 71 of 448
75.0 4200
Processing waveform 72 of 448
75.0 1950
Processing waveform 73 of 448
75.0 5775
Processing waveform 74 of 448
75.0 9075
Processing waveform 75 of 448
75.0 4200
Processing waveform 76 of 448
75.0 2625
Processing waveform 77 of 448
75.0 4875
Processing waveform 78 of 448
75.0 5025
Processing waveform 79 of 448
75.0 14625
Processing waveform 80 of 448
75.0 11025
Processing waveform 81 of 448
75.0 1950
Processing waveform 82 of 448
75.0 3975
Processing waveform 83 of 448
75.0 2550
Processing waveform 84 of 448
75.0 7500
Processing waveform 85 of 448
75.0 14925
Processing waveform 86 of 448
75.0 13275
Processing waveform 87 of 448
75.0 1725
Processing waveform 88 of 448
75.0 2400
Processing waveform 89 of 448
75.0 6375
Processing waveform 90 of 448
75.0 6450
Processing waveform 91 of 448
75.0 16725
Processing waveform 92 of 448
75.0 6075
Processing waveform 93 of 448
75.0 3450
Processing waveform 94 of 448
75.0 2325
Processing waveform 95 of 448
75.0 2925
Processing waveform 96 of 448
75.0 4575
Processing waveform 97 of 448
75.0 7050
Processing waveform 98 of 448
75.0 2025
Processing waveform 99 of 448
75.0 2925
Processing waveform 100 of 448
75.0 2850
Processing waveform 101 of 448
75.0 3600
Processing waveform 102 of 448
75.0 3750
Processing waveform 103 of 448
75.0 2100
Processing waveform 104 of 448
75.0 1650
Processing waveform 105 of 448
75.0 4200
Processing waveform 106 of 448
75.0 9075
Processing waveform 107 of 448
75.0 4125
Processing waveform 108 of 448
75.0 2925
Processing waveform 109 of 448
75.0 2250
Processing waveform 110 of 448
75.0 5475
Processing waveform 111 of 448
75.0 2625
Processing waveform 112 of 448
75.0 2400
Processing waveform 113 of 448
75.0 2700
Processing waveform 114 of 448
75.0 9075
Processing waveform 115 of 448
75.0 9075
Processing waveform 116 of 448
75.0 6225
Processing waveform 117 of 448
75.0 7125
Processing waveform 118 of 448
75.0 5850
Processing waveform 119 of 448
75.0 9075
Processing waveform 120 of 448
75.0 3225

Processing waveform 121 of 448
75.0 8100
Processing waveform 122 of 448
75.0 4575
Processing waveform 123 of 448
75.0 2850
Processing waveform 124 of 448
75.0 3375
Processing waveform 125 of 448
75.0 2100
Processing waveform 126 of 448
75.0 9075
Processing waveform 127 of 448
75.0 5850
Processing waveform 128 of 448
75.0 2775
Processing waveform 129 of 448
75.0 2700
Processing waveform 130 of 448
75.0 2100
Processing waveform 131 of 448
75.0 9075
Processing waveform 132 of 448
75.0 3225
Processing waveform 133 of 448
75.0 3300
Processing waveform 134 of 448
75.0 2475
Processing waveform 135 of 448
75.0 3075
Processing waveform 136 of 448
75.0 6450
Processing waveform 137 of 448
75.0 3600
Processing waveform 138 of 448
75.0 4650
Processing waveform 139 of 448
75.0 2475
Processing waveform 140 of 448
75.0 1875
Processing waveform 141 of 448
75.0 3450
Processing waveform 142 of 448
75.0 1500
Processing waveform 143 of 448
75.0 2625
Processing waveform 144 of 448
75.0 3525
Processing waveform 145 of 448
75.0 5625
Processing waveform 146 of 448
75.0 3675
Processing waveform 147 of 448
75.0 9075
Processing waveform 148 of 448
75.0 2700
Processing waveform 149 of 448
75.0 9075
Processing waveform 150 of 448
75.0 3000
Processing waveform 151 of 448
75.0 3525
Processing waveform 152 of 448
75.0 7200
Processing waveform 153 of 448
75.0 4875
Processing waveform 154 of 448
75.0 6075
Processing waveform 155 of 448
75.0 2700
Processing waveform 156 of 448
75.0 5250
Processing waveform 157 of 448
75.0 12750
Processing waveform 158 of 448
75.0 5100
Processing waveform 159 of 448
75.0 2475
Processing waveform 160 of 448
75.0 6525
Processing waveform 161 of 448
75.0 7200
Processing waveform 162 of 448
75.0 4500
Processing waveform 163 of 448
75.0 4950
Processing waveform 164 of 448
75.0 3150
Processing waveform 165 of 448
75.0 5850
Processing waveform 166 of 448
75.0 2325
Processing waveform 167 of 448
75.0 4200
Processing waveform 168 of 448
75.0 13350
Processing waveform 169 of 448
75.0 9525
Processing waveform 170 of 448
75.0 2325
Processing waveform 171 of 448
75.0 3975
Processing waveform 172 of 448
75.0 2400
Processing waveform 173 of 448
75.0 5025
Processing waveform 174 of 448
75.0 6825
Processing waveform 175 of 448
75.0 3750
Processing waveform 176 of 448
75.0 9075
Processing waveform 177 of 448
75.0 4500
Processing waveform 178 of 448
75.0 2175
Processing waveform 179 of 448
75.0 3300
Processing waveform 180 of 448
75.0 4275
Processing waveform 181 of 448
75.0 2850
Processing waveform 182 of 448
75.0 2850
Processing waveform 183 of 448
75.0 2700
Processing waveform 184 of 448
75.0 5250
Processing waveform 185 of 448
75.0 3000
Processing waveform 186 of 448
75.0 2775
Processing waveform 187 of 448
75.0 6450
Processing waveform 188 of 448
75.0 3150
Processing waveform 189 of 448
75.0 1650
Processing waveform 190 of 448
75.0 4500
Processing waveform 191 of 448
75.0 2250
Processing waveform 192 of 448
75.0 2175
Processing waveform 193 of 448
75.0 3225
Processing waveform 194 of 448
75.0 4050
Processing waveform 195 of 448
75.0 3300
Processing waveform 196 of 448
75.0 2850
Processing waveform 197 of 448
75.0 2025
Processing waveform 198 of 448
75.0 4800
Processing waveform 199 of 448
75.0 2325
Processing waveform 200 of 448
75.0 6150
Processing waveform 201 of 448
75.0 2775
Processing waveform 202 of 448
75.0 2850
Processing waveform 203 of 448
75.0 4425
Processing waveform 204 of 448
75.0 2250
Processing waveform 205 of 448
75.0 3075
Processing waveform 206 of 448
75.0 2775
Processing waveform 207 of 448
75.0 1875
Processing waveform 208 of 448
75.0 3300
Processing waveform 209 of 448
75.0 1875
Processing waveform 210 of 448
75.0 5550
Processing waveform 211 of 448
75.0 9075
Processing waveform 212 of 448
75.0 4650
Processing waveform 213 of 448
75.0 6525
Processing waveform 214 of 448
75.0 9075
Processing waveform 215 of 448
75.0 3225
Processing waveform 216 of 448
75.0 3150
Processing waveform 217 of 448
75.0 4275
Processing waveform 218 of 448
75.0 4050
Processing waveform 219 of 448
75.0 5400
Processing waveform 220 of 448
75.0 4725
Processing waveform 221 of 448
75.0 3450
Processing waveform 222 of 448
75.0 6150
Processing waveform 223 of 448
75.0 2250
Processing waveform 224 of 448
75.0 9000
Processing waveform 225 of 448
75.0 3150
Processing waveform 226 of 448
75.0 4950
Processing waveform 227 of 448
75.0 6750
Processing waveform 228 of 448
75.0 3150
Processing waveform 229 of 448
75.0 7050
Processing waveform 230 of 448
75.0 3675
Processing waveform 231 of 448
75.0 3225
Processing waveform 232 of 448
75.0 2325
Processing waveform 233 of 448
75.0 7500
Processing waveform 234 of 448
75.0 3975
Processing waveform 235 of 448
75.0 6150
Processing waveform 236 of 448
75.0 7875
Processing waveform 237 of 448
75.0 7275
Processing waveform 238 of 448
75.0 6075
Processing waveform 239 of 448
75.0 3900
Processing waveform 240 of 448
75.0 3000
Processing waveform 241 of 448
75.0 3900
Processing waveform 242 of 448
75.0 6525
Processing waveform 243 of 448
75.0 2925
Processing waveform 244 of 448
75.0 2700
Processing waveform 245 of 448
75.0 2250
Processing waveform 246 of 448
75.0 4425
Processing waveform 247 of 448
75.0 7500
Processing waveform 248 of 448
75.0 2250
Processing waveform 249 of 448
75.0 9600
Processing waveform 250 of 448
75.0 4650
Processing waveform 251 of 448
75.0 6750
Processing waveform 252 of 448
75.0 8925
Processing waveform 253 of 448
75.0 7125
Processing waveform 254 of 448
75.0 3000
Processing waveform 255 of 448
75.0 1800
Processing waveform 256 of 448
75.0 4425
Processing waveform 257 of 448
75.0 9450
Processing waveform 258 of 448
75.0 6525
Processing waveform 259 of 448
75.0 3225
Processing waveform 260 of 448
75.0 2100
Processing waveform 261 of 448
75.0 2700
Processing waveform 262 of 448
75.0 2325
Processing waveform 263 of 448
75.0 2775
Processing waveform 264 of 448
75.0 5850
Processing waveform 265 of 448
75.0 3825
Processing waveform 266 of 448
75.0 6000
Processing waveform 267 of 448
75.0 2025
Processing waveform 268 of 448
75.0 1725
Processing waveform 269 of 448
75.0 2175
Processing waveform 270 of 448
75.0 5400
Processing waveform 271 of 448
75.0 3000
Processing waveform 272 of 448
75.0 5325
Processing waveform 273 of 448
75.0 3750
Processing waveform 274 of 448
75.0 3075
Processing waveform 275 of 448
75.0 5550
Processing waveform 276 of 448
75.0 7350
Processing waveform 277 of 448
75.0 4650
Processing waveform 278 of 448
75.0 2775
Processing waveform 279 of 448
75.0 2325
Processing waveform 280 of 448
75.0 2925
Processing waveform 281 of 448
75.0 3075
Processing waveform 282 of 448
75.0 6375
Processing waveform 283 of 448
75.0 2475
Processing waveform 284 of 448
75.0 3975
Processing waveform 285 of 448
75.0 3150
Processing waveform 286 of 448
75.0 5925
Processing waveform 287 of 448
75.0 5400
Processing waveform 288 of 448
75.0 1875
Processing waveform 289 of 448
75.0 5850
Processing waveform 290 of 448
75.0 1725
Processing waveform 291 of 448
75.0 7650
Processing waveform 292 of 448
75.0 2700
Processing waveform 293 of 448
75.0 9075
Processing waveform 294 of 448
75.0 6975
Processing waveform 295 of 448
75.0 4050
Processing waveform 296 of 448
75.0 3450
Processing waveform 297 of 448
75.0 5925
Processing waveform 298 of 448
75.0 2250
Processing waveform 299 of 448
75.0 3675
Processing waveform 300 of 448
75.0 2625
Processing waveform 301 of 448
75.0 4575
Processing waveform 302 of 448
75.0 2025
Processing waveform 303 of 448
75.0 5175
Processing waveform 304 of 448
75.0 3225
Processing waveform 305 of 448
75.0 4950
Processing waveform 306 of 448
75.0 2925
Processing waveform 307 of 448
75.0 1800
Processing waveform 308 of 448
75.0 8925
Processing waveform 309 of 448
75.0 7350
Processing waveform 310 of 448
75.0 4125
Processing waveform 311 of 448
75.0 3225
Processing waveform 312 of 448
75.0 9075
Processing waveform 313 of 448
75.0 5025
Processing waveform 314 of 448
75.0 8925
Processing waveform 315 of 448
75.0 4800
Processing waveform 316 of 448
75.0 7800
Processing waveform 317 of 448
75.0 2775
Processing waveform 318 of 448
75.0 6600
Processing waveform 319 of 448
75.0 1800
Processing waveform 320 of 448
75.0 2625

Processing waveform 321 of 448
75.0 9075
Processing waveform 322 of 448
75.0 7200
Processing waveform 323 of 448
75.0 5550
Processing waveform 324 of 448
75.0 2775
Processing waveform 325 of 448
75.0 5250
Processing waveform 326 of 448
75.0 1875
Processing waveform 327 of 448
75.0 1875
Processing waveform 328 of 448
75.0 5025
Processing waveform 329 of 448
75.0 3600
Processing waveform 330 of 448
75.0 3375
Processing waveform 331 of 448
75.0 4050
Processing waveform 332 of 448
75.0 3375
Processing waveform 333 of 448
75.0 2925
Processing waveform 334 of 448
75.0 6675
Processing waveform 335 of 448
75.0 3750
Processing waveform 336 of 448
75.0 4575
Processing waveform 337 of 448
75.0 9075
Processing waveform 338 of 448
75.0 3900
Processing waveform 339 of 448
75.0 9075
Processing waveform 340 of 448
75.0 4875
Processing waveform 341 of 448
75.0 3075
Processing waveform 342 of 448
75.0 3150
Processing waveform 343 of 448
75.0 2550
Processing waveform 344 of 448
75.0 9075
Processing waveform 345 of 448
75.0 7425
Processing waveform 346 of 448
75.0 5025
Processing waveform 347 of 448
75.0 2850
Processing waveform 348 of 448
75.0 1875
Processing waveform 349 of 448
75.0 7125
Processing waveform 350 of 448
75.0 4500
Processing waveform 351 of 448
75.0 6075
Processing waveform 352 of 448
75.0 2475
Processing waveform 353 of 448
75.0 4125
Processing waveform 354 of 448
75.0 2100
Processing waveform 355 of 448
75.0 3975
Processing waveform 356 of 448
75.0 3075
Processing waveform 357 of 448
75.0 5550
Processing waveform 358 of 448
75.0 2250
Processing waveform 359 of 448
75.0 3600
Processing waveform 360 of 448
75.0 5775
Processing waveform 361 of 448
75.0 2325
Processing waveform 362 of 448
75.0 2325
Processing waveform 363 of 448
75.0 3525
Processing waveform 364 of 448
75.0 4500
Processing waveform 365 of 448
75.0 4275
Processing waveform 366 of 448
75.0 3450
Processing waveform 367 of 448
75.0 2550
Processing waveform 368 of 448
75.0 3300
Processing waveform 369 of 448
75.0 9075
Processing waveform 370 of 448
75.0 9075
Processing waveform 371 of 448
75.0 6525
Processing waveform 372 of 448
75.0 6975
Processing waveform 373 of 448
75.0 6750
Processing waveform 374 of 448
75.0 7275
Processing waveform 375 of 448
75.0 2400
Processing waveform 376 of 448
75.0 4125
Processing waveform 377 of 448
75.0 3525
Processing waveform 378 of 448
75.0 5175
Processing waveform 379 of 448
75.0 3750
Processing waveform 380 of 448
75.0 4875
Processing waveform 381 of 448
75.0 2250
Processing waveform 382 of 448
75.0 4125
Processing waveform 383 of 448
75.0 4425
Processing waveform 384 of 448
75.0 5250
Processing waveform 385 of 448
75.0 9075
Processing waveform 386 of 448
75.0 4725
Processing waveform 387 of 448
75.0 2475
Processing waveform 388 of 448
75.0 2925
Processing waveform 389 of 448
75.0 7350
Processing waveform 390 of 448
75.0 2550
Processing waveform 391 of 448
75.0 2850
Processing waveform 392 of 448
75.0 2550
Processing waveform 393 of 448
75.0 5775
Processing waveform 394 of 448
75.0 4200
Processing waveform 395 of 448
75.0 2175
Processing waveform 396 of 448
75.0 2925
Processing waveform 397 of 448
75.0 10350
Processing waveform 398 of 448
75.0 15600
Processing waveform 399 of 448
75.0 6450
Processing waveform 400 of 448
75.0 3450
Processing waveform 401 of 448
75.0 11175
Processing waveform 402 of 448
75.0 4725
Processing waveform 403 of 448
75.0 4200
Processing waveform 404 of 448
75.0 3750
Processing waveform 405 of 448
75.0 3975
Processing waveform 406 of 448
75.0 4575
Processing waveform 407 of 448
75.0 3225
Processing waveform 408 of 448
75.0 13500
Processing waveform 409 of 448
75.0 4050
Processing waveform 410 of 448
75.0 5250
Processing waveform 411 of 448
75.0 7050
Processing waveform 412 of 448
75.0 3450
Processing waveform 413 of 448
75.0 6675
Processing waveform 414 of 448
75.0 6750
Processing waveform 415 of 448
75.0 2475
Processing waveform 416 of 448
75.0 7650
Processing waveform 417 of 448
75.0 6750
Processing waveform 418 of 448
75.0 4125
Processing waveform 419 of 448
75.0 7425
Processing waveform 420 of 448
75.0 5400
Processing waveform 421 of 448
75.0 4800
Processing waveform 422 of 448
75.0 10200
Processing waveform 423 of 448
75.0 18225
Processing waveform 424 of 448
75.0 5925
Processing waveform 425 of 448
75.0 3525
Processing waveform 426 of 448
75.0 3300
Processing waveform 427 of 448
75.0 3600
Processing waveform 428 of 448
75.0 4650
Processing waveform 429 of 448
75.0 7875
Processing waveform 430 of 448
75.0 6300
Processing waveform 431 of 448
75.0 6750
Processing waveform 432 of 448
75.0 2925
Processing waveform 433 of 448
75.0 3525
Processing waveform 434 of 448
75.0 7875
Processing waveform 435 of 448
75.0 8925
Processing waveform 436 of 448
75.0 2250
Processing waveform 437 of 448
75.0 2700
Processing waveform 438 of 448
75.0 3000
Processing waveform 439 of 448
75.0 3300
Processing waveform 440 of 448
75.0 4425
Processing waveform 441 of 448
75.0 4500
Processing waveform 442 of 448
75.0 7500
Processing waveform 443 of 448
75.0 4200
Processing waveform 444 of 448
75.0 4800
Processing waveform 445 of 448
75.0 1950
Processing waveform 446 of 448
75.0 4650
Processing waveform 447 of 448
75.0 5475
Training data have been read and features have been extracted  (448, 120)
Computation time:  600.7623279094696
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.6417849063873291
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    78                         
        h          98                   
        l                88             
        r                      93       
        t                            91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  20.9375  +/-  2.22125319894  %
          Predicted class
              e     h     l     r     t 
        e     5    10     5     9    10 
        h     6    12    10    10    11 
        l     5    13    10     9     7 
        r     6    12    10    10     9 
        t     7    11     8     8    11 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['l', 't'] 2
class
l    81
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 172
75.0 3975
Processing waveform 1 of 172
75.0 2400
Processing waveform 2 of 172
75.0 9075
Processing waveform 3 of 172
75.0 2175
Processing waveform 4 of 172
75.0 3300
Processing waveform 5 of 172
75.0 4275
Processing waveform 6 of 172
75.0 2850
Processing waveform 7 of 172
75.0 2850
Processing waveform 8 of 172
75.0 2700
Processing waveform 9 of 172
75.0 5250
Processing waveform 10 of 172
75.0 3000
Processing waveform 11 of 172
75.0 2775
Processing waveform 12 of 172
75.0 6450
Processing waveform 13 of 172
75.0 3150
Processing waveform 14 of 172
75.0 1650
Processing waveform 15 of 172
75.0 4500
Processing waveform 16 of 172
75.0 2250
Processing waveform 17 of 172
75.0 2175
Processing waveform 18 of 172
75.0 3225
Processing waveform 19 of 172
75.0 4050
Processing waveform 20 of 172
75.0 3300
Processing waveform 21 of 172
75.0 2850
Processing waveform 22 of 172
75.0 2025
Processing waveform 23 of 172
75.0 4800
Processing waveform 24 of 172
75.0 2325
Processing waveform 25 of 172
75.0 6150
Processing waveform 26 of 172
75.0 2775
Processing waveform 27 of 172
75.0 2850
Processing waveform 28 of 172
75.0 4425
Processing waveform 29 of 172
75.0 2250
Processing waveform 30 of 172
75.0 3075
Processing waveform 31 of 172
75.0 2775
Processing waveform 32 of 172
75.0 1875
Processing waveform 33 of 172
75.0 3300
Processing waveform 34 of 172
75.0 1875
Processing waveform 35 of 172
75.0 5550
Processing waveform 36 of 172
75.0 9075
Processing waveform 37 of 172
75.0 4650
Processing waveform 38 of 172
75.0 6525
Processing waveform 39 of 172
75.0 9075
Processing waveform 40 of 172
75.0 3225
Processing waveform 41 of 172
75.0 3150
Processing waveform 42 of 172
75.0 4275

Processing waveform 43 of 172
75.0 4050
Processing waveform 44 of 172
75.0 5400
Processing waveform 45 of 172
75.0 4725
Processing waveform 46 of 172
75.0 3450
Processing waveform 47 of 172
75.0 6150
Processing waveform 48 of 172
75.0 2250
Processing waveform 49 of 172
75.0 9000
Processing waveform 50 of 172
75.0 3150
Processing waveform 51 of 172
75.0 4950
Processing waveform 52 of 172
75.0 6750
Processing waveform 53 of 172
75.0 3150
Processing waveform 54 of 172
75.0 7050
Processing waveform 55 of 172
75.0 3675
Processing waveform 56 of 172
75.0 3225
Processing waveform 57 of 172
75.0 2325
Processing waveform 58 of 172
75.0 7500
Processing waveform 59 of 172
75.0 3975
Processing waveform 60 of 172
75.0 6150
Processing waveform 61 of 172
75.0 7875
Processing waveform 62 of 172
75.0 7275
Processing waveform 63 of 172
75.0 6075
Processing waveform 64 of 172
75.0 3900
Processing waveform 65 of 172
75.0 3000
Processing waveform 66 of 172
75.0 3900
Processing waveform 67 of 172
75.0 6525
Processing waveform 68 of 172
75.0 2925
Processing waveform 69 of 172
75.0 2700
Processing waveform 70 of 172
75.0 2250
Processing waveform 71 of 172
75.0 4425
Processing waveform 72 of 172
75.0 2250
Processing waveform 73 of 172
75.0 9600
Processing waveform 74 of 172
75.0 4650
Processing waveform 75 of 172
75.0 6750
Processing waveform 76 of 172
75.0 7125
Processing waveform 77 of 172
75.0 3000
Processing waveform 78 of 172
75.0 1800
Processing waveform 79 of 172
75.0 4425
Processing waveform 80 of 172
75.0 9450
Processing waveform 81 of 172
75.0 5550
Processing waveform 82 of 172
75.0 2250
Processing waveform 83 of 172
75.0 3600
Processing waveform 84 of 172
75.0 5775
Processing waveform 85 of 172
75.0 2325
Processing waveform 86 of 172
75.0 2325
Processing waveform 87 of 172
75.0 3525
Processing waveform 88 of 172
75.0 4500
Processing waveform 89 of 172
75.0 4275
Processing waveform 90 of 172
75.0 3450
Processing waveform 91 of 172
75.0 2550
Processing waveform 92 of 172
75.0 3300
Processing waveform 93 of 172
75.0 9075
Processing waveform 94 of 172
75.0 9075
Processing waveform 95 of 172
75.0 6525
Processing waveform 96 of 172
75.0 6975
Processing waveform 97 of 172
75.0 6750
Processing waveform 98 of 172
75.0 7275
Processing waveform 99 of 172
75.0 2400
Processing waveform 100 of 172
75.0 4125
Processing waveform 101 of 172
75.0 3525
Processing waveform 102 of 172
75.0 5175
Processing waveform 103 of 172
75.0 3750
Processing waveform 104 of 172
75.0 4875
Processing waveform 105 of 172
75.0 2250
Processing waveform 106 of 172
75.0 4125
Processing waveform 107 of 172
75.0 4425
Processing waveform 108 of 172
75.0 5250
Processing waveform 109 of 172
75.0 9075
Processing waveform 110 of 172
75.0 4725
Processing waveform 111 of 172
75.0 2475
Processing waveform 112 of 172
75.0 2925
Processing waveform 113 of 172
75.0 7350
Processing waveform 114 of 172
75.0 2550
Processing waveform 115 of 172
75.0 2850
Processing waveform 116 of 172
75.0 2550
Processing waveform 117 of 172
75.0 5775
Processing waveform 118 of 172
75.0 4200
Processing waveform 119 of 172
75.0 2175
Processing waveform 120 of 172
75.0 2925
Processing waveform 121 of 172
75.0 10350
Processing waveform 122 of 172
75.0 15600
Processing waveform 123 of 172
75.0 6450
Processing waveform 124 of 172
75.0 3450
Processing waveform 125 of 172
75.0 11175
Processing waveform 126 of 172
75.0 4725
Processing waveform 127 of 172
75.0 4200
Processing waveform 128 of 172
75.0 3750
Processing waveform 129 of 172
75.0 3975
Processing waveform 130 of 172
75.0 4575
Processing waveform 131 of 172
75.0 3225
Processing waveform 132 of 172
75.0 13500
Processing waveform 133 of 172
75.0 4050
Processing waveform 134 of 172
75.0 5250
Processing waveform 135 of 172
75.0 7050
Processing waveform 136 of 172
75.0 3450
Processing waveform 137 of 172
75.0 6675
Processing waveform 138 of 172
75.0 6750
Processing waveform 139 of 172
75.0 2475
Processing waveform 140 of 172
75.0 7650
Processing waveform 141 of 172
75.0 6750
Processing waveform 142 of 172
75.0 4125
Processing waveform 143 of 172
75.0 7425
Processing waveform 144 of 172
75.0 5400
Processing waveform 145 of 172
75.0 4800
Processing waveform 146 of 172
75.0 10200
Processing waveform 147 of 172
75.0 18225
Processing waveform 148 of 172
75.0 5925
Processing waveform 149 of 172
75.0 3525
Processing waveform 150 of 172
75.0 3300
Processing waveform 151 of 172
75.0 3600
Processing waveform 152 of 172
75.0 4650
Processing waveform 153 of 172
75.0 7875
Processing waveform 154 of 172
75.0 6300
Processing waveform 155 of 172
75.0 6750
Processing waveform 156 of 172
75.0 2925
Processing waveform 157 of 172
75.0 3525
Processing waveform 158 of 172
75.0 7875
Processing waveform 159 of 172
75.0 8925
Processing waveform 160 of 172
75.0 2250
Processing waveform 161 of 172
75.0 2700
Processing waveform 162 of 172
75.0 3000
Processing waveform 163 of 172
75.0 3300
Processing waveform 164 of 172
75.0 4425
Processing waveform 165 of 172
75.0 4500
Processing waveform 166 of 172
75.0 7500
Processing waveform 167 of 172
75.0 4200
Processing waveform 168 of 172
75.0 4800
Processing waveform 169 of 172
75.0 1950
Processing waveform 170 of 172
75.0 4650
Processing waveform 171 of 172
75.0 5475
Training data have been read and features have been extracted  (172, 120)
Computation time:  241.60809659957886
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.13846826553344727
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l    81       
        t          91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  60.3875968992  +/-  6.01911902878  %
          Predicted class
              l     t 
        l    23    18 
        t    17    29 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'r'] 2
class
e    75
r    83
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 158
75.0 3450
Processing waveform 1 of 158
75.0 4125
Processing waveform 2 of 158
75.0 2400
Processing waveform 3 of 158
75.0 2700
Processing waveform 4 of 158
75.0 2700
Processing waveform 5 of 158
75.0 4350
Processing waveform 6 of 158
75.0 2775
Processing waveform 7 of 158
75.0 2250
Processing waveform 8 of 158
75.0 2250
Processing waveform 9 of 158
75.0 2475
Processing waveform 10 of 158
75.0 3300
Processing waveform 11 of 158
75.0 5100
Processing waveform 12 of 158
75.0 9075
Processing waveform 13 of 158
75.0 3225
Processing waveform 14 of 158
75.0 4125
Processing waveform 15 of 158
75.0 3300
Processing waveform 16 of 158
75.0 3225
Processing waveform 17 of 158
75.0 2850
Processing waveform 18 of 158
75.0 2400
Processing waveform 19 of 158
75.0 3900
Processing waveform 20 of 158
75.0 2775
Processing waveform 21 of 158
75.0 2325
Processing waveform 22 of 158
75.0 4425
Processing waveform 23 of 158
75.0 3525
Processing waveform 24 of 158
75.0 6300
Processing waveform 25 of 158
75.0 2625
Processing waveform 26 of 158
75.0 4050
Processing waveform 27 of 158
75.0 2775
Processing waveform 28 of 158
75.0 2400
Processing waveform 29 of 158
75.0 2850
Processing waveform 30 of 158
75.0 2250
Processing waveform 31 of 158
75.0 2325
Processing waveform 32 of 158
75.0 3750
Processing waveform 33 of 158
75.0 3525
Processing waveform 34 of 158
75.0 5700
Processing waveform 35 of 158
75.0 2700
Processing waveform 36 of 158
75.0 3000
Processing waveform 37 of 158
75.0 4350
Processing waveform 38 of 158
75.0 4950
Processing waveform 39 of 158
75.0 2925
Processing waveform 40 of 158
75.0 2775
Processing waveform 41 of 158
75.0 3450
Processing waveform 42 of 158
75.0 4875
Processing waveform 43 of 158
75.0 9075
Processing waveform 44 of 158
75.0 5175
Processing waveform 45 of 158
75.0 3075
Processing waveform 46 of 158
75.0 4125
Processing waveform 47 of 158
75.0 9075
Processing waveform 48 of 158
75.0 2775
Processing waveform 49 of 158
75.0 4575
Processing waveform 50 of 158
75.0 2700

Processing waveform 51 of 158
75.0 1950
Processing waveform 52 of 158
75.0 2100
Processing waveform 53 of 158
75.0 4575
Processing waveform 54 of 158
75.0 5850
Processing waveform 55 of 158
75.0 3150
Processing waveform 56 of 158
75.0 4275
Processing waveform 57 of 158
75.0 9075
Processing waveform 58 of 158
75.0 3600
Processing waveform 59 of 158
75.0 2625
Processing waveform 60 of 158
75.0 2700
Processing waveform 61 of 158
75.0 2700
Processing waveform 62 of 158
75.0 3525
Processing waveform 63 of 158
75.0 4200
Processing waveform 64 of 158
75.0 4200
Processing waveform 65 of 158
75.0 1950
Processing waveform 66 of 158
75.0 5775
Processing waveform 67 of 158
75.0 9075
Processing waveform 68 of 158
75.0 4875
Processing waveform 69 of 158
75.0 14625
Processing waveform 70 of 158
75.0 11025
Processing waveform 71 of 158
75.0 1950
Processing waveform 72 of 158
75.0 3975
Processing waveform 73 of 158
75.0 2550
Processing waveform 74 of 158
75.0 7500
Processing waveform 75 of 158
75.0 14925
Processing waveform 76 of 158
75.0 13275
Processing waveform 77 of 158
75.0 1725
Processing waveform 78 of 158
75.0 2400
Processing waveform 79 of 158
75.0 6375
Processing waveform 80 of 158
75.0 6450
Processing waveform 81 of 158
75.0 16725
Processing waveform 82 of 158
75.0 6075
Processing waveform 83 of 158
75.0 3450
Processing waveform 84 of 158
75.0 2325
Processing waveform 85 of 158
75.0 2925
Processing waveform 86 of 158
75.0 4575
Processing waveform 87 of 158
75.0 7050
Processing waveform 88 of 158
75.0 2025
Processing waveform 89 of 158
75.0 2925
Processing waveform 90 of 158
75.0 2850
Processing waveform 91 of 158
75.0 3600
Processing waveform 92 of 158
75.0 3750
Processing waveform 93 of 158
75.0 2100
Processing waveform 94 of 158
75.0 1650
Processing waveform 95 of 158
75.0 4200
Processing waveform 96 of 158
75.0 9075
Processing waveform 97 of 158
75.0 4125
Processing waveform 98 of 158
75.0 2925
Processing waveform 99 of 158
75.0 2250
Processing waveform 100 of 158
75.0 5475
Processing waveform 101 of 158
75.0 2625
Processing waveform 102 of 158
75.0 2400
Processing waveform 103 of 158
75.0 2700
Processing waveform 104 of 158
75.0 9075
Processing waveform 105 of 158
75.0 9075
Processing waveform 106 of 158
75.0 6225
Processing waveform 107 of 158
75.0 7125
Processing waveform 108 of 158
75.0 5850
Processing waveform 109 of 158
75.0 9075
Processing waveform 110 of 158
75.0 3225
Processing waveform 111 of 158
75.0 8100
Processing waveform 112 of 158
75.0 4575
Processing waveform 113 of 158
75.0 2850
Processing waveform 114 of 158
75.0 3375
Processing waveform 115 of 158
75.0 2100
Processing waveform 116 of 158
75.0 9075
Processing waveform 117 of 158
75.0 5850
Processing waveform 118 of 158
75.0 2775
Processing waveform 119 of 158
75.0 2700
Processing waveform 120 of 158
75.0 2100
Processing waveform 121 of 158
75.0 9075
Processing waveform 122 of 158
75.0 3225
Processing waveform 123 of 158
75.0 3300
Processing waveform 124 of 158
75.0 2475
Processing waveform 125 of 158
75.0 3075
Processing waveform 126 of 158
75.0 6450
Processing waveform 127 of 158
75.0 3600
Processing waveform 128 of 158
75.0 4650
Processing waveform 129 of 158
75.0 2475
Processing waveform 130 of 158
75.0 1875
Processing waveform 131 of 158
75.0 3450
Processing waveform 132 of 158
75.0 1500
Processing waveform 133 of 158
75.0 2625
Processing waveform 134 of 158
75.0 3525
Processing waveform 135 of 158
75.0 5625
Processing waveform 136 of 158
75.0 3675
Processing waveform 137 of 158
75.0 9075
Processing waveform 138 of 158
75.0 2700
Processing waveform 139 of 158
75.0 3000
Processing waveform 140 of 158
75.0 3525
Processing waveform 141 of 158
75.0 7200
Processing waveform 142 of 158
75.0 4875
Processing waveform 143 of 158
75.0 6075
Processing waveform 144 of 158
75.0 2700
Processing waveform 145 of 158
75.0 5250
Processing waveform 146 of 158
75.0 12750
Processing waveform 147 of 158
75.0 5100
Processing waveform 148 of 158
75.0 2475
Processing waveform 149 of 158
75.0 6525
Processing waveform 150 of 158
75.0 7200
Processing waveform 151 of 158
75.0 4950
Processing waveform 152 of 158
75.0 3150
Processing waveform 153 of 158
75.0 2325
Processing waveform 154 of 158
75.0 4200
Processing waveform 155 of 158
75.0 13350
Processing waveform 156 of 158
75.0 9525
Processing waveform 157 of 158
75.0 2325
Training data have been read and features have been extracted  (158, 120)
Computation time:  215.86644291877747
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.13682150840759277
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    75       
        r          83 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  47.2995780591  +/-  4.75297305972  %
          Predicted class
              e     r 
        e    16    21 
        r    20    21 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't'] 2
class
h    97
l    81
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 269
75.0 3975
Processing waveform 1 of 269
75.0 2400
Processing waveform 2 of 269
75.0 9075
Processing waveform 3 of 269
75.0 2175
Processing waveform 4 of 269
75.0 3300
Processing waveform 5 of 269
75.0 4275
Processing waveform 6 of 269
75.0 2850
Processing waveform 7 of 269
75.0 2850
Processing waveform 8 of 269
75.0 2700
Processing waveform 9 of 269
75.0 5250
Processing waveform 10 of 269
75.0 3000
Processing waveform 11 of 269
75.0 2775
Processing waveform 12 of 269
75.0 6450
Processing waveform 13 of 269
75.0 3150
Processing waveform 14 of 269
75.0 1650
Processing waveform 15 of 269
75.0 4500
Processing waveform 16 of 269
75.0 2250
Processing waveform 17 of 269
75.0 2175
Processing waveform 18 of 269
75.0 3225
Processing waveform 19 of 269
75.0 4050
Processing waveform 20 of 269
75.0 3300
Processing waveform 21 of 269
75.0 2850
Processing waveform 22 of 269
75.0 2025
Processing waveform 23 of 269
75.0 4800
Processing waveform 24 of 269
75.0 2325
Processing waveform 25 of 269
75.0 6150
Processing waveform 26 of 269
75.0 2775
Processing waveform 27 of 269
75.0 2850
Processing waveform 28 of 269
75.0 4425
Processing waveform 29 of 269
75.0 2250
Processing waveform 30 of 269
75.0 3075
Processing waveform 31 of 269
75.0 2775
Processing waveform 32 of 269
75.0 1875
Processing waveform 33 of 269
75.0 3300
Processing waveform 34 of 269
75.0 1875
Processing waveform 35 of 269
75.0 5550
Processing waveform 36 of 269
75.0 9075
Processing waveform 37 of 269
75.0 4650
Processing waveform 38 of 269
75.0 6525
Processing waveform 39 of 269
75.0 9075
Processing waveform 40 of 269
75.0 3225
Processing waveform 41 of 269
75.0 3150
Processing waveform 42 of 269
75.0 4275
Processing waveform 43 of 269
75.0 4050
Processing waveform 44 of 269
75.0 5400
Processing waveform 45 of 269
75.0 4725
Processing waveform 46 of 269
75.0 3450
Processing waveform 47 of 269
75.0 6150
Processing waveform 48 of 269
75.0 2250
Processing waveform 49 of 269
75.0 9000
Processing waveform 50 of 269
75.0 3150
Processing waveform 51 of 269
75.0 4950
Processing waveform 52 of 269
75.0 6750
Processing waveform 53 of 269
75.0 3150
Processing waveform 54 of 269
75.0 7050
Processing waveform 55 of 269
75.0 3675
Processing waveform 56 of 269
75.0 3225
Processing waveform 57 of 269
75.0 2325
Processing waveform 58 of 269
75.0 7500
Processing waveform 59 of 269
75.0 3975
Processing waveform 60 of 269
75.0 6150
Processing waveform 61 of 269
75.0 7875
Processing waveform 62 of 269
75.0 7275
Processing waveform 63 of 269
75.0 6075
Processing waveform 64 of 269
75.0 3900
Processing waveform 65 of 269
75.0 3000
Processing waveform 66 of 269
75.0 3900
Processing waveform 67 of 269
75.0 6525
Processing waveform 68 of 269
75.0 2925
Processing waveform 69 of 269
75.0 2700
Processing waveform 70 of 269
75.0 2250
Processing waveform 71 of 269
75.0 4425
Processing waveform 72 of 269
75.0 2250

Processing waveform 73 of 269
75.0 9600
Processing waveform 74 of 269
75.0 4650
Processing waveform 75 of 269
75.0 6750
Processing waveform 76 of 269
75.0 7125
Processing waveform 77 of 269
75.0 3000
Processing waveform 78 of 269
75.0 1800
Processing waveform 79 of 269
75.0 4425
Processing waveform 80 of 269
75.0 9450
Processing waveform 81 of 269
75.0 3225
Processing waveform 82 of 269
75.0 2100
Processing waveform 83 of 269
75.0 2700
Processing waveform 84 of 269
75.0 2325
Processing waveform 85 of 269
75.0 2775
Processing waveform 86 of 269
75.0 5850
Processing waveform 87 of 269
75.0 3825
Processing waveform 88 of 269
75.0 6000
Processing waveform 89 of 269
75.0 2025
Processing waveform 90 of 269
75.0 1725
Processing waveform 91 of 269
75.0 2175
Processing waveform 92 of 269
75.0 5400
Processing waveform 93 of 269
75.0 3000
Processing waveform 94 of 269
75.0 5325
Processing waveform 95 of 269
75.0 3750
Processing waveform 96 of 269
75.0 3075
Processing waveform 97 of 269
75.0 5550
Processing waveform 98 of 269
75.0 7350
Processing waveform 99 of 269
75.0 4650
Processing waveform 100 of 269
75.0 2775
Processing waveform 101 of 269
75.0 2325
Processing waveform 102 of 269
75.0 2925
Processing waveform 103 of 269
75.0 3075
Processing waveform 104 of 269
75.0 6375
Processing waveform 105 of 269
75.0 2475
Processing waveform 106 of 269
75.0 3975
Processing waveform 107 of 269
75.0 3150
Processing waveform 108 of 269
75.0 5925
Processing waveform 109 of 269
75.0 5400
Processing waveform 110 of 269
75.0 1875
Processing waveform 111 of 269
75.0 5850
Processing waveform 112 of 269
75.0 1725
Processing waveform 113 of 269
75.0 7650
Processing waveform 114 of 269
75.0 2700
Processing waveform 115 of 269
75.0 9075
Processing waveform 116 of 269
75.0 6975
Processing waveform 117 of 269
75.0 4050
Processing waveform 118 of 269
75.0 3450
Processing waveform 119 of 269
75.0 5925
Processing waveform 120 of 269
75.0 2250
Processing waveform 121 of 269
75.0 3675
Processing waveform 122 of 269
75.0 2625
Processing waveform 123 of 269
75.0 4575
Processing waveform 124 of 269
75.0 2025
Processing waveform 125 of 269
75.0 5175
Processing waveform 126 of 269
75.0 3225
Processing waveform 127 of 269
75.0 4950
Processing waveform 128 of 269
75.0 2925
Processing waveform 129 of 269
75.0 1800
Processing waveform 130 of 269
75.0 8925
Processing waveform 131 of 269
75.0 7350
Processing waveform 132 of 269
75.0 4125
Processing waveform 133 of 269
75.0 3225
Processing waveform 134 of 269
75.0 9075
Processing waveform 135 of 269
75.0 5025
Processing waveform 136 of 269
75.0 8925
Processing waveform 137 of 269
75.0 4800
Processing waveform 138 of 269
75.0 7800
Processing waveform 139 of 269
75.0 2775
Processing waveform 140 of 269
75.0 6600
Processing waveform 141 of 269
75.0 1800
Processing waveform 142 of 269
75.0 2625
Processing waveform 143 of 269
75.0 9075
Processing waveform 144 of 269
75.0 7200
Processing waveform 145 of 269
75.0 5550
Processing waveform 146 of 269
75.0 2775
Processing waveform 147 of 269
75.0 5250
Processing waveform 148 of 269
75.0 1875
Processing waveform 149 of 269
75.0 1875
Processing waveform 150 of 269
75.0 5025
Processing waveform 151 of 269
75.0 3600
Processing waveform 152 of 269
75.0 3375
Processing waveform 153 of 269
75.0 4050
Processing waveform 154 of 269
75.0 3375
Processing waveform 155 of 269
75.0 2925
Processing waveform 156 of 269
75.0 6675
Processing waveform 157 of 269
75.0 3750
Processing waveform 158 of 269
75.0 4575
Processing waveform 159 of 269
75.0 9075
Processing waveform 160 of 269
75.0 3900
Processing waveform 161 of 269
75.0 9075
Processing waveform 162 of 269
75.0 4875
Processing waveform 163 of 269
75.0 3075
Processing waveform 164 of 269
75.0 3150
Processing waveform 165 of 269
75.0 9075
Processing waveform 166 of 269
75.0 7425
Processing waveform 167 of 269
75.0 5025
Processing waveform 168 of 269
75.0 2850
Processing waveform 169 of 269
75.0 1875
Processing waveform 170 of 269
75.0 7125
Processing waveform 171 of 269
75.0 4500
Processing waveform 172 of 269
75.0 6075
Processing waveform 173 of 269
75.0 2475
Processing waveform 174 of 269
75.0 4125
Processing waveform 175 of 269
75.0 2100
Processing waveform 176 of 269
75.0 3975
Processing waveform 177 of 269
75.0 3075
Processing waveform 178 of 269
75.0 5550
Processing waveform 179 of 269
75.0 2250
Processing waveform 180 of 269
75.0 3600
Processing waveform 181 of 269
75.0 5775
Processing waveform 182 of 269
75.0 2325
Processing waveform 183 of 269
75.0 2325
Processing waveform 184 of 269
75.0 3525
Processing waveform 185 of 269
75.0 4500
Processing waveform 186 of 269
75.0 4275
Processing waveform 187 of 269
75.0 3450
Processing waveform 188 of 269
75.0 2550
Processing waveform 189 of 269
75.0 3300
Processing waveform 190 of 269
75.0 9075
Processing waveform 191 of 269
75.0 9075
Processing waveform 192 of 269
75.0 6525
Processing waveform 193 of 269
75.0 6975
Processing waveform 194 of 269
75.0 6750
Processing waveform 195 of 269
75.0 7275
Processing waveform 196 of 269
75.0 2400
Processing waveform 197 of 269
75.0 4125
Processing waveform 198 of 269
75.0 3525
Processing waveform 199 of 269
75.0 5175
Processing waveform 200 of 269
75.0 3750
Processing waveform 201 of 269
75.0 4875
Processing waveform 202 of 269
75.0 2250
Processing waveform 203 of 269
75.0 4125
Processing waveform 204 of 269
75.0 4425
Processing waveform 205 of 269
75.0 5250
Processing waveform 206 of 269
75.0 9075
Processing waveform 207 of 269
75.0 4725
Processing waveform 208 of 269
75.0 2475
Processing waveform 209 of 269
75.0 2925
Processing waveform 210 of 269
75.0 7350
Processing waveform 211 of 269
75.0 2550
Processing waveform 212 of 269
75.0 2850
Processing waveform 213 of 269
75.0 2550
Processing waveform 214 of 269
75.0 5775
Processing waveform 215 of 269
75.0 4200
Processing waveform 216 of 269
75.0 2175
Processing waveform 217 of 269
75.0 2925
Processing waveform 218 of 269
75.0 10350
Processing waveform 219 of 269
75.0 15600
Processing waveform 220 of 269
75.0 6450
Processing waveform 221 of 269
75.0 3450
Processing waveform 222 of 269
75.0 11175
Processing waveform 223 of 269
75.0 4725
Processing waveform 224 of 269
75.0 4200
Processing waveform 225 of 269
75.0 3750
Processing waveform 226 of 269
75.0 3975
Processing waveform 227 of 269
75.0 4575
Processing waveform 228 of 269
75.0 3225
Processing waveform 229 of 269
75.0 13500
Processing waveform 230 of 269
75.0 4050
Processing waveform 231 of 269
75.0 5250
Processing waveform 232 of 269
75.0 7050
Processing waveform 233 of 269
75.0 3450
Processing waveform 234 of 269
75.0 6675
Processing waveform 235 of 269
75.0 6750
Processing waveform 236 of 269
75.0 2475
Processing waveform 237 of 269
75.0 7650
Processing waveform 238 of 269
75.0 6750
Processing waveform 239 of 269
75.0 4125
Processing waveform 240 of 269
75.0 7425
Processing waveform 241 of 269
75.0 5400
Processing waveform 242 of 269
75.0 4800
Processing waveform 243 of 269
75.0 10200
Processing waveform 244 of 269
75.0 18225
Processing waveform 245 of 269
75.0 5925
Processing waveform 246 of 269
75.0 3525
Processing waveform 247 of 269
75.0 3300
Processing waveform 248 of 269
75.0 3600
Processing waveform 249 of 269
75.0 4650
Processing waveform 250 of 269
75.0 7875
Processing waveform 251 of 269
75.0 6300
Processing waveform 252 of 269
75.0 6750
Processing waveform 253 of 269
75.0 2925
Processing waveform 254 of 269
75.0 3525
Processing waveform 255 of 269
75.0 7875
Processing waveform 256 of 269
75.0 8925
Processing waveform 257 of 269
75.0 2250
Processing waveform 258 of 269
75.0 2700
Processing waveform 259 of 269
75.0 3000
Processing waveform 260 of 269
75.0 3300
Processing waveform 261 of 269
75.0 4425
Processing waveform 262 of 269
75.0 4500
Processing waveform 263 of 269
75.0 7500
Processing waveform 264 of 269
75.0 4200
Processing waveform 265 of 269
75.0 4800
Processing waveform 266 of 269
75.0 1950
Processing waveform 267 of 269
75.0 4650
Processing waveform 268 of 269
75.0 5475
Training data have been read and features have been extracted  (269, 120)
Computation time:  351.7281610965729
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled

Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.27211952209472656
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h    97             
        l          81       
        t                91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  36.5925925926  +/-  3.59138327616  %
          Predicted class
              h     l     t 
        h    18    14    17 
        l    16    12    12 
        t    19     8    19 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't', 'r'] 2
class
h    97
l    81
r    83
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 352
75.0 3450
Processing waveform 1 of 352
75.0 4125
Processing waveform 2 of 352
75.0 2400
Processing waveform 3 of 352
75.0 2700
Processing waveform 4 of 352
75.0 2700
Processing waveform 5 of 352
75.0 4350
Processing waveform 6 of 352
75.0 2775
Processing waveform 7 of 352
75.0 2250
Processing waveform 8 of 352
75.0 2250
Processing waveform 9 of 352
75.0 2475
Processing waveform 10 of 352
75.0 3300
Processing waveform 11 of 352
75.0 5100
Processing waveform 12 of 352
75.0 9075
Processing waveform 13 of 352
75.0 3225
Processing waveform 14 of 352
75.0 4125
Processing waveform 15 of 352
75.0 3300
Processing waveform 16 of 352
75.0 3225
Processing waveform 17 of 352
75.0 2850
Processing waveform 18 of 352
75.0 2400
Processing waveform 19 of 352
75.0 3900
Processing waveform 20 of 352
75.0 2775
Processing waveform 21 of 352
75.0 2325
Processing waveform 22 of 352
75.0 4425
Processing waveform 23 of 352
75.0 3525
Processing waveform 24 of 352
75.0 6300
Processing waveform 25 of 352
75.0 2625
Processing waveform 26 of 352
75.0 4050
Processing waveform 27 of 352
75.0 2775
Processing waveform 28 of 352
75.0 2400
Processing waveform 29 of 352
75.0 2850
Processing waveform 30 of 352
75.0 2250
Processing waveform 31 of 352
75.0 2325
Processing waveform 32 of 352
75.0 3750
Processing waveform 33 of 352
75.0 3525
Processing waveform 34 of 352
75.0 5700
Processing waveform 35 of 352
75.0 2700
Processing waveform 36 of 352
75.0 3000
Processing waveform 37 of 352
75.0 4350
Processing waveform 38 of 352
75.0 4950
Processing waveform 39 of 352
75.0 2925
Processing waveform 40 of 352
75.0 2775
Processing waveform 41 of 352
75.0 3450
Processing waveform 42 of 352
75.0 4875
Processing waveform 43 of 352
75.0 9075
Processing waveform 44 of 352
75.0 5175
Processing waveform 45 of 352
75.0 3075
Processing waveform 46 of 352
75.0 4125
Processing waveform 47 of 352
75.0 9075
Processing waveform 48 of 352
75.0 2775
Processing waveform 49 of 352
75.0 4575
Processing waveform 50 of 352
75.0 2700
Processing waveform 51 of 352
75.0 1950
Processing waveform 52 of 352
75.0 2100
Processing waveform 53 of 352
75.0 4575
Processing waveform 54 of 352
75.0 5850
Processing waveform 55 of 352
75.0 3150
Processing waveform 56 of 352
75.0 4275
Processing waveform 57 of 352
75.0 9075
Processing waveform 58 of 352
75.0 3600
Processing waveform 59 of 352
75.0 2625
Processing waveform 60 of 352
75.0 2700
Processing waveform 61 of 352
75.0 2700
Processing waveform 62 of 352
75.0 3525
Processing waveform 63 of 352
75.0 4200
Processing waveform 64 of 352
75.0 4200
Processing waveform 65 of 352
75.0 1950
Processing waveform 66 of 352
75.0 5775
Processing waveform 67 of 352
75.0 9075
Processing waveform 68 of 352
75.0 4875
Processing waveform 69 of 352
75.0 14625
Processing waveform 70 of 352
75.0 11025
Processing waveform 71 of 352
75.0 1950
Processing waveform 72 of 352
75.0 3975
Processing waveform 73 of 352
75.0 2550
Processing waveform 74 of 352
75.0 7500
Processing waveform 75 of 352
75.0 14925
Processing waveform 76 of 352
75.0 13275
Processing waveform 77 of 352
75.0 1725
Processing waveform 78 of 352
75.0 2400
Processing waveform 79 of 352
75.0 6375
Processing waveform 80 of 352
75.0 6450
Processing waveform 81 of 352
75.0 16725
Processing waveform 82 of 352
75.0 6075
Processing waveform 83 of 352
75.0 3975
Processing waveform 84 of 352
75.0 2400
Processing waveform 85 of 352
75.0 9075
Processing waveform 86 of 352
75.0 2175
Processing waveform 87 of 352
75.0 3300
Processing waveform 88 of 352
75.0 4275
Processing waveform 89 of 352
75.0 2850
Processing waveform 90 of 352
75.0 2850
Processing waveform 91 of 352
75.0 2700
Processing waveform 92 of 352
75.0 5250
Processing waveform 93 of 352
75.0 3000
Processing waveform 94 of 352
75.0 2775
Processing waveform 95 of 352
75.0 6450
Processing waveform 96 of 352
75.0 3150
Processing waveform 97 of 352
75.0 1650
Processing waveform 98 of 352
75.0 4500
Processing waveform 99 of 352
75.0 2250
Processing waveform 100 of 352
75.0 2175
Processing waveform 101 of 352
75.0 3225
Processing waveform 102 of 352
75.0 4050
Processing waveform 103 of 352
75.0 3300
Processing waveform 104 of 352
75.0 2850
Processing waveform 105 of 352
75.0 2025
Processing waveform 106 of 352
75.0 4800
Processing waveform 107 of 352
75.0 2325
Processing waveform 108 of 352
75.0 6150
Processing waveform 109 of 352
75.0 2775
Processing waveform 110 of 352
75.0 2850
Processing waveform 111 of 352
75.0 4425
Processing waveform 112 of 352
75.0 2250
Processing waveform 113 of 352
75.0 3075
Processing waveform 114 of 352
75.0 2775
Processing waveform 115 of 352
75.0 1875
Processing waveform 116 of 352
75.0 3300
Processing waveform 117 of 352
75.0 1875
Processing waveform 118 of 352
75.0 5550
Processing waveform 119 of 352
75.0 9075
Processing waveform 120 of 352
75.0 4650
Processing waveform 121 of 352
75.0 6525
Processing waveform 122 of 352
75.0 9075
Processing waveform 123 of 352
75.0 3225
Processing waveform 124 of 352
75.0 3150
Processing waveform 125 of 352
75.0 4275
Processing waveform 126 of 352
75.0 4050
Processing waveform 127 of 352
75.0 5400
Processing waveform 128 of 352
75.0 4725
Processing waveform 129 of 352
75.0 3450
Processing waveform 130 of 352
75.0 6150
Processing waveform 131 of 352
75.0 2250
Processing waveform 132 of 352
75.0 9000
Processing waveform 133 of 352
75.0 3150
Processing waveform 134 of 352
75.0 4950
Processing waveform 135 of 352
75.0 6750
Processing waveform 136 of 352
75.0 3150
Processing waveform 137 of 352
75.0 7050
Processing waveform 138 of 352
75.0 3675
Processing waveform 139 of 352
75.0 3225
Processing waveform 140 of 352
75.0 2325
Processing waveform 141 of 352
75.0 7500
Processing waveform 142 of 352
75.0 3975
Processing waveform 143 of 352
75.0 6150
Processing waveform 144 of 352
75.0 7875
Processing waveform 145 of 352
75.0 7275
Processing waveform 146 of 352
75.0 6075
Processing waveform 147 of 352
75.0 3900
Processing waveform 148 of 352
75.0 3000
Processing waveform 149 of 352
75.0 3900
Processing waveform 150 of 352
75.0 6525
Processing waveform 151 of 352
75.0 2925
Processing waveform 152 of 352
75.0 2700
Processing waveform 153 of 352
75.0 2250
Processing waveform 154 of 352
75.0 4425
Processing waveform 155 of 352
75.0 2250
Processing waveform 156 of 352
75.0 9600
Processing waveform 157 of 352
75.0 4650
Processing waveform 158 of 352
75.0 6750
Processing waveform 159 of 352
75.0 7125
Processing waveform 160 of 352
75.0 3000
Processing waveform 161 of 352
75.0 1800
Processing waveform 162 of 352
75.0 4425
Processing waveform 163 of 352
75.0 9450
Processing waveform 164 of 352
75.0 3225
Processing waveform 165 of 352
75.0 2100
Processing waveform 166 of 352
75.0 2700
Processing waveform 167 of 352
75.0 2325
Processing waveform 168 of 352
75.0 2775
Processing waveform 169 of 352
75.0 5850
Processing waveform 170 of 352
75.0 3825
Processing waveform 171 of 352
75.0 6000
Processing waveform 172 of 352
75.0 2025
Processing waveform 173 of 352
75.0 1725
Processing waveform 174 of 352
75.0 2175
Processing waveform 175 of 352
75.0 5400
Processing waveform 176 of 352
75.0 3000
Processing waveform 177 of 352
75.0 5325
Processing waveform 178 of 352
75.0 3750
Processing waveform 179 of 352
75.0 3075
Processing waveform 180 of 352
75.0 5550
Processing waveform 181 of 352
75.0 7350

Processing waveform 182 of 352
75.0 4650
Processing waveform 183 of 352
75.0 2775
Processing waveform 184 of 352
75.0 2325
Processing waveform 185 of 352
75.0 2925
Processing waveform 186 of 352
75.0 3075
Processing waveform 187 of 352
75.0 6375
Processing waveform 188 of 352
75.0 2475
Processing waveform 189 of 352
75.0 3975
Processing waveform 190 of 352
75.0 3150
Processing waveform 191 of 352
75.0 5925
Processing waveform 192 of 352
75.0 5400
Processing waveform 193 of 352
75.0 1875
Processing waveform 194 of 352
75.0 5850
Processing waveform 195 of 352
75.0 1725
Processing waveform 196 of 352
75.0 7650
Processing waveform 197 of 352
75.0 2700
Processing waveform 198 of 352
75.0 9075
Processing waveform 199 of 352
75.0 6975
Processing waveform 200 of 352
75.0 4050
Processing waveform 201 of 352
75.0 3450
Processing waveform 202 of 352
75.0 5925
Processing waveform 203 of 352
75.0 2250
Processing waveform 204 of 352
75.0 3675
Processing waveform 205 of 352
75.0 2625
Processing waveform 206 of 352
75.0 4575
Processing waveform 207 of 352
75.0 2025
Processing waveform 208 of 352
75.0 5175
Processing waveform 209 of 352
75.0 3225
Processing waveform 210 of 352
75.0 4950
Processing waveform 211 of 352
75.0 2925
Processing waveform 212 of 352
75.0 1800
Processing waveform 213 of 352
75.0 8925
Processing waveform 214 of 352
75.0 7350
Processing waveform 215 of 352
75.0 4125
Processing waveform 216 of 352
75.0 3225
Processing waveform 217 of 352
75.0 9075
Processing waveform 218 of 352
75.0 5025
Processing waveform 219 of 352
75.0 8925
Processing waveform 220 of 352
75.0 4800
Processing waveform 221 of 352
75.0 7800
Processing waveform 222 of 352
75.0 2775
Processing waveform 223 of 352
75.0 6600
Processing waveform 224 of 352
75.0 1800
Processing waveform 225 of 352
75.0 2625
Processing waveform 226 of 352
75.0 9075
Processing waveform 227 of 352
75.0 7200
Processing waveform 228 of 352
75.0 5550
Processing waveform 229 of 352
75.0 2775
Processing waveform 230 of 352
75.0 5250
Processing waveform 231 of 352
75.0 1875
Processing waveform 232 of 352
75.0 1875
Processing waveform 233 of 352
75.0 5025
Processing waveform 234 of 352
75.0 3600
Processing waveform 235 of 352
75.0 3375
Processing waveform 236 of 352
75.0 4050
Processing waveform 237 of 352
75.0 3375
Processing waveform 238 of 352
75.0 2925
Processing waveform 239 of 352
75.0 6675
Processing waveform 240 of 352
75.0 3750
Processing waveform 241 of 352
75.0 4575
Processing waveform 242 of 352
75.0 9075
Processing waveform 243 of 352
75.0 3900
Processing waveform 244 of 352
75.0 9075
Processing waveform 245 of 352
75.0 4875
Processing waveform 246 of 352
75.0 3075
Processing waveform 247 of 352
75.0 3150
Processing waveform 248 of 352
75.0 9075
Processing waveform 249 of 352
75.0 7425
Processing waveform 250 of 352
75.0 5025
Processing waveform 251 of 352
75.0 2850
Processing waveform 252 of 352
75.0 1875
Processing waveform 253 of 352
75.0 7125
Processing waveform 254 of 352
75.0 4500
Processing waveform 255 of 352
75.0 6075
Processing waveform 256 of 352
75.0 2475
Processing waveform 257 of 352
75.0 4125
Processing waveform 258 of 352
75.0 2100
Processing waveform 259 of 352
75.0 3975
Processing waveform 260 of 352
75.0 3075
Processing waveform 261 of 352
75.0 5550
Processing waveform 262 of 352
75.0 2250
Processing waveform 263 of 352
75.0 3600
Processing waveform 264 of 352
75.0 5775
Processing waveform 265 of 352
75.0 2325
Processing waveform 266 of 352
75.0 2325
Processing waveform 267 of 352
75.0 3525
Processing waveform 268 of 352
75.0 4500
Processing waveform 269 of 352
75.0 4275
Processing waveform 270 of 352
75.0 3450
Processing waveform 271 of 352
75.0 2550
Processing waveform 272 of 352
75.0 3300
Processing waveform 273 of 352
75.0 9075
Processing waveform 274 of 352
75.0 9075
Processing waveform 275 of 352
75.0 6525
Processing waveform 276 of 352
75.0 6975
Processing waveform 277 of 352
75.0 6750
Processing waveform 278 of 352
75.0 7275
Processing waveform 279 of 352
75.0 2400
Processing waveform 280 of 352
75.0 4125
Processing waveform 281 of 352
75.0 3525
Processing waveform 282 of 352
75.0 5175
Processing waveform 283 of 352
75.0 3750
Processing waveform 284 of 352
75.0 4875
Processing waveform 285 of 352
75.0 2250
Processing waveform 286 of 352
75.0 4125
Processing waveform 287 of 352
75.0 4425
Processing waveform 288 of 352
75.0 5250
Processing waveform 289 of 352
75.0 9075
Processing waveform 290 of 352
75.0 4725
Processing waveform 291 of 352
75.0 2475
Processing waveform 292 of 352
75.0 2925
Processing waveform 293 of 352
75.0 7350
Processing waveform 294 of 352
75.0 2550
Processing waveform 295 of 352
75.0 2850
Processing waveform 296 of 352
75.0 2550
Processing waveform 297 of 352
75.0 5775
Processing waveform 298 of 352
75.0 4200
Processing waveform 299 of 352
75.0 2175
Processing waveform 300 of 352
75.0 2925
Processing waveform 301 of 352
75.0 10350
Processing waveform 302 of 352
75.0 15600
Processing waveform 303 of 352
75.0 6450
Processing waveform 304 of 352
75.0 3450
Processing waveform 305 of 352
75.0 11175
Processing waveform 306 of 352
75.0 4725
Processing waveform 307 of 352
75.0 4200
Processing waveform 308 of 352
75.0 3750
Processing waveform 309 of 352
75.0 3975
Processing waveform 310 of 352
75.0 4575
Processing waveform 311 of 352
75.0 3225
Processing waveform 312 of 352
75.0 13500
Processing waveform 313 of 352
75.0 4050
Processing waveform 314 of 352
75.0 5250
Processing waveform 315 of 352
75.0 7050
Processing waveform 316 of 352
75.0 3450
Processing waveform 317 of 352
75.0 6675
Processing waveform 318 of 352
75.0 6750
Processing waveform 319 of 352
75.0 2475
Processing waveform 320 of 352
75.0 7650
Processing waveform 321 of 352
75.0 6750
Processing waveform 322 of 352
75.0 4125
Processing waveform 323 of 352
75.0 7425
Processing waveform 324 of 352
75.0 5400
Processing waveform 325 of 352
75.0 4800
Processing waveform 326 of 352
75.0 10200
Processing waveform 327 of 352
75.0 18225
Processing waveform 328 of 352
75.0 5925
Processing waveform 329 of 352
75.0 3525
Processing waveform 330 of 352
75.0 3300
Processing waveform 331 of 352
75.0 3600
Processing waveform 332 of 352
75.0 4650
Processing waveform 333 of 352
75.0 7875
Processing waveform 334 of 352
75.0 6300
Processing waveform 335 of 352
75.0 6750
Processing waveform 336 of 352
75.0 2925
Processing waveform 337 of 352
75.0 3525
Processing waveform 338 of 352
75.0 7875
Processing waveform 339 of 352
75.0 8925
Processing waveform 340 of 352
75.0 2250
Processing waveform 341 of 352
75.0 2700
Processing waveform 342 of 352
75.0 3000
Processing waveform 343 of 352
75.0 3300
Processing waveform 344 of 352
75.0 4425
Processing waveform 345 of 352
75.0 4500
Processing waveform 346 of 352
75.0 7500
Processing waveform 347 of 352
75.0 4200
Processing waveform 348 of 352
75.0 4800
Processing waveform 349 of 352
75.0 1950
Processing waveform 350 of 352
75.0 4650
Processing waveform 351 of 352
75.0 5475
Training data have been read and features have been extracted  (352, 120)
Computation time:  468.320974111557
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.42241621017456055
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h    97                   
        l          81             
        r                83       
        t                      91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  28.1060606061  +/-  2.85124382827  %
          Predicted class
              h     l     r     t 
        h    14    11     9    15 
        l    15     9     8     9 
        r    15     8     8    11 
        t    15     6     7    18 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'h', 'l', 't', 'r'] 2
class
e    75
h    97
l    81
r    83
t    91
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 427
75.0 3450

Processing waveform 1 of 427
75.0 4125
Processing waveform 2 of 427
75.0 2400
Processing waveform 3 of 427
75.0 2700
Processing waveform 4 of 427
75.0 2700
Processing waveform 5 of 427
75.0 4350
Processing waveform 6 of 427
75.0 2775
Processing waveform 7 of 427
75.0 2250
Processing waveform 8 of 427
75.0 2250
Processing waveform 9 of 427
75.0 2475
Processing waveform 10 of 427
75.0 3300
Processing waveform 11 of 427
75.0 5100
Processing waveform 12 of 427
75.0 9075
Processing waveform 13 of 427
75.0 3225
Processing waveform 14 of 427
75.0 4125
Processing waveform 15 of 427
75.0 3300
Processing waveform 16 of 427
75.0 3225
Processing waveform 17 of 427
75.0 2850
Processing waveform 18 of 427
75.0 2400
Processing waveform 19 of 427
75.0 3900
Processing waveform 20 of 427
75.0 2775
Processing waveform 21 of 427
75.0 2325
Processing waveform 22 of 427
75.0 4425
Processing waveform 23 of 427
75.0 3525
Processing waveform 24 of 427
75.0 6300
Processing waveform 25 of 427
75.0 2625
Processing waveform 26 of 427
75.0 4050
Processing waveform 27 of 427
75.0 2775
Processing waveform 28 of 427
75.0 2400
Processing waveform 29 of 427
75.0 2850
Processing waveform 30 of 427
75.0 2250
Processing waveform 31 of 427
75.0 2325
Processing waveform 32 of 427
75.0 3750
Processing waveform 33 of 427
75.0 3525
Processing waveform 34 of 427
75.0 5700
Processing waveform 35 of 427
75.0 2700
Processing waveform 36 of 427
75.0 3000
Processing waveform 37 of 427
75.0 4350
Processing waveform 38 of 427
75.0 4950
Processing waveform 39 of 427
75.0 2925
Processing waveform 40 of 427
75.0 2775
Processing waveform 41 of 427
75.0 3450
Processing waveform 42 of 427
75.0 4875
Processing waveform 43 of 427
75.0 9075
Processing waveform 44 of 427
75.0 5175
Processing waveform 45 of 427
75.0 3075
Processing waveform 46 of 427
75.0 4125
Processing waveform 47 of 427
75.0 9075
Processing waveform 48 of 427
75.0 2775
Processing waveform 49 of 427
75.0 4575
Processing waveform 50 of 427
75.0 2700
Processing waveform 51 of 427
75.0 1950
Processing waveform 52 of 427
75.0 2100
Processing waveform 53 of 427
75.0 4575
Processing waveform 54 of 427
75.0 5850
Processing waveform 55 of 427
75.0 3150
Processing waveform 56 of 427
75.0 4275
Processing waveform 57 of 427
75.0 9075
Processing waveform 58 of 427
75.0 3600
Processing waveform 59 of 427
75.0 2625
Processing waveform 60 of 427
75.0 2700
Processing waveform 61 of 427
75.0 2700
Processing waveform 62 of 427
75.0 3525
Processing waveform 63 of 427
75.0 4200
Processing waveform 64 of 427
75.0 4200
Processing waveform 65 of 427
75.0 1950
Processing waveform 66 of 427
75.0 5775
Processing waveform 67 of 427
75.0 9075
Processing waveform 68 of 427
75.0 4875
Processing waveform 69 of 427
75.0 14625
Processing waveform 70 of 427
75.0 11025
Processing waveform 71 of 427
75.0 1950
Processing waveform 72 of 427
75.0 3975
Processing waveform 73 of 427
75.0 2550
Processing waveform 74 of 427
75.0 7500
Processing waveform 75 of 427
75.0 14925
Processing waveform 76 of 427
75.0 13275
Processing waveform 77 of 427
75.0 1725
Processing waveform 78 of 427
75.0 2400
Processing waveform 79 of 427
75.0 6375
Processing waveform 80 of 427
75.0 6450
Processing waveform 81 of 427
75.0 16725
Processing waveform 82 of 427
75.0 6075
Processing waveform 83 of 427
75.0 3450
Processing waveform 84 of 427
75.0 2325
Processing waveform 85 of 427
75.0 2925
Processing waveform 86 of 427
75.0 4575
Processing waveform 87 of 427
75.0 7050
Processing waveform 88 of 427
75.0 2025
Processing waveform 89 of 427
75.0 2925
Processing waveform 90 of 427
75.0 2850
Processing waveform 91 of 427
75.0 3600
Processing waveform 92 of 427
75.0 3750
Processing waveform 93 of 427
75.0 2100
Processing waveform 94 of 427
75.0 1650
Processing waveform 95 of 427
75.0 4200
Processing waveform 96 of 427
75.0 9075
Processing waveform 97 of 427
75.0 4125
Processing waveform 98 of 427
75.0 2925
Processing waveform 99 of 427
75.0 2250
Processing waveform 100 of 427
75.0 5475
Processing waveform 101 of 427
75.0 2625
Processing waveform 102 of 427
75.0 2400
Processing waveform 103 of 427
75.0 2700
Processing waveform 104 of 427
75.0 9075
Processing waveform 105 of 427
75.0 9075
Processing waveform 106 of 427
75.0 6225
Processing waveform 107 of 427
75.0 7125
Processing waveform 108 of 427
75.0 5850
Processing waveform 109 of 427
75.0 9075
Processing waveform 110 of 427
75.0 3225
Processing waveform 111 of 427
75.0 8100
Processing waveform 112 of 427
75.0 4575
Processing waveform 113 of 427
75.0 2850
Processing waveform 114 of 427
75.0 3375
Processing waveform 115 of 427
75.0 2100
Processing waveform 116 of 427
75.0 9075
Processing waveform 117 of 427
75.0 5850
Processing waveform 118 of 427
75.0 2775
Processing waveform 119 of 427
75.0 2700
Processing waveform 120 of 427
75.0 2100
Processing waveform 121 of 427
75.0 9075
Processing waveform 122 of 427
75.0 3225
Processing waveform 123 of 427
75.0 3300
Processing waveform 124 of 427
75.0 2475
Processing waveform 125 of 427
75.0 3075
Processing waveform 126 of 427
75.0 6450
Processing waveform 127 of 427
75.0 3600
Processing waveform 128 of 427
75.0 4650
Processing waveform 129 of 427
75.0 2475
Processing waveform 130 of 427
75.0 1875
Processing waveform 131 of 427
75.0 3450
Processing waveform 132 of 427
75.0 1500
Processing waveform 133 of 427
75.0 2625
Processing waveform 134 of 427
75.0 3525
Processing waveform 135 of 427
75.0 5625
Processing waveform 136 of 427
75.0 3675
Processing waveform 137 of 427
75.0 9075
Processing waveform 138 of 427
75.0 2700
Processing waveform 139 of 427
75.0 3000
Processing waveform 140 of 427
75.0 3525
Processing waveform 141 of 427
75.0 7200
Processing waveform 142 of 427
75.0 4875
Processing waveform 143 of 427
75.0 6075
Processing waveform 144 of 427
75.0 2700
Processing waveform 145 of 427
75.0 5250
Processing waveform 146 of 427
75.0 12750
Processing waveform 147 of 427
75.0 5100
Processing waveform 148 of 427
75.0 2475
Processing waveform 149 of 427
75.0 6525
Processing waveform 150 of 427
75.0 7200
Processing waveform 151 of 427
75.0 4950
Processing waveform 152 of 427
75.0 3150
Processing waveform 153 of 427
75.0 2325
Processing waveform 154 of 427
75.0 4200
Processing waveform 155 of 427
75.0 13350
Processing waveform 156 of 427
75.0 9525
Processing waveform 157 of 427
75.0 2325
Processing waveform 158 of 427
75.0 3975
Processing waveform 159 of 427
75.0 2400
Processing waveform 160 of 427
75.0 9075
Processing waveform 161 of 427
75.0 2175
Processing waveform 162 of 427
75.0 3300
Processing waveform 163 of 427
75.0 4275
Processing waveform 164 of 427
75.0 2850
Processing waveform 165 of 427
75.0 2850
Processing waveform 166 of 427
75.0 2700
Processing waveform 167 of 427
75.0 5250
Processing waveform 168 of 427
75.0 3000
Processing waveform 169 of 427
75.0 2775
Processing waveform 170 of 427
75.0 6450
Processing waveform 171 of 427
75.0 3150
Processing waveform 172 of 427
75.0 1650
Processing waveform 173 of 427
75.0 4500
Processing waveform 174 of 427
75.0 2250
Processing waveform 175 of 427
75.0 2175
Processing waveform 176 of 427
75.0 3225
Processing waveform 177 of 427
75.0 4050
Processing waveform 178 of 427
75.0 3300
Processing waveform 179 of 427
75.0 2850
Processing waveform 180 of 427
75.0 2025
Processing waveform 181 of 427
75.0 4800
Processing waveform 182 of 427
75.0 2325
Processing waveform 183 of 427
75.0 6150
Processing waveform 184 of 427
75.0 2775
Processing waveform 185 of 427
75.0 2850
Processing waveform 186 of 427
75.0 4425
Processing waveform 187 of 427
75.0 2250
Processing waveform 188 of 427
75.0 3075
Processing waveform 189 of 427
75.0 2775
Processing waveform 190 of 427
75.0 1875
Processing waveform 191 of 427
75.0 3300
Processing waveform 192 of 427
75.0 1875
Processing waveform 193 of 427
75.0 5550
Processing waveform 194 of 427
75.0 9075
Processing waveform 195 of 427
75.0 4650
Processing waveform 196 of 427
75.0 6525
Processing waveform 197 of 427
75.0 9075
Processing waveform 198 of 427
75.0 3225
Processing waveform 199 of 427
75.0 3150
Processing waveform 200 of 427
75.0 4275
Processing waveform 201 of 427
75.0 4050
Processing waveform 202 of 427
75.0 5400
Processing waveform 203 of 427
75.0 4725

Processing waveform 204 of 427
75.0 3450
Processing waveform 205 of 427
75.0 6150
Processing waveform 206 of 427
75.0 2250
Processing waveform 207 of 427
75.0 9000
Processing waveform 208 of 427
75.0 3150
Processing waveform 209 of 427
75.0 4950
Processing waveform 210 of 427
75.0 6750
Processing waveform 211 of 427
75.0 3150
Processing waveform 212 of 427
75.0 7050
Processing waveform 213 of 427
75.0 3675
Processing waveform 214 of 427
75.0 3225
Processing waveform 215 of 427
75.0 2325
Processing waveform 216 of 427
75.0 7500
Processing waveform 217 of 427
75.0 3975
Processing waveform 218 of 427
75.0 6150
Processing waveform 219 of 427
75.0 7875
Processing waveform 220 of 427
75.0 7275
Processing waveform 221 of 427
75.0 6075
Processing waveform 222 of 427
75.0 3900
Processing waveform 223 of 427
75.0 3000
Processing waveform 224 of 427
75.0 3900
Processing waveform 225 of 427
75.0 6525
Processing waveform 226 of 427
75.0 2925
Processing waveform 227 of 427
75.0 2700
Processing waveform 228 of 427
75.0 2250
Processing waveform 229 of 427
75.0 4425
Processing waveform 230 of 427
75.0 2250
Processing waveform 231 of 427
75.0 9600
Processing waveform 232 of 427
75.0 4650
Processing waveform 233 of 427
75.0 6750
Processing waveform 234 of 427
75.0 7125
Processing waveform 235 of 427
75.0 3000
Processing waveform 236 of 427
75.0 1800
Processing waveform 237 of 427
75.0 4425
Processing waveform 238 of 427
75.0 9450
Processing waveform 239 of 427
75.0 3225
Processing waveform 240 of 427
75.0 2100
Processing waveform 241 of 427
75.0 2700
Processing waveform 242 of 427
75.0 2325
Processing waveform 243 of 427
75.0 2775
Processing waveform 244 of 427
75.0 5850
Processing waveform 245 of 427
75.0 3825
Processing waveform 246 of 427
75.0 6000
Processing waveform 247 of 427
75.0 2025
Processing waveform 248 of 427
75.0 1725
Processing waveform 249 of 427
75.0 2175
Processing waveform 250 of 427
75.0 5400
Processing waveform 251 of 427
75.0 3000
Processing waveform 252 of 427
75.0 5325
Processing waveform 253 of 427
75.0 3750
Processing waveform 254 of 427
75.0 3075
Processing waveform 255 of 427
75.0 5550
Processing waveform 256 of 427
75.0 7350
Processing waveform 257 of 427
75.0 4650
Processing waveform 258 of 427
75.0 2775
Processing waveform 259 of 427
75.0 2325
Processing waveform 260 of 427
75.0 2925
Processing waveform 261 of 427
75.0 3075
Processing waveform 262 of 427
75.0 6375
Processing waveform 263 of 427
75.0 2475
Processing waveform 264 of 427
75.0 3975
Processing waveform 265 of 427
75.0 3150
Processing waveform 266 of 427
75.0 5925
Processing waveform 267 of 427
75.0 5400
Processing waveform 268 of 427
75.0 1875
Processing waveform 269 of 427
75.0 5850
Processing waveform 270 of 427
75.0 1725
Processing waveform 271 of 427
75.0 7650
Processing waveform 272 of 427
75.0 2700
Processing waveform 273 of 427
75.0 9075
Processing waveform 274 of 427
75.0 6975
Processing waveform 275 of 427
75.0 4050
Processing waveform 276 of 427
75.0 3450
Processing waveform 277 of 427
75.0 5925
Processing waveform 278 of 427
75.0 2250
Processing waveform 279 of 427
75.0 3675
Processing waveform 280 of 427
75.0 2625
Processing waveform 281 of 427
75.0 4575
Processing waveform 282 of 427
75.0 2025
Processing waveform 283 of 427
75.0 5175
Processing waveform 284 of 427
75.0 3225
Processing waveform 285 of 427
75.0 4950
Processing waveform 286 of 427
75.0 2925
Processing waveform 287 of 427
75.0 1800
Processing waveform 288 of 427
75.0 8925
Processing waveform 289 of 427
75.0 7350
Processing waveform 290 of 427
75.0 4125
Processing waveform 291 of 427
75.0 3225
Processing waveform 292 of 427
75.0 9075
Processing waveform 293 of 427
75.0 5025
Processing waveform 294 of 427
75.0 8925
Processing waveform 295 of 427
75.0 4800
Processing waveform 296 of 427
75.0 7800
Processing waveform 297 of 427
75.0 2775
Processing waveform 298 of 427
75.0 6600
Processing waveform 299 of 427
75.0 1800
Processing waveform 300 of 427
75.0 2625
Processing waveform 301 of 427
75.0 9075
Processing waveform 302 of 427
75.0 7200
Processing waveform 303 of 427
75.0 5550
Processing waveform 304 of 427
75.0 2775
Processing waveform 305 of 427
75.0 5250
Processing waveform 306 of 427
75.0 1875
Processing waveform 307 of 427
75.0 1875
Processing waveform 308 of 427
75.0 5025
Processing waveform 309 of 427
75.0 3600
Processing waveform 310 of 427
75.0 3375
Processing waveform 311 of 427
75.0 4050
Processing waveform 312 of 427
75.0 3375
Processing waveform 313 of 427
75.0 2925
Processing waveform 314 of 427
75.0 6675
Processing waveform 315 of 427
75.0 3750
Processing waveform 316 of 427
75.0 4575
Processing waveform 317 of 427
75.0 9075
Processing waveform 318 of 427
75.0 3900
Processing waveform 319 of 427
75.0 9075
Processing waveform 320 of 427
75.0 4875
Processing waveform 321 of 427
75.0 3075
Processing waveform 322 of 427
75.0 3150
Processing waveform 323 of 427
75.0 9075
Processing waveform 324 of 427
75.0 7425
Processing waveform 325 of 427
75.0 5025
Processing waveform 326 of 427
75.0 2850
Processing waveform 327 of 427
75.0 1875
Processing waveform 328 of 427
75.0 7125
Processing waveform 329 of 427
75.0 4500
Processing waveform 330 of 427
75.0 6075
Processing waveform 331 of 427
75.0 2475
Processing waveform 332 of 427
75.0 4125
Processing waveform 333 of 427
75.0 2100
Processing waveform 334 of 427
75.0 3975
Processing waveform 335 of 427
75.0 3075
Processing waveform 336 of 427
75.0 5550
Processing waveform 337 of 427
75.0 2250
Processing waveform 338 of 427
75.0 3600
Processing waveform 339 of 427
75.0 5775
Processing waveform 340 of 427
75.0 2325
Processing waveform 341 of 427
75.0 2325
Processing waveform 342 of 427
75.0 3525
Processing waveform 343 of 427
75.0 4500
Processing waveform 344 of 427
75.0 4275
Processing waveform 345 of 427
75.0 3450
Processing waveform 346 of 427
75.0 2550
Processing waveform 347 of 427
75.0 3300
Processing waveform 348 of 427
75.0 9075
Processing waveform 349 of 427
75.0 9075
Processing waveform 350 of 427
75.0 6525
Processing waveform 351 of 427
75.0 6975
Processing waveform 352 of 427
75.0 6750
Processing waveform 353 of 427
75.0 7275
Processing waveform 354 of 427
75.0 2400
Processing waveform 355 of 427
75.0 4125
Processing waveform 356 of 427
75.0 3525
Processing waveform 357 of 427
75.0 5175
Processing waveform 358 of 427
75.0 3750
Processing waveform 359 of 427
75.0 4875
Processing waveform 360 of 427
75.0 2250
Processing waveform 361 of 427
75.0 4125
Processing waveform 362 of 427
75.0 4425
Processing waveform 363 of 427
75.0 5250
Processing waveform 364 of 427
75.0 9075
Processing waveform 365 of 427
75.0 4725
Processing waveform 366 of 427
75.0 2475
Processing waveform 367 of 427
75.0 2925
Processing waveform 368 of 427
75.0 7350
Processing waveform 369 of 427
75.0 2550
Processing waveform 370 of 427
75.0 2850
Processing waveform 371 of 427
75.0 2550
Processing waveform 372 of 427
75.0 5775
Processing waveform 373 of 427
75.0 4200
Processing waveform 374 of 427
75.0 2175
Processing waveform 375 of 427
75.0 2925
Processing waveform 376 of 427
75.0 10350
Processing waveform 377 of 427
75.0 15600
Processing waveform 378 of 427
75.0 6450
Processing waveform 379 of 427
75.0 3450
Processing waveform 380 of 427
75.0 11175
Processing waveform 381 of 427
75.0 4725
Processing waveform 382 of 427
75.0 4200
Processing waveform 383 of 427
75.0 3750
Processing waveform 384 of 427
75.0 3975
Processing waveform 385 of 427
75.0 4575
Processing waveform 386 of 427
75.0 3225
Processing waveform 387 of 427
75.0 13500
Processing waveform 388 of 427
75.0 4050
Processing waveform 389 of 427
75.0 5250
Processing waveform 390 of 427
75.0 7050
Processing waveform 391 of 427
75.0 3450
Processing waveform 392 of 427
75.0 6675
Processing waveform 393 of 427
75.0 6750
Processing waveform 394 of 427
75.0 2475
Processing waveform 395 of 427
75.0 7650
Processing waveform 396 of 427
75.0 6750
Processing waveform 397 of 427
75.0 4125
Processing waveform 398 of 427
75.0 7425
Processing waveform 399 of 427
75.0 5400
Processing waveform 400 of 427
75.0 4800
Processing waveform 401 of 427
75.0 10200
Processing waveform 402 of 427
75.0 18225
Processing waveform 403 of 427
75.0 5925

Processing waveform 404 of 427
75.0 3525
Processing waveform 405 of 427
75.0 3300
Processing waveform 406 of 427
75.0 3600
Processing waveform 407 of 427
75.0 4650
Processing waveform 408 of 427
75.0 7875
Processing waveform 409 of 427
75.0 6300
Processing waveform 410 of 427
75.0 6750
Processing waveform 411 of 427
75.0 2925
Processing waveform 412 of 427
75.0 3525
Processing waveform 413 of 427
75.0 7875
Processing waveform 414 of 427
75.0 8925
Processing waveform 415 of 427
75.0 2250
Processing waveform 416 of 427
75.0 2700
Processing waveform 417 of 427
75.0 3000
Processing waveform 418 of 427
75.0 3300
Processing waveform 419 of 427
75.0 4425
Processing waveform 420 of 427
75.0 4500
Processing waveform 421 of 427
75.0 7500
Processing waveform 422 of 427
75.0 4200
Processing waveform 423 of 427
75.0 4800
Processing waveform 424 of 427
75.0 1950
Processing waveform 425 of 427
75.0 4650
Processing waveform 426 of 427
75.0 5475
Training data have been read and features have been extracted  (427, 120)
Computation time:  566.9208197593689
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.5916342735290527
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    75                         
        h          97                   
        l                81             
        r                      83       
        t                            91 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  21.292834891  +/-  2.71050416235  %
          Predicted class
              e     h     l     r     t 
        e     5    10     6     6    10 
        h     5    12    12     7    13 
        l     4    12     9     7     7 
        r     6    13     8     6     9 
        t     8    13     7     5    13 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['l', 't'] 3
class
l    74
t    89
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 163
75.0 2400
Processing waveform 1 of 163
75.0 9075
Processing waveform 2 of 163
75.0 2175
Processing waveform 3 of 163
75.0 3300
Processing waveform 4 of 163
75.0 2700
Processing waveform 5 of 163
75.0 5250
Processing waveform 6 of 163
75.0 3000
Processing waveform 7 of 163
75.0 2775
Processing waveform 8 of 163
75.0 6450
Processing waveform 9 of 163
75.0 3150
Processing waveform 10 of 163
75.0 1650
Processing waveform 11 of 163
75.0 4500
Processing waveform 12 of 163
75.0 2250
Processing waveform 13 of 163
75.0 2175
Processing waveform 14 of 163
75.0 3225
Processing waveform 15 of 163
75.0 4050
Processing waveform 16 of 163
75.0 3300
Processing waveform 17 of 163
75.0 2850
Processing waveform 18 of 163
75.0 2025
Processing waveform 19 of 163
75.0 4800
Processing waveform 20 of 163
75.0 2325
Processing waveform 21 of 163
75.0 6150
Processing waveform 22 of 163
75.0 2775
Processing waveform 23 of 163
75.0 2850
Processing waveform 24 of 163
75.0 4425
Processing waveform 25 of 163
75.0 2250
Processing waveform 26 of 163
75.0 3075
Processing waveform 27 of 163
75.0 2775
Processing waveform 28 of 163
75.0 1875
Processing waveform 29 of 163
75.0 3300
Processing waveform 30 of 163
75.0 1875
Processing waveform 31 of 163
75.0 5550
Processing waveform 32 of 163
75.0 9075
Processing waveform 33 of 163
75.0 6525
Processing waveform 34 of 163
75.0 9075
Processing waveform 35 of 163
75.0 3225
Processing waveform 36 of 163
75.0 3150
Processing waveform 37 of 163
75.0 4275
Processing waveform 38 of 163
75.0 4050
Processing waveform 39 of 163
75.0 5400
Processing waveform 40 of 163
75.0 4725
Processing waveform 41 of 163
75.0 3450
Processing waveform 42 of 163
75.0 6150
Processing waveform 43 of 163
75.0 2250
Processing waveform 44 of 163
75.0 9000
Processing waveform 45 of 163
75.0 3150
Processing waveform 46 of 163
75.0 4950
Processing waveform 47 of 163
75.0 6750
Processing waveform 48 of 163
75.0 3150
Processing waveform 49 of 163
75.0 7050
Processing waveform 50 of 163
75.0 3675
Processing waveform 51 of 163
75.0 3225
Processing waveform 52 of 163
75.0 2325
Processing waveform 53 of 163
75.0 3975
Processing waveform 54 of 163
75.0 6150
Processing waveform 55 of 163
75.0 7875
Processing waveform 56 of 163
75.0 7275
Processing waveform 57 of 163
75.0 6075
Processing waveform 58 of 163
75.0 3900
Processing waveform 59 of 163
75.0 3000
Processing waveform 60 of 163
75.0 3900
Processing waveform 61 of 163
75.0 6525
Processing waveform 62 of 163
75.0 2925
Processing waveform 63 of 163
75.0 2700
Processing waveform 64 of 163
75.0 2250
Processing waveform 65 of 163
75.0 2250
Processing waveform 66 of 163
75.0 9600
Processing waveform 67 of 163
75.0 4650
Processing waveform 68 of 163
75.0 6750
Processing waveform 69 of 163
75.0 7125
Processing waveform 70 of 163
75.0 3000
Processing waveform 71 of 163
75.0 1800
Processing waveform 72 of 163
75.0 4425
Processing waveform 73 of 163
75.0 9450
Processing waveform 74 of 163
75.0 5550
Processing waveform 75 of 163
75.0 2250
Processing waveform 76 of 163
75.0 3600
Processing waveform 77 of 163
75.0 5775
Processing waveform 78 of 163
75.0 2325
Processing waveform 79 of 163
75.0 2325
Processing waveform 80 of 163
75.0 3525
Processing waveform 81 of 163
75.0 4500
Processing waveform 82 of 163
75.0 4275
Processing waveform 83 of 163
75.0 3450
Processing waveform 84 of 163
75.0 2550
Processing waveform 85 of 163
75.0 3300
Processing waveform 86 of 163
75.0 9075
Processing waveform 87 of 163
75.0 9075
Processing waveform 88 of 163
75.0 6525
Processing waveform 89 of 163
75.0 6975
Processing waveform 90 of 163
75.0 6750
Processing waveform 91 of 163
75.0 7275
Processing waveform 92 of 163
75.0 2400
Processing waveform 93 of 163
75.0 4125
Processing waveform 94 of 163
75.0 3525
Processing waveform 95 of 163
75.0 5175
Processing waveform 96 of 163
75.0 3750
Processing waveform 97 of 163
75.0 4875
Processing waveform 98 of 163
75.0 2250
Processing waveform 99 of 163
75.0 4125
Processing waveform 100 of 163
75.0 4425
Processing waveform 101 of 163
75.0 5250
Processing waveform 102 of 163
75.0 9075
Processing waveform 103 of 163
75.0 4725
Processing waveform 104 of 163
75.0 2925
Processing waveform 105 of 163
75.0 7350
Processing waveform 106 of 163
75.0 2550
Processing waveform 107 of 163
75.0 2850
Processing waveform 108 of 163
75.0 2550
Processing waveform 109 of 163
75.0 5775
Processing waveform 110 of 163
75.0 4200
Processing waveform 111 of 163
75.0 2175
Processing waveform 112 of 163
75.0 10350
Processing waveform 113 of 163
75.0 15600
Processing waveform 114 of 163
75.0 6450
Processing waveform 115 of 163
75.0 3450
Processing waveform 116 of 163
75.0 11175
Processing waveform 117 of 163
75.0 4725
Processing waveform 118 of 163
75.0 4200
Processing waveform 119 of 163
75.0 3750
Processing waveform 120 of 163
75.0 3975
Processing waveform 121 of 163
75.0 4575
Processing waveform 122 of 163
75.0 3225
Processing waveform 123 of 163
75.0 13500
Processing waveform 124 of 163
75.0 4050
Processing waveform 125 of 163
75.0 5250
Processing waveform 126 of 163
75.0 7050
Processing waveform 127 of 163
75.0 3450
Processing waveform 128 of 163
75.0 6675
Processing waveform 129 of 163
75.0 6750
Processing waveform 130 of 163
75.0 2475
Processing waveform 131 of 163
75.0 7650
Processing waveform 132 of 163
75.0 6750
Processing waveform 133 of 163
75.0 4125
Processing waveform 134 of 163
75.0 7425
Processing waveform 135 of 163
75.0 5400
Processing waveform 136 of 163
75.0 4800
Processing waveform 137 of 163
75.0 10200
Processing waveform 138 of 163
75.0 18225
Processing waveform 139 of 163
75.0 5925
Processing waveform 140 of 163
75.0 3525
Processing waveform 141 of 163
75.0 3300
Processing waveform 142 of 163
75.0 3600
Processing waveform 143 of 163
75.0 4650
Processing waveform 144 of 163
75.0 7875
Processing waveform 145 of 163
75.0 6300
Processing waveform 146 of 163
75.0 6750
Processing waveform 147 of 163
75.0 2925

Processing waveform 148 of 163
75.0 3525
Processing waveform 149 of 163
75.0 7875
Processing waveform 150 of 163
75.0 8925
Processing waveform 151 of 163
75.0 2250
Processing waveform 152 of 163
75.0 2700
Processing waveform 153 of 163
75.0 3000
Processing waveform 154 of 163
75.0 3300
Processing waveform 155 of 163
75.0 4425
Processing waveform 156 of 163
75.0 4500
Processing waveform 157 of 163
75.0 7500
Processing waveform 158 of 163
75.0 4200
Processing waveform 159 of 163
75.0 4800
Processing waveform 160 of 163
75.0 1950
Processing waveform 161 of 163
75.0 4650
Processing waveform 162 of 163
75.0 5475
Training data have been read and features have been extracted  (163, 120)
Computation time:  232.30904126167297
Model will be trained on 2 classes [0 1] ['l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.13512921333312988
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              l     t 
        l    74       
        t          89 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  59.4308943089  +/-  4.01925433961  %
          Predicted class
              l     t 
        l    20    17 
        t    16    29 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'r'] 3
class
e    71
r    73
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 144
75.0 3450
Processing waveform 1 of 144
75.0 4125
Processing waveform 2 of 144
75.0 2400
Processing waveform 3 of 144
75.0 2700
Processing waveform 4 of 144
75.0 2700
Processing waveform 5 of 144
75.0 4350
Processing waveform 6 of 144
75.0 2775
Processing waveform 7 of 144
75.0 2250
Processing waveform 8 of 144
75.0 2475
Processing waveform 9 of 144
75.0 3300
Processing waveform 10 of 144
75.0 9075
Processing waveform 11 of 144
75.0 4125
Processing waveform 12 of 144
75.0 3300
Processing waveform 13 of 144
75.0 3225
Processing waveform 14 of 144
75.0 2850
Processing waveform 15 of 144
75.0 2400
Processing waveform 16 of 144
75.0 3900
Processing waveform 17 of 144
75.0 2775
Processing waveform 18 of 144
75.0 2325
Processing waveform 19 of 144
75.0 4425
Processing waveform 20 of 144
75.0 3525
Processing waveform 21 of 144
75.0 6300
Processing waveform 22 of 144
75.0 2625
Processing waveform 23 of 144
75.0 2400
Processing waveform 24 of 144
75.0 2250
Processing waveform 25 of 144
75.0 3750
Processing waveform 26 of 144
75.0 3525
Processing waveform 27 of 144
75.0 5700
Processing waveform 28 of 144
75.0 2700
Processing waveform 29 of 144
75.0 4350
Processing waveform 30 of 144
75.0 4950
Processing waveform 31 of 144
75.0 2925
Processing waveform 32 of 144
75.0 2775
Processing waveform 33 of 144
75.0 3450
Processing waveform 34 of 144
75.0 4875
Processing waveform 35 of 144
75.0 9075
Processing waveform 36 of 144
75.0 5175
Processing waveform 37 of 144
75.0 3075
Processing waveform 38 of 144
75.0 4125
Processing waveform 39 of 144
75.0 9075
Processing waveform 40 of 144
75.0 2775
Processing waveform 41 of 144
75.0 4575
Processing waveform 42 of 144
75.0 2700
Processing waveform 43 of 144
75.0 1950
Processing waveform 44 of 144
75.0 2100
Processing waveform 45 of 144
75.0 4575
Processing waveform 46 of 144
75.0 5850
Processing waveform 47 of 144
75.0 3150
Processing waveform 48 of 144
75.0 4275
Processing waveform 49 of 144
75.0 9075
Processing waveform 50 of 144
75.0 3600
Processing waveform 51 of 144
75.0 2625
Processing waveform 52 of 144
75.0 2700
Processing waveform 53 of 144
75.0 2700
Processing waveform 54 of 144
75.0 3525
Processing waveform 55 of 144
75.0 4200
Processing waveform 56 of 144
75.0 4200
Processing waveform 57 of 144
75.0 1950
Processing waveform 58 of 144
75.0 5775
Processing waveform 59 of 144
75.0 9075
Processing waveform 60 of 144
75.0 4875
Processing waveform 61 of 144
75.0 14625
Processing waveform 62 of 144
75.0 1950
Processing waveform 63 of 144
75.0 3975
Processing waveform 64 of 144
75.0 2550
Processing waveform 65 of 144
75.0 7500
Processing waveform 66 of 144
75.0 14925
Processing waveform 67 of 144
75.0 13275
Processing waveform 68 of 144
75.0 1725
Processing waveform 69 of 144
75.0 2400
Processing waveform 70 of 144
75.0 6375
Processing waveform 71 of 144
75.0 16725
Processing waveform 72 of 144
75.0 6075
Processing waveform 73 of 144
75.0 3450
Processing waveform 74 of 144
75.0 2925
Processing waveform 75 of 144
75.0 4575
Processing waveform 76 of 144
75.0 7050
Processing waveform 77 of 144
75.0 2025
Processing waveform 78 of 144
75.0 2925
Processing waveform 79 of 144
75.0 2850
Processing waveform 80 of 144
75.0 3600
Processing waveform 81 of 144
75.0 3750
Processing waveform 82 of 144
75.0 2100
Processing waveform 83 of 144
75.0 1650
Processing waveform 84 of 144
75.0 4200
Processing waveform 85 of 144
75.0 9075
Processing waveform 86 of 144
75.0 4125
Processing waveform 87 of 144
75.0 2925
Processing waveform 88 of 144
75.0 2250
Processing waveform 89 of 144
75.0 5475
Processing waveform 90 of 144
75.0 2625
Processing waveform 91 of 144
75.0 2400
Processing waveform 92 of 144
75.0 2700
Processing waveform 93 of 144
75.0 9075
Processing waveform 94 of 144
75.0 9075
Processing waveform 95 of 144
75.0 6225
Processing waveform 96 of 144
75.0 7125
Processing waveform 97 of 144
75.0 5850
Processing waveform 98 of 144
75.0 9075
Processing waveform 99 of 144
75.0 3225
Processing waveform 100 of 144
75.0 8100
Processing waveform 101 of 144
75.0 4575
Processing waveform 102 of 144
75.0 2850
Processing waveform 103 of 144
75.0 3375
Processing waveform 104 of 144
75.0 2100
Processing waveform 105 of 144
75.0 9075
Processing waveform 106 of 144
75.0 5850
Processing waveform 107 of 144
75.0 2775
Processing waveform 108 of 144
75.0 2700
Processing waveform 109 of 144
75.0 2100
Processing waveform 110 of 144
75.0 9075
Processing waveform 111 of 144
75.0 3225
Processing waveform 112 of 144
75.0 3300
Processing waveform 113 of 144
75.0 2475
Processing waveform 114 of 144
75.0 3075
Processing waveform 115 of 144
75.0 6450
Processing waveform 116 of 144
75.0 3600
Processing waveform 117 of 144
75.0 4650
Processing waveform 118 of 144
75.0 2475
Processing waveform 119 of 144
75.0 1875
Processing waveform 120 of 144
75.0 3450
Processing waveform 121 of 144
75.0 1500
Processing waveform 122 of 144
75.0 2625
Processing waveform 123 of 144
75.0 3525
Processing waveform 124 of 144
75.0 5625
Processing waveform 125 of 144
75.0 3675
Processing waveform 126 of 144
75.0 9075
Processing waveform 127 of 144
75.0 2700
Processing waveform 128 of 144
75.0 3000
Processing waveform 129 of 144
75.0 3525
Processing waveform 130 of 144
75.0 7200
Processing waveform 131 of 144
75.0 4875
Processing waveform 132 of 144
75.0 6075
Processing waveform 133 of 144
75.0 2700
Processing waveform 134 of 144
75.0 5250
Processing waveform 135 of 144
75.0 12750
Processing waveform 136 of 144
75.0 5100
Processing waveform 137 of 144
75.0 2475
Processing waveform 138 of 144
75.0 6525
Processing waveform 139 of 144
75.0 7200
Processing waveform 140 of 144
75.0 4950
Processing waveform 141 of 144
75.0 13350
Processing waveform 142 of 144
75.0 9525
Processing waveform 143 of 144
75.0 2325
Training data have been read and features have been extracted  (144, 120)
Computation time:  202.42961192131042
Model will be trained on 2 classes [0 1] ['e' 'r']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.1266334056854248
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     r 
        e    71       
        r          73 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  48.287037037  +/-  5.33935006551  %
          Predicted class
              e     r 
        e    18    18 
        r    19    17 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't'] 3
class
h    97
l    74
t    89
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 260
75.0 2400

Processing waveform 1 of 260
75.0 9075
Processing waveform 2 of 260
75.0 2175
Processing waveform 3 of 260
75.0 3300
Processing waveform 4 of 260
75.0 2700
Processing waveform 5 of 260
75.0 5250
Processing waveform 6 of 260
75.0 3000
Processing waveform 7 of 260
75.0 2775
Processing waveform 8 of 260
75.0 6450
Processing waveform 9 of 260
75.0 3150
Processing waveform 10 of 260
75.0 1650
Processing waveform 11 of 260
75.0 4500
Processing waveform 12 of 260
75.0 2250
Processing waveform 13 of 260
75.0 2175
Processing waveform 14 of 260
75.0 3225
Processing waveform 15 of 260
75.0 4050
Processing waveform 16 of 260
75.0 3300
Processing waveform 17 of 260
75.0 2850
Processing waveform 18 of 260
75.0 2025
Processing waveform 19 of 260
75.0 4800
Processing waveform 20 of 260
75.0 2325
Processing waveform 21 of 260
75.0 6150
Processing waveform 22 of 260
75.0 2775
Processing waveform 23 of 260
75.0 2850
Processing waveform 24 of 260
75.0 4425
Processing waveform 25 of 260
75.0 2250
Processing waveform 26 of 260
75.0 3075
Processing waveform 27 of 260
75.0 2775
Processing waveform 28 of 260
75.0 1875
Processing waveform 29 of 260
75.0 3300
Processing waveform 30 of 260
75.0 1875
Processing waveform 31 of 260
75.0 5550
Processing waveform 32 of 260
75.0 9075
Processing waveform 33 of 260
75.0 6525
Processing waveform 34 of 260
75.0 9075
Processing waveform 35 of 260
75.0 3225
Processing waveform 36 of 260
75.0 3150
Processing waveform 37 of 260
75.0 4275
Processing waveform 38 of 260
75.0 4050
Processing waveform 39 of 260
75.0 5400
Processing waveform 40 of 260
75.0 4725
Processing waveform 41 of 260
75.0 3450
Processing waveform 42 of 260
75.0 6150
Processing waveform 43 of 260
75.0 2250
Processing waveform 44 of 260
75.0 9000
Processing waveform 45 of 260
75.0 3150
Processing waveform 46 of 260
75.0 4950
Processing waveform 47 of 260
75.0 6750
Processing waveform 48 of 260
75.0 3150
Processing waveform 49 of 260
75.0 7050
Processing waveform 50 of 260
75.0 3675
Processing waveform 51 of 260
75.0 3225
Processing waveform 52 of 260
75.0 2325
Processing waveform 53 of 260
75.0 3975
Processing waveform 54 of 260
75.0 6150
Processing waveform 55 of 260
75.0 7875
Processing waveform 56 of 260
75.0 7275
Processing waveform 57 of 260
75.0 6075
Processing waveform 58 of 260
75.0 3900
Processing waveform 59 of 260
75.0 3000
Processing waveform 60 of 260
75.0 3900
Processing waveform 61 of 260
75.0 6525
Processing waveform 62 of 260
75.0 2925
Processing waveform 63 of 260
75.0 2700
Processing waveform 64 of 260
75.0 2250
Processing waveform 65 of 260
75.0 2250
Processing waveform 66 of 260
75.0 9600
Processing waveform 67 of 260
75.0 4650
Processing waveform 68 of 260
75.0 6750
Processing waveform 69 of 260
75.0 7125
Processing waveform 70 of 260
75.0 3000
Processing waveform 71 of 260
75.0 1800
Processing waveform 72 of 260
75.0 4425
Processing waveform 73 of 260
75.0 9450
Processing waveform 74 of 260
75.0 3225
Processing waveform 75 of 260
75.0 2100
Processing waveform 76 of 260
75.0 2700
Processing waveform 77 of 260
75.0 2325
Processing waveform 78 of 260
75.0 2775
Processing waveform 79 of 260
75.0 5850
Processing waveform 80 of 260
75.0 3825
Processing waveform 81 of 260
75.0 6000
Processing waveform 82 of 260
75.0 2025
Processing waveform 83 of 260
75.0 1725
Processing waveform 84 of 260
75.0 2175
Processing waveform 85 of 260
75.0 5400
Processing waveform 86 of 260
75.0 3000
Processing waveform 87 of 260
75.0 5325
Processing waveform 88 of 260
75.0 3750
Processing waveform 89 of 260
75.0 3075
Processing waveform 90 of 260
75.0 5550
Processing waveform 91 of 260
75.0 7350
Processing waveform 92 of 260
75.0 4650
Processing waveform 93 of 260
75.0 2775
Processing waveform 94 of 260
75.0 2325
Processing waveform 95 of 260
75.0 2925
Processing waveform 96 of 260
75.0 3075
Processing waveform 97 of 260
75.0 6375
Processing waveform 98 of 260
75.0 2475
Processing waveform 99 of 260
75.0 3975
Processing waveform 100 of 260
75.0 3150
Processing waveform 101 of 260
75.0 5925
Processing waveform 102 of 260
75.0 5400
Processing waveform 103 of 260
75.0 1875
Processing waveform 104 of 260
75.0 5850
Processing waveform 105 of 260
75.0 1725
Processing waveform 106 of 260
75.0 7650
Processing waveform 107 of 260
75.0 2700
Processing waveform 108 of 260
75.0 9075
Processing waveform 109 of 260
75.0 6975
Processing waveform 110 of 260
75.0 4050
Processing waveform 111 of 260
75.0 3450
Processing waveform 112 of 260
75.0 5925
Processing waveform 113 of 260
75.0 2250
Processing waveform 114 of 260
75.0 3675
Processing waveform 115 of 260
75.0 2625
Processing waveform 116 of 260
75.0 4575
Processing waveform 117 of 260
75.0 2025
Processing waveform 118 of 260
75.0 5175
Processing waveform 119 of 260
75.0 3225
Processing waveform 120 of 260
75.0 4950
Processing waveform 121 of 260
75.0 2925
Processing waveform 122 of 260
75.0 1800
Processing waveform 123 of 260
75.0 8925
Processing waveform 124 of 260
75.0 7350
Processing waveform 125 of 260
75.0 4125
Processing waveform 126 of 260
75.0 3225
Processing waveform 127 of 260
75.0 9075
Processing waveform 128 of 260
75.0 5025
Processing waveform 129 of 260
75.0 8925
Processing waveform 130 of 260
75.0 4800
Processing waveform 131 of 260
75.0 7800
Processing waveform 132 of 260
75.0 2775
Processing waveform 133 of 260
75.0 6600
Processing waveform 134 of 260
75.0 1800
Processing waveform 135 of 260
75.0 2625
Processing waveform 136 of 260
75.0 9075
Processing waveform 137 of 260
75.0 7200
Processing waveform 138 of 260
75.0 5550
Processing waveform 139 of 260
75.0 2775
Processing waveform 140 of 260
75.0 5250
Processing waveform 141 of 260
75.0 1875
Processing waveform 142 of 260
75.0 1875
Processing waveform 143 of 260
75.0 5025
Processing waveform 144 of 260
75.0 3600
Processing waveform 145 of 260
75.0 3375
Processing waveform 146 of 260
75.0 4050
Processing waveform 147 of 260
75.0 3375
Processing waveform 148 of 260
75.0 2925
Processing waveform 149 of 260
75.0 6675
Processing waveform 150 of 260
75.0 3750
Processing waveform 151 of 260
75.0 4575
Processing waveform 152 of 260
75.0 9075
Processing waveform 153 of 260
75.0 3900
Processing waveform 154 of 260
75.0 9075
Processing waveform 155 of 260
75.0 4875
Processing waveform 156 of 260
75.0 3075
Processing waveform 157 of 260
75.0 3150
Processing waveform 158 of 260
75.0 9075
Processing waveform 159 of 260
75.0 7425
Processing waveform 160 of 260
75.0 5025
Processing waveform 161 of 260
75.0 2850
Processing waveform 162 of 260
75.0 1875
Processing waveform 163 of 260
75.0 7125
Processing waveform 164 of 260
75.0 4500
Processing waveform 165 of 260
75.0 6075
Processing waveform 166 of 260
75.0 2475
Processing waveform 167 of 260
75.0 4125
Processing waveform 168 of 260
75.0 2100
Processing waveform 169 of 260
75.0 3975
Processing waveform 170 of 260
75.0 3075
Processing waveform 171 of 260
75.0 5550
Processing waveform 172 of 260
75.0 2250
Processing waveform 173 of 260
75.0 3600
Processing waveform 174 of 260
75.0 5775
Processing waveform 175 of 260
75.0 2325
Processing waveform 176 of 260
75.0 2325
Processing waveform 177 of 260
75.0 3525
Processing waveform 178 of 260
75.0 4500
Processing waveform 179 of 260
75.0 4275
Processing waveform 180 of 260
75.0 3450
Processing waveform 181 of 260
75.0 2550
Processing waveform 182 of 260
75.0 3300
Processing waveform 183 of 260
75.0 9075
Processing waveform 184 of 260
75.0 9075
Processing waveform 185 of 260
75.0 6525
Processing waveform 186 of 260
75.0 6975
Processing waveform 187 of 260
75.0 6750
Processing waveform 188 of 260
75.0 7275
Processing waveform 189 of 260
75.0 2400
Processing waveform 190 of 260
75.0 4125
Processing waveform 191 of 260
75.0 3525
Processing waveform 192 of 260
75.0 5175
Processing waveform 193 of 260
75.0 3750
Processing waveform 194 of 260
75.0 4875
Processing waveform 195 of 260
75.0 2250
Processing waveform 196 of 260
75.0 4125
Processing waveform 197 of 260
75.0 4425
Processing waveform 198 of 260
75.0 5250
Processing waveform 199 of 260
75.0 9075
Processing waveform 200 of 260
75.0 4725
Processing waveform 201 of 260
75.0 2925
Processing waveform 202 of 260
75.0 7350
Processing waveform 203 of 260
75.0 2550

Processing waveform 204 of 260
75.0 2850
Processing waveform 205 of 260
75.0 2550
Processing waveform 206 of 260
75.0 5775
Processing waveform 207 of 260
75.0 4200
Processing waveform 208 of 260
75.0 2175
Processing waveform 209 of 260
75.0 10350
Processing waveform 210 of 260
75.0 15600
Processing waveform 211 of 260
75.0 6450
Processing waveform 212 of 260
75.0 3450
Processing waveform 213 of 260
75.0 11175
Processing waveform 214 of 260
75.0 4725
Processing waveform 215 of 260
75.0 4200
Processing waveform 216 of 260
75.0 3750
Processing waveform 217 of 260
75.0 3975
Processing waveform 218 of 260
75.0 4575
Processing waveform 219 of 260
75.0 3225
Processing waveform 220 of 260
75.0 13500
Processing waveform 221 of 260
75.0 4050
Processing waveform 222 of 260
75.0 5250
Processing waveform 223 of 260
75.0 7050
Processing waveform 224 of 260
75.0 3450
Processing waveform 225 of 260
75.0 6675
Processing waveform 226 of 260
75.0 6750
Processing waveform 227 of 260
75.0 2475
Processing waveform 228 of 260
75.0 7650
Processing waveform 229 of 260
75.0 6750
Processing waveform 230 of 260
75.0 4125
Processing waveform 231 of 260
75.0 7425
Processing waveform 232 of 260
75.0 5400
Processing waveform 233 of 260
75.0 4800
Processing waveform 234 of 260
75.0 10200
Processing waveform 235 of 260
75.0 18225
Processing waveform 236 of 260
75.0 5925
Processing waveform 237 of 260
75.0 3525
Processing waveform 238 of 260
75.0 3300
Processing waveform 239 of 260
75.0 3600
Processing waveform 240 of 260
75.0 4650
Processing waveform 241 of 260
75.0 7875
Processing waveform 242 of 260
75.0 6300
Processing waveform 243 of 260
75.0 6750
Processing waveform 244 of 260
75.0 2925
Processing waveform 245 of 260
75.0 3525
Processing waveform 246 of 260
75.0 7875
Processing waveform 247 of 260
75.0 8925
Processing waveform 248 of 260
75.0 2250
Processing waveform 249 of 260
75.0 2700
Processing waveform 250 of 260
75.0 3000
Processing waveform 251 of 260
75.0 3300
Processing waveform 252 of 260
75.0 4425
Processing waveform 253 of 260
75.0 4500
Processing waveform 254 of 260
75.0 7500
Processing waveform 255 of 260
75.0 4200
Processing waveform 256 of 260
75.0 4800
Processing waveform 257 of 260
75.0 1950
Processing waveform 258 of 260
75.0 4650
Processing waveform 259 of 260
75.0 5475
Training data have been read and features have been extracted  (260, 120)
Computation time:  344.2882580757141
Model will be trained on 3 classes [0 1 2] ['h' 'l' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.26029276847839355
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     t 
        h    97             
        l          74       
        t                89 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  36.1538461538  +/-  3.33530513672  %
          Predicted class
              h     l     t 
        h    20    11    17 
        l    17     9    11 
        t    21     6    18 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['h', 'l', 't', 'r'] 3
class
h    97
l    74
r    73
t    89
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 333
75.0 3450
Processing waveform 1 of 333
75.0 4125
Processing waveform 2 of 333
75.0 2400
Processing waveform 3 of 333
75.0 2700
Processing waveform 4 of 333
75.0 2700
Processing waveform 5 of 333
75.0 4350
Processing waveform 6 of 333
75.0 2775
Processing waveform 7 of 333
75.0 2250
Processing waveform 8 of 333
75.0 2475
Processing waveform 9 of 333
75.0 3300
Processing waveform 10 of 333
75.0 9075
Processing waveform 11 of 333
75.0 4125
Processing waveform 12 of 333
75.0 3300
Processing waveform 13 of 333
75.0 3225
Processing waveform 14 of 333
75.0 2850
Processing waveform 15 of 333
75.0 2400
Processing waveform 16 of 333
75.0 3900
Processing waveform 17 of 333
75.0 2775
Processing waveform 18 of 333
75.0 2325
Processing waveform 19 of 333
75.0 4425
Processing waveform 20 of 333
75.0 3525
Processing waveform 21 of 333
75.0 6300
Processing waveform 22 of 333
75.0 2625
Processing waveform 23 of 333
75.0 2400
Processing waveform 24 of 333
75.0 2250
Processing waveform 25 of 333
75.0 3750
Processing waveform 26 of 333
75.0 3525
Processing waveform 27 of 333
75.0 5700
Processing waveform 28 of 333
75.0 2700
Processing waveform 29 of 333
75.0 4350
Processing waveform 30 of 333
75.0 4950
Processing waveform 31 of 333
75.0 2925
Processing waveform 32 of 333
75.0 2775
Processing waveform 33 of 333
75.0 3450
Processing waveform 34 of 333
75.0 4875
Processing waveform 35 of 333
75.0 9075
Processing waveform 36 of 333
75.0 5175
Processing waveform 37 of 333
75.0 3075
Processing waveform 38 of 333
75.0 4125
Processing waveform 39 of 333
75.0 9075
Processing waveform 40 of 333
75.0 2775
Processing waveform 41 of 333
75.0 4575
Processing waveform 42 of 333
75.0 2700
Processing waveform 43 of 333
75.0 1950
Processing waveform 44 of 333
75.0 2100
Processing waveform 45 of 333
75.0 4575
Processing waveform 46 of 333
75.0 5850
Processing waveform 47 of 333
75.0 3150
Processing waveform 48 of 333
75.0 4275
Processing waveform 49 of 333
75.0 9075
Processing waveform 50 of 333
75.0 3600
Processing waveform 51 of 333
75.0 2625
Processing waveform 52 of 333
75.0 2700
Processing waveform 53 of 333
75.0 2700
Processing waveform 54 of 333
75.0 3525
Processing waveform 55 of 333
75.0 4200
Processing waveform 56 of 333
75.0 4200
Processing waveform 57 of 333
75.0 1950
Processing waveform 58 of 333
75.0 5775
Processing waveform 59 of 333
75.0 9075
Processing waveform 60 of 333
75.0 4875
Processing waveform 61 of 333
75.0 14625
Processing waveform 62 of 333
75.0 1950
Processing waveform 63 of 333
75.0 3975
Processing waveform 64 of 333
75.0 2550
Processing waveform 65 of 333
75.0 7500
Processing waveform 66 of 333
75.0 14925
Processing waveform 67 of 333
75.0 13275
Processing waveform 68 of 333
75.0 1725
Processing waveform 69 of 333
75.0 2400
Processing waveform 70 of 333
75.0 6375
Processing waveform 71 of 333
75.0 16725
Processing waveform 72 of 333
75.0 6075
Processing waveform 73 of 333
75.0 2400
Processing waveform 74 of 333
75.0 9075
Processing waveform 75 of 333
75.0 2175
Processing waveform 76 of 333
75.0 3300
Processing waveform 77 of 333
75.0 2700
Processing waveform 78 of 333
75.0 5250
Processing waveform 79 of 333
75.0 3000
Processing waveform 80 of 333
75.0 2775
Processing waveform 81 of 333
75.0 6450
Processing waveform 82 of 333
75.0 3150
Processing waveform 83 of 333
75.0 1650
Processing waveform 84 of 333
75.0 4500
Processing waveform 85 of 333
75.0 2250
Processing waveform 86 of 333
75.0 2175
Processing waveform 87 of 333
75.0 3225
Processing waveform 88 of 333
75.0 4050
Processing waveform 89 of 333
75.0 3300
Processing waveform 90 of 333
75.0 2850
Processing waveform 91 of 333
75.0 2025
Processing waveform 92 of 333
75.0 4800
Processing waveform 93 of 333
75.0 2325
Processing waveform 94 of 333
75.0 6150
Processing waveform 95 of 333
75.0 2775
Processing waveform 96 of 333
75.0 2850
Processing waveform 97 of 333
75.0 4425
Processing waveform 98 of 333
75.0 2250
Processing waveform 99 of 333
75.0 3075
Processing waveform 100 of 333
75.0 2775
Processing waveform 101 of 333
75.0 1875
Processing waveform 102 of 333
75.0 3300
Processing waveform 103 of 333
75.0 1875
Processing waveform 104 of 333
75.0 5550
Processing waveform 105 of 333
75.0 9075
Processing waveform 106 of 333
75.0 6525
Processing waveform 107 of 333
75.0 9075
Processing waveform 108 of 333
75.0 3225
Processing waveform 109 of 333
75.0 3150
Processing waveform 110 of 333
75.0 4275
Processing waveform 111 of 333
75.0 4050
Processing waveform 112 of 333
75.0 5400
Processing waveform 113 of 333
75.0 4725
Processing waveform 114 of 333
75.0 3450
Processing waveform 115 of 333
75.0 6150
Processing waveform 116 of 333
75.0 2250
Processing waveform 117 of 333
75.0 9000
Processing waveform 118 of 333
75.0 3150
Processing waveform 119 of 333
75.0 4950
Processing waveform 120 of 333
75.0 6750

Processing waveform 121 of 333
75.0 3150
Processing waveform 122 of 333
75.0 7050
Processing waveform 123 of 333
75.0 3675
Processing waveform 124 of 333
75.0 3225
Processing waveform 125 of 333
75.0 2325
Processing waveform 126 of 333
75.0 3975
Processing waveform 127 of 333
75.0 6150
Processing waveform 128 of 333
75.0 7875
Processing waveform 129 of 333
75.0 7275
Processing waveform 130 of 333
75.0 6075
Processing waveform 131 of 333
75.0 3900
Processing waveform 132 of 333
75.0 3000
Processing waveform 133 of 333
75.0 3900
Processing waveform 134 of 333
75.0 6525
Processing waveform 135 of 333
75.0 2925
Processing waveform 136 of 333
75.0 2700
Processing waveform 137 of 333
75.0 2250
Processing waveform 138 of 333
75.0 2250
Processing waveform 139 of 333
75.0 9600
Processing waveform 140 of 333
75.0 4650
Processing waveform 141 of 333
75.0 6750
Processing waveform 142 of 333
75.0 7125
Processing waveform 143 of 333
75.0 3000
Processing waveform 144 of 333
75.0 1800
Processing waveform 145 of 333
75.0 4425
Processing waveform 146 of 333
75.0 9450
Processing waveform 147 of 333
75.0 3225
Processing waveform 148 of 333
75.0 2100
Processing waveform 149 of 333
75.0 2700
Processing waveform 150 of 333
75.0 2325
Processing waveform 151 of 333
75.0 2775
Processing waveform 152 of 333
75.0 5850
Processing waveform 153 of 333
75.0 3825
Processing waveform 154 of 333
75.0 6000
Processing waveform 155 of 333
75.0 2025
Processing waveform 156 of 333
75.0 1725
Processing waveform 157 of 333
75.0 2175
Processing waveform 158 of 333
75.0 5400
Processing waveform 159 of 333
75.0 3000
Processing waveform 160 of 333
75.0 5325
Processing waveform 161 of 333
75.0 3750
Processing waveform 162 of 333
75.0 3075
Processing waveform 163 of 333
75.0 5550
Processing waveform 164 of 333
75.0 7350
Processing waveform 165 of 333
75.0 4650
Processing waveform 166 of 333
75.0 2775
Processing waveform 167 of 333
75.0 2325
Processing waveform 168 of 333
75.0 2925
Processing waveform 169 of 333
75.0 3075
Processing waveform 170 of 333
75.0 6375
Processing waveform 171 of 333
75.0 2475
Processing waveform 172 of 333
75.0 3975
Processing waveform 173 of 333
75.0 3150
Processing waveform 174 of 333
75.0 5925
Processing waveform 175 of 333
75.0 5400
Processing waveform 176 of 333
75.0 1875
Processing waveform 177 of 333
75.0 5850
Processing waveform 178 of 333
75.0 1725
Processing waveform 179 of 333
75.0 7650
Processing waveform 180 of 333
75.0 2700
Processing waveform 181 of 333
75.0 9075
Processing waveform 182 of 333
75.0 6975
Processing waveform 183 of 333
75.0 4050
Processing waveform 184 of 333
75.0 3450
Processing waveform 185 of 333
75.0 5925
Processing waveform 186 of 333
75.0 2250
Processing waveform 187 of 333
75.0 3675
Processing waveform 188 of 333
75.0 2625
Processing waveform 189 of 333
75.0 4575
Processing waveform 190 of 333
75.0 2025
Processing waveform 191 of 333
75.0 5175
Processing waveform 192 of 333
75.0 3225
Processing waveform 193 of 333
75.0 4950
Processing waveform 194 of 333
75.0 2925
Processing waveform 195 of 333
75.0 1800
Processing waveform 196 of 333
75.0 8925
Processing waveform 197 of 333
75.0 7350
Processing waveform 198 of 333
75.0 4125
Processing waveform 199 of 333
75.0 3225
Processing waveform 200 of 333
75.0 9075
Processing waveform 201 of 333
75.0 5025
Processing waveform 202 of 333
75.0 8925
Processing waveform 203 of 333
75.0 4800
Processing waveform 204 of 333
75.0 7800
Processing waveform 205 of 333
75.0 2775
Processing waveform 206 of 333
75.0 6600
Processing waveform 207 of 333
75.0 1800
Processing waveform 208 of 333
75.0 2625
Processing waveform 209 of 333
75.0 9075
Processing waveform 210 of 333
75.0 7200
Processing waveform 211 of 333
75.0 5550
Processing waveform 212 of 333
75.0 2775
Processing waveform 213 of 333
75.0 5250
Processing waveform 214 of 333
75.0 1875
Processing waveform 215 of 333
75.0 1875
Processing waveform 216 of 333
75.0 5025
Processing waveform 217 of 333
75.0 3600
Processing waveform 218 of 333
75.0 3375
Processing waveform 219 of 333
75.0 4050
Processing waveform 220 of 333
75.0 3375
Processing waveform 221 of 333
75.0 2925
Processing waveform 222 of 333
75.0 6675
Processing waveform 223 of 333
75.0 3750
Processing waveform 224 of 333
75.0 4575
Processing waveform 225 of 333
75.0 9075
Processing waveform 226 of 333
75.0 3900
Processing waveform 227 of 333
75.0 9075
Processing waveform 228 of 333
75.0 4875
Processing waveform 229 of 333
75.0 3075
Processing waveform 230 of 333
75.0 3150
Processing waveform 231 of 333
75.0 9075
Processing waveform 232 of 333
75.0 7425
Processing waveform 233 of 333
75.0 5025
Processing waveform 234 of 333
75.0 2850
Processing waveform 235 of 333
75.0 1875
Processing waveform 236 of 333
75.0 7125
Processing waveform 237 of 333
75.0 4500
Processing waveform 238 of 333
75.0 6075
Processing waveform 239 of 333
75.0 2475
Processing waveform 240 of 333
75.0 4125
Processing waveform 241 of 333
75.0 2100
Processing waveform 242 of 333
75.0 3975
Processing waveform 243 of 333
75.0 3075
Processing waveform 244 of 333
75.0 5550
Processing waveform 245 of 333
75.0 2250
Processing waveform 246 of 333
75.0 3600
Processing waveform 247 of 333
75.0 5775
Processing waveform 248 of 333
75.0 2325
Processing waveform 249 of 333
75.0 2325
Processing waveform 250 of 333
75.0 3525
Processing waveform 251 of 333
75.0 4500
Processing waveform 252 of 333
75.0 4275
Processing waveform 253 of 333
75.0 3450
Processing waveform 254 of 333
75.0 2550
Processing waveform 255 of 333
75.0 3300
Processing waveform 256 of 333
75.0 9075
Processing waveform 257 of 333
75.0 9075
Processing waveform 258 of 333
75.0 6525
Processing waveform 259 of 333
75.0 6975
Processing waveform 260 of 333
75.0 6750
Processing waveform 261 of 333
75.0 7275
Processing waveform 262 of 333
75.0 2400
Processing waveform 263 of 333
75.0 4125
Processing waveform 264 of 333
75.0 3525
Processing waveform 265 of 333
75.0 5175
Processing waveform 266 of 333
75.0 3750
Processing waveform 267 of 333
75.0 4875
Processing waveform 268 of 333
75.0 2250
Processing waveform 269 of 333
75.0 4125
Processing waveform 270 of 333
75.0 4425
Processing waveform 271 of 333
75.0 5250
Processing waveform 272 of 333
75.0 9075
Processing waveform 273 of 333
75.0 4725
Processing waveform 274 of 333
75.0 2925
Processing waveform 275 of 333
75.0 7350
Processing waveform 276 of 333
75.0 2550
Processing waveform 277 of 333
75.0 2850
Processing waveform 278 of 333
75.0 2550
Processing waveform 279 of 333
75.0 5775
Processing waveform 280 of 333
75.0 4200
Processing waveform 281 of 333
75.0 2175
Processing waveform 282 of 333
75.0 10350
Processing waveform 283 of 333
75.0 15600
Processing waveform 284 of 333
75.0 6450
Processing waveform 285 of 333
75.0 3450
Processing waveform 286 of 333
75.0 11175
Processing waveform 287 of 333
75.0 4725
Processing waveform 288 of 333
75.0 4200
Processing waveform 289 of 333
75.0 3750
Processing waveform 290 of 333
75.0 3975
Processing waveform 291 of 333
75.0 4575
Processing waveform 292 of 333
75.0 3225
Processing waveform 293 of 333
75.0 13500
Processing waveform 294 of 333
75.0 4050
Processing waveform 295 of 333
75.0 5250
Processing waveform 296 of 333
75.0 7050
Processing waveform 297 of 333
75.0 3450
Processing waveform 298 of 333
75.0 6675
Processing waveform 299 of 333
75.0 6750
Processing waveform 300 of 333
75.0 2475
Processing waveform 301 of 333
75.0 7650
Processing waveform 302 of 333
75.0 6750
Processing waveform 303 of 333
75.0 4125
Processing waveform 304 of 333
75.0 7425
Processing waveform 305 of 333
75.0 5400
Processing waveform 306 of 333
75.0 4800
Processing waveform 307 of 333
75.0 10200
Processing waveform 308 of 333
75.0 18225
Processing waveform 309 of 333
75.0 5925
Processing waveform 310 of 333
75.0 3525
Processing waveform 311 of 333
75.0 3300
Processing waveform 312 of 333
75.0 3600
Processing waveform 313 of 333
75.0 4650
Processing waveform 314 of 333
75.0 7875
Processing waveform 315 of 333
75.0 6300
Processing waveform 316 of 333
75.0 6750
Processing waveform 317 of 333
75.0 2925
Processing waveform 318 of 333
75.0 3525
Processing waveform 319 of 333
75.0 7875
Processing waveform 320 of 333
75.0 8925

Processing waveform 321 of 333
75.0 2250
Processing waveform 322 of 333
75.0 2700
Processing waveform 323 of 333
75.0 3000
Processing waveform 324 of 333
75.0 3300
Processing waveform 325 of 333
75.0 4425
Processing waveform 326 of 333
75.0 4500
Processing waveform 327 of 333
75.0 7500
Processing waveform 328 of 333
75.0 4200
Processing waveform 329 of 333
75.0 4800
Processing waveform 330 of 333
75.0 1950
Processing waveform 331 of 333
75.0 4650
Processing waveform 332 of 333
75.0 5475
Training data have been read and features have been extracted  (333, 120)
Computation time:  446.7432026863098
Model will be trained on 4 classes [0 1 2 3] ['h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.39943456649780273
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              h     l     r     t 
        h    97                   
        l          74             
        r                73       
        t                      89 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  27.2255489022  +/-  3.31505620631  %
          Predicted class
              h     l     r     t 
        h    17     9     8    15 
        l    15     7     6     9 
        r    15     6     6    10 
        t    17     6     6    15 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBLG..SHZ ['e', 'h', 'l', 't', 'r'] 3
class
e    71
h    97
l    74
r    73
t    89
dtype: int64
./AAA-master/MONTSERRAT/catalog/30_MVO_labelled_events_filtered.pd


 *** ANALYZER ***
Processing waveform 0 of 404
75.0 3450
Processing waveform 1 of 404
75.0 4125
Processing waveform 2 of 404
75.0 2400
Processing waveform 3 of 404
75.0 2700
Processing waveform 4 of 404
75.0 2700
Processing waveform 5 of 404
75.0 4350
Processing waveform 6 of 404
75.0 2775
Processing waveform 7 of 404
75.0 2250
Processing waveform 8 of 404
75.0 2475
Processing waveform 9 of 404
75.0 3300
Processing waveform 10 of 404
75.0 9075
Processing waveform 11 of 404
75.0 4125
Processing waveform 12 of 404
75.0 3300
Processing waveform 13 of 404
75.0 3225
Processing waveform 14 of 404
75.0 2850
Processing waveform 15 of 404
75.0 2400
Processing waveform 16 of 404
75.0 3900
Processing waveform 17 of 404
75.0 2775
Processing waveform 18 of 404
75.0 2325
Processing waveform 19 of 404
75.0 4425
Processing waveform 20 of 404
75.0 3525
Processing waveform 21 of 404
75.0 6300
Processing waveform 22 of 404
75.0 2625
Processing waveform 23 of 404
75.0 2400
Processing waveform 24 of 404
75.0 2250
Processing waveform 25 of 404
75.0 3750
Processing waveform 26 of 404
75.0 3525
Processing waveform 27 of 404
75.0 5700
Processing waveform 28 of 404
75.0 2700
Processing waveform 29 of 404
75.0 4350
Processing waveform 30 of 404
75.0 4950
Processing waveform 31 of 404
75.0 2925
Processing waveform 32 of 404
75.0 2775
Processing waveform 33 of 404
75.0 3450
Processing waveform 34 of 404
75.0 4875
Processing waveform 35 of 404
75.0 9075
Processing waveform 36 of 404
75.0 5175
Processing waveform 37 of 404
75.0 3075
Processing waveform 38 of 404
75.0 4125
Processing waveform 39 of 404
75.0 9075
Processing waveform 40 of 404
75.0 2775
Processing waveform 41 of 404
75.0 4575
Processing waveform 42 of 404
75.0 2700
Processing waveform 43 of 404
75.0 1950
Processing waveform 44 of 404
75.0 2100
Processing waveform 45 of 404
75.0 4575
Processing waveform 46 of 404
75.0 5850
Processing waveform 47 of 404
75.0 3150
Processing waveform 48 of 404
75.0 4275
Processing waveform 49 of 404
75.0 9075
Processing waveform 50 of 404
75.0 3600
Processing waveform 51 of 404
75.0 2625
Processing waveform 52 of 404
75.0 2700
Processing waveform 53 of 404
75.0 2700
Processing waveform 54 of 404
75.0 3525
Processing waveform 55 of 404
75.0 4200
Processing waveform 56 of 404
75.0 4200
Processing waveform 57 of 404
75.0 1950
Processing waveform 58 of 404
75.0 5775
Processing waveform 59 of 404
75.0 9075
Processing waveform 60 of 404
75.0 4875
Processing waveform 61 of 404
75.0 14625
Processing waveform 62 of 404
75.0 1950
Processing waveform 63 of 404
75.0 3975
Processing waveform 64 of 404
75.0 2550
Processing waveform 65 of 404
75.0 7500
Processing waveform 66 of 404
75.0 14925
Processing waveform 67 of 404
75.0 13275
Processing waveform 68 of 404
75.0 1725
Processing waveform 69 of 404
75.0 2400
Processing waveform 70 of 404
75.0 6375
Processing waveform 71 of 404
75.0 16725
Processing waveform 72 of 404
75.0 6075
Processing waveform 73 of 404
75.0 3450
Processing waveform 74 of 404
75.0 2925
Processing waveform 75 of 404
75.0 4575
Processing waveform 76 of 404
75.0 7050
Processing waveform 77 of 404
75.0 2025
Processing waveform 78 of 404
75.0 2925
Processing waveform 79 of 404
75.0 2850
Processing waveform 80 of 404
75.0 3600
Processing waveform 81 of 404
75.0 3750
Processing waveform 82 of 404
75.0 2100
Processing waveform 83 of 404
75.0 1650
Processing waveform 84 of 404
75.0 4200
Processing waveform 85 of 404
75.0 9075
Processing waveform 86 of 404
75.0 4125
Processing waveform 87 of 404
75.0 2925
Processing waveform 88 of 404
75.0 2250
Processing waveform 89 of 404
75.0 5475
Processing waveform 90 of 404
75.0 2625
Processing waveform 91 of 404
75.0 2400
Processing waveform 92 of 404
75.0 2700
Processing waveform 93 of 404
75.0 9075
Processing waveform 94 of 404
75.0 9075
Processing waveform 95 of 404
75.0 6225
Processing waveform 96 of 404
75.0 7125
Processing waveform 97 of 404
75.0 5850
Processing waveform 98 of 404
75.0 9075
Processing waveform 99 of 404
75.0 3225
Processing waveform 100 of 404
75.0 8100
Processing waveform 101 of 404
75.0 4575
Processing waveform 102 of 404
75.0 2850
Processing waveform 103 of 404
75.0 3375
Processing waveform 104 of 404
75.0 2100
Processing waveform 105 of 404
75.0 9075
Processing waveform 106 of 404
75.0 5850
Processing waveform 107 of 404
75.0 2775
Processing waveform 108 of 404
75.0 2700
Processing waveform 109 of 404
75.0 2100
Processing waveform 110 of 404
75.0 9075
Processing waveform 111 of 404
75.0 3225
Processing waveform 112 of 404
75.0 3300
Processing waveform 113 of 404
75.0 2475
Processing waveform 114 of 404
75.0 3075
Processing waveform 115 of 404
75.0 6450
Processing waveform 116 of 404
75.0 3600
Processing waveform 117 of 404
75.0 4650
Processing waveform 118 of 404
75.0 2475
Processing waveform 119 of 404
75.0 1875
Processing waveform 120 of 404
75.0 3450
Processing waveform 121 of 404
75.0 1500
Processing waveform 122 of 404
75.0 2625
Processing waveform 123 of 404
75.0 3525
Processing waveform 124 of 404
75.0 5625
Processing waveform 125 of 404
75.0 3675
Processing waveform 126 of 404
75.0 9075
Processing waveform 127 of 404
75.0 2700
Processing waveform 128 of 404
75.0 3000
Processing waveform 129 of 404
75.0 3525
Processing waveform 130 of 404
75.0 7200
Processing waveform 131 of 404
75.0 4875
Processing waveform 132 of 404
75.0 6075
Processing waveform 133 of 404
75.0 2700
Processing waveform 134 of 404
75.0 5250
Processing waveform 135 of 404
75.0 12750
Processing waveform 136 of 404
75.0 5100
Processing waveform 137 of 404
75.0 2475
Processing waveform 138 of 404
75.0 6525
Processing waveform 139 of 404
75.0 7200
Processing waveform 140 of 404
75.0 4950
Processing waveform 141 of 404
75.0 13350
Processing waveform 142 of 404
75.0 9525
Processing waveform 143 of 404
75.0 2325
Processing waveform 144 of 404
75.0 2400
Processing waveform 145 of 404
75.0 9075
Processing waveform 146 of 404
75.0 2175
Processing waveform 147 of 404
75.0 3300
Processing waveform 148 of 404
75.0 2700
Processing waveform 149 of 404
75.0 5250
Processing waveform 150 of 404
75.0 3000
Processing waveform 151 of 404
75.0 2775
Processing waveform 152 of 404
75.0 6450
Processing waveform 153 of 404
75.0 3150
Processing waveform 154 of 404
75.0 1650
Processing waveform 155 of 404
75.0 4500
Processing waveform 156 of 404
75.0 2250
Processing waveform 157 of 404
75.0 2175
Processing waveform 158 of 404
75.0 3225
Processing waveform 159 of 404
75.0 4050
Processing waveform 160 of 404
75.0 3300
Processing waveform 161 of 404
75.0 2850

Processing waveform 162 of 404
75.0 2025
Processing waveform 163 of 404
75.0 4800
Processing waveform 164 of 404
75.0 2325
Processing waveform 165 of 404
75.0 6150
Processing waveform 166 of 404
75.0 2775
Processing waveform 167 of 404
75.0 2850
Processing waveform 168 of 404
75.0 4425
Processing waveform 169 of 404
75.0 2250
Processing waveform 170 of 404
75.0 3075
Processing waveform 171 of 404
75.0 2775
Processing waveform 172 of 404
75.0 1875
Processing waveform 173 of 404
75.0 3300
Processing waveform 174 of 404
75.0 1875
Processing waveform 175 of 404
75.0 5550
Processing waveform 176 of 404
75.0 9075
Processing waveform 177 of 404
75.0 6525
Processing waveform 178 of 404
75.0 9075
Processing waveform 179 of 404
75.0 3225
Processing waveform 180 of 404
75.0 3150
Processing waveform 181 of 404
75.0 4275
Processing waveform 182 of 404
75.0 4050
Processing waveform 183 of 404
75.0 5400
Processing waveform 184 of 404
75.0 4725
Processing waveform 185 of 404
75.0 3450
Processing waveform 186 of 404
75.0 6150
Processing waveform 187 of 404
75.0 2250
Processing waveform 188 of 404
75.0 9000
Processing waveform 189 of 404
75.0 3150
Processing waveform 190 of 404
75.0 4950
Processing waveform 191 of 404
75.0 6750
Processing waveform 192 of 404
75.0 3150
Processing waveform 193 of 404
75.0 7050
Processing waveform 194 of 404
75.0 3675
Processing waveform 195 of 404
75.0 3225
Processing waveform 196 of 404
75.0 2325
Processing waveform 197 of 404
75.0 3975
Processing waveform 198 of 404
75.0 6150
Processing waveform 199 of 404
75.0 7875
Processing waveform 200 of 404
75.0 7275
Processing waveform 201 of 404
75.0 6075
Processing waveform 202 of 404
75.0 3900
Processing waveform 203 of 404
75.0 3000
Processing waveform 204 of 404
75.0 3900
Processing waveform 205 of 404
75.0 6525
Processing waveform 206 of 404
75.0 2925
Processing waveform 207 of 404
75.0 2700
Processing waveform 208 of 404
75.0 2250
Processing waveform 209 of 404
75.0 2250
Processing waveform 210 of 404
75.0 9600
Processing waveform 211 of 404
75.0 4650
Processing waveform 212 of 404
75.0 6750
Processing waveform 213 of 404
75.0 7125
Processing waveform 214 of 404
75.0 3000
Processing waveform 215 of 404
75.0 1800
Processing waveform 216 of 404
75.0 4425
Processing waveform 217 of 404
75.0 9450
Processing waveform 218 of 404
75.0 3225
Processing waveform 219 of 404
75.0 2100
Processing waveform 220 of 404
75.0 2700
Processing waveform 221 of 404
75.0 2325
Processing waveform 222 of 404
75.0 2775
Processing waveform 223 of 404
75.0 5850
Processing waveform 224 of 404
75.0 3825
Processing waveform 225 of 404
75.0 6000
Processing waveform 226 of 404
75.0 2025
Processing waveform 227 of 404
75.0 1725
Processing waveform 228 of 404
75.0 2175
Processing waveform 229 of 404
75.0 5400
Processing waveform 230 of 404
75.0 3000
Processing waveform 231 of 404
75.0 5325
Processing waveform 232 of 404
75.0 3750
Processing waveform 233 of 404
75.0 3075
Processing waveform 234 of 404
75.0 5550
Processing waveform 235 of 404
75.0 7350
Processing waveform 236 of 404
75.0 4650
Processing waveform 237 of 404
75.0 2775
Processing waveform 238 of 404
75.0 2325
Processing waveform 239 of 404
75.0 2925
Processing waveform 240 of 404
75.0 3075
Processing waveform 241 of 404
75.0 6375
Processing waveform 242 of 404
75.0 2475
Processing waveform 243 of 404
75.0 3975
Processing waveform 244 of 404
75.0 3150
Processing waveform 245 of 404
75.0 5925
Processing waveform 246 of 404
75.0 5400
Processing waveform 247 of 404
75.0 1875
Processing waveform 248 of 404
75.0 5850
Processing waveform 249 of 404
75.0 1725
Processing waveform 250 of 404
75.0 7650
Processing waveform 251 of 404
75.0 2700
Processing waveform 252 of 404
75.0 9075
Processing waveform 253 of 404
75.0 6975
Processing waveform 254 of 404
75.0 4050
Processing waveform 255 of 404
75.0 3450
Processing waveform 256 of 404
75.0 5925
Processing waveform 257 of 404
75.0 2250
Processing waveform 258 of 404
75.0 3675
Processing waveform 259 of 404
75.0 2625
Processing waveform 260 of 404
75.0 4575
Processing waveform 261 of 404
75.0 2025
Processing waveform 262 of 404
75.0 5175
Processing waveform 263 of 404
75.0 3225
Processing waveform 264 of 404
75.0 4950
Processing waveform 265 of 404
75.0 2925
Processing waveform 266 of 404
75.0 1800
Processing waveform 267 of 404
75.0 8925
Processing waveform 268 of 404
75.0 7350
Processing waveform 269 of 404
75.0 4125
Processing waveform 270 of 404
75.0 3225
Processing waveform 271 of 404
75.0 9075
Processing waveform 272 of 404
75.0 5025
Processing waveform 273 of 404
75.0 8925
Processing waveform 274 of 404
75.0 4800
Processing waveform 275 of 404
75.0 7800
Processing waveform 276 of 404
75.0 2775
Processing waveform 277 of 404
75.0 6600
Processing waveform 278 of 404
75.0 1800
Processing waveform 279 of 404
75.0 2625
Processing waveform 280 of 404
75.0 9075
Processing waveform 281 of 404
75.0 7200
Processing waveform 282 of 404
75.0 5550
Processing waveform 283 of 404
75.0 2775
Processing waveform 284 of 404
75.0 5250
Processing waveform 285 of 404
75.0 1875
Processing waveform 286 of 404
75.0 1875
Processing waveform 287 of 404
75.0 5025
Processing waveform 288 of 404
75.0 3600
Processing waveform 289 of 404
75.0 3375
Processing waveform 290 of 404
75.0 4050
Processing waveform 291 of 404
75.0 3375
Processing waveform 292 of 404
75.0 2925
Processing waveform 293 of 404
75.0 6675
Processing waveform 294 of 404
75.0 3750
Processing waveform 295 of 404
75.0 4575
Processing waveform 296 of 404
75.0 9075
Processing waveform 297 of 404
75.0 3900
Processing waveform 298 of 404
75.0 9075
Processing waveform 299 of 404
75.0 4875
Processing waveform 300 of 404
75.0 3075
Processing waveform 301 of 404
75.0 3150
Processing waveform 302 of 404
75.0 9075
Processing waveform 303 of 404
75.0 7425
Processing waveform 304 of 404
75.0 5025
Processing waveform 305 of 404
75.0 2850
Processing waveform 306 of 404
75.0 1875
Processing waveform 307 of 404
75.0 7125
Processing waveform 308 of 404
75.0 4500
Processing waveform 309 of 404
75.0 6075
Processing waveform 310 of 404
75.0 2475
Processing waveform 311 of 404
75.0 4125
Processing waveform 312 of 404
75.0 2100
Processing waveform 313 of 404
75.0 3975
Processing waveform 314 of 404
75.0 3075
Processing waveform 315 of 404
75.0 5550
Processing waveform 316 of 404
75.0 2250
Processing waveform 317 of 404
75.0 3600
Processing waveform 318 of 404
75.0 5775
Processing waveform 319 of 404
75.0 2325
Processing waveform 320 of 404
75.0 2325
Processing waveform 321 of 404
75.0 3525
Processing waveform 322 of 404
75.0 4500
Processing waveform 323 of 404
75.0 4275
Processing waveform 324 of 404
75.0 3450
Processing waveform 325 of 404
75.0 2550
Processing waveform 326 of 404
75.0 3300
Processing waveform 327 of 404
75.0 9075
Processing waveform 328 of 404
75.0 9075
Processing waveform 329 of 404
75.0 6525
Processing waveform 330 of 404
75.0 6975
Processing waveform 331 of 404
75.0 6750
Processing waveform 332 of 404
75.0 7275
Processing waveform 333 of 404
75.0 2400
Processing waveform 334 of 404
75.0 4125
Processing waveform 335 of 404
75.0 3525
Processing waveform 336 of 404
75.0 5175
Processing waveform 337 of 404
75.0 3750
Processing waveform 338 of 404
75.0 4875
Processing waveform 339 of 404
75.0 2250
Processing waveform 340 of 404
75.0 4125
Processing waveform 341 of 404
75.0 4425
Processing waveform 342 of 404
75.0 5250
Processing waveform 343 of 404
75.0 9075
Processing waveform 344 of 404
75.0 4725
Processing waveform 345 of 404
75.0 2925
Processing waveform 346 of 404
75.0 7350
Processing waveform 347 of 404
75.0 2550
Processing waveform 348 of 404
75.0 2850
Processing waveform 349 of 404
75.0 2550
Processing waveform 350 of 404
75.0 5775
Processing waveform 351 of 404
75.0 4200
Processing waveform 352 of 404
75.0 2175
Processing waveform 353 of 404
75.0 10350
Processing waveform 354 of 404
75.0 15600
Processing waveform 355 of 404
75.0 6450
Processing waveform 356 of 404
75.0 3450
Processing waveform 357 of 404
75.0 11175
Processing waveform 358 of 404
75.0 4725
Processing waveform 359 of 404
75.0 4200
Processing waveform 360 of 404
75.0 3750
Processing waveform 361 of 404
75.0 3975

Processing waveform 362 of 404
75.0 4575
Processing waveform 363 of 404
75.0 3225
Processing waveform 364 of 404
75.0 13500
Processing waveform 365 of 404
75.0 4050
Processing waveform 366 of 404
75.0 5250
Processing waveform 367 of 404
75.0 7050
Processing waveform 368 of 404
75.0 3450
Processing waveform 369 of 404
75.0 6675
Processing waveform 370 of 404
75.0 6750
Processing waveform 371 of 404
75.0 2475
Processing waveform 372 of 404
75.0 7650
Processing waveform 373 of 404
75.0 6750
Processing waveform 374 of 404
75.0 4125
Processing waveform 375 of 404
75.0 7425
Processing waveform 376 of 404
75.0 5400
Processing waveform 377 of 404
75.0 4800
Processing waveform 378 of 404
75.0 10200
Processing waveform 379 of 404
75.0 18225
Processing waveform 380 of 404
75.0 5925
Processing waveform 381 of 404
75.0 3525
Processing waveform 382 of 404
75.0 3300
Processing waveform 383 of 404
75.0 3600
Processing waveform 384 of 404
75.0 4650
Processing waveform 385 of 404
75.0 7875
Processing waveform 386 of 404
75.0 6300
Processing waveform 387 of 404
75.0 6750
Processing waveform 388 of 404
75.0 2925
Processing waveform 389 of 404
75.0 3525
Processing waveform 390 of 404
75.0 7875
Processing waveform 391 of 404
75.0 8925
Processing waveform 392 of 404
75.0 2250
Processing waveform 393 of 404
75.0 2700
Processing waveform 394 of 404
75.0 3000
Processing waveform 395 of 404
75.0 3300
Processing waveform 396 of 404
75.0 4425
Processing waveform 397 of 404
75.0 4500
Processing waveform 398 of 404
75.0 7500
Processing waveform 399 of 404
75.0 4200
Processing waveform 400 of 404
75.0 4800
Processing waveform 401 of 404
75.0 1950
Processing waveform 402 of 404
75.0 4650
Processing waveform 403 of 404
75.0 5475
Training data have been read and features have been extracted  (404, 120)
Computation time:  544.6862058639526
Model will be trained on 5 classes [0 1 2 3 4] ['e' 'h' 'l' 'r' 't']
Features have been scaled
Model has been trained:  RandomForestClassifier(criterion='entropy')
Computation time:  0.568462610244751
Model score is:  1.0
and associated confusion matrix is:
          Predicted class
              e     h     l     r     t 
        e    71                         
        h          97                   
        l                74             
        r                      73       
        t                            89 
StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5)
Cross-validation results:  21.9801980198  +/-  2.89959867058  %
          Predicted class
              e     h     l     r     t 
        e     4    11     6     4    11 
        h     6    15     9     5    14 
        l     4    14     7     5     8 
        r     5    13     6     5     8 
        t     6    14     5     5    14 
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['l', 't'] 0
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'r'] 0
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't'] 0
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't', 'r'] 0
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'h', 'l', 't', 'r'] 0
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['l', 't'] 1
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'r'] 1
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't'] 1
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't', 'r'] 1
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'h', 'l', 't', 'r'] 1
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['l', 't'] 2
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'r'] 2
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't'] 2
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't', 'r'] 2
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'h', 'l', 't', 'r'] 2
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['l', 't'] 3
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'r'] 3
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't'] 3
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['h', 'l', 't', 'r'] 3
r    115
h    107
l    106
t    102
e     93
Name: class, dtype: int64
MV.MBRY..SHZ ['e', 'h', 'l', 't', 'r'] 3
Done

print('Done')

